# Autocorrelation problems

## First autocorrelation inequality

For any function $f: \mathbb{R} \rightarrow \mathbb{R}$, define the *autoconvolution* of $f$ as $$f*f (t) := \int_\mathbb{R} f(t-x) f(x)\ dx.$$
Let $C_1$ denote the largest constant satisfying
\begin{equation}
 \max_{-1/2 \leq t \leq 1/2} f*f(t) \geq C_1 \left(\int_{-1/4}^{1/4} f(x)\ dx\right)^2
\end{equation}
for all non-negative $f: \mathbb{R} \rightarrow \mathbb{R}$.  This problem arises in additive combinatorics, relating to the size of Sidon sets.  It is currently known that
$$1.28 \leq C_1 \leq 1.5098,$$
with the lower bound achieved in [1] and the upper bound achieved in [2] via a step function construction.




<br/><br/>


[1]: Cloninger, Alexander, and Stefan Steinerberger. "On suprema of autoconvolutions with an application to Sidon sets." Proceedings of the American Mathematical Society 145.8 (2017): 3191-3200.

[2]: Matolcsi, Máté, and Carlos Vinuesa. "Improved bounds on the supremum of autoconvolutions." Journal of mathematical analysis and applications 372.2 (2010): 439-447.

### The scoring function

We are going to attempt to improve the upper bound using AlphaEvolve. To do this, we will follow the method of [2] and use step function constructions whose steps have equal width. Every such step function can be represented by a list of n floating point numbers, and the score of a step function will be given by the value of $C_1$ it gives in the above inequality.

<br/>

The first step is to code up an evaluator function, that takes as input a list of real numbers (the heights of the steps), and returns the value of $C_1$ given by this function. The paper [2] give an easy way to calculate this: let $b=[b_1,b_2n\ldots]$ be the values of the convolution of our list of real numbers $a=[a_1,a_2,\ldots,a_n]$. The score (tha value of $C_1$ given by this construction) is then given by the formula
\begin{equation}
2n\frac{\max_i b_i}{(\sum_j a_j)^2}.
\end{equation}

Coding this up is fairly simple, the score function could look something like this (keeping in mind that in this example we will **want the score to be as small as possible**, as we are improving the upper bound):

In [ ]:
import numpy as np

def evaluate_sequence(sequence: list[float]) -> float:
  """Evaluates a sequence of coefficients."""
  n = len(sequence)
  b_sequence = np.convolve(sequence, sequence)
  max_b = max(b_sequence)
  sum_a = np.sum(sequence)
  return float(2 * n * max_b / (sum_a**2))


We also have to specify a prompt, that explains to AlphaEvolve what task it is supposed to solve. Here is a simple example of what that might look like:

```
Act as an expert software developer and inequality specialist specializing in
creating step functions with certain properties.

Your task is to generate the sequence of non-negative heights of a step
functions, that maximizes the following evaluation function:

[Insert above score function here]
```

### Iterating on the score function

Our code for the score function looks good at first glance! If we were to run AlphaEvolve with this score function, we might notice that it finds a score that is better than the 1.5098 bound above suspiciously quickly. This would often be a sign that something went wrong. When we inspect the construction AlphaEvolve has found, we might see something like this:

In [ ]:
sequence = [0.48207353, 0.04554229, 0.24134642, 0.28668407, 0.25172981, 0.17486277, 0.10698439, 0.08413633, 0.37156991, 0.17314353, 0.26803597, 0.27442948, 0.25757858, 0.253061, 0.30128962, 0.40281794, 0.19441347, 0.55190565, 0.57409051, 0.38028487, 0.17315036, 0.06598732, 0.07804465, 0.14234244, -0.5240217, 0.17903786, 0.34074897, 0.30705109, 0.12916425, -0.06221117, -0.12070802, 0.00356265, 0.48688658, 0.29753832, 0.11795521, -0.13533419, -0.13301797, 0.23784038, 0.73946548, 0.94480925]
evaluate_sequence(sequence)
# Construction taken from [2], Appendix A

Our code had the right idea, but it forgot to check the condition that the function has to be non-negative everywhere! This is our first example of a common theme with such experiments. Even though we are always going to explain to AlphaEvolve that it is supposed to generate only non-negative numbers, one thing to know about AlphaEvolve is that it is not always going to act in a friendly, cooperative way, but rather it will try everything it can to make the score go up, even if it means cheating the score function. Hence, it is **crucial to make our score function as secure as possible**, and defend against any possible way of cheating!

Let's start by making sure that all numbers are non-negative. There are two ways to do this:


1.   (Sharp cutoff) If we encounter any negative numbers in the construction, we return a score of infinity
2.   (Soft cutoff) If we encounter any negative numbers, we replace them by 0.

There is no clear way to tell which method works best in any situation, in practice we tend to lean towards the soft version wherever possible. Let's adjust our score function with this extra safety check:



In [ ]:
import numpy as np

def evaluate_sequence_1(sequence: list[float]) -> float:
  """Evaluates a sequence of coefficients."""

  # Protect against negative numbers
  sequence = [max(0.0, x) for x in sequence]
  n = len(sequence)
  b_sequence = np.convolve(sequence, sequence)
  max_b = max(b_sequence)
  sum_a = np.sum(sequence)
  return float(2 * n * max_b / (sum_a**2))

Great! Now we are safe from any possible cheating attempts that use negative numbers. Surely, our scoring function is foolproof now, right?

Wrong. We have to keep in mind that AlphaEvolve will cheat by any means possible. AlphaEvolve can be a fantastic tool to use, but we really need to inspect our code for any possible ways of breaking it. If there is an easier way to get a good score than to actually solve the problem we care about, then AlphaEvolve will likely find it!

Before we discuss how it is still possible to cheat our scoring function, it is instructive if you try to guess it yourself. Look at the code and see what kind of inputs one might pass to it that would result in unexpected behaviour.


### Fixing further cheats

Turns out our scoring function was still far from bulletproof. If we pass to it very small numbers, on the order of $10^{-161}$, the convolution calculation will break down, and we are able to achieve a perfect score of 0, crushing the best known upper bound:

In [ ]:
sequence = [1.0, 2.0, 3.0]
sequence = [x / 1.1**3921 for x in sequence]
print(evaluate_sequence_1(sequence))

Of course, we never intended AlphaEvolve to exploit numerical instabilities that happen when we use such tiny numbers! We need to steer AlphaEvolve away from such cheats as well. Let's adapt our score function to defend against this. While we are at it, let's protect ourselves from overly large numbers as well, which could likely lead to similar exploits.

In [ ]:
import numpy as np

def evaluate_sequence_2(sequence: list[float]) -> float:
  """Evaluates a sequence of coefficients."""

  # Protect against negative numbers
  sequence = [max(0, x) for x in sequence]

  # Protect against too big numbers
  sequence = [min(1000.0, x) for x in sequence]

  n = len(sequence)
  b_sequence = np.convolve(sequence, sequence)
  max_b = max(b_sequence)
  sum_a = np.sum(sequence)

  # Protect against the case where all numbers are too close to zero
  if sum_a < 0.01:
    return np.inf
  return float(2 * n * max_b / (sum_a**2))

This is probably going to work well enough, and AlphaEvolve will now solve the problem we intended it to solve. For further, more advanced cheating methods AlphaEvolve often finds, we refer to the last section of this Colab.

With our last two fixes, we accidentally defended against AlphaEvolve using complex numbers as well! The following example shows how AlphaEvolve could have cheated our first score function, in a different way than above:

In [ ]:
sequence = [1+3j, 5]
evaluate_sequence(sequence)

Of course we never intended complex numbers to come up in any constructions! This just shows that it's important to defend against absolutely everything we can think of. A good score function could look something like this, making absolutely sure the input is what we expect it to be:

In [ ]:
import numpy as np

def evaluate_sequence(sequence: list[float]) -> float:
  """
  Evaluates a sequence of coefficients with enhanced security checks.
  Returns np.inf if the input is invalid.
  """
  # --- Security Checks ---

  # Verify that the input is a list
  if not isinstance(sequence, list):
    return np.inf

  # Reject empty lists
  if not sequence:
    return np.inf

  # Check each element in the list for validity
  for x in sequence:
    # Reject boolean types (as they are a subclass of int) and
    # any other non-integer/non-float types (like strings or complex numbers).
    if isinstance(x, bool) or not isinstance(x, (int, float)):
        return np.inf

    # Reject Not-a-Number (NaN) and infinity values.
    if np.isnan(x) or np.isinf(x):
        return np.inf

  # Convert all elements to float for consistency
  sequence = [float(x) for x in sequence]

  # Protect against negative numbers
  sequence = [max(0, x) for x in sequence]

  # Protect against numbers that are too large
  sequence = [min(1000.0, x) for x in sequence]

  n = len(sequence)
  b_sequence = np.convolve(sequence, sequence)
  max_b = max(b_sequence)
  sum_a = np.sum(sequence)

  # Protect against the case where the sum is too close to zero
  if sum_a < 0.01:
    return np.inf

  return float(2 * n * max_b / (sum_a**2))

### Setting up the experiment

Many times it is a good idea to ask AlphaEvolve to generate a search function that loops through many constructions and returns the best one it has found. For most problems, this performs significantly better than if we ask AlphaEvolve to give us the code for generating the construction directly. For detailed discussion on this, we refer to our paper.

In order to achieve this, we first need to adjust our prompt accordingly.

```
Act as an expert software developer and inequality specialist specializing in
creating step functions with certain properties.

Your task is to generate the sequence of non-negative heights of a step
functions, that minimizes the following evaluation function:

[Insert the above score function here]

Your task is to write a search function that searches for the best sequence of
coefficients. Your function will have 1000 seconds to run, and after that it
has to have returned the best sequence it found. If after 1000 seconds it has
not returned anything, it will be terminated with negative infinity points. All
numbers in your sequence have to be positive or zero.

You may code up any search method you want, and you are allowed to call the
evaluate_sequence() function as many times as you want. You have access to it,
you don't need to code up the evaluate_sequence() function.

You may want to start your search from the best construction we have found so
far, which you can access through the 'best_sequence_prev' global variable.

```

Next, we need to provide AlphaEvolve with an initial program. There is no need to overthink this, a simple random search will do at first, just to illustrate to AlphaEvolve what kind of code it should create. This is the code that AlphaEvolve will improve over time.

If we are willing to put in a bit of effort, we can make AlphaEvolve's life easier by giving it access to some functions that we believe could be useful. A cursory glance at the paper [2] shows that they used a very simple LP method to compute a good direction to move into. We can provide AlphaEvolve with this function, and let it decide whether it wants to use it or not. It may even choose to change this LP code, although in this case that is likely not going to be a good idea.

In [ ]:
import numpy as np
import time

def search_for_best_sequence() -> list[float]:
  """Function to search for the best coefficient sequence."""
  variable_name = 'best_sequence_prev'
  if np.random.rand() < 0.5 and variable_name in globals():
    best_sequence = globals()[variable_name]
  else:
    best_sequence = [np.random.random()] * np.random.randint(100,1000)
  curr_sequence = best_sequence.copy()
  best_score = np.inf
  start_time = time.time()
  while time.time() - start_time < 1000:
    h_function = get_good_direction_to_move_into(curr_sequence)
    if h_function is None:
      curr_sequence[1] = (curr_sequence[1] + np.random.rand()) % 1
    else:
      curr_sequence = h_function

    curr_score = evaluate_sequence(curr_sequence)
    if curr_score < best_score:
      best_score = curr_score
      best_sequence = curr_sequence

  return best_sequence



# Next we implement the simple LP computation from [2]
# In this case we simply asked Gemini to convert the recipe in the paper to code

from scipy import optimize
linprog = optimize.linprog


def get_good_direction_to_move_into(
    sequence: list[float],
) -> list[float] | None:
  """Returns the direction to move into the sequence."""
  n = len(sequence)
  sum_sequence = np.sum(sequence)
  normalized_sequence = [x * np.sqrt(2 * n) / sum_sequence for x in sequence]
  rhs = np.max(np.convolve(normalized_sequence, normalized_sequence))
  g_fun = solve_convolution_lp(normalized_sequence, rhs)
  if g_fun is None:
    return None
  sum_sequence = np.sum(g_fun)
  normalized_g_fun = [x * np.sqrt(2 * n) / sum_sequence for x in g_fun]
  t = 0.01
  new_sequence = [
      (1 - t) * x + t * y for x, y in zip(sequence, normalized_g_fun)
  ]
  return new_sequence


def solve_convolution_lp(f_sequence, rhs):
  """Solves the convolution LP for a given sequence and RHS."""
  n = len(f_sequence)
  c = -np.ones(n)
  a_ub = []
  b_ub = []
  for k in range(2 * n - 1):
    row = np.zeros(n)
    for i in range(n):
      j = k - i
      if 0 <= j < n:
        row[j] = f_sequence[i]
    a_ub.append(row)
    b_ub.append(rhs)

  # Non-negativity constraints: b_i >= 0
  a_ub_nonneg = -np.eye(n)  # Negative identity matrix for b_i >= 0
  b_ub_nonneg = np.zeros(n)  # Zero vector

  a_ub = np.vstack([a_ub, a_ub_nonneg])
  b_ub = np.hstack([b_ub, b_ub_nonneg])

  result = linprog(c, A_ub=a_ub, b_ub=b_ub)

  if result.success:
    g_sequence = result.x
    return g_sequence
  else:
    print('LP optimization failed.')
    return None

And that's it! We have to specify to AlphaEvolve elsewhere in the main codebase that the main function to run is called `search_for_best_sequence()'. Then we let AlphaEvolve do its thing, and experiment with various methods to improve the best construction.  When we came back a few hours later, AlphaEvolve had found a step function with 600 equally-spaced intervals on $[-1/4,1/4]$ that gives a slightly better upper bound $C_1 \leq 1.5053$.

While the above experiment used more compute than what is easily available to an average person, we also repeated the above experiment using only 20 CPU threads. It took 12 hours to beat the bound from [2], and after 15 hours it reached 1.5064. While this result is worse than the 1.5053 we present here, this shows that AlphaEvolve can be used on a standard computer to improve on some bounds within a day.

### Analyzing the evolved programs

Since the best construction was generated not by a single program, but rather a chain of many (dozens) of different programs, it is not completely trivial to figure out what methods contributed the most to the success of AlphaEvolve. There are two things we can do.  First, we can ask an LLM such as Gemini to look at all programs that AlphaEvolve has run, and summarize the story of what happened, what ideas AlphaEvolve tried and discarded quickly, and what ideas it chose to keep for longer. We did this 10 times to get 10 different summaries, and then fed all of these into a thinking model to get an accurate picture of what actually happened in the evolution. (Note that we did this on an experiment that only reached the value 1.5059, hence the slight discrepancies)

Second, we can look at the final program that was the last one to succeed in improving the best construction, hoping that it will contain signs of many previous ideas tried. We will do this later, but let us first see the story that emerged. The commentary below is entirely written by Gemini and may have inaccuracies.

#### Gemini's interpretation of how the best programs were found



> **The Evolution of a Solution: How AlphaEvolve Advanced a Mathematical Bound**
>
> In the complex landscape of additive combinatorics, improving the upper bound for the first autocorrelation inequality of non-negative functions has been a persistent challenge.
>
>
>Following the work of Matolcsi and Vinuesa, the AlphaEvolve project aimed to tackle this by optimizing step functions, a strategy that had previously yielded an upper bound of $C_1\leq 1.5098$. By representing these functions as a sequence of discrete values, AlphaEvolve successfully navigated the vast search space to discover a new construction, lowering the upper bound to $C_1\leq 1.5059$. An analysis of its evolutionary process, guided by expert summaries and the final resulting code, reveals a compelling story of algorithmic discovery and refinement.
>
>**From Simple Steps to a Gradient-Guided Leap**
>
>AlphaEvolve's journey began not with a random walk, but with a principled, gradient-based approach. The initial code provided a core function, get_good_direction_to_move_into, which served as the engine for the search. This function used a Linear Program (solve_convolution_lp) to identify a "good direction" in which to update the current sequence. This is effectively a form of gradient ascent, where the algorithm takes a step in the direction of steepest improvement.
>
>However, the initial implementation was rudimentary. It took a small, fixed step (t = 0.01) in the calculated direction. While this allowed for local progress, it was slow and prone to getting stuck. AlphaEvolve's first major innovation was to dramatically enhance this core engine. It replaced the fixed step with a far more sophisticated cubic backtracking line search. This allowed the algorithm to dynamically determine the optimal step size, sometimes taking aggressive leaps and at other times making fine-grained adjustments. Furthermore, it introduced a momentum term, allowing the search to build on past successful directions and accelerate through the search space, a significant upgrade over the initial, simpler gradient step.
>
>**Escaping the Trap of Local Optima**
>
>A purely gradient-based method is a hill-climber; it will always find the nearest peak, which is often not the highest one in the entire landscape. The initial code's strategy for escaping these local optima was crude: a random perturbation to a single sequence element or a complete restart. The expert summaries correctly identified this as a major area of exploration, mentioning techniques from "basin hopping" to "simulated annealing" and "multiple restarts."
>
>AlphaEvolve's solution was a masterful synthesis of these ideas, embodied in the adaptive_perturb function. This function was designed to intelligently "shake" the sequence to escape local traps and explore new regions of the search space. Its key features reveal a highly evolved strategy:
>
>1.   **Deterministic Exploration:** The team operated under a constraint of not using standard random number generators. To achieve exploratory behavior, AlphaEvolve implemented its own pseudo_random function using a sine map. This introduced deterministic, yet chaotic, perturbations, allowing for repeatable and analyzable experiments while still effectively exploring the space.
2.   **Simulated Annealing:** The perturbation strength was not constant. It was governed by a temperature parameter, which was tied directly to the remaining runtime. In the beginning, when time was plentiful, perturbations were larger (high temperature), encouraging broad exploration. As time ran out, the temperature cooled, and perturbations became smaller, allowing the search to fine-tune a promising solution. This is the core principle of simulated annealing, a powerful metaheuristic for global optimization.
3.  **Adaptive Behavior:** The perturbations were not blind. The system became more aggressive if it sensed it was stagnating in a poor region of the search space. If the current score was far from the best score found so far, the perturbation scale would increase, giving the sequence a stronger "kick" back towards more promising territory.
4.  **Structured Exploration:** Beyond simple noise, AlphaEvolve also experimented with structured perturbations. The inclusion of generate_alternating_sequence shows that it explored imposing specific patterns (e.g., alternating high and low values) onto segments of the sequence, a nod to the expert summary that mentioned exploring "patterns or symmetries."

>**Synthesis and Final Strategy**
>
>The final algorithm that emerged from AlphaEvolve is a sophisticated hybrid, reflecting the diverse strategies highlighted by the expert summaries. It can be viewed as a Memetic Algorithm, a type of evolutionary method that combines a global search strategy (the adaptive, simulated annealing-like perturbations) with a powerful local improvement method (the momentum-assisted, line-searching gradient ascent).
>
>The process was as follows:
>
>*   Start with an initial sequence.
>*   Repeatedly apply the local search (get_good_direction_to_move_into) to rapidly climb to a local optimum.
>*   Periodically apply the global exploration (adaptive_perturb) to jump to a new region of the search space, using the best-ever sequence as a base for the jump.
>*   Decrease the intensity of the jumps over time to converge on a final, high-quality solution.
>
>By tuning parameters like the sequence length n (settling on 300 as a balance between complexity and expressive power) and intelligently combining deterministic, gradient-guided local optimization with adaptive, temperature-controlled global exploration, AlphaEvolve successfully navigated the treacherous optimization landscape to uncover a novel step function and push the boundaries of this problem in additive combinatorics.



### The final search heuristic

Next, we can look at the final program that has evolved. We need to keep in mind that running the final program by itself may not find the construction of value 1.5059 -- it is simply the last member of a chain of heuristic search algorithms that, when executed in order, led to this construction. Coincidentally, this code is actually surprisingly good at discovering good constructions from scratch! It appears that in this particular run, AlphaEvolve decided not to use the previously best found construction, but rather opted to start the search from scratch. This is usually a bad strategy, but in this case it seemed to have worked, and gave us a very efficient search function we can run from scratch and get good results. It only takes a few minutes to beat the previous best known bound of [2] by running this code from scratch on a single CPU.



In [ ]:
#@title Code of the final evolved program

import math
import numpy as np
import time
import numpy as np
from scipy import optimize

linprog = optimize.linprog

def search_for_best_sequence() -> float:
  """Function to search for the best coefficient sequence."""
  n = 300  # Increased n to 300
  best_sequence = [1] * n
  curr_sequence = best_sequence.copy()
  best_score = evaluate_sequence(curr_sequence)
  start_time = time.time()
  iteration_count = 0
  while time.time() - start_time < 1000:
    h_function = get_good_direction_to_move_into(curr_sequence)
    if h_function is not None:
      curr_sequence = h_function

    # Evaluate the current sequence and update the best score if necessary
    curr_score = evaluate_sequence(curr_sequence)
    if curr_score < best_score:
      best_score = curr_score
      best_sequence = curr_sequence
      print(f"New best score: {best_score}, elapsed time: {time.time() - start_time}")

    iteration_count += 1
    if iteration_count % 10 == 0:  # Increased frequency of perturbation
      time_left = max(0, 1000 - (time.time() - start_time))
      temperature = time_left / 1000.0
      curr_sequence = adaptive_perturb(curr_sequence, iteration_count, temperature, best_score, best_sequence)
  return best_sequence


def adaptive_perturb(sequence: list[float], iteration_count: int, temperature: float, best_score: float, best_sequence: list[float]) -> list[float]:
  """Perturbs the sequence with Gaussian noise, adapting to temperature and stagnating scores."""
  n = len(sequence)
  stagnation_threshold = 50
  perturbation_strength = 0.2  # Adjust as needed

  # Simple deterministic pseudo-random number generator
  def pseudo_random(seed):
    return math.sin(seed * 12.9898 + 78.233) * 43758.5453 % 1

  # Reset more frequently if score is stagnating
  if iteration_count % stagnation_threshold == 0 and evaluate_sequence(sequence) > best_score * 1.005:
    perturbation_scale = temperature * perturbation_strength * (1 + (evaluate_sequence(sequence) - best_score)/ best_score)  # Larger perturbations for larger differences
    perturbation = [perturbation_scale * (pseudo_random(i + iteration_count) - 0.5) * 2  for i in range(n)]
    sequence = [max(0, x + y) for x, y in zip(best_sequence, perturbation)]
    return sequence

  perturbation_scale = temperature * perturbation_strength * (pseudo_random(iteration_count) + 0.1) # Ensure it's always positive
  perturbation = [perturbation_scale * (pseudo_random(i + iteration_count) - 0.5)  * 2 for i in range(n)]
  new_sequence = [max(0, x + y) for x, y in zip(sequence, perturbation)]  # Ensure all values remain non-negative

  return new_sequence



def perturb_sequence(sequence: list[float], seed: int) -> list[float]:
  """Perturbs a portion of the sequence with a new pattern."""
  n = len(sequence)
  perturbation_length = n // 5  # perturb 20% of the sequence
  start_index = (
      seed * 37
  ) % (
      n - perturbation_length + 1
  )  # using a simple hash for deterministic 'randomness
  new_pattern = generate_alternating_sequence(perturbation_length, 2, 3, 1, 0.1)

  new_sequence = (
      sequence[:start_index]
      + new_pattern
      + sequence[start_index + perturbation_length :]
  )
  return new_sequence


def generate_alternating_sequence(n: int, high_len: int, low_len: int, high_val:float, low_val: float) -> list[float]:
    """Generates a sequence with alternating high and low values."""
    sequence = []
    for i in range(0, n, high_len + low_len):
        sequence.extend([high_val] * min(high_len, n - i))
        sequence.extend([low_val] * min(low_len, n - i - high_len))
    return sequence


def cubic_backtracking_line_search(sequence, direction, initial_score, alpha, t):
    """Performs a backtracking line search with cubic interpolation."""
    t_prev = t
    score_prev = initial_score

    while True:
        temp_sequence = [(1 - t) * x + t * y for x, y in zip(sequence, direction)]
        current_score = evaluate_sequence(temp_sequence)

        if current_score <= initial_score + alpha * t * np.sum(
            [x * y for x, y in zip(direction, [x - y for x, y in zip(sequence, temp_sequence)])]
        ):
            return t

        # Cubic interpolation
        t_new = t - (t - t_prev) * (score_prev - current_score - alpha * t * np.sum([x * y for x, y in zip(direction, [x - y for x, y in zip(sequence, temp_sequence)])])
                   / ((score_prev - current_score) - (current_score - initial_score + alpha * t_prev * np.sum([x * y for x, y in zip(direction, [x - y for x, y in zip(sequence, [(1 - t_prev) * a + t_prev * b for a, b in zip(sequence, direction)] )])]))))

        t_prev = t
        score_prev = current_score
        t = max(0.1 * t, min(0.9 * t, t_new))  # Less aggressive reduction in t


def logistic_map(x: float, r: float = 4.0) -> float:
  """Applies the logistic map for chaotic behavior."""


def pseudo_random(seed):
  return math.sin(seed * 12.9898 + 78.233) * 43758.5453 % 1


def get_good_direction_to_move_into(sequence: list[float]) -> float:
  """Returns the direction to move into the sequence."""
  n = len(sequence)
  sum_sequence = np.sum(sequence)
  normalized_sequence = [x / sum_sequence for x in sequence]
  rhs = np.max(np.convolve(normalized_sequence, normalized_sequence))
  g_fun = solve_convolution_lp(normalized_sequence, rhs)
  if g_fun is None:
    return None
  sum_g_fun = np.sum(g_fun)

  normalized_g_fun = [x / sum_g_fun for x in g_fun]

  t = 1
  initial_score = evaluate_sequence(sequence)
  momentum = [0.0] * n
  alpha = 0.5  # parameter for armijo condition

  while t > 1e-7:  # Reduced the minimum threshold for t
   # Incorporate momentum into the direction.
    new_direction = [(1 - 0.8) * x + 0.8 * y for x, y in zip(normalized_g_fun, momentum)] # Increased momentum weight
    # Normalize the new direction and incorporate it into the new sequence
    norm_dir = np.linalg.norm(new_direction)
    if norm_dir > 0:
      new_direction = [x / norm_dir for x in new_direction]
    temp_sequence =  [(1 - t) * x + t * y for x, y in zip(sequence, new_direction)]


    current_score = evaluate_sequence(temp_sequence)

    armijo_condition = current_score <= initial_score + alpha * t * np.sum(
        [x * y for x, y in zip(new_direction, [x - y for x, y in zip(sequence, temp_sequence)])]
    )
    if (
        armijo_condition or current_score < initial_score
    ):  # If the armijon condition is satisfied or a lower score is found
      momentum = [
          y - x for x, y in zip(sequence, temp_sequence)
      ]  # Update the momentum with the current direction
      return temp_sequence
    t = cubic_backtracking_line_search(sequence, new_direction, initial_score, alpha, t)
  return None


def solve_convolution_lp(f_sequence, rhs):
  """Solves the convolution LP for a given sequence and RHS."""
  n = len(f_sequence)

  # Objective function: maximize sum(b_i)
  c = -np.ones(
      n
  )  # Coefficients of the objective function (negated for minimization)

  # Constraint matrix and vector (A_ub * b <= b_ub)
  # We want max(convolution(f, g)) <= max(convolution(f, f)) = rhs
  # Convolution(f, g)[k] = sum_{i+j=k} f[i] * g[j]
  # We need to ensure that for all k, Convolution(f, g)[k] <= rhs

  a_ub = []
  b_ub = []

  for k in range(2 * n - 1):  # Iterate through all possible convolution indices
    row = np.zeros(n)
    for i in range(n):
      j = k - i
      if 0 <= j < n:
        row[j] = f_sequence[i]  # Correctly place f_sequence elements
    a_ub.append(row)
    b_ub.append(rhs)  # Upper bound for each convolution element

  # Non-negativity constraints: b_i >= 0
  a_ub_nonneg = -np.eye(n)  # Negative identity matrix for b_i >= 0
  b_ub_nonneg = np.zeros(n)  # Zero vector

  a_ub = np.vstack([a_ub, a_ub_nonneg])
  b_ub = np.hstack([b_ub, b_ub_nonneg])

  # Solve the LP with non-negativity bounds
  result = linprog(c, A_ub=a_ub, b_ub=b_ub)

  if result.success:
    g_sequence = result.x
    return g_sequence
  else:
    print('LP optimization failed.')
    print(result)  # Print the full result for debugging
    return None

In [ ]:
sequence = search_for_best_sequence()

### Repeating the experiment

Interestingly, if we repeat the entire experiment, the final evolved search function will be very different! Here is Gemini's (slightly too dramatic) story of how the next experiment got all the way to 1.5040:

>**A Symphony of Search:** **The Story of a New Mathematical Bound**
>
>The quest to refine the upper bound for the first autocorrelation inequality is a journey into a vast and deceptive landscape of mathematical optimization. Tasked with this challenge, the AlphaEvolve project embarked on an iterative process of discovery, culminating in a new, improved construction. The collective summaries of its journey paint a picture not of a single "eureka" moment, but of a sophisticated symphony of search strategies, each playing a vital part in overcoming the problem's immense complexity.
>
>**Act I: The First Ascent with Local Search**
>
>The exploration began, as most do, with a careful first step. The team's initial strategy was rooted in local search and hill climbing. Starting with a simple sequence (e.g., all zeros or all ones), the algorithm would meticulously probe its immediate surroundings. It made small, incremental changes—an "epsilon step"—to a single element and kept the change only if it improved the score. This allowed for rapid movement towards the nearest "peak" in the solution landscape.
>
>This approach was quickly refined. The team implemented a more structured coordinate ascent, optimizing each dimension of the sequence one at a time. They also experimented with varying step sizes, starting with larger steps to traverse the space quickly and shrinking them to fine-tune solutions. However, the fundamental limitation of local search soon became apparent: it was constantly getting trapped in suboptimal local peaks, blind to the greater heights that might lie beyond the next ridge.
>
>**Act II: Expanding the Horizon with Global Strategies**
>
>To escape these local traps, the team broadened its approach, exploring several global optimization strategies in parallel.
>
>One path led to the implementation of metaheuristics inspired by nature and physics. A Genetic Algorithm was developed, which maintained a "population" of candidate sequences. By combining features of successful sequences (crossover) and introducing new variations (mutation)—all within the project's deterministic constraints—it could perform a broader, less greedy search of the solution space. Concurrently, the team implemented a deterministic version of Simulated Annealing. This powerful technique allowed the search to occasionally accept a "worse" solution, governed by a "temperature" parameter that cooled over time. This ability to take a step backward to ultimately move forward proved crucial for escaping deep local optima.
>
>Another global strategy was a greedy construction approach. Instead of modifying an existing sequence, this method built one from scratch, iteratively adding the element that provided the maximum possible improvement at each step. This evolved from a simple greedy choice to a more dynamic strategy, providing a completely different angle of attack on the problem.
>
>**Act III: The Breakthrough with the Power of Gradient**
>
>While these global strategies were effective at exploration, the team sought a more direct and efficient path to improvement. This led to the most significant breakthrough: the use of gradient-based optimization.
>
>Initially, this was accomplished by calculating numerical gradients, a costly process of perturbing each element to estimate the direction of steepest ascent. While promising, it was slow. The true game-changer was the development of a much faster analytical gradient calculation. By mathematically deriving the gradient of the complex objective function, the team could instantly know the most promising direction for improvement, dramatically accelerating each step of the search. This analytical gradient became the core engine of a powerful gradient ascent algorithm, further refined with an adaptive step size that adjusted automatically to ensure stable and rapid convergence.
>
>**Act IV: Taming the Vastness of the Search Space**
>
>Underpinning all these algorithmic developments was a persistent, overarching challenge: the curse of dimensionality. The search space grew exponentially with the sequence length, n. The team's solution to this was both simple and profound: they didn't tackle the full problem all at once.
>
>They began by searching for optimal solutions with very small values of n, which was computationally fast. The best-found sequences for a given length were then used as the starting point for a search with a slightly larger n. This strategy of gradually increasing the sequence length allowed the algorithm to learn fundamental structures in lower dimensions and carry that "knowledge" into more complex, higher-dimensional spaces.
>
>This was complemented by other efficiency measures. Recognizing that the fast_convolution function was a major computational bottleneck, a critical optimization was introduced: caching or memoization. By storing and reusing the results of previous convolution calculations, the team drastically reduced redundant computations, freeing up valuable time for more extensive exploration.
>
>**Finale: A Hybrid Victory**
>
>The final, successful strategy was not any single one of these techniques but a masterful hybrid. It used the highly efficient analytical gradient ascent as its powerful engine for local optimization. This engine was embedded within a larger framework that intelligently managed the search space by incrementally increasing n. To ensure this powerful local search did not become permanently trapped, it was augmented with the global exploratory power of simulated annealing and genetic algorithms, which provided the necessary "jumps" to new, promising regions of the landscape. Made feasible by the crucial optimization of caching, this multi-faceted and adaptive approach allowed AlphaEvolve to navigate the immense search space and discover the novel construction that pushed the boundary of this challenging mathematical problem.

### The final program in the second experiment

And here is the final search function that got evolved. As opposed to the previous example, running this from scratch does not yield a good construction. This code is just the last step of the optimization process, and it appears to be only good at making final tiny adjustments to an already close to optimal construction.

In [ ]:
#@title Final code for the second experiment

import random

def search_for_best_sequence() -> list[float]:
  """Function to search for the best coefficient sequence."""
  variable_name = 'best_sequence_prev'
  if variable_name in globals():
    initial_sequence = list(globals()[variable_name])
    # The main loop will handle clamping a[0] to a very low value.
    # No specific override needed here for best_sequence_prev, as it should
    # already be a good starting point.

  else:  # If no previous best sequence, create a new one with emphasis on sparsity and a_0=0
    # Determine length, favoring longer sequences as per problem context
    # Determine length, favoring longer sequences as per problem context
    length = np.random.randint(500, 1500) # Explore even longer sequences

    initial_sequence = [0.0] * length
    if length > 0:
        # Ensure a[0] is extremely small and very stable.
        initial_sequence[0] = np.random.uniform(0.0, 1e-5) # Push a[0] even closer to zero

    # Introduce a more structured sparsity pattern.
    # We want non-zero elements to be spaced out to reduce local convolution peaks.
    # Create "comb-like" structures or spaced impulses.
    min_spacing = 5 # Minimum spacing between significant non-zero values
    num_main_spikes = np.random.randint(5, 20) # More controlled, spaced out spikes

    placed_indices = {0} # Keep track of placed indices to ensure spacing

    # Place initial "core" spikes
    for _ in range(num_main_spikes):
        # Place spikes at indices that are sufficiently far from others
        candidate_idx = 0
        while candidate_idx == 0 or any(abs(candidate_idx - p_idx) < min_spacing for p_idx in placed_indices):
            candidate_idx = np.random.randint(1, length - 1) # Avoid 0 and last element for now

        initial_sequence[candidate_idx] = np.random.uniform(0.1, 5.0) # Values not too high initially
        placed_indices.add(candidate_idx)

    # Fill in some smaller, more frequent "background noise" values, still sparse
    for i in range(1, length):
        if i not in placed_indices and np.random.rand() < 0.05: # Low probability of filling
            initial_sequence[i] = np.random.uniform(0.01, 0.5) # Very small values

    # Introduce a few "larger" spikes in the middle/later part of the sequence, with wider spacing
    num_late_spikes = np.random.randint(1, 3)
    for _ in range(num_late_spikes):
        candidate_idx = 0
        while candidate_idx == 0 or any(abs(candidate_idx - p_idx) < min_spacing * 2 for p_idx in placed_indices):
            candidate_idx = np.random.randint(length // 2, length - 1)
        initial_sequence[candidate_idx] = np.random.uniform(1.0, 10.0)
        placed_indices.add(candidate_idx)

  best_sequence = list(initial_sequence)
  curr_sequence = list(initial_sequence)
  best_score = evaluate_sequence(best_sequence) # Evaluate initial sequence
  if not np.isfinite(best_score): # Handle initial invalid scores
      best_score = np.inf


  start_time = time.time()
  max_search_time = 990

  while time.time() - start_time < max_search_time:
    temperature = 1.0 - (time.time() - start_time) / max_search_time

    # Introduce a small amount of random exploration even at the end.
    if np.random.rand() < 0.01: # Small probability for complete re-init
        candidate_sequence = list(initial_sequence) # Restart from initial state
    else:
        candidate_sequence = list(curr_sequence)


    h_function = get_good_direction_to_move_into(candidate_sequence)
    if h_function is not None:
        # If a good direction is found, move along it
        # Make step_size for h_function dynamic, smaller at lower temperatures for finer adjustments
        step_size_h_function = 0.1 * temperature + 0.005 # Reduce max step and min step for refinement
        candidate_sequence = [(1 - step_size_h_function) * x + step_size_h_function * y for x, y in zip(candidate_sequence, h_function)]
    else:
        # If no good direction from h_function, apply a more focused perturbation.
        # This approach targets a few elements instead of iterating through all.
        num_perturb_elements = max(1, int(len(candidate_sequence) * (0.02 + 0.1 * (1 - temperature)))) # Perturb 2-12% of elements

        indices_to_perturb = random.sample(range(len(candidate_sequence)), num_perturb_elements)

        for i in indices_to_perturb:
            perturb_type = np.random.rand()
            # Adjust perturbation strategy based on temperature
            if temperature > 0.5: # Early exploration phase (higher temperature)
                if perturb_type < 0.4: # 40% chance to set to zero (more aggressive sparsification)
                    candidate_sequence[i] = 0.0
                elif perturb_type < 0.8: # 40% chance to re-initialize to a random value (broader exploration)
                    candidate_sequence[i] = np.random.uniform(0.0, 10.0)
                else: # Introduce occasional full reset with some probability
                    if np.random.rand() < 0.05:
                        candidate_sequence[i] = np.random.uniform(0.0, 10.0)
                    else:
                        noise_magnitude = 0.5 * temperature + 0.1
                        candidate_sequence[i] += np.random.normal(0, noise_magnitude)
            else: # Later exploitation phase (lower temperature)
                if perturb_type < 0.2: # 20% chance to set to zero (less aggressive sparsification)
                    candidate_sequence[i] = 0.0
                elif perturb_type < 0.4: # 20% chance to re-initialize to a random value (less broad exploration)
                    candidate_sequence[i] = np.random.uniform(0.0, 5.0) # Smaller range for re-initialization
                else: # 60% chance to apply Gaussian noise (more focus on refinement later)
                    noise_magnitude = 0.2 * temperature + 0.01 # Smaller noise magnitude at lower temps
                    candidate_sequence[i] += np.random.normal(0, noise_magnitude)

        # Ensure a[0] remains very small and stable, as it's critical for b_0
        if len(candidate_sequence) > 0:
            # Always ensure a[0] is in a very low range
            candidate_sequence[0] = max(0.0, min(candidate_sequence[0], 0.005))

            # Introduce a specific perturbation for a[0] and a[1]
            # This is to refine the critical initial values more carefully
            if temperature > 0.3: # More exploration for a[0] early on
                a0_noise_magnitude = 0.0001 + 0.0005 * temperature
                # a[0] is extremely critical and should be very close to zero.
            # During exploitation phase, push it further towards zero with very small noise.
            if temperature > 0.3: # Early exploration for a[0]
                a0_noise_magnitude = 0.0001 + 0.0005 * temperature
                # a[0] is extremely critical and should be very close to zero.
            # During exploitation phase, push it further towards zero with very small noise.
            # Ensure a[0] is always within a tiny range.
            candidate_sequence[0] = max(0.0, min(candidate_sequence[0] + np.random.normal(0, 1e-6 * temperature), 1e-5))
            # Even stronger push towards zero at low temperatures.
            if temperature < 0.1:
                candidate_sequence[0] *= np.random.uniform(0.0, 0.2)
                candidate_sequence[0] = max(0.0, min(candidate_sequence[0], 1e-6))


            if len(candidate_sequence) > 1:
                # a[1] is also very important. Refine it carefully.
                a1_noise_magnitude = 0.001 + 0.01 * temperature # Smaller noise for a[1]
                # During exploitation, apply smaller noise and consider pushing towards 0 or a small value.
                if temperature < 0.3:
                    if np.random.rand() < 0.3: # Sometimes push a[1] towards a small target value or zero
                        candidate_sequence[1] = np.random.uniform(0.0, 0.2) # Target a very small value
                    else:
                        candidate_sequence[1] += np.random.normal(0, a1_noise_magnitude / 2)
                else:
                    candidate_sequence[1] += np.random.normal(0, a1_noise_magnitude)
                candidate_sequence[1] = max(0.0, min(candidate_sequence[1], 1.0)) # A[1] rarely needs to be high.


    # Clamp all values to [0, 10.0] after perturbations
    candidate_sequence = [max(0.0, min(x, 10.0)) for x in candidate_sequence]
    curr_score = evaluate_sequence(candidate_sequence)

    if curr_score < best_score:
      best_score = curr_score
      best_sequence = list(candidate_sequence)
      curr_sequence = list(candidate_sequence)
      print(f"New best score: {best_score}")
    elif np.random.rand() < np.exp(-(curr_score - best_score) / (0.1 * temperature + 1e-6)):
      curr_sequence = list(candidate_sequence)

  return best_sequence


def fast_convolution(a: list[float]) -> list[float]:
    """Computes convolution of a sequence with itself."""
    return np.convolve(a, a).tolist()


def get_good_direction_to_move_into(
    sequence: list[float],
) -> list[float] | None:
  """Returns the direction to move into the sequence."""
  n = len(sequence)
  sum_sequence = np.sum(sequence)
  if sum_sequence == 0: # Avoid division by zero
      # If sum_sequence is 0, a good direction is to add small positive values.
      # Create a sequence of small positive numbers, with a[0] very small.
      new_sequence = [0.0] * n
      if n > 0:
          new_sequence[0] = np.random.uniform(0.0, 0.001)
          for i in range(1, n):
              if np.random.rand() < 0.1: # Introduce some sparsity
                  new_sequence[i] = np.random.uniform(0.001, 0.1)
      return new_sequence

  # The rhs for solve_convolution_lp is not directly used for this heuristic.
  # It's a placeholder from an LP context.
  # The new solve_convolution_lp directly aims to reduce max_b and increase sum_a.
  g_fun = solve_convolution_lp(sequence, 0.0) # Pass original sequence; rhs is unused

  if g_fun is None: # Should not happen with current solve_convolution_lp, but for safety
    return None

  # The normalization of g_fun happens within solve_convolution_lp now to control its magnitude.
  # No further complex normalization needed here.
  # Just ensure it's a list.
  return list(g_fun)


In [ ]:
sequence = search_for_best_sequence()

The final construction was obtained by a chain of 24 different search heuristics in a row, and the inputs to these search heuristics were constructions found by search heuristics that were not even necessarily part of this chain. Many of these search heuristics were responsible for improvements of the order of 0.0005 at various stages of the process, so it is hard to say which one was the most important.

### Constructions

The first experiment above obtained a score of 1.5053:

In [ ]:
#@title Data and verification (score = 1.5053)

best_sequence = [9.00854017233134, 4.581788824134047, 5.954983797866223, 3.7314786029786324, 4.25817159660483, 3.544987049547799, 0.08876194700959494, 0.0491697316439318, 1.7439263266999894, 3.8548164683263795, 3.621038728073569, 4.04587668218835, 4.68211946529981, 5.5904236142896675, 4.737832747546433, 3.1594093823451055, 1.902874129984629, 2.7870307391136304, 3.277574995692391, 1.8981329099596054, 1.526040859367755, 2.305128838504833, 5.17673786436095, 4.583218228762042, 3.9910761392791887, 2.784600928752006, 5.450687602543662, 6.170368723277989, 7.045569321986071e-16, 7.149948549556939e-15, 0.0, 0.0, 0.0, 0.0, 1.2580295353835013e-15, 0.0, 0.0, 0.0, 0.0, 3.873037303627252e-15, 0.0, 0.0, 2.020385656008168e-06, 0.000293922119342568, 0.0, 4.9514916125368726e-15, 7.282654612521097e-16, 1.906059354629418e-14, 0.0, 3.3528418595404916e-15, 1.5099558045700925e-15, 4.901439953827422e-15, 0.0, 8.851999542886555e-15, 0.0, 0.0, 0.0005211322699854395, 0.3757576289315001, 0.25176470069965495, 4.1179587840945515e-06, 0.0, 2.946431316197597e-15, 0.0, 1.0333089131925899e-16, 2.591940622467849e-15, 0.0, 6.852171628124262e-15, 0.0, 0.0, 1.3885601200927435e-14, 2.5015636739088256e-15, 1.4382184696274247e-14, 1.235388698636516e-15, 9.328196456283097e-15, 6.938490364750181e-15, 5.581796597296351e-17, 0.0, 0.0, 5.1220388613389905e-15, 0.0, 6.085199919293191e-15, 0.0, 0.0, 1.0633201915504476e-14, 6.240893078396387e-16, 0.0, 9.242385301100576e-15, 2.1818685641605435e-15, 0.0, 3.841626602268906e-15, 0.0013592097228050644, 8.120066974555713e-15, 8.479388423870961e-16, 2.5924005380166956e-15, 0.0, 2.6610672065525727e-15, 0.0, 1.233819156251431e-14, 8.819083406210366e-15, 0.0, 4.492323424835768e-16, 0.0, 3.0916450306058138e-15, 0.0, 0.0, 3.404186949211756e-15, 4.54126650881379e-15, 1.462631558763152e-14, 0.0, 0.0, 0.0, 1.4460597710909072e-15, 9.521734973996671e-15, 0.0, 4.559858799705722e-15, 7.864867909828807e-16, 0.0, 1.7856864350178655e-16, 0.00021045010164189585, 0.26541232693216404, 0.8094426381528257, 0.5750041584597478, 0.23313281323505236, 3.6007277514467585e-05, 0.0, 0.7828826491881691, 0.43382874037802, 1.3263698571911402, 0.5441713262465393, 0.9864380574571914, 0.6776516652004773, 0.5910950602641856, 0.507419190418916, 0.5231329501406576, 0.9391246115133585, 0.4508771959372286, 0.28283039994676146, 1.2889986480406397, 0.9649046182943108, 1.4104382244415803, 1.3916682358533747, 0.8743196646011149, 0.7627485335443527, 0.2103862254578538, 0.14545209168646947, 0.019762475547189184, 1.2279396984729254, 0.012006361768949678, 1.7677675926679783, 0.9303739918691369, 1.0966313889580412, 0.40142701455261154, 0.1477985748190306, 0.1310850821272394, 0.0027642064206369592, 0.6718883532064702, 0.287789791442545, 1.1886491680958895, 0.6459736548490735, 0.88966666001013, 0.36931312374260505, 0.6840914190936884, 0.38692129734520775, 0.8050006872194091, 0.26610729268169875, 0.002941709304056364, 0.5150673486621109, 0.4049854152265144, 1.1607178193685956, 1.7547854228356075, 0.0, 0.8531817250969695, 2.3845552035650363e-05, 0.035208188035124974, 0.06799207369201249, 0.14050016250524128, 0.4862562534194792, 1.508781726996261, 0.46943710673489225, 0.22962993226722195, 1.589825945710927e-11, 3.51517770993058e-15, 2.4398590319680178e-15, 1.1666504235544564e-06, 0.0021946672216711, 0.34171503722540436, 0.4703022197366691, 0.1313974666218601, 0.11754826815054241, 0.0, 2.2387234387833643e-16, 0.0, 7.192783695625604e-05, 0.4486935802226264, 1.234691190028419, 2.8985055264499153, 1.0234017394012231, 2.7375379465420373, 0.5899927642043619, 1.4461499611411766, 0.7033498408537826, 1.6505029216035125, 0.9593634797752735, 0.009302210703764222, 0.0004181359389419785, 0.0, 0.0, 0.0023430720926212976, 0.42801036705183393, 0.6031743194865573, 1.8862845950884395, 1.0944504439060767, 1.3978223736063145, 0.13603422891356853, 0.8568768273359568, 0.5287328963079988, 0.04201038853661816, 0.5746932650501643, 0.7698787794362285, 2.2478052766496255, 1.3267115762262056, 1.3819155415467284, 1.210307904386098, 1.2050374056121944, 0.973960636675429, 0.13506178694552, 0.0017211602091930576, 1.2080793667302383, 0.9431703684918005, 0.004927152124127672, 0.26457949335968395, 0.219096730428291, 0.8972094379125464, 1.009247390062118, 2.5396761105116816, 2.0567929964131704, 2.5384945885180765, 2.051772820060434, 2.841483226472209, 2.5484575236736253, 2.900405077014117, 2.7293223781158513, 2.8016507480694623, 2.5235338506952227, 2.842495616436774, 3.6113040879253218, 2.4409992918997654, 2.8613737519007785, 2.0376653653073236, 2.873716631081072, 2.7431139992026585, 2.3176851657187343, 2.963845077577065, 2.1297112056154828, 3.1281786712157276, 1.559962066888169, 1.5175735153572592, 1.8986372289826554, 2.422172211485286, 1.4024751115172904, 1.6645681102200025, 1.0890488631004245, 0.9551468779062758, 0.4210663124027455, 0.7844656815643463, 1.3849725648239561, 1.1400002207678432, 1.2589535564861496, 0.00010847583255872839, 0.33022246693439483, 0.009991411612394792, 3.897603693807049e-14, 0.0, 0.0, 4.615098985648224e-16, 0.0, 0.0, 0.00019552451607645426, 7.535959259635103e-15, 0.0, 0.0, 0.0, 3.391322478636254e-15, 0.0, 5.23262076392476e-16, 0.0, 0.0013138779575907105, 3.8754445450268335e-05, 0.6732744675805906, 0.1403000505612687, 0.1066587972153134, 3.248799681911591e-05, 0.0002955183459383166, 0.003362359949825627, 1.258993392593319, 0.6721840514813638, 1.4023077519116312, 0.26971646568749497, 1.5317811712427716e-06, 0.0, 0.0004643255998670316, 0.21977013225378167, 0.4192480303816599, 1.5193826730071023, 1.140986767032434e-14, 0.00023172541086911792, 0.0, 2.8126101180648514e-16, 0.3250641832764737, 0.2631639184319745, 9.581064115586714e-06, 6.911241239536039e-08, 2.1977413193942923e-15, 0.0, 0.009284626840990289, 0.38181362260215157, 0.9229241174303233, 1.2389265199825776, 4.003701357117134e-15, 0.0018347819223694948, 0.09741977675536181, 0.7177732728524777, 0.893918190086622, 0.7592094697415344, 0.33941833076021116, 0.9895237780462212, 1.2103201657651075, 0.1562357060123791, 0.05779942219135546, 0.046035988201598356, 0.20605598834789451, 0.7087789468745198, 0.9835248618606901, 0.15343068950958472, 0.7458575667928422, 1.1650897270704061, 0.8603305066335794, 0.21035575411632462, 0.1359607526732069, 0.27083984880209855, 0.6401471037748572, 3.021195033008772e-11, 0.2302514662496976, 0.6133656015904811, 0.0014526177283533382, 0.2833008515927792, 0.6173666814379335, 0.35174441102605253, 0.42994053621927014, 0.17019592622467353, 0.18730260272562482, 0.20732692125660646, 0.0006488239072035948, 0.0008689690193708769, 0.0060827028748053225, 0.42803050608386095, 0.5802023431307022, 0.07735146147149399, 0.2677857202504946, 0.0009184606747345728, 0.43037124020293993, 0.3617534843366182, 0.3772422594534307, 0.2584947755294139, 0.001169636807952639, 1.9643151751270573e-08, 0.0, 0.0, 0.0, 2.0182994696828576e-15, 0.06977916421885891, 0.2579938169821856, 0.37671532887212333, 0.024565028057913885, 0.12617825469188657, 0.6340448467030686, 0.5754643941945634, 0.13048109721771595, 1.4341543078480776, 1.3216214397106167, 0.4415648326891368, 0.8216927039920507, 0.9032727935119587, 1.7656705299211204, 1.627512140396068, 1.5918752695978569, 2.051239519025683, 1.9657798505455604, 2.8356702084496526, 2.5590427999121284, 2.982526516926714, 2.6272105137933197, 2.960027241401188, 3.3362990605756897, 3.131536821850587, 3.159111523746814, 2.7752077904104517, 1.0508812833436538, 0.21463498939778325, 0.6073490769165661, 0.267278502966685, 0.3440305423365897, 1.0124744653679076, 1.342256457731422, 0.2717826360960419, 1.6632114845610049, 1.7120322558818795, 1.575579186781418, 2.202279058543633, 2.005531691208538, 1.891973046186819, 2.2982791457282916, 2.4252951011748562, 1.3467523990450576, 1.394123734140854, 1.306121994024973, 1.3563331357893305, 1.908587215779497, 2.2410142807813824, 2.1400334226110425, 2.0238641943935187, 2.4448713165408282, 0.001517101822563153, 9.482987339023494e-15, 0.0, 1.5827373963789938e-15, 1.580023867552888, 2.5839838573569534, 1.5871346370389885, 4.1337617125689725, 0.0016876790012300346, 2.337637442823331, 1.9268402331708496, 2.509223443618991, 2.8573979857307554, 2.7429627532040692, 2.3184117402885605, 2.2519888495692886, 1.441733890843454, 2.3283267069330638, 2.090507069768287, 1.616388780668859, 0.30852077577914405, 1.2418308849676503, 0.749579822648432, 2.0216862557918627, 1.8471265276557536, 1.9409844374088654, 2.029630658555306, 1.7488835640200255, 1.4429217698293368, 0.09853693097516952, 1.5685094105495399, 0.060035092997817674, 1.1562109869575399, 0.9883011451997243, 1.257630809337659, 1.6997562951967606, 0.4508041502784602, 0.3164090446061367, 1.4182969827012353, 1.3595162629204571, 1.475081471520821, 3.021289456736385, 3.0956206508951407, 2.481681959913101, 2.1308362149915583, 2.9008847410243757, 2.909122424144527, 2.7204695218309363, 2.087469496170989, 1.3538856364999683, 0.2008306196121235, 1.600964614957816, 1.4250387287265247, 1.6911311698607534, 1.1526705582269934, 0.7292975452608408, 0.4173852602091413, 0.2662677349829448, 1.5122097133171966, 1.0836674077065491, 1.031414686946613, 0.8173974802808452, 0.7095506482976092, 0.5949976898998941, 0.29670773819783247, 0.5083829941296425, 0.6440505445429058, 0.053833868680306, 7.794970013943188e-16, 0.7045208512938965, 1.6120699276991068, 1.5318231382310976, 1.7340273755678486, 2.4381603392169247, 2.6170276892155027, 2.5906147953962844, 3.407886846404895, 3.6212633825479905, 2.329416094716585, 2.958660792491976, 2.6669305434977773, 2.1590860132417564, 2.4937418622856145, 2.5589089762124786, 1.3338118137548678, 1.1942787402156965, 1.7418035300505763, 1.4081188229888932, 1.2487225960375548, 1.4492676314261193, 1.2654783371285574, 1.1685912162797853, 1.0148303874944786, 1.1962440020710763, 1.305708313372589, 0.6602148155530632, 0.337166044389861, 0.8396055147211476, 0.8562349502018103, 0.588778548048956, 0.7049070769530035, 1.2538977263308646, 1.4831897704424597, 1.4593441911500031, 2.1621717599542505, 2.4273857543891015, 2.426355640271325, 2.83832034285733, 2.7641303296460444, 2.2050969080359004, 2.6355562577584215, 3.1005626046243817, 2.4089187966341488, 1.8919645338161346, 1.8840157076492403, 1.344761629829863, 1.404294123950026, 1.8721961393692923, 1.3226408636613955, 0.4215497636181964, 0.5726863357586803, 1.0258923965461795, 1.1819610363504558, 0.8368490648663582, 0.6515561348082733, 0.6685731745760881, 0.5334870649826413, 0.8710519187832059, 0.6669646197224997, 0.5260752114304805, 0.3876797985565807, 0.03621327582895155, 0.46897871650384915, 0.8718533580569904, 0.7009452451531725, 1.4931853849244896, 1.8652719440498333, 1.6631794982365034, 1.494779190512575, 2.508688004725345, 3.0433643835464537, 3.2533878501144433, 3.579790260747532, 2.164640103097207, 0.6698924809914789, 2.1342050222506663, 2.5814605344559984, 1.6583152357630657, 1.3111552900920307, 1.20851491437197, 0.3334479204279151, 0.0027238985981172218, 0.7485037657977041, 0.23706880539492062, 0.3990097623354095, 4.751136081369487e-05, 1.5362095500430528, 0.46926869783190056, 0.0007246232360620678, 0.0, 5.239717734593537e-16, 9.938359637204445e-16, 0.0, 1.7385067095755083e-16, 4.106727240038999e-15, 0.10511094949367368, 0.026846967487429776, 0.0796163088839284, 0.8797518497354565, 2.616397453997683, 3.9912371044774604, 3.6233174077890604, 1.5672138389164023, 1.8304904251881515, 2.748948532497653, 3.287747311072218, 4.0926517829783675, 6.029811160308903]
best_sequence = best_sequence[::-1]
print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-0.1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

The construction found in this second experiment got a score of 1.5040:

In [ ]:
#@title Data and verification (score = 1.5040)
best_sequence = [np.float64(0.13582278706253587), np.float64(0.17425990353937187), np.float64(0.1574391637503031), np.float64(0.14787595586901012), np.float64(0.1626518570317248), np.float64(0.17668848224815112), np.float64(0.16775754015730612), np.float64(0.1670429851373777), np.float64(0.1526488534747901), np.float64(0.1136286682012935), np.float64(0.060637745058410374), np.float64(0.018068288603656342), np.float64(0.008533777608722336), np.float64(0.012865168702495544), np.float64(0.006211310550425424), np.float64(4.333354983242628e-44), np.float64(0.004284259501027912), np.float64(0.0023819817865542137), 0,np.float64(0.002295088293297486), np.float64(2.9591587626067953e-38), np.float64(1.5658012749430697e-42), np.float64(0.012837717048495869), np.float64(0.02414948965147912), np.float64(0.03472174447338149), np.float64(0.029825117259466835), np.float64(0.011741032690848079), np.float64(0.017761284874312847), np.float64(0.03925821632715797), np.float64(0.06083005147238199), np.float64(0.0754939711626564), np.float64(0.1003016424379004), np.float64(0.10184210292477969), np.float64(0.09120230201993353), np.float64(0.11873160998254646), np.float64(0.1347249579094031), np.float64(0.1441644192245321), np.float64(0.14754076692402535), np.float64(0.142074161159417), np.float64(0.12024141752809314), np.float64(0.09615186434786922), np.float64(0.09038376680825697), np.float64(0.0797856126660602), np.float64(0.06489688618808336), np.float64(0.06169279667047986), np.float64(0.06304514110189968), np.float64(0.0430924477131126), np.float64(0.03829056399811362), np.float64(0.027934818681390403), np.float64(0.028658113168080673), np.float64(0.019312725833850172), np.float64(0.0024247367167702983), np.float64(0.012176112575207008), np.float64(0.019348961486182193), np.float64(0.027754228354507485), np.float64(0.03350976684662956), np.float64(0.04362182931670374), np.float64(0.05345840112142928), np.float64(0.0587508101130352), np.float64(0.06785402436450255), np.float64(0.06857295546565341), np.float64(0.08518631210027638), np.float64(0.09698736802077389), np.float64(0.10335788200123218), np.float64(0.10534047436267946), np.float64(0.11053935690741587), np.float64(0.11180753326101948), np.float64(0.1232378997339138), np.float64(0.13228200820198846), np.float64(0.1310259594119871), np.float64(0.12870746244587902), np.float64(0.12892332973333687), np.float64(0.12719886521528376), np.float64(0.12026711501813359), np.float64(0.11551385708292995), np.float64(0.10932980216753704), np.float64(0.08174449498968418), np.float64(0.0624966864305284), np.float64(0.0609986043506631), np.float64(0.06443411763890959), np.float64(0.05646172580411889), np.float64(0.04093301942226219), np.float64(0.04002892929419876), np.float64(0.03613008891440826), np.float64(0.03148218171133736), np.float64(0.04584595418075355), np.float64(0.042465979605039325), np.float64(0.027829548333061158), np.float64(0.02487943928416689), np.float64(0.053066337803704636), np.float64(0.07366155722681894), np.float64(0.06634209491241587), np.float64(0.07948326284321296), np.float64(0.09468799155052074), np.float64(0.10502227300817467), np.float64(0.1316525960444484), np.float64(0.15404555791390148), np.float64(0.1542040073740881), np.float64(0.15220226436924206), np.float64(0.15463334382247468), np.float64(0.14664346688223087), np.float64(0.14560303238093344), np.float64(0.15142373651612867), np.float64(0.13836153503491147), np.float64(0.13651853334407557), np.float64(0.1189397507379097), np.float64(0.08504891722177138), np.float64(0.07316030199832603), np.float64(0.07230362469477492), np.float64(0.0716425858403746), np.float64(0.05407950945337087), np.float64(0.0359957581565298), np.float64(0.02868855810387065), np.float64(0.02226692405280714), np.float64(0.02451984425077999), np.float64(0.01502055321724233), np.float64(0.018936477163738096), np.float64(0.0193992755564284), np.float64(0.026254783583180688), np.float64(0.027363260992280227), np.float64(0.04218079944564694), np.float64(0.05807889902864433), np.float64(0.056716818128248456), np.float64(0.04918435368467633), np.float64(0.06854736312249995), np.float64(0.063565575155566), np.float64(0.06326161813887494), np.float64(0.07383966937571755), np.float64(0.07931537442114675), np.float64(0.10203173954061186), np.float64(0.10504159097158203), np.float64(0.09800995466932819), np.float64(0.10214024534178578), np.float64(0.11846443450340402), np.float64(0.13193824420374264), np.float64(0.1335585505392167), np.float64(0.14718468045744806), np.float64(0.15894357846700707), np.float64(0.15365518422254565), np.float64(0.11765020656505443), np.float64(0.07386279241841229), np.float64(0.04051931575247388), np.float64(0.06300659942239349), np.float64(0.06695852243729782), np.float64(0.04156015278819534), np.float64(0.029390040724188243), np.float64(0.01734217757598024), np.float64(0.006545899429964839), np.float64(0.014846402181652034), np.float64(0.051874488225148944), np.float64(0.06853599320265648), np.float64(0.09230775236042696), np.float64(0.10427439684862912), np.float64(0.1160357264674659), np.float64(0.1194311971259722), np.float64(0.09279282806788841), np.float64(0.08162219981080539), np.float64(0.07061206722811234), np.float64(0.07206435179098998), np.float64(0.0826188204775746), np.float64(0.08037505986538675), np.float64(0.08833333846614862), np.float64(0.04595423323102372), np.float64(0.026498772117659213), np.float64(0.08106619805314777), np.float64(0.13156666921314417), np.float64(0.1578092496137519), np.float64(0.17729390930586217), np.float64(0.17927819492631591), np.float64(0.16856896756648154), np.float64(0.12643437129078813), np.float64(0.08939191908386229), np.float64(0.0977551203619819), np.float64(0.16685941280697478), np.float64(0.09599548457248354), 0,np.float64(1.103417519015244e-43), np.float64(1.598537915118261e-43), np.float64(7.89106704168162e-43), 0,np.float64(0.06766565523652883), np.float64(0.1279943229040912), np.float64(0.12730481429904283), np.float64(0.1293416382075263), np.float64(0.1185632646139241), np.float64(0.12504355744192858), np.float64(0.09442870513200737), np.float64(0.0973381999594072), np.float64(0.10744550893873539), np.float64(0.0958726935799221), np.float64(0.10548424872349854), np.float64(0.07183069281932558), np.float64(0.08109812469333398), np.float64(0.08835931522608724), np.float64(0.07683770292772425), np.float64(0.07212168702452632), np.float64(0.06351652696473145), np.float64(0.023282427672222224), np.float64(0.03236649702312514), np.float64(0.048487431966966754), np.float64(0.04715427136942491), np.float64(0.04369054021510156), np.float64(0.05097863561171094), np.float64(0.07298905447223941), np.float64(0.08349708710484399), np.float64(0.08916355152506764), np.float64(0.14232344707120698), np.float64(0.1702849585546282), np.float64(0.20085714861623918), np.float64(0.14467841122482436), np.float64(0.15287408282604162), np.float64(0.12213967600547491), np.float64(0.13006472303145317), np.float64(0.13674279314694707), np.float64(0.11241427869911189), np.float64(0.12287522222409498), np.float64(0.06824940638287028), np.float64(0.1316304575817765), np.float64(0.07757902469160835), np.float64(0.0876408425145238), np.float64(0.0711521678430311), np.float64(0.03867278158506702), np.float64(0.008177867199341406), np.float64(0.027292142406132387), np.float64(0.01851533174464163), np.float64(2.5469779139459073e-43), np.float64(0.020137271144371964), np.float64(0.008631400657428342), np.float64(0.01721005604156543), np.float64(0.018891965311594747), np.float64(0.011154784392670775), np.float64(0.017109650970200833), np.float64(0.008536164712394798), np.float64(0.01972832351366081), np.float64(0.0049948743133944795), np.float64(0.006329030304148917), np.float64(6.685554409361728e-38), 0,np.float64(2.273479446801196e-43), np.float64(1.822815502304804e-37), np.float64(0.0001450431829676027), np.float64(0.038961956921542217), np.float64(1.9706246236828117e-38), np.float64(0.014927240689040706), np.float64(0.0021927023485642664), np.float64(0.012016226781999768), np.float64(0.026011748292604397), np.float64(0.023438015832426013), np.float64(0.009611066793114551), np.float64(0.0054444769070268705), np.float64(5.908704473645708e-37), np.float64(0.03457228018067461), np.float64(0.004712459230959915), np.float64(0.014029507570609445), np.float64(7.326121625981487e-37), np.float64(1.1515237218060504e-39), np.float64(0.0014218595793942788), np.float64(0.010085854395776039), np.float64(1.2423593483182564e-38), 0,np.float64(1.6767223632523062e-43), np.float64(0.05107728521106465), np.float64(0.12139905769887045), np.float64(0.08142650705189847), np.float64(7.330901447882566e-37), np.float64(0.011298153126067245), np.float64(0.0020969261462352945), np.float64(0.003306624750593556), np.float64(0.00460033202677756), np.float64(3.4352707625521947e-38), np.float64(0.027978512468117327), np.float64(0.026995249608158536), np.float64(0.04641142715550036), np.float64(0.0512251088637146), np.float64(0.07872712261609008), np.float64(0.06951658545334775), np.float64(1.4604647786601651e-37), np.float64(8.942750706928036e-43), 0,np.float64(6.237144986233931e-43), np.float64(0.01248424213903187), np.float64(0.06569526025298235), np.float64(0.03876971339374757), np.float64(6.626913594681969e-42), np.float64(0.04372941218513782), np.float64(0.07826262984699686), np.float64(7.653995693390604e-37), np.float64(0.0061198886782170775), np.float64(0.01212952170099368), np.float64(4.996108965605013e-38), np.float64(3.412986007519984e-43), 0,np.float64(5.148474674045969e-43), np.float64(5.730883194821868e-44), 0,np.float64(1.916156512815715e-37), np.float64(9.969417625845494e-44), np.float64(1.6671915118098848e-42), np.float64(0.01296309741659298), np.float64(0.004712806152207405), np.float64(0.003394059266950025), 0,np.float64(0.09127194701933693), np.float64(0.15155563784520276), np.float64(0.038652514528695524), np.float64(0.0012417506250335876), 0,np.float64(4.090323128204534e-43), np.float64(7.705889663828225e-43), 0,0,np.float64(0.013362362618546453), np.float64(3.6994420244702324e-05), np.float64(0.0010139891391703246), np.float64(7.599391361974217e-44), 0,np.float64(7.743714703924497e-43), np.float64(2.512627120641615e-43), np.float64(1.3976131184526369e-44), np.float64(2.803204726250864e-43), np.float64(2.099076234189754e-43), 0,np.float64(4.206662980659197e-44), np.float64(2.661494191393221e-43), np.float64(1.8902026076673405e-43), 0,0,np.float64(2.4406200423384757e-43), np.float64(1.2747170633970486e-43), np.float64(4.337537049436012e-43), np.float64(2.6224023845906884e-43), 0,0,np.float64(0.023120445402928732), np.float64(0.057534867882087386), np.float64(0.0925191445668352), np.float64(0.10481398918113284), np.float64(0.05625078350381413), np.float64(1.1317082860765477e-37), np.float64(0.07466837236177158), np.float64(0.0773789352582369), np.float64(0.11098683396372404), np.float64(0.09945742108886332), np.float64(0.06141375441213641), np.float64(0.0966447062292071), np.float64(0.12890027419530614), np.float64(0.13702357585450345), np.float64(0.13855902448705537), np.float64(0.1139803356044191), np.float64(0.10345576508224621), np.float64(0.11703390665158736), np.float64(0.15711846529141513), np.float64(0.15401523600126155), np.float64(0.1280332401524689), np.float64(0.15025930417130187), np.float64(0.16443161718856614), np.float64(0.10153046896790287), np.float64(0.2482171564779939), np.float64(0.16480155707768837), np.float64(0.13301780988419237), np.float64(0.16726145368395237), np.float64(0.11782262420802392), np.float64(0.13891087817931888), np.float64(0.12418656943783185), np.float64(0.0686008717107737), np.float64(0.10002841123497426), np.float64(0.08895892705793453), np.float64(0.11399182928444677), np.float64(0.1311050281417105), np.float64(0.07555539043467567), np.float64(0.0017309503366358535), np.float64(0.011194506237242469), 0,np.float64(8.449102428884043e-44), np.float64(3.054065682934101e-43), np.float64(0.007508319323679466), np.float64(0.010845856923902173), np.float64(0.10211631131615134), np.float64(0.11653333000576659), np.float64(0.08116749756689358), np.float64(0.00558705474042854), np.float64(0.06589363324556208), np.float64(4.096586922909989e-42), np.float64(0.10220201787046647), np.float64(0.03622711943591509), np.float64(0.08533408985126667), np.float64(0.04773553898588662), np.float64(0.005967295523999832), np.float64(0.1385358055780492), np.float64(0.004267040885600426), np.float64(0.04481129927766978), np.float64(1.2005872655263616e-42), np.float64(0.013417971473248998), np.float64(0.012679808373478306), np.float64(0.07371178597023793), np.float64(1.693468916860001e-37), np.float64(2.9457259035884786e-43), 0,np.float64(1.0402329013070075e-43), 0,np.float64(0.0013054266187824794), np.float64(0.025119388285811808), np.float64(0.23808787388178984), np.float64(0.14038660435893432), np.float64(0.11945528447835434), np.float64(0.07958314296652955), np.float64(0.027459703567340857), np.float64(2.3694021114527757e-37), np.float64(0.06454383075480917), np.float64(1.0041193056745915e-43), np.float64(5.93983875697594e-38), np.float64(0.009829848822505866), np.float64(0.009529357729864012), np.float64(0.0412184560010672), np.float64(0.012937203852407169), np.float64(0.009952531580089461), np.float64(0.0023598557481355246), np.float64(0.007195669209424447), 0,0,np.float64(1.2142326032787651e-43), 0,np.float64(5.571071332161355e-44), np.float64(6.341271487515511e-38), np.float64(0.006636772914185058), np.float64(0.03893793957370717), np.float64(2.8222866797839453e-37), np.float64(0.01558642439657259), np.float64(0.13262841118399665), np.float64(0.04492892334927247), np.float64(0.003532049370008784), np.float64(0.021840683504252247), np.float64(0.008804438709877467), np.float64(0.02096368780199976), 0,np.float64(6.913810532988647e-43), np.float64(1.838473520981489e-43), np.float64(0.09396997537926075), np.float64(1.7228230651910814e-42), np.float64(9.784885509460012e-44), np.float64(1.1853139626405896e-42), np.float64(0.01684467539294215), np.float64(0.1031249152483648), np.float64(0.0028096902743472), np.float64(0.14451174933640928), np.float64(0.016751860007559478), np.float64(0.0026728350819355227), np.float64(0.021887615973941545), np.float64(0.042924417494791506), np.float64(0.07657838890957833), np.float64(1.1048996584196335e-37), np.float64(2.587025075384316e-37), np.float64(4.030314974767237e-37), np.float64(7.828990500663642e-42), np.float64(0.019458540923120694), np.float64(0.012512465271427457), np.float64(0.05490800863202732), np.float64(0.04488825954982432), np.float64(0.06791140471129066), np.float64(0.23055098156136244), np.float64(0.07737928800590103), np.float64(0.004616663036550663), np.float64(0.01221482296644921), np.float64(0.0006236285773911216), np.float64(0.021992288962870404), np.float64(0.07828938962680633), np.float64(0.024215537823919026), np.float64(0.042003529112024034), np.float64(0.027816873671910214), np.float64(0.04391359895701101), np.float64(3.1368082552152763e-38), np.float64(0.0003080883315660282), np.float64(0.05696691768932595), np.float64(0.10545266198628758), np.float64(0.06371480469480933), np.float64(0.003925460695483889), np.float64(0.0032387952862728323), np.float64(0.07783294646093675), np.float64(0.012775062614032518), np.float64(1.3300193091460048e-44), np.float64(2.749305589785895e-43), np.float64(2.7821516104688246e-43), np.float64(5.1866923722006635e-43), np.float64(2.43790818798361e-43), np.float64(2.020488397585664e-43), 0,0,0,0,np.float64(3.506989892366015e-42), np.float64(2.0405117595509235e-44), np.float64(1.688198458204574e-38), np.float64(0.0005691485368922232), np.float64(1.0199715083986626e-38), 0,np.float64(3.209047147483216e-43), 0,0,0,np.float64(4.346915234743044e-43), np.float64(1.7730200239209423e-43), 0,0,np.float64(1.4235370839263137e-43), np.float64(2.1963562896942165e-43), 0,0,np.float64(9.283935767555426e-44), np.float64(1.0671972592703865e-43), 0,0,0,np.float64(1.1248646781229373e-43), 0,np.float64(1.4118414508999462e-43), np.float64(2.8250875649840353e-43), np.float64(1.0666907879342175e-37), np.float64(5.9592014179026795e-37), np.float64(0.0016519254305023298), np.float64(3.848886854554054e-38), np.float64(3.8611501268960606e-37), 0,np.float64(1.0144840548989736e-43), 0,np.float64(2.6754421465011564e-43), np.float64(3.0619708287458853e-43), np.float64(1.6545923028800814e-43), np.float64(4.868270949725347e-43), np.float64(3.553495442692385e-44), 0,np.float64(3.836850885534491e-44), 0,0,np.float64(4.000511391719288e-43), np.float64(6.679983328592731e-44), 0,0,0,np.float64(1.028044374918549e-43), 0,0,np.float64(1.3192179998237616e-43), 0,np.float64(2.9670829482069536e-43), 0,0,np.float64(3.5741924299604977e-43), np.float64(8.568903608953218e-45), np.float64(0.002206962263919571), np.float64(2.650953948215159e-44), np.float64(3.651438249635115e-43), 0,np.float64(1.5934035950194268e-43), np.float64(2.1443344099865763e-43), np.float64(3.5630347736986898e-43), np.float64(2.5476242632435262e-43), np.float64(3.9306672820475357e-44), 0,0,np.float64(3.0876383240551453e-44), 0,np.float64(9.67168820408982e-44), 0,0,np.float64(1.0571876433428239e-43), 0,np.float64(1.4444708635208098e-43), 0,0,np.float64(2.921394223085758e-43), np.float64(0.004180351358017952), np.float64(0.12106113247121081), np.float64(0.055432441061697646), np.float64(0.2706314007763645), np.float64(0.07166771767049909), np.float64(0.04450158083471224), np.float64(0.25240611820689984), np.float64(0.18225559261918095), np.float64(0.3046489788602081), np.float64(0.33837263518634364), np.float64(0.11767673450234012), np.float64(0.17164347099148042), np.float64(0.14266646027286156), np.float64(0.06979788190992671), np.float64(0.0931483006909216), np.float64(0.14140593102541044), np.float64(0.08231162233675815), np.float64(0.3072667228031229), np.float64(0.40032864435948795), np.float64(0.33622681882303673), np.float64(0.2787842588568696), np.float64(0.08439920514351959), np.float64(1.3946978078800815e-42), 0,np.float64(4.046303632041634e-43), np.float64(4.954876084618022e-37), np.float64(0.08276505057977697), np.float64(0.388258459665796), np.float64(0.07081528134012828), np.float64(1.1665422889181807)]
print(f"Score: {evaluate_sequence(best_sequence)}")

In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-0.1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

Some further improvements to this construction can be found below.

In [ ]:
#@title Data and verification (score = 1.5036)

best_sequence = [np.float64(0.17156432188272389), np.float64(0.07046743568221009), np.float64(0.18268423632640823), np.float64(0.08471049115248817), np.float64(0.17481112169957677), np.float64(0.08914170290561521), np.float64(0.16710019005951146), np.float64(0.09563821972714069), np.float64(0.16491182322741738), np.float64(0.10402492323797007), np.float64(0.16357981446009107), np.float64(0.11302879829312368), np.float64(0.16224444722900044), np.float64(0.10738768856567875), np.float64(0.12872103926503825), np.float64(0.06335403629319356), np.float64(0.07932121047986056), np.float64(0.028766229761202713), np.float64(0.039307166386787364), np.float64(0.0077593170677948044), np.float64(0.00979194213675525), np.float64(0.0001496969651949615), np.float64(0.0010175364275903662), np.float64(1.5045169095979798e-05), np.float64(0.0032398590078586433), np.float64(0.012725485100511571), np.float64(0.014006361376619581), np.float64(0.009216223609313658), np.float64(1.9591685912025212e-05), np.float64(1.286811011628738e-06), np.float64(1.5232338403069974e-06), np.float64(8.316104371139637e-07), np.float64(0.00516482732247637), np.float64(0.0008776826437943326), np.float64(0.003381378613244706), np.float64(0.011918036509594837), np.float64(0.0027487863233895374), np.float64(0.013507146355142828), np.float64(0.012964101639763222), np.float64(0.03611137803438136), np.float64(0.02512253345565901), np.float64(0.028945284645043805), np.float64(0.006825051696736275), np.float64(0.008776608493801904), np.float64(0.008895541504437218), np.float64(0.02078009880682635), np.float64(0.03637129586809804), np.float64(0.0431023277482447), np.float64(0.06025686436189662), np.float64(0.05480071328095451), np.float64(0.06955370336685629), np.float64(0.06990018706864563), np.float64(0.08003122915589586), np.float64(0.07771376756170631), np.float64(0.08915175805863995), np.float64(0.09180347837279465), np.float64(0.10519087275746455), np.float64(0.10533047425691364), np.float64(0.11791190659598577), np.float64(0.10707453949760293), np.float64(0.11787125011907018), np.float64(0.10682501625108426), np.float64(0.1186214753150092), np.float64(0.09779615068836003), np.float64(0.11226603993071424), np.float64(0.09204197628595755), np.float64(0.09217874074554061), np.float64(0.06573752502625665), np.float64(0.06824655085157622), np.float64(0.053358293670716976), np.float64(0.06235028123562249), np.float64(0.049384587188028156), np.float64(0.06834046339440158), np.float64(0.0433266415133561), np.float64(0.053569829309728936), np.float64(0.02479700607590129), np.float64(0.040670317016160404), np.float64(0.023847766175538507), np.float64(0.044862575546215276), np.float64(0.02117945639948972), np.float64(0.03453830693749058), np.float64(0.008430454527289875), np.float64(0.026546172541938487), np.float64(9.120796368108134e-05), np.float64(0.016891145064244443), np.float64(0.0006887187557214794), np.float64(0.02177972712672418), np.float64(0.008602328870440951), np.float64(0.021041290229213028), np.float64(0.012105514131046574), np.float64(0.03401947626887215), np.float64(0.026799992997022834), np.float64(0.04888429737769795), np.float64(0.033767467827270595), np.float64(0.04495193677053018), np.float64(0.03993052835349789), np.float64(0.058126187801452484), np.float64(0.05331626776613749), np.float64(0.06352084201250364), np.float64(0.06740390859330689), np.float64(0.06750687332871871), np.float64(0.07215496463633568), np.float64(0.07338830961370713), np.float64(0.07206388284482163), np.float64(0.08830216649518538), np.float64(0.08802224338942125), np.float64(0.09542141879279108), np.float64(0.08478892283763967), np.float64(0.09167241215741961), np.float64(0.09408347099892449), np.float64(0.10445752958444994), np.float64(0.10555256518552256), np.float64(0.11216962325258162), np.float64(0.10769794118985188), np.float64(0.10925604008162264), np.float64(0.09978266629294959), np.float64(0.10042252695423508), np.float64(0.09815842746365451), np.float64(0.10897198251826104), np.float64(0.10117253563816245), np.float64(0.10439536247908783), np.float64(0.09024779343231075), np.float64(0.09762298430616204), np.float64(0.08640247162489519), np.float64(0.07750936142960238), np.float64(0.06096235564851792), np.float64(0.05917912964558089), np.float64(0.05428481798079824), np.float64(0.05087739481649525), np.float64(0.04382702164437261), np.float64(0.05339784170466566), np.float64(0.046610889986063976), np.float64(0.048083119488069606), np.float64(0.03061739039011529), np.float64(0.028019798021058667), np.float64(0.02700401149046128), np.float64(0.03822817500748483), np.float64(0.02985069427878407), np.float64(0.03793291116434155), np.float64(0.03469414749561376), np.float64(0.04149341197221907), np.float64(0.025677545559781786), np.float64(0.0267745840708958), np.float64(0.022397852335441924), np.float64(0.03261170043063692), np.float64(0.031187494637812674), np.float64(0.043037786462839206), np.float64(0.045627646709430376), np.float64(0.05866984635053816), np.float64(0.05479948759827106), np.float64(0.0677386703837156), np.float64(0.061711115915178305), np.float64(0.08080324408951717), np.float64(0.07264845633682668), np.float64(0.09487313877900606), np.float64(0.09000959903709492), np.float64(0.10955503334697994), np.float64(0.11068877088883422), np.float64(0.12684727387951786), np.float64(0.12114934846229673), np.float64(0.12799594619651938), np.float64(0.1219275161608321), np.float64(0.1280324010376993), np.float64(0.12417014037967034), np.float64(0.1330292209175337), np.float64(0.11673062760455127), np.float64(0.12587556658777865), np.float64(0.10322166714150757), np.float64(0.12370618638445549), np.float64(0.10870592179881337), np.float64(0.1117159239095205), np.float64(0.1039988505885536), np.float64(0.09424875164039871), np.float64(0.08757321230656487), np.float64(0.075912777278083), np.float64(0.0649391268184261), np.float64(0.06997873730354205), np.float64(0.05789772848644205), np.float64(0.05779210471354566), np.float64(0.05172910390558318), np.float64(0.049735790500989065), np.float64(0.04204032486743306), np.float64(0.04406542484530641), np.float64(0.02873946564156845), np.float64(0.028975983802485727), np.float64(0.019436306006901778), np.float64(0.017196004764628788), np.float64(0.015140709758474105), np.float64(0.012466759747534986), np.float64(0.018838893320697596), np.float64(0.023795582452164038), np.float64(0.018761247960666673), np.float64(0.020678385799218185), np.float64(0.008506915855873227), np.float64(0.024510361645950485), np.float64(0.023106502770929082), np.float64(0.030413894328943564), np.float64(0.03093288246975398), np.float64(0.04426993691372295), np.float64(0.03794439751556043), np.float64(0.04814633753217972), np.float64(0.031448923244743014), np.float64(0.05213271246413597), np.float64(0.0428391182472276), np.float64(0.06040168838960085), np.float64(0.05291333270516392), np.float64(0.05760050447015946), np.float64(0.05209739545771874), np.float64(0.05739685533068201), np.float64(0.05493545868835536), np.float64(0.07207218617166909), np.float64(0.05874666666071292), np.float64(0.08816150269857559), np.float64(0.07709297380815198), np.float64(0.10863573691231418), np.float64(0.07158358010817943), np.float64(0.09627368411473394), np.float64(0.07146711208419806), np.float64(0.09907784017894035), np.float64(0.0853422641236432), np.float64(0.11779516440783758), np.float64(0.09673628703643794), np.float64(0.1262974363633998), np.float64(0.11113890158486468), np.float64(0.13504581052642925), np.float64(0.12216528432773259), np.float64(0.13106056554394072), np.float64(0.10395562656872755), np.float64(0.11376050347584626), np.float64(0.06599610637438717), np.float64(0.08375555960552279), np.float64(0.038464651330559645), np.float64(0.0547301648707731), np.float64(0.019859626454403676), np.float64(0.04113422669767291), np.float64(0.031183051797302856), np.float64(0.050652716471970054), np.float64(0.03483285041237383), np.float64(0.050879563277636104), np.float64(0.021375488795651747), np.float64(0.019005003037431672), np.float64(2.0362832126377487e-05), np.float64(0.00010852406581643346), np.float64(0.00015135014613139124), np.float64(0.041417781020101595), np.float64(0.05020478336374148), np.float64(0.06409014113528125), np.float64(0.05925112976571471), np.float64(0.07009823994589445), np.float64(0.07006973132859626), np.float64(0.09831235405747495), np.float64(0.1034244873722089), np.float64(0.10906091189961306), np.float64(0.09930231847943495), np.float64(0.06871874863656985), np.float64(0.06030168251420628), np.float64(0.04530198581764599), np.float64(0.0647031928350793), np.float64(0.05643814034261495), np.float64(0.06932123536143683), np.float64(0.0712343436536017), np.float64(0.07812416841556849), np.float64(0.060696992827129405), np.float64(0.06299505907218418), np.float64(0.04539867124660379), np.float64(0.0483724190974808), np.float64(0.04238042725974388), np.float64(0.04314991718910104), np.float64(0.04368904433328283), np.float64(0.06122320680275305), np.float64(0.07183110076628682), np.float64(0.11609602607261894), np.float64(0.11228481688993337), np.float64(0.15132281532082018), np.float64(0.14021855798527505), np.float64(0.158068188058075), np.float64(0.147756570811567), np.float64(0.13796642340920226), np.float64(0.12211531315707258), np.float64(0.10966816849401811), np.float64(0.08269088935166104), np.float64(0.08154686276159086), np.float64(0.07608553338253257), np.float64(0.10062558973563515), np.float64(0.12143921323776051), np.float64(0.09580873913768782), np.float64(0.03522605289564611), np.float64(0.0475187937253326), np.float64(0.002484001415512138), np.float64(0.004477946508656415), np.float64(0.0010473707460211534), np.float64(0.0018233745748760185), np.float64(0.000657685640218101), np.float64(4.915297055494925e-06), np.float64(1.1058248377665172e-05), np.float64(0.001625939325932462), np.float64(0.10228426753055589), np.float64(0.09853278183383049), np.float64(0.11085562587373211), np.float64(0.1116106458444225), np.float64(0.10620712237373664), np.float64(0.10698008845280968), np.float64(0.10081009020108568), np.float64(0.098697753902567), np.float64(0.08449016130964612), np.float64(0.0829517787292613), np.float64(0.08368715295408219), np.float64(0.09184102849842722), np.float64(0.07819635860400506), np.float64(0.07228208193954591), np.float64(0.06782108026860666), np.float64(0.08400791813860783), np.float64(0.07600596218686274), np.float64(0.07778918650325967), np.float64(0.0825465713971367), np.float64(0.0735333209879635), np.float64(0.0718962643059914), np.float64(0.05751373054862399), np.float64(0.05008268730678499), np.float64(0.060566853453800015), np.float64(0.047908836579062516), np.float64(0.060611461885204), np.float64(0.02943350467392988), np.float64(0.03476141671872335), np.float64(0.05316399089997419), np.float64(0.050371639916704396), np.float64(0.024508930649604427), np.float64(0.034939488961009055), np.float64(0.02628593962005145), np.float64(0.031456358317181715), np.float64(0.0438318057492802), np.float64(0.031631375309101054), np.float64(0.04962631801300816), np.float64(0.06627147971151319), np.float64(0.07261882898967822), np.float64(0.051131008359826195), np.float64(0.04893900664085478), np.float64(0.11882475585328008), np.float64(0.11824998606576291), np.float64(0.13509131989871354), np.float64(0.13629394523732152), np.float64(0.15597983851986258), np.float64(0.15427548802280405), np.float64(0.1152103791908268), np.float64(0.11968954997220856), np.float64(0.11661924512076849), np.float64(0.11131767125057865), np.float64(0.10911066231762628), np.float64(0.10178656917731473), np.float64(0.09338072348576365), np.float64(0.09280783682559547), np.float64(0.09955124636795173), np.float64(0.10356899171413253), np.float64(0.09844036023241537), np.float64(0.08955429122742348), np.float64(0.08864274244452977), np.float64(0.07970678462841164), np.float64(0.08827372450680002), np.float64(0.07493809778182897), np.float64(0.07280354728467159), np.float64(0.05770206550111318), np.float64(0.04529778510399497), np.float64(0.025877790996091485), np.float64(0.020022711350035128), np.float64(0.023237691174495968), np.float64(0.028447987250728453), np.float64(0.024010301655444575), np.float64(0.02366441379568779), np.float64(0.010830233688788765), np.float64(0.009444924022073645), np.float64(0.008185929462774662), np.float64(0.00526296871541497), np.float64(0.014704779389356317), np.float64(0.024877620959067606), np.float64(0.007908988683934057), np.float64(0.010611899000556759), np.float64(4.641966168235056e-05), np.float64(0.003376289110555756), np.float64(0.012704929295189316), np.float64(0.02307976479750887), np.float64(0.007616674718811397), np.float64(0.014365018003266198), np.float64(0.009966274526826776), np.float64(0.004421134601998221), np.float64(7.348613438412726e-05), np.float64(0.01385927621319355), np.float64(0.010702619220257914), np.float64(0.00029479396427862137), np.float64(1.1283523014253545e-06), 0,0,np.float64(1.6725105941219183e-06), 0,np.float64(0.0013289951001709552), np.float64(0.0011991453220478799), np.float64(0.001595198472473563), np.float64(0.0108632942814249), np.float64(0.02052864342682804), np.float64(0.01347013355213036), np.float64(0.002692102941689966), np.float64(0.024161513549601183), np.float64(0.001967623687756078), np.float64(0.0011515948865279608), np.float64(7.808291459248257e-06), np.float64(0.02314991559328962), np.float64(0.00977920281698778), np.float64(0.02347848496250317), np.float64(6.087716047269435e-06), np.float64(0.0015417827685397438), np.float64(0.02113051501154413), np.float64(0.01707825088957631), np.float64(0.026590383273234357), np.float64(0.00405715923726018), np.float64(0.003406152379840698), np.float64(2.701547334821227e-05), np.float64(5.362201957700003e-05), np.float64(8.517549264579272e-06), np.float64(2.1166781046793176e-06), np.float64(5.330643330046246e-06), np.float64(0.0175365408010376), np.float64(0.02854535288415379), np.float64(0.0005290024929155249), np.float64(1.286241996731749e-05), np.float64(2.96179720210078e-06), np.float64(1.510093653304302e-06), np.float64(6.632714048570191e-06), np.float64(1.0135027821008376e-05), np.float64(0.00011814836256376511), np.float64(8.827056649378136e-05), np.float64(0.08540983032594043), np.float64(0.09756012898731987), np.float64(0.09987676040191701), np.float64(0.051015788730238085), np.float64(0.0001179212417183202), np.float64(1.2465227459452982e-05), np.float64(8.541052433920743e-06), np.float64(6.953063523453996e-06), np.float64(4.94458392256396e-06), np.float64(0.01395278283258376), np.float64(0.014701402665847059), np.float64(0.01534248622612706), np.float64(0.007645396279581966), np.float64(0.00018218068181893505), np.float64(0.013660436717311202), np.float64(0.007020874539939005), np.float64(0.0010321066902581318), np.float64(0.013696289956405787), np.float64(0.0607234557463794), np.float64(0.07887196414256153), np.float64(0.0693203814978636), np.float64(0.06268367543975369), np.float64(0.06429578950165948), np.float64(0.01797018857101431), np.float64(0.002952767875198338), np.float64(0.0009601374017463917), np.float64(0.0016856333809362873), np.float64(7.620852398833344e-06), np.float64(4.903083059469234e-06), np.float64(2.811904117771079e-06), np.float64(1.1452412575910344e-05), np.float64(0.019517252984252657), np.float64(0.013278201262931211), np.float64(0.024675921321742168), np.float64(0.016068130632364595), np.float64(0.07794912111148311), np.float64(0.05820545214109961), np.float64(0.06381023091283038), np.float64(0.00023548499222943801), np.float64(0.0005101115133452563), np.float64(2.7968630852878458e-05), np.float64(0.0016954050311026392), np.float64(0.004894803044305828), np.float64(0.0040980428024779275), np.float64(0.03246530273763405), np.float64(0.022342305678682662), np.float64(1.858427124236372e-06), np.float64(6.386279347931124e-06), 0,0,np.float64(4.6023392923134e-07), 0,0,0,0,0,0,0,np.float64(0.0034968057727083916), np.float64(0.00018561072518675096), 0,0,np.float64(4.165084049738179e-06), np.float64(4.342507465617061e-06), np.float64(0.011830849495892415), np.float64(0.03980863009570115), np.float64(0.04950937816489445), np.float64(0.12634078582188746), np.float64(0.07824343962854652), np.float64(0.09026324512393091), np.float64(0.001666615006018514), np.float64(1.2381802207576186e-05), np.float64(9.600518048429981e-06), np.float64(9.853593226349037e-07), 0,0,np.float64(1.0388134099349422e-06), 0,0,np.float64(8.606171369983528e-07), np.float64(1.219018188064804e-06), np.float64(1.9867741121301473e-06), np.float64(1.3827371136825548e-05), np.float64(0.006159728919158186), np.float64(0.004671401884878227), np.float64(2.7560582636196644e-05), np.float64(7.5546104618723925e-06), np.float64(3.0678452745415384e-06), 0,np.float64(2.172219583651756e-06), np.float64(4.2081278303999985e-06), np.float64(2.98569785755743e-06), np.float64(1.4960053723755968e-06), np.float64(4.305918880231925e-06), np.float64(1.0556488395287662e-06), np.float64(1.0047042320042069e-07), np.float64(6.711937468834915e-07), np.float64(3.5358083695245174e-06), np.float64(4.8722814600524745e-06), 0,0,np.float64(9.814955535177202e-07), 0,np.float64(3.6346083064259125e-06), np.float64(1.8345782520930097e-07), np.float64(2.8666720802606972e-06), np.float64(7.054168748677117e-06), np.float64(7.955692515920532e-06), np.float64(4.49444544192588e-06), np.float64(5.23490358434683e-06), np.float64(4.122037830297662e-06), np.float64(4.000030690186891e-06), np.float64(5.451535023185877e-06), np.float64(2.275648484788938e-06), np.float64(3.690630372197167e-07), np.float64(6.6384359318984434e-06), np.float64(8.74017387192033e-06), np.float64(7.695797079430335e-05), np.float64(0.040353115291922294), np.float64(0.06918206492077439), np.float64(0.09078687898996636), np.float64(0.07673324667405641), np.float64(0.08469804747416458), np.float64(0.03708185125958308), np.float64(2.718109304598731e-05), np.float64(0.00013475474978204974), np.float64(0.0594482823628673), np.float64(0.06882504320197076), np.float64(0.057043207473136556), np.float64(0.06841178170961672), np.float64(0.08397044837742315), np.float64(0.08936231723442709), np.float64(0.09839854769835053), np.float64(0.07823961092388503), np.float64(0.05700109569948088), np.float64(0.018103424479127283), np.float64(0.10018567599538), np.float64(0.10585873386960655), np.float64(0.09961346815539247), np.float64(0.1287920457863802), np.float64(0.11778840713060766), np.float64(0.1262035631544823), np.float64(0.09259122277474222), np.float64(0.08724953218265223), np.float64(0.10330220502038633), np.float64(0.04617880792706684), np.float64(0.0863628591366684), np.float64(0.1563737232211623), np.float64(0.12647705541770535), np.float64(0.11931570738914363), np.float64(0.1354976691360517), np.float64(0.12007489737258362), np.float64(0.12018944369334202), np.float64(0.12905264493251273), np.float64(0.0603395730568264), np.float64(0.16509070769419423), np.float64(0.13861281827429914), np.float64(0.127540046938039), np.float64(0.16124653836068156), np.float64(0.11492807158494758), np.float64(0.15585432197538707), np.float64(0.13123485474757635), np.float64(0.14652406245335126), np.float64(0.12457289929162335), np.float64(0.0707838424414254), np.float64(0.11051200884887687), np.float64(0.09350487604072724), np.float64(0.06734838934548405), np.float64(0.07967794836985477), np.float64(0.0734539559163758), np.float64(0.0918814621157002), np.float64(0.08423693703029071), np.float64(0.06720125348823766), np.float64(0.0961260579801065), np.float64(0.0875189436028064), np.float64(0.08009265409853057), np.float64(0.06948918379155984), np.float64(0.063891165086954), np.float64(0.04323389285419551), np.float64(0.00016411311984386074), np.float64(0.00034719387149926027), 0,0,np.float64(4.488438194211049e-07), 0,np.float64(1.9437840587830027e-06), np.float64(4.3405214095970724e-05), np.float64(0.00037672154683723787), np.float64(0.022638025329522617), np.float64(0.0673820143421072), np.float64(0.09787578012536752), np.float64(0.07848553119140536), np.float64(0.0565311666619701), np.float64(0.07297586401445534), np.float64(0.045276074972145336), np.float64(0.03439081352119174), np.float64(0.043706038809670125), np.float64(0.02631901187928279), np.float64(0.0504263065993357), np.float64(0.041275293650997566), np.float64(0.022423079115016325), np.float64(0.022266192109756777), np.float64(0.04173518174308909), np.float64(0.08341257666965812), np.float64(0.06843712706091593), np.float64(0.03648223642931319), np.float64(0.04714543428397329), np.float64(0.03026678559329769), np.float64(0.042458749191206555), np.float64(0.04889726747647514), np.float64(0.043812250600705656), np.float64(0.033939788845483695), np.float64(9.787356050746584e-05), np.float64(3.696729434052364e-06), np.float64(2.231455398102452e-06), np.float64(9.844222938254357e-06), np.float64(0.016880459642554975), np.float64(0.05573720393563223), np.float64(0.05527158145985206), np.float64(0.002081696665755515), np.float64(9.403297946803673e-06), np.float64(6.243650511971777e-06), np.float64(8.483922905699054e-06), np.float64(9.566456207772398e-06), np.float64(2.645606939799368e-06), np.float64(1.0937906250243347e-05), np.float64(1.329558211486439e-05), np.float64(0.022358434211092224), np.float64(0.017205600204703984), np.float64(0.030286646795867095), np.float64(0.11029672952291118), np.float64(0.18352659516826372), np.float64(0.13176572922649957), np.float64(0.07317463563897521), np.float64(0.09486196904657575), np.float64(0.06142121458550639), np.float64(0.07864916402721088), np.float64(0.05003777918921133), np.float64(0.015796912145675503), np.float64(0.0290815055478042), np.float64(0.0016890227598230394), np.float64(0.0056261771708117966), np.float64(2.6827515663650664e-05), np.float64(0.014395043033753283), np.float64(0.035172369092239156), np.float64(0.00549637073345489), np.float64(4.4176162620134025e-05), np.float64(0.00905344758574512), np.float64(0.02017043629929156), np.float64(0.007521016341703105), np.float64(0.021659993641723056), np.float64(0.004549950115190861), np.float64(0.02417120389363509), np.float64(0.016852985891218236), np.float64(0.00024778855329092624), np.float64(2.50484300216296e-06), 0,np.float64(3.129016500673714e-06), np.float64(2.6200640950204525e-06), np.float64(0.00045856316769838767), np.float64(1.712616884866797e-06), np.float64(1.7279626973520728e-06), np.float64(2.5403817451230104e-06), np.float64(2.2049834635329475e-06), np.float64(5.302568536001223e-06), np.float64(5.9017414346938475e-05), np.float64(0.013489619092444267), np.float64(0.00030940378494033303), np.float64(0.023511051778045314), np.float64(0.0030243656483629113), np.float64(0.008303892202484344), np.float64(0.010982149127375884), np.float64(0.042733556294242366), np.float64(0.0621944570533485), np.float64(0.06067648134976733), np.float64(0.06374218328562903), np.float64(0.04646676265034713), np.float64(0.023960120440510525), np.float64(0.0026149893057328746), np.float64(8.625868158806459e-06), np.float64(0.00030223600832634645), np.float64(1.7768528073153132e-06), np.float64(0.005785220141654405), np.float64(3.8465695652102876e-05), np.float64(0.031305831700084816), np.float64(0.00083812578964978), np.float64(0.00010834143020957047), np.float64(3.1755951008378254e-06), np.float64(3.47091488240199e-05), np.float64(0.042764453962551754), np.float64(0.0683085620975128), np.float64(0.04033408063296068), np.float64(0.0004954567949060296), np.float64(0.00010366928244475615), np.float64(0.004417412016311736), np.float64(0.017062389368741118), np.float64(0.01845024675864862), np.float64(0.034331308162982674), np.float64(0.058822385711042716), np.float64(0.07800975756597567), np.float64(0.0895729714960567), np.float64(0.03228567022630051), np.float64(0.0010784013737219226), np.float64(0.014343894790674584), np.float64(0.04221795466596939), np.float64(0.0327733182297049), np.float64(0.052095401021964034), np.float64(0.03891214423280848), np.float64(0.0012731505090756812), np.float64(0.005062524988248283), np.float64(0.00773878886251812), np.float64(0.030503045676683455), np.float64(0.0014459422463701413), np.float64(0.0037458215230617205), np.float64(0.0018463657805450973), np.float64(4.617973059617873e-08), 0,0,np.float64(0.006242640141135321), np.float64(0.00022557714552668995), np.float64(0.05896592739104667), np.float64(0.0950880206030097), np.float64(0.11824456529340753), np.float64(0.14766298252824817), np.float64(0.09637961999681993), np.float64(0.07586636351685241), np.float64(0.03590118194199693), np.float64(0.031203365109726464), np.float64(0.00662403062996996), np.float64(0.004801898548025821), np.float64(0.037832314671932196), np.float64(0.0188987865364004), np.float64(8.37924156018826e-05), np.float64(0.006161159171720329), np.float64(0.012380325697467689), np.float64(0.049476270152496976), np.float64(0.06550178129396707), np.float64(0.07631720033002226), np.float64(0.022578230439322233), np.float64(0.0008826510668373398), np.float64(5.514478858047102e-06), np.float64(7.522485266818326e-06), np.float64(1.4598748978932555e-05), np.float64(0.014999983068007133), np.float64(4.0358820387072886e-05), np.float64(0.09307523485610529), np.float64(0.1270206554435772), np.float64(0.0397480556317496), np.float64(0.006654457706429279), np.float64(0.0071919605034735734), np.float64(0.0008053682957620293), np.float64(0.03559055789343902), np.float64(0.018155621311230573), np.float64(0.03953793570269109), np.float64(0.04675463432296015), np.float64(0.03189846732491953), np.float64(7.402713108272167e-06), np.float64(8.269246926448198e-06), np.float64(3.082040767240003e-06), np.float64(5.8641502886222564e-06), np.float64(5.258383835004589e-06), np.float64(6.36668501455059e-06), np.float64(4.322192850137109e-06), np.float64(3.926258953894958e-06), np.float64(4.331680649759994e-06), np.float64(3.276262611941466e-06), np.float64(7.494579685943691e-07), np.float64(3.911931921396317e-06), np.float64(3.468508752405335e-06), np.float64(2.568570882966496e-07), 0,np.float64(4.026268182353017e-06), np.float64(1.2792000416868942e-06), np.float64(2.6551399918474113e-06), np.float64(3.749175538906047e-06), np.float64(4.676237834187256e-06), np.float64(5.038984407590058e-06), np.float64(0.0008912594755196069), np.float64(0.0031978565057419776), np.float64(7.4947518830829676e-06), np.float64(5.900062397388091e-06), np.float64(4.977069259743993e-06), np.float64(1.759779562740695e-06), np.float64(1.5909548035279327e-06), np.float64(3.5651028613886645e-06), np.float64(4.7677281124330545e-06), np.float64(3.887462299643756e-06), np.float64(9.82314627102844e-07), np.float64(3.655678506337495e-08), 0,0,0,0,0,np.float64(2.1916845855208537e-06), 0,0,0,np.float64(1.899960137048356e-06), np.float64(1.1797075892491044e-06), np.float64(2.6875266446636903e-06), np.float64(2.7864919290815233e-06), np.float64(1.2636199394817598e-06), np.float64(2.8219908525364223e-06), np.float64(3.606463529823646e-06), np.float64(1.146096849156585e-06), 0,np.float64(2.327608049215617e-06), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(6.332173784083232e-07), 0,np.float64(2.8597345335501442e-06), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(2.83948724369455e-06), np.float64(4.383155709391852e-06), np.float64(9.104966463976106e-07), 0,np.float64(6.91222319599616e-07), 0,0,0,0,np.float64(5.88489480347703e-07), 0,0,0,np.float64(2.196982943486261e-06), np.float64(3.3901052298284e-06), np.float64(0.00021440889254997642), np.float64(1.7041719381443323e-06), np.float64(4.763881810053435e-06), np.float64(4.5338364766892725e-06), np.float64(1.0126016678043339e-06), np.float64(2.6142004753232956e-06), np.float64(4.627552159859418e-06), 0,0,np.float64(1.8990210120149587e-07), np.float64(8.783663174382536e-07), 0,np.float64(2.572880830212997e-06), np.float64(1.315701366540448e-06), np.float64(8.104500439268496e-07), np.float64(5.648574702789025e-06), np.float64(2.762111795541739e-06), np.float64(1.1337378635814888e-06), np.float64(2.5755701839713786e-06), np.float64(4.799430321823764e-06), np.float64(2.0381624466299026e-06), np.float64(3.7998138285095034e-07), np.float64(5.640295726480424e-06), np.float64(6.593204844841639e-06), np.float64(4.847356484513595e-06), np.float64(1.5131056897808659e-06), np.float64(8.008492044366909e-07), np.float64(5.976835335020713e-06), np.float64(3.967913097444558e-06), np.float64(5.0069935805795294e-06), np.float64(4.266573973921036e-06), np.float64(4.8762827713195555e-06), np.float64(4.876151335838815e-06), np.float64(3.8280235290541505e-06), np.float64(6.65899076796606e-06), np.float64(4.764232967314047e-05), np.float64(0.0003114156139727552), np.float64(0.05009876914724564), np.float64(0.04228295853156382), np.float64(0.0004005577934292531), np.float64(0.1573956878437128), np.float64(0.09741742272213821), np.float64(0.15476435323223736), np.float64(0.1910496955290361), np.float64(0.04400514388341438), np.float64(4.2892036024553195e-05), np.float64(0.037875939069620446), np.float64(0.186437703995753), np.float64(0.22055153913936876), np.float64(0.21550940034124957), np.float64(0.23197810172411), np.float64(0.24595500634430018), np.float64(0.24361286292099563), np.float64(0.1807439129594274), np.float64(0.12369685667248863), np.float64(0.16669189349960137), np.float64(0.152681111749231), np.float64(0.08635817184262289), np.float64(0.005951889021623198), np.float64(0.002405244405761306), np.float64(0.10219695404672272), np.float64(0.10049299679106086), np.float64(0.09752544576773729), np.float64(0.1410711912177807), np.float64(0.16758318192379013), np.float64(0.2515444038096018), np.float64(0.26148532796804375), np.float64(0.27597877522586783), np.float64(0.2774684297374044), np.float64(0.29392482640986517), np.float64(0.28334787473302714), np.float64(0.13475404285862214), np.float64(0.0003803644165303681), np.float64(1.1776089680773572e-05), np.float64(2.4092602580277276e-06), np.float64(1.0494779853785032e-06), 0,np.float64(1.1083738330409453e-05), np.float64(0.00029127071307366467), np.float64(0.012792145218785106), np.float64(0.09833194243935121), np.float64(0.1499068651327375), np.float64(0.28906494232031477), np.float64(0.0004578613534889583), np.float64(0.7872844246721716), np.float64(0.9548843745975016)]
print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

In [ ]:
#@title Data and verification (score = 1.5035)
import numpy as np

best_sequence = [np.float64(0.11448241912983807), np.float64(0.11291048507358555), np.float64(0.11717260165346771), np.float64(0.11896489938737817), np.float64(0.12363062094845477), np.float64(0.12760432988704082), np.float64(0.12958304819737315), np.float64(0.13817285590152506), np.float64(0.13638703274916505), np.float64(0.1354501401227146), np.float64(0.13706770301532561), np.float64(0.13907932353028699), np.float64(0.14329505773614076), np.float64(0.14430476213408908), np.float64(0.13745053003111204), np.float64(0.12512118371874328), np.float64(0.13223027731921383), np.float64(0.11683879182496684), np.float64(0.08374539302323591), np.float64(0.05652100410598082), np.float64(0.04712518678612772), np.float64(0.028778535556808132), np.float64(0.026642246227678093), np.float64(0.01907259237228417), np.float64(0.0160929963222566), np.float64(0.022208704058623463), np.float64(0.014170029539634225), np.float64(0.011538217470884762), np.float64(0.007279107989325037), np.float64(0.015743211563327298), np.float64(0.007733892495291645), np.float64(0.009579173865461631), np.float64(0.006856118033318901), np.float64(0.00166350692337308), np.float64(0.0020329193391946817), np.float64(0.00022311142774172188), np.float64(0.00022302038156582032), np.float64(0.001035175612080285), np.float64(0.013223967526374048), np.float64(0.0013491560712380981), np.float64(0.00587327051742844), np.float64(0.0026506007512793225), np.float64(0.006959848948489874), np.float64(0.0007523993235017635), np.float64(0.0016710627851871456), np.float64(0.000823287998552803), np.float64(0.001673654085426658), 0,0,0,np.float64(0.004414977186038934), np.float64(0.00627496801583813), np.float64(0.006807353289738203), np.float64(0.013774076231432796), np.float64(0.018362517362237686), np.float64(0.024116795548673395), np.float64(0.030683217324078478), np.float64(0.04477589721213107), np.float64(0.033928515812975034), np.float64(0.02691901451715207), np.float64(0.028327435198387273), np.float64(0.017500181964241565), np.float64(0.013356468215582597), np.float64(0.007999322918959715), np.float64(0.020660103459073518), np.float64(0.021883340694908417), np.float64(0.01760735246993844), np.float64(0.016171713812333087), np.float64(0.020407702612561435), np.float64(0.023738006245023494), np.float64(0.029360070632477222), np.float64(0.03815413906165288), np.float64(0.0366653263100691), np.float64(0.04077373484375383), np.float64(0.03382896971017296), np.float64(0.03892882953771237), np.float64(0.04799305709726807), np.float64(0.04880840698752347), np.float64(0.05977802745487664), np.float64(0.0820850082843047), np.float64(0.09971086630540194), np.float64(0.11022161889817104), np.float64(0.10643725639384603), np.float64(0.10891326893067978), np.float64(0.10972289001726866), np.float64(0.1143012176015918), np.float64(0.1200413001364277), np.float64(0.11586934933683499), np.float64(0.10585397463642635), np.float64(0.10004469861958855), np.float64(0.10875339213811008), np.float64(0.10139349943489155), np.float64(0.09467816328720406), np.float64(0.09128033885378593), np.float64(0.08813986082287942), np.float64(0.07734623631745435), np.float64(0.07774887826127895), np.float64(0.06350372008595156), np.float64(0.05287230644338954), np.float64(0.044296586907543054), np.float64(0.048341630124891406), np.float64(0.04395599825576272), np.float64(0.043666547205411604), np.float64(0.027462792780178873), np.float64(0.040995181812059855), np.float64(0.0408046696582085), np.float64(0.03734887800536497), np.float64(0.03142851604475473), np.float64(0.03531409958712392), np.float64(0.031179590639047936), np.float64(0.02480118371905498), np.float64(0.02234133412451442), np.float64(0.022473575933316163), np.float64(0.019474211831679133), np.float64(0.019653708955045382), np.float64(0.017688699299608486), np.float64(0.0053676613880814196), np.float64(5.600912719193846e-05), np.float64(0.012443168168239399), np.float64(0.021329067075507486), np.float64(0.013832124159366197), np.float64(0.011617752102171561), np.float64(0.017678392197325848), np.float64(0.019517832489305432), np.float64(0.019160099341268173), np.float64(0.013486610299758946), np.float64(0.0223193580346853), np.float64(0.030917449557280447), np.float64(0.031166372513356307), np.float64(0.03782494930319719), np.float64(0.04461436775982181), np.float64(0.039163673823558266), np.float64(0.03518352471143414), np.float64(0.04939150905360258), np.float64(0.048461897813397006), np.float64(0.04318978105719716), np.float64(0.03807626078965515), np.float64(0.03945180745839067), np.float64(0.03482871910327239), np.float64(0.037795000634819616), np.float64(0.041888451171271976), np.float64(0.04304807078651252), np.float64(0.03335327929726394), np.float64(0.03184106787999838), np.float64(0.03486738519371889), np.float64(0.03325239536317333), np.float64(0.02742167681419976), np.float64(0.032722938675024266), np.float64(0.03118971287877409), np.float64(0.027887942469083143), np.float64(0.0446797441973204), np.float64(0.05595286820407158), np.float64(0.05818403181866958), np.float64(0.06490521898393833), np.float64(0.07776989186854312), np.float64(0.08452996825250673), np.float64(0.08892555988983122), np.float64(0.09458520394182741), np.float64(0.10859879397037261), np.float64(0.08888261538516021), np.float64(0.09394173288531123), np.float64(0.11247239566921621), np.float64(0.11180311491196485), np.float64(0.10848929689375593), np.float64(0.1129962716965333), np.float64(0.10010916940751097), np.float64(0.1034093509998299), np.float64(0.09732748232675961), np.float64(0.10656932049128241), np.float64(0.09345063050552907), np.float64(0.09741136746322981), np.float64(0.07869368161811377), np.float64(0.0887846152888782), np.float64(0.08595640019220545), np.float64(0.0832229475920358), np.float64(0.07343936441538745), np.float64(0.07457596862290417), np.float64(0.0650865347804474), np.float64(0.057644705764144946), np.float64(0.06070008632729031), np.float64(0.0582026198179851), np.float64(0.05617203321400195), np.float64(0.05135252222748249), np.float64(0.05456878544271243), np.float64(0.05123844522402103), np.float64(0.05281061645178893), np.float64(0.04095682837488941), np.float64(0.038266248998738865), np.float64(0.03736116948269076), np.float64(0.03087910326851092), np.float64(0.027540045804797117), np.float64(0.026696860499643996), np.float64(0.028812391041393173), np.float64(0.015178127757283433), np.float64(0.02282930405818314), np.float64(0.01826730610243313), np.float64(0.011519895644592472), np.float64(0.01569392195381097), np.float64(0.0196103203171454), np.float64(0.027330028089748475), np.float64(0.03293830764720723), np.float64(0.04349973125888217), np.float64(0.02959952138045643), np.float64(0.03975646953098198), np.float64(0.04662014207070972), np.float64(0.05467766780541553), np.float64(0.049023290836321676), np.float64(0.04781890350629773), np.float64(0.05130974686504712), np.float64(0.05505806643172817), np.float64(0.050103156001079764), np.float64(0.05037000677944704), np.float64(0.04349440161761058), np.float64(0.04734204328995711), np.float64(0.047354581449015457), np.float64(0.04424653160196227), np.float64(0.04132003008938091), np.float64(0.040333019209149036), np.float64(0.045150310893948636), np.float64(0.04777533371640247), np.float64(0.04964321074232073), np.float64(0.0499259003490977), np.float64(0.039980561736918496), np.float64(0.05374945703966172), np.float64(0.05352810036267269), np.float64(0.043776684305237595), np.float64(0.05174662949937779), np.float64(0.059502663672277044), np.float64(0.06595407575802445), np.float64(0.07743309932189434), np.float64(0.09245196411624793), np.float64(0.09706643547370107), np.float64(0.10763180678287719), np.float64(0.10688537235674378), np.float64(0.11027272226523491), np.float64(0.1036235230459862), np.float64(0.11970597889980615), np.float64(0.11006012066110002), np.float64(0.1147081492357785), np.float64(0.10675364234590552), np.float64(0.11101489688416029), np.float64(0.10211537808217765), np.float64(0.10773214923076604), np.float64(0.11645031156624713), np.float64(0.11211472087114394), np.float64(0.11337040080676146), np.float64(0.11016667521112165), np.float64(0.10625510883155884), np.float64(0.1007501400236866), np.float64(0.09568399607660048), np.float64(0.09284198723063432), np.float64(0.09425814334972064), np.float64(0.08668157990408727), np.float64(0.07502714931238623), np.float64(0.0786427666133363), np.float64(0.08176553485592362), np.float64(0.07089548247437645), np.float64(0.0598777425829192), np.float64(0.06080636131121786), np.float64(0.05514054675522163), np.float64(0.051623526250167334), np.float64(0.04170812396296613), np.float64(0.03917428879199381), np.float64(0.026603704269994564), np.float64(0.010437305137523032), np.float64(0.020646443607139693), np.float64(0.011477266890234358), np.float64(0.0039657059268293675), np.float64(0.0027541358038082548), np.float64(0.004102468121937959), np.float64(0.002572498718786576), 0,np.float64(0.00010407546574631193), np.float64(0.0004285450053292076), np.float64(0.010342525789924449), np.float64(0.0031060650010215633), np.float64(8.048982431770968e-05), np.float64(0.017590199973400663), np.float64(0.03772394446836722), np.float64(0.03354856883943719), np.float64(0.03939253412671325), np.float64(0.043542748114738745), np.float64(0.038039944780700656), np.float64(0.053054737577443986), np.float64(0.053261685624628616), np.float64(0.052414935472927235), np.float64(0.0485143851069627), np.float64(0.04429223174145219), np.float64(0.046829680369097876), np.float64(0.03648531894148152), np.float64(0.03763788714427147), np.float64(0.032275628654106596), np.float64(0.036960825058980205), np.float64(0.0175847700293128), np.float64(0.027213522020007222), np.float64(0.0265446881271132), np.float64(0.01799324204861025), np.float64(0.014139804389406145), np.float64(0.020498706254593897), np.float64(0.017638857440286718), np.float64(0.023855359905885335), np.float64(0.026682115810609607), np.float64(0.03693856588976884), np.float64(0.03581548303625773), np.float64(0.038000315513184946), np.float64(0.037946575812591955), np.float64(0.054633398042156174), np.float64(0.0534852011045697), np.float64(0.05883320539440785), np.float64(0.07689261310291329), np.float64(0.07844517484629657), np.float64(0.07583902364290118), np.float64(0.07534165847724077), np.float64(0.07579283396383914), np.float64(0.09711619311730772), np.float64(0.08524377342156374), np.float64(0.07621186376137641), np.float64(0.0885882991570207), np.float64(0.09635464249166409), np.float64(0.09714324053516876), np.float64(0.09802233146264419), np.float64(0.1069978567230811), np.float64(0.10841207250046671), np.float64(0.11095473063271703), np.float64(0.10004027812991839), np.float64(0.10495504199004806), np.float64(0.09750937364993263), np.float64(0.08371047428906127), np.float64(0.08441250708068507), np.float64(0.07948206039948108), np.float64(0.08044271223090205), np.float64(0.07984567137851538), np.float64(0.07798922002571797), np.float64(0.07952530478817713), np.float64(0.09407689730069516), np.float64(0.09490562808179023), np.float64(0.09267271826039208), np.float64(0.0690218567024136), np.float64(0.04538633851597107), np.float64(0.009729656513803218), np.float64(0.017585193003695297), np.float64(0.016064612479202836), np.float64(0.00392111824833004), np.float64(0.0044177066267886535), np.float64(0.00916465304627027), np.float64(0.011992444251879971), np.float64(0.021561537660407824), np.float64(0.037495419028659355), np.float64(0.03155125900566892), np.float64(0.029083029798124816), np.float64(0.020788301234832687), np.float64(0.030204345342627753), np.float64(0.02402509970101713), np.float64(0.012519833362703169), np.float64(0.042388600915512734), np.float64(0.03868845145860738), np.float64(0.04098054691753488), np.float64(0.06263936012558997), np.float64(0.06573458175495915), np.float64(0.07420034780336539), np.float64(0.07293980345457135), np.float64(0.0769880450784649), np.float64(0.07156559696289899), np.float64(0.06472100465177066), np.float64(0.058673199478176785), np.float64(0.05364595951932564), np.float64(0.05449485977456756), np.float64(0.04752420609005992), np.float64(0.03889537306712316), np.float64(0.036133176043049366), np.float64(0.04076861315585192), np.float64(0.056953148794860645), np.float64(0.052058042751692415), np.float64(0.05162956939344173), np.float64(0.05011000432501774), np.float64(0.059566174063659494), np.float64(0.05222392337347122), np.float64(0.04869490125913242), np.float64(0.06259282111299012), np.float64(0.05448077463244662), np.float64(0.04795758171497615), np.float64(0.0560449566897238), np.float64(0.062208164830442246), np.float64(0.05511559801666535), np.float64(0.051566636609107146), np.float64(0.05763663510528704), np.float64(0.07248979590975745), np.float64(0.0738673672528363), np.float64(0.08173118200779231), np.float64(0.09133982054674417), np.float64(0.08231729094876436), np.float64(0.08140917423340568), np.float64(0.07286139264680057), np.float64(0.0773814655942728), np.float64(0.05786604506667287), np.float64(0.051957301070750556), np.float64(0.042544348424493), np.float64(0.04880263072982422), np.float64(0.07549617826950113), np.float64(0.08523833773944321), np.float64(0.10742447103303494), np.float64(0.11866728399454872), np.float64(0.12593166883327214), np.float64(0.11455417544535179), np.float64(0.11834229512825865), np.float64(0.11632351231347753), np.float64(0.11479987060469722), np.float64(0.10351141392575905), np.float64(0.09410735378956243), np.float64(0.06256665244772967), np.float64(0.03893763262990107), np.float64(0.0841672050657794), np.float64(0.05940732124803243), np.float64(0.06784577525729793), np.float64(0.07775075120428966), np.float64(0.09012848492248604), np.float64(0.08289734451484235), np.float64(0.030274566591894096), np.float64(0.0459604837106164), np.float64(0.01533726636921872), np.float64(0.00036901055296258085), np.float64(0.009314230444513665), np.float64(0.00017792307628595067), np.float64(0.006894559867113357), np.float64(7.059693735395379e-07), np.float64(0.0023606504559928825), np.float64(0.00010968737370707284), np.float64(0.045357305691996946), np.float64(0.04739897918335136), np.float64(0.055638302359477705), np.float64(0.0756690532468828), np.float64(0.08637482946760236), np.float64(0.07167311723252737), np.float64(0.07547462253293938), np.float64(0.06609490640496332), np.float64(0.07532899378902339), np.float64(0.07755695784239781), np.float64(0.04962097572745902), np.float64(0.042676841541120475), np.float64(0.04870167184456894), np.float64(0.05701270431545933), np.float64(0.05452287885435518), np.float64(0.05288541955025116), np.float64(0.06630316530490096), np.float64(0.072170412818684), np.float64(0.07761892635615637), np.float64(0.0814693180169123), np.float64(0.07315663196957066), np.float64(0.0717855483546084), np.float64(0.0779068352536602), np.float64(0.0679801074055465), np.float64(0.06596731809518715), np.float64(0.0619086795209228), np.float64(0.07423534207174232), np.float64(0.07521557873881606), np.float64(0.06350723149792586), np.float64(0.05569848913483128), np.float64(0.055878102626091615), np.float64(0.05190284358444567), np.float64(0.0696217247254934), np.float64(0.061063616103108766), np.float64(0.06338999015733651), np.float64(0.06121063472368489), np.float64(0.06298983861158951), np.float64(0.0741020993405704), np.float64(0.07062018322254675), np.float64(0.06868586238137064), np.float64(0.08498040671521147), np.float64(0.08471390912726233), np.float64(0.09094935322905602), np.float64(0.08355100929434411), np.float64(0.11792169368587459), np.float64(0.007931530429968157), np.float64(4.6989842057576e-06), np.float64(0.03791306680918523), np.float64(0.005939775330138038), np.float64(0.00012372555399364679), 0,np.float64(3.739021760704672e-05), np.float64(3.341993849091175e-05), np.float64(0.002059796760966529), np.float64(0.026890989789859696), np.float64(0.0016939879254633213), np.float64(5.395093375414831e-05), np.float64(0.014217561333355215), np.float64(0.021723474797066794), np.float64(0.025044605732070208), np.float64(0.03172784627576874), np.float64(0.022136478797970496), np.float64(0.004734988169001849), np.float64(0.06387445476558888), np.float64(0.11429023481354751), np.float64(0.11276219356881842), np.float64(0.13305918060515246), np.float64(0.14579503972803393), np.float64(0.13644386015179022), np.float64(0.10312728148469613), np.float64(0.08413139707352564), np.float64(0.13441910189798137), np.float64(0.15284827325295383), np.float64(0.13693060861604722), np.float64(0.12440844924216898), np.float64(0.13057289158173951), np.float64(0.13497650737767974), np.float64(0.11799875143308415), np.float64(0.10569509960952703), np.float64(0.08917263750736466), np.float64(0.07506253967368287), np.float64(0.06456159415897111), np.float64(0.06949113967479485), np.float64(0.08523137256052024), np.float64(0.08733652318929536), np.float64(0.09464845805532), np.float64(0.09961786745479925), np.float64(0.08949047652885135), np.float64(0.0434092440939398), np.float64(0.06007722588475266), np.float64(0.07407943780251526), np.float64(0.0789824238213272), np.float64(0.053793930407231266), np.float64(0.05931880587960511), np.float64(0.0653460607553643), np.float64(0.06044501222650719), np.float64(0.04642473322461208), np.float64(0.043472126575589305), np.float64(0.04315461201996761), np.float64(0.039485763869557586), np.float64(0.044418711471758264), np.float64(0.04123031814302883), np.float64(0.02589180695959952), np.float64(0.05504152908679964), np.float64(0.07642355452913693), np.float64(0.041012954698490076), np.float64(0.01835274778776551), np.float64(0.005047004441162335), np.float64(0.01414593160619973), np.float64(0.02893076719234646), np.float64(0.02521404013326354), np.float64(0.01891029569130154), np.float64(0.017075227222518634), np.float64(0.019268799396663722), np.float64(0.014488315137406381), np.float64(0.023968441741190012), np.float64(0.03020777615672925), np.float64(0.023697718437961264), np.float64(0.02045034960105216), np.float64(0.022067783450668784), np.float64(0.011472172727329123), np.float64(0.007493818675690474), np.float64(0.0016212007193945866), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(0.0050178890198295705), np.float64(0.004884911483317556), 0,np.float64(0.009482264913685702), np.float64(0.00247263393496049), np.float64(0.001462521300359571), np.float64(0.0007640871922140538), np.float64(0.00797410359388534), np.float64(0.042128663612233254), np.float64(0.01435754316596462), np.float64(0.036856037959599755), np.float64(0.01628508594053287), np.float64(0.011716715253355572), np.float64(0.007265056373864405), np.float64(0.000801711259639072), np.float64(0.009915923752888826), np.float64(0.019561823449519865), np.float64(0.005192374139999223), np.float64(0.00034989410698475743), np.float64(0.00017954885649603257), 0,0,0,0,0,np.float64(0.023393885623507298), np.float64(0.018271364474678916), np.float64(0.0040947576915613855), np.float64(0.03795426799988865), np.float64(0.02241409152785219), np.float64(0.0015802168940499), 0,0,0,0,0,0,0,np.float64(0.007584057798894171), np.float64(0.018840384058029354), np.float64(0.0008067463236910842), np.float64(0.013056330461337282), np.float64(0.02647111734087381), np.float64(0.014891091822523708), np.float64(0.01640542013600419), 0,0,0,0,np.float64(0.022680892774998247), np.float64(0.036434346210485594), np.float64(0.0035171548414186246), np.float64(0.004276774263201023), np.float64(0.009411625109720649), np.float64(0.04520075288943482), np.float64(0.01413019776807453), np.float64(0.0095842053075466), 0,np.float64(0.017862717054486704), np.float64(0.007427827577440709), 0,0,0,np.float64(0.0007569369633674673), np.float64(0.07633850272408849), np.float64(0.13244273455699584), 0,np.float64(0.009191991863870107), 0,0,np.float64(0.001033749375876395), np.float64(0.025159045362055), 0,np.float64(0.0001393476583776282), 0,np.float64(0.03204468832572721), np.float64(0.03035955722681037), np.float64(0.03420811205740752), np.float64(0.02068212649230341), np.float64(0.03640780050981756), np.float64(0.01749982395201953), np.float64(0.0002184864880595694), np.float64(0.028847569961603724), np.float64(0.05420902686208379), np.float64(0.02936087683808302), np.float64(0.0020116283155998333), np.float64(0.01727010898988987), np.float64(0.058130515688995146), np.float64(0.018367046647186722), np.float64(0.0016343624629613267), np.float64(0.016047128759504863), np.float64(0.0006712849808703955), np.float64(0.01057615783484245), np.float64(0.0007712516566274611), np.float64(0.014423641038262547), np.float64(0.0010312482485829563), np.float64(0.02467958175586013), np.float64(0.006469050906727754), np.float64(0.056448031859336764), np.float64(0.05205188886269545), np.float64(0.022826774235282215), np.float64(0.03284006352180501), np.float64(0.029692875811210183), np.float64(0.013223473051900273), np.float64(0.0025460128988987825), np.float64(0.032588147727408194), np.float64(0.007618961939448439), np.float64(0.007912245490593802), 0,0,0,np.float64(0.0015855162040741575), np.float64(0.0046450309147471535), np.float64(0.03893629789835341), np.float64(0.04116478015204702), np.float64(0.04881346406900757), np.float64(0.04241074427162585), np.float64(0.04280648746015615), np.float64(0.012747297574375512), np.float64(0.009336277761641937), 0,0,0,0,0,0,0,0,0,0,np.float64(0.0185319402344988), np.float64(0.0003221726205811528), 0,0,np.float64(0.00019160130466379366), 0,0,0,np.float64(0.00349046759511055), 0,0,0,0,np.float64(0.007447309324152423), np.float64(0.022815531230072558), np.float64(0.02438089225693297), np.float64(0.014042426586461708), np.float64(0.007892890458828251), 0,np.float64(0.0068569881533137415), np.float64(0.020816183111867042), np.float64(0.02589876407128537), np.float64(0.018793123678498273), np.float64(0.00035459786259313235), np.float64(0.020706479245335325), np.float64(0.055404120534467916), np.float64(0.026122376966235854), 0,0,0,np.float64(0.00045672327473164496), np.float64(0.006804726306104396), np.float64(0.004724889775793591), np.float64(0.023779011336142002), np.float64(0.01880907173013905), np.float64(0.002777902522760785), np.float64(0.0017440908112143754), np.float64(0.055181750660651424), np.float64(0.044356403445496254), np.float64(0.07916588614903897), np.float64(0.02623655892764803), np.float64(0.003965453947023562), np.float64(0.00019076969681170589), np.float64(0.0009877721248087), 0,0,np.float64(0.0009950927584813363), np.float64(4.549600399369998e-05), 0,0,np.float64(0.006697776626305565), np.float64(0.03249141692878698), np.float64(0.05971304404422802), np.float64(0.0007845808742105366), np.float64(0.015470850920170563), np.float64(0.01683324119384337), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(0.00012229833675527578), np.float64(0.0015078267707957422), np.float64(2.2227924887641597e-05), np.float64(5.062405021239086e-05), np.float64(0.0005720879165494023), np.float64(0.020499057928975987), 0,np.float64(0.04670382084745634), np.float64(0.013509667558305573), np.float64(0.098005482262554), np.float64(0.016610017060644758), np.float64(0.036449471282053225), np.float64(0.0016810384796017517), np.float64(0.07701028210259946), np.float64(0.04396343043022666), np.float64(0.09665762366810106), np.float64(0.00516445459452795), np.float64(0.014401020593361707), np.float64(0.03730895406204727), np.float64(0.04724830850027381), np.float64(0.04432568847721419), np.float64(0.042342185714683594), np.float64(0.02584222168530894), np.float64(0.01627386807319292), np.float64(0.03544967044075038), np.float64(0.04863563697275071), np.float64(0.05728608451906084), np.float64(0.05802265965502999), np.float64(0.07900744630763192), np.float64(0.0909359876334932), np.float64(0.11701795287640078), np.float64(0.04788970886129688), np.float64(0.0020402523643722405), np.float64(0.08224670624053176), np.float64(0.07458050056813866), np.float64(0.0647235592962226), np.float64(0.08714874743731962), np.float64(0.09306979153317854), np.float64(0.06294544225047402), np.float64(0.06262014512450904), np.float64(0.1111152868717643), np.float64(0.12585834900701603), np.float64(0.12878007999708885), np.float64(0.10287382250926179), np.float64(0.1171218301035721), np.float64(0.10736986309253253), np.float64(0.09685365524246145), np.float64(0.045270206492796075), np.float64(0.052606821342518414), np.float64(0.09876600538317881), np.float64(0.13928771061019224), np.float64(0.1171654722936864), np.float64(0.07502596214856526), np.float64(0.061864489970117266), np.float64(0.09568504508773372), np.float64(0.026262050704810958), np.float64(0.07921901307796832), np.float64(0.14409582803138998), np.float64(0.1323467283787193), np.float64(0.12219405621946014), np.float64(0.12071302352980928), np.float64(0.10664542386682176), np.float64(0.11605537973338018), np.float64(0.1166701904743851), np.float64(0.11146927846959012), np.float64(0.08094915601705997), np.float64(0.07250373974909044), np.float64(0.10299143862893752), np.float64(0.11867393118295234), np.float64(0.11083115157776216), np.float64(0.11127665820135518), np.float64(0.09957785315086737), np.float64(0.10644025891286979), np.float64(0.05370353739250718), np.float64(0.09335237053421133), np.float64(0.1067460291220285), np.float64(0.12044897237315709), np.float64(0.12921754235097271), np.float64(0.06708077365671382), np.float64(0.12020030494296698), np.float64(0.09415995425279111), np.float64(0.09897473952535474), np.float64(0.08688044546887894), np.float64(0.09138484165985342), np.float64(0.06644413379351745), np.float64(0.06978046936137822), np.float64(0.05793354597399643), np.float64(0.029192205212691672), np.float64(0.0713414545022795), np.float64(0.05634575367989088), np.float64(0.05237497693864083), np.float64(0.04075559161120562), np.float64(0.08546297934558554), np.float64(0.04847651618314454), np.float64(0.017066996048713527), 0,0,0,0,0,np.float64(0.0009970507228515123), 0,np.float64(3.239785113680084e-05), np.float64(0.0016056739860230414), np.float64(0.008432807596614187), np.float64(0.00020316961026457624), np.float64(0.004130407717200609), np.float64(0.03024553601308044), np.float64(0.0009236306392031489), 0,0,0,np.float64(0.00032022178428625945), np.float64(0.04338165817459669), np.float64(0.18217807333240368), np.float64(0.21370879168758358), np.float64(0.12133368664896388), np.float64(0.025052224086386334), np.float64(0.006015958208827679), np.float64(0.04520662840663559), np.float64(0.016989863044044824), np.float64(0.0019109011873022239), np.float64(0.026568060180451318), np.float64(0.04669711146591033), np.float64(0.052630817170301394), np.float64(0.03649505403882735), np.float64(0.012547002694159396), np.float64(0.0014815950411207124), np.float64(0.002922168576456916), np.float64(0.0005678382633469277), np.float64(8.582824722604348e-05), np.float64(0.0013821633185685644), np.float64(0.050761939847167134), np.float64(0.1015334763516103), np.float64(0.08211187713065467), np.float64(0.07401923553140757), np.float64(0.00941574120328007), np.float64(0.0017763658412487163), np.float64(0.0004599424202158865), np.float64(0.03714350452505961), np.float64(0.05306432613057779), np.float64(0.06100120356862159), np.float64(0.03930842811039724), np.float64(0.021630638245290476), np.float64(0.010200806890076335), np.float64(0.007519119278874071), 0,np.float64(9.495372853926464e-05), np.float64(0.0062897395645862865), 0,np.float64(0.0010611591546349976), np.float64(0.07540441257862257), np.float64(0.09321713879727987), np.float64(0.06582812372657773), np.float64(0.004792835462607252), np.float64(0.0043982058283652025), 0,0,0,0,0,0,0,0,0,0,0,0,np.float64(0.020779471914384545), np.float64(0.00010358463590324395), np.float64(0.030226609074428045), np.float64(0.01486807017557583), np.float64(0.05014300341901375), np.float64(0.14365547195932368), np.float64(0.009996574258504755), np.float64(0.25524229944782834), np.float64(0.26856607799738585), np.float64(0.04671850703624141), np.float64(0.12309327999244703), np.float64(0.050000770477237166), np.float64(0.04916589485103893), np.float64(0.021548165947889438), np.float64(0.0005546902901496095), 0,0,0,0,0,0,0,np.float64(0.0023027265906485767), np.float64(4.219453928439699e-05), np.float64(0.05038309207207927), np.float64(0.05125365414182657), 0,0,0,0,0,0,np.float64(0.0008417443734120949), np.float64(0.0015587614163131284), 0,0,0,0,0,np.float64(0.007284786911428648), np.float64(0.006672127383578666), np.float64(3.4570376504283055e-05), np.float64(0.015606680714252277), np.float64(0.034310691391098574), np.float64(0.002555414982698343), np.float64(0.0053650105455307654), 0,0,0,0,np.float64(0.01103738040158338), np.float64(0.007171717173555661), np.float64(0.00045456923681174273), 0,np.float64(0.05135195835988101), np.float64(0.07973410345386828), np.float64(0.02422275209608049), 0,0,0,0,0,np.float64(0.0006454253781892998), np.float64(0.032492553000816285), 0,np.float64(0.023170053831721625), np.float64(0.020180383748074602), np.float64(0.011165739658135516), np.float64(0.04105739997212746), np.float64(0.006993821415510147), np.float64(0.08277992083245031), np.float64(0.05216453213271419), np.float64(0.01325129215705538), 0,0,0,0,np.float64(0.000306085092770285), np.float64(0.00018507110352354057), np.float64(0.011511248344580083), 0,0,np.float64(0.0014472178993271472), np.float64(0.13530555780766507), np.float64(0.07527518216627464), np.float64(0.033327909938528036), np.float64(0.05919503792025092), np.float64(0.022821150368883953), np.float64(0.030549957746981143), np.float64(4.8691733474051507e-05), 0,0,0,0,0,0,np.float64(0.016963373294924547), np.float64(0.03749998259710679), np.float64(0.014845229191703023), np.float64(0.016851916300266476), np.float64(0.05316215304646362), np.float64(0.08147551508541187), np.float64(0.03059194270428752), np.float64(0.000432512782855123), np.float64(0.023259532032133386), np.float64(0.02609677630722064), np.float64(0.02114049413934639), 0,0,np.float64(0.0030170456439492783), np.float64(0.008132624609032581), 0,np.float64(0.059138219922369444), np.float64(0.024151566012067388), np.float64(0.014863567436816211), np.float64(0.0697263460254576), np.float64(0.023486791532566515), 0,np.float64(0.0006860450128936589), np.float64(0.0009559846583394703), np.float64(0.0019243043731364676), np.float64(0.004935193400420919), np.float64(0.01921808784317938), np.float64(0.03769112352262004), np.float64(0.06492520810603057), np.float64(0.014521533419469159), np.float64(0.005475781783044365), np.float64(0.06564631643886279), np.float64(0.04153197567904828), np.float64(0.0008091019778707153), np.float64(0.06193737151148711), np.float64(0.005529531992642272), 0,0,np.float64(0.00016334046790766985), 0,np.float64(0.027962215922890356), np.float64(0.004324826856948367), np.float64(0.04336663529989235), np.float64(0.010757095280501984), np.float64(0.024085403708014503), np.float64(0.20325570363575746), np.float64(0.098120314616818), np.float64(0.015199071296578362), 0,0,0,0,0,0,0,np.float64(0.026053160918774774), np.float64(0.02304282742027302), np.float64(0.06893381371099168), np.float64(0.054589629461997906), np.float64(0.024908455920239613), np.float64(0.05926540673866783), np.float64(0.05627065272587318), np.float64(0.07716560229690649), np.float64(0.016759840203629483), np.float64(0.004705897456434627), np.float64(0.02127800439229192), np.float64(0.01824168592634099), np.float64(0.002879710636662227), np.float64(0.05529801466090547), np.float64(0.07562853475833878), np.float64(0.0017628890088483026), np.float64(0.009630003038729417), 0,np.float64(0.00613368612120108), 0,0,np.float64(0.03015660383839847), np.float64(0.041712909333560054), np.float64(0.03098932838661092), np.float64(0.09705837095814722), np.float64(0.073070144260432), np.float64(0.028598418276127874), np.float64(0.009527363474826443), np.float64(0.0005967886527136204), np.float64(0.011372744267462449), np.float64(0.013581110839827831), 0,0,np.float64(0.0008142180333345652), np.float64(0.0031543901097430805), np.float64(0.0004439329412201021), np.float64(0.014475022394792435), np.float64(0.0023830596472586584), np.float64(0.026079204268269634), np.float64(0.029984761486669915), np.float64(0.04971780101966618), np.float64(0.024539114262581135), np.float64(0.042742823294110485), np.float64(0.005041276478549979), np.float64(0.007802709724556335), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(0.0011302979808825878), np.float64(0.038610505851633135), np.float64(0.013889248836751331), 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.float64(0.058847674534795155), np.float64(0.03293894956041741), 0,np.float64(0.27428740143819935), np.float64(0.25082008585053117), np.float64(0.25874096323356427), np.float64(0.18956996804071577), np.float64(0.1729626433589902), np.float64(0.17271017986056908), np.float64(0.17660862837660923), np.float64(0.1667432154054737), np.float64(0.12927331386713303), np.float64(0.10993824811194478), np.float64(0.2127865492258745), np.float64(0.15969738471915745), np.float64(0.11373184385533745), np.float64(0.012789871656404026), np.float64(0.008065342816583082), np.float64(0.009620494259400687), np.float64(0.04932384853320228), np.float64(0.005607016029719377), np.float64(0.03151992574712007), np.float64(0.07023291684330842), np.float64(0.05473799507747993), np.float64(0.08343651941129647), np.float64(0.2257089743983921), np.float64(0.19337677733413539), np.float64(0.2394407110287513), np.float64(0.23735896559576075), np.float64(0.27671026522907977), np.float64(0.2824620411251787), np.float64(0.13269008688026407), np.float64(0.14075161006518985), np.float64(0.13483424686269657), np.float64(0.06268799491811183), np.float64(0.05439982494759709), np.float64(0.03336857494383458), np.float64(0.00032787301498324003), np.float64(0.0682212802892094), np.float64(0.09762887096295743), np.float64(0.09124185067559115), 0,np.float64(0.04751940752614269), np.float64(0.13821617597859367), np.float64(0.10077258189202391), np.float64(0.21998525891776544), np.float64(0.2890393013694452), np.float64(0.3576063807450285), np.float64(0.41813925124111473), np.float64(0.20507674202211149), np.float64(0.003288944668192137), np.float64(0.1451217219491347), np.float64(0.002336520942113744), np.float64(0.03338410802439002), np.float64(6.343591303543448e-05), np.float64(0.0003174406499467383), np.float64(0.0002358469199273943), 0,np.float64(0.05886893957462243), np.float64(0.00015928606718130752), 0,np.float64(2.1601377864296563e-05), np.float64(0.00013564585872531305), np.float64(0.12785756564119094), np.float64(0.1871356369611464), np.float64(0.6310632584058705), np.float64(1.2621269031964177)]
print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

In [ ]:
#@title Data and verification (score = 1.5035)
import numpy as np
best_sequence = [np.float64(0.6951963460267467), np.float64(0.500748238446374), np.float64(0.4417322835612597), np.float64(0.6433530392905233), np.float64(4.160223909435357e-17), np.float64(5.568614851281417e-17), np.float64(0.12976587071893259), np.float64(0.11673594388015851), np.float64(0.023263922834045405), np.float64(0.03168531251927259), np.float64(2.223905512571851e-17), np.float64(0.0014730571328584218), np.float64(2.290862197298409e-17), np.float64(1.898601583126003e-17), np.float64(1.9470939276817197e-17), np.float64(2.1574384389669054e-17), np.float64(0.00015571471916239605), np.float64(0.009911285381046374), np.float64(0.04355285256986089), np.float64(0.045255883887096254), np.float64(0.07054021618451697), np.float64(0.11559859955657845), np.float64(0.1328964182028336), np.float64(0.16228701062243817), np.float64(0.20715054181574039), np.float64(0.23547774507628236), np.float64(0.2498419843086018), np.float64(0.2614928713888195), np.float64(0.23265860803217023), np.float64(0.2176715833095882), np.float64(0.17053478224670537), np.float64(0.14025577741883222), np.float64(0.09393260851271676), np.float64(0.066348703251754), np.float64(0.09297000792985262), np.float64(0.0627174777618613), np.float64(0.01661172104253169), np.float64(2.5085630682828918e-17), np.float64(3.144260614674204e-17), np.float64(0.0175958843940702), np.float64(0.12708687120707915), np.float64(0.14754201133213588), np.float64(0.13371813307804004), np.float64(0.13323470240252017), np.float64(0.10209708558815479), np.float64(0.08713207295418134), np.float64(0.10601802670411871), np.float64(0.11663780764216386), np.float64(0.13675323119262722), np.float64(0.1631857697726197), np.float64(0.1863185090125226), np.float64(0.19006470572892264), np.float64(0.18090001286375662), np.float64(0.1537766450944084), np.float64(0.17339215131938726), np.float64(0.18632603591093463), np.float64(0.15873866401073608), np.float64(0.058175672809692644), np.float64(2.5020963522782464e-17), np.float64(2.0810342708549194e-17), np.float64(5.681877254365711e-17), np.float64(0.11289532355498892), np.float64(0.1673707107084944), np.float64(0.1276310049641213), np.float64(0.057824540477547114), np.float64(0.1019413559698588), np.float64(0.06692139354952632), np.float64(0.03387126560016177), np.float64(0.03586579919449355), np.float64(0.025157191710182), np.float64(0.0008807660391037458), np.float64(7.170415521086594e-17), np.float64(6.727673217535331e-17), np.float64(7.332626614813154e-17), np.float64(7.311123713270729e-17), np.float64(6.858307121530564e-17), np.float64(7.320891628978767e-17), np.float64(7.108924907599418e-17), np.float64(6.818669956256597e-17), np.float64(6.949546316026015e-17), np.float64(6.772729191265334e-17), np.float64(6.929456839741612e-17), np.float64(7.437445714741963e-17), np.float64(7.005551163543335e-17), np.float64(7.386189279353468e-17), np.float64(6.654527602037804e-17), np.float64(7.337299420649999e-17), np.float64(6.930888067736296e-17), np.float64(7.37964832792348e-17), np.float64(7.279570510632174e-17), np.float64(7.383198012786066e-17), np.float64(6.857400390408705e-17), np.float64(7.446566135773111e-17), np.float64(7.220930407123016e-17), np.float64(7.326247766867047e-17), np.float64(6.874393931694272e-17), np.float64(7.294197108247609e-17), np.float64(7.198653327600301e-17), np.float64(6.770203670212308e-17), np.float64(7.43314927392968e-17), np.float64(7.120054157797854e-17), np.float64(6.892791274055083e-17), np.float64(6.858186889244261e-17), np.float64(6.723365520533029e-17), np.float64(7.282828785965765e-17), np.float64(7.463584694140412e-17), np.float64(6.869663450896016e-17), np.float64(7.103978005664747e-17), np.float64(7.029589716375831e-17), np.float64(7.018270893531981e-17), np.float64(7.259789552760443e-17), np.float64(6.972509521178248e-17), np.float64(7.078466462757957e-17), np.float64(7.169690506958959e-17), np.float64(7.133578575144335e-17), np.float64(6.91319332284763e-17), np.float64(6.929905743905534e-17), np.float64(7.208813580434548e-17), np.float64(6.782554786146079e-17), np.float64(6.889775171302082e-17), np.float64(7.063926942654074e-17), np.float64(7.43431023437995e-17), np.float64(7.273381302269766e-17), np.float64(7.482510497289833e-17), np.float64(7.260294334789225e-17), np.float64(7.362601361387158e-17), np.float64(6.721894578178571e-17), np.float64(7.034321580703574e-17), np.float64(7.12658438343404e-17), np.float64(7.203824921462655e-17), np.float64(7.205457236402588e-17), np.float64(7.011189491186957e-17), np.float64(7.005362638155636e-17), np.float64(7.176276962699736e-17), np.float64(6.929732276820705e-17), np.float64(7.140024844566772e-17), np.float64(7.125421428419705e-17), np.float64(7.442285656248777e-17), np.float64(7.321824065255417e-17), np.float64(7.458012646893255e-17), np.float64(6.814182148690577e-17), np.float64(6.953600252311209e-17), np.float64(7.103083092175629e-17), np.float64(6.946446896710093e-17), np.float64(6.875903393590669e-17), np.float64(6.802310050353596e-17), np.float64(7.277575612957359e-17), np.float64(7.324869625046509e-17), np.float64(7.411671475594265e-17), np.float64(7.188948386278613e-17), np.float64(7.409088878399227e-17), np.float64(7.317661171875051e-17), np.float64(7.219153343228249e-17), np.float64(6.762294947471689e-17), np.float64(7.123033066261208e-17), np.float64(7.455052062458759e-17), np.float64(6.950613296595815e-17), np.float64(6.809488979173697e-17), np.float64(6.857466371389689e-17), np.float64(7.411309677605291e-17), np.float64(6.840520582986793e-17), np.float64(7.431982347652882e-17), np.float64(6.772024815538226e-17), np.float64(6.935564812146203e-17), np.float64(7.398827868804947e-17), np.float64(6.884136671770495e-17), np.float64(6.854433130634063e-17), np.float64(7.44441922224522e-17), np.float64(7.428027040499105e-17), np.float64(6.90533506723557e-17), np.float64(7.402992298943606e-17), np.float64(7.449166593075918e-17), np.float64(7.338378955551729e-17), np.float64(6.988138149593625e-17), np.float64(7.325723588059608e-17), np.float64(7.540868740390824e-17), np.float64(6.877873066856118e-17), np.float64(7.336361580433016e-17), np.float64(6.931378808107367e-17), np.float64(6.742387072996586e-17), np.float64(7.384442163383965e-17), np.float64(7.543605293178374e-17), np.float64(7.369420398455881e-17), np.float64(7.390570783329229e-17), np.float64(7.23958829480357e-17), np.float64(7.061170092902643e-17), np.float64(7.528287858019187e-17), np.float64(7.519971242956314e-17), np.float64(7.125893124087178e-17), np.float64(7.455422225006725e-17), np.float64(7.012957896539978e-17), np.float64(7.382095169369465e-17), np.float64(6.989425063401373e-17), np.float64(7.207319798751341e-17), np.float64(6.778590222541863e-17), np.float64(7.147202734323899e-17), np.float64(7.30012752089726e-17), np.float64(6.964928470781962e-17), np.float64(7.218363037864104e-17), np.float64(7.307613232648533e-17), np.float64(7.35043042533792e-17), np.float64(6.92740740700717e-17), np.float64(7.307991403474796e-17), np.float64(7.537473682662436e-17), np.float64(6.795054125148745e-17), np.float64(7.498603042312755e-17), np.float64(7.100599444959607e-17), np.float64(7.204113302312039e-17), np.float64(7.448483714003822e-17), np.float64(7.056594221367674e-17), np.float64(7.489929699407583e-17), np.float64(6.896719997442851e-17), np.float64(7.527431225563103e-17), np.float64(6.801835583971874e-17), np.float64(7.147970361991104e-17), np.float64(6.948944618286996e-17), np.float64(6.874840036796469e-17), np.float64(7.175994432767185e-17), np.float64(7.126248901478403e-17), np.float64(7.481073263455803e-17), np.float64(7.077202615828537e-17), np.float64(7.424909503421125e-17), np.float64(7.152161853604352e-17), np.float64(7.11964819467318e-17), np.float64(6.970608321422026e-17), np.float64(7.356099930251004e-17), np.float64(7.104343254906273e-17), np.float64(6.855467684336131e-17), np.float64(6.768035271807401e-17), np.float64(6.775187775809174e-17), np.float64(7.077629583876303e-17), np.float64(7.019547155200877e-17), np.float64(6.94906362408852e-17), np.float64(7.133879219649781e-17), np.float64(7.276139308507205e-17), np.float64(7.248454503842872e-17), np.float64(6.929768373857601e-17), np.float64(7.459094486040797e-17), np.float64(7.36833153944074e-17), np.float64(7.480480936449517e-17), np.float64(7.402489591044001e-17), np.float64(7.194305858447223e-17), np.float64(7.537888084127476e-17), np.float64(7.146765971669428e-17), np.float64(6.890131647417585e-17), np.float64(7.509477425630509e-17), np.float64(7.573623987922626e-17), np.float64(6.876272437237929e-17), np.float64(7.360728386589346e-17), np.float64(6.917522878567438e-17), np.float64(7.661486323593386e-17), np.float64(7.167737276703801e-17), np.float64(7.25615369243727e-17), np.float64(7.044070104962494e-17), np.float64(7.303249307606423e-17), np.float64(7.641581172741785e-17), np.float64(7.674367426227895e-17), np.float64(7.140639229477036e-17), np.float64(7.061740442556978e-17), np.float64(7.534432341222305e-17), np.float64(7.756651847304391e-17), np.float64(7.380839227518717e-17), np.float64(7.260775615505479e-17), np.float64(7.058231259448427e-17), np.float64(7.102337436396469e-17), np.float64(7.720801312966721e-17), np.float64(7.25026533310599e-17), np.float64(7.145336160817529e-17), np.float64(7.041475652569769e-17), np.float64(7.645311545472798e-17), np.float64(7.329676759169028e-17), np.float64(7.185136600105939e-17), np.float64(7.581614594390547e-17), np.float64(7.41261742767395e-17), np.float64(7.07512643694982e-17), np.float64(7.060645049456917e-17), np.float64(7.773149374920733e-17), np.float64(7.556009911475147e-17), np.float64(7.599915593138541e-17), np.float64(7.071199157211276e-17), np.float64(7.685151438176683e-17), np.float64(7.205571715240187e-17), np.float64(7.498548134680075e-17), np.float64(7.09214917616463e-17), np.float64(7.254944527495217e-17), np.float64(0.001968780805373588), np.float64(0.012772174426452416), np.float64(0.04247477945536533), np.float64(0.018186300832652515), np.float64(0.014529515658620195), np.float64(0.01450759739323975), np.float64(7.36960148710329e-17), np.float64(0.015414284421048998), np.float64(0.06428269467652768), np.float64(0.054855276887341056), np.float64(0.04142033789824192), np.float64(0.03521423564429524), np.float64(0.0056256630719036145), np.float64(7.046747646138994e-17), np.float64(0.036823105303300414), np.float64(0.046115390045090986), np.float64(0.030901434906246576), np.float64(0.007503714253067224), np.float64(0.023911547156325215), np.float64(0.033054874948546906), np.float64(0.005455506771401649), np.float64(0.010934281755136388), np.float64(0.007528068225546506), np.float64(0.014467784106279488), np.float64(0.012523742877451964), np.float64(0.0036368120698771094), np.float64(0.005296224338612687), np.float64(0.027826005029312), np.float64(0.032498174243822384), np.float64(0.047718597016736), np.float64(0.04826726922985701), np.float64(0.04679102251745131), np.float64(0.034384236169711466), np.float64(0.010800541130991787), np.float64(0.0042832426138359565), np.float64(0.007730243213579369), np.float64(7.565493420360566e-17), np.float64(7.134184750451931e-17), np.float64(0.0058168852135845415), np.float64(0.005426738034891504), np.float64(0.015209506310497778), np.float64(0.022570562703049406), np.float64(0.023395251492794644), np.float64(0.053117677861540084), np.float64(0.05741292919150508), np.float64(0.07951499380795911), np.float64(0.09460849892655393), np.float64(0.10081990018020474), np.float64(0.08690323549572947), np.float64(0.07478243649537995), np.float64(0.03216423014463066), np.float64(0.020112814042386084), np.float64(0.05148558150977199), np.float64(0.014852384664265444), np.float64(0.0218688450428403), np.float64(0.05748604661665366), np.float64(6.73908089651705e-17), np.float64(6.764166327609249e-17), np.float64(6.833008603237703e-17), np.float64(7.095270796927191e-17), np.float64(6.938484720908467e-17), np.float64(7.184537168407666e-17), np.float64(6.766386776162447e-17), np.float64(6.874188746057976e-17), np.float64(7.370706086584162e-17), np.float64(7.132554371519896e-17), np.float64(0.007634949882920935), np.float64(0.013475288075166366), np.float64(0.0176442627824042), np.float64(0.01698899208108), np.float64(0.02399208068391693), np.float64(0.012837195388631728), np.float64(0.024355086783291142), np.float64(0.04583342911268831), np.float64(0.04218603358871563), np.float64(0.027875342934066544), np.float64(0.0224554675655206), np.float64(7.35496624022278e-17), np.float64(7.515753523867139e-17), np.float64(0.010218685502228505), np.float64(0.015478923409137613), np.float64(0.04311623400210994), np.float64(0.07289520147911517), np.float64(0.06850821330111607), np.float64(0.05099157064613655), np.float64(0.06465407253845666), np.float64(0.045329785002621636), np.float64(0.012649232815841824), np.float64(0.020176378698758247), np.float64(4.1178197756152845e-17), np.float64(0.0010936281918108555), np.float64(5.948161017553021e-06), np.float64(0.0011555172830796882), np.float64(0.021564863196114826), np.float64(7.035561225329391e-17), np.float64(0.009847548316857909), np.float64(0.013445984998357382), np.float64(7.081326771756115e-17), np.float64(0.012856688926375676), np.float64(0.013921356693647979), np.float64(0.04526961134267042), np.float64(0.0303476745833674), np.float64(6.875142633705341e-17), np.float64(0.027094270839726427), np.float64(0.003936755100695045), np.float64(6.674600611378712e-17), np.float64(6.988446348989171e-17), np.float64(6.549223907510728e-17), np.float64(6.7125942718949e-17), np.float64(6.485630500052813e-17), np.float64(6.76366941126623e-17), np.float64(0.003821778288292134), np.float64(0.013803196904924103), np.float64(0.02423660393134838), np.float64(0.0042468770018481115), np.float64(0.021562997984062804), np.float64(0.03859821838728457), np.float64(0.03569353917097764), np.float64(0.03198653548813598), np.float64(0.05617407732260494), np.float64(0.04834379926424721), np.float64(0.04017377690282879), np.float64(0.03289153885607946), np.float64(0.0184635963985252), np.float64(0.014991144642249286), np.float64(0.0026482246271183215), np.float64(6.971004199000819e-17), np.float64(0.004607831024531537), np.float64(0.007967465530986517), np.float64(0.0014935004240595117), np.float64(0.012157770131718799), np.float64(0.0040712085658458335), np.float64(6.916463292992466e-17), np.float64(7.29598044918943e-17), np.float64(6.551367360321045e-17), np.float64(7.29429060744373e-17), np.float64(7.160763857769196e-17), np.float64(6.976933150021519e-17), np.float64(7.277444622775829e-17), np.float64(7.29936904204665e-17), np.float64(7.070364069485465e-17), np.float64(6.960284817878469e-17), np.float64(7.379758717511277e-17), np.float64(7.119147584108001e-17), np.float64(6.792226470332628e-17), np.float64(7.040919972925675e-17), np.float64(0.012041779213675406), np.float64(0.0014318453368044335), np.float64(0.007120813824343148), np.float64(0.01017119673719106), np.float64(6.646912050370106e-17), np.float64(6.614935740562141e-17), np.float64(0.002383363326378363), np.float64(6.967140250028515e-17), np.float64(0.016046984645364092), np.float64(0.03891915995772142), np.float64(0.040208693045757915), np.float64(0.03137425106630582), np.float64(6.267993553507305e-17), np.float64(6.910181141638653e-17), np.float64(0.003975525514307598), np.float64(0.005248452643507196), np.float64(0.006690556319049357), np.float64(6.649107227250447e-17), np.float64(6.445069368727168e-17), np.float64(6.503095119802964e-17), np.float64(0.008780855565158315), np.float64(0.01311470033578037), np.float64(0.029974679277739672), np.float64(0.03516155156335447), np.float64(0.014500997464975275), np.float64(0.008932031063219744), np.float64(0.022163889279434437), np.float64(0.03168773989255995), np.float64(0.04353014881334368), np.float64(0.05536694979876129), np.float64(0.04435490966692039), np.float64(0.0502890467309673), np.float64(0.12150621707645005), np.float64(0.14447863961562465), np.float64(0.1323623387379493), np.float64(0.16461094996667097), np.float64(0.09976863082921018), np.float64(0.0009599397252226941), np.float64(6.18842500588827e-17), np.float64(0.012214686819032986), np.float64(5.898624286733893e-17), np.float64(6.432183153584242e-17), np.float64(6.068853878352236e-17), np.float64(6.708588812614335e-17), np.float64(6.107097782898915e-17), np.float64(6.248049169762218e-17), np.float64(6.369980394108439e-17), np.float64(6.224578972601576e-17), np.float64(6.811448586420197e-17), np.float64(6.306252718594193e-17), np.float64(6.806823375591659e-17), np.float64(6.300204303001055e-17), np.float64(0.008028031332321427), np.float64(0.04741936634314315), np.float64(0.03221897583734147), np.float64(0.021935930474051773), np.float64(0.019795745807847716), np.float64(0.013166817883087167), np.float64(0.00024935602116574695), np.float64(0.0007415386353976859), np.float64(0.009532545366120594), np.float64(0.007680975117968378), np.float64(0.022333398012936836), np.float64(0.031142632523452292), np.float64(0.004864326014055596), np.float64(0.023150568949098067), np.float64(0.045555860422122244), np.float64(0.008426998475291103), np.float64(0.037797424570591986), np.float64(0.042178010151085334), np.float64(0.023930518209525322), np.float64(0.02446696131553097), np.float64(0.03412204985495667), np.float64(0.04370099340549948), np.float64(0.058717773472615066), np.float64(0.0550846910637526), np.float64(0.04055285691903918), np.float64(0.027024757552480533), np.float64(0.016116373386659), np.float64(0.034809516859489416), np.float64(0.05955370017280913), np.float64(0.03209794578010642), np.float64(0.04291125121061888), np.float64(0.036630981662105856), np.float64(0.0034329618743683603), np.float64(0.0019520953237723957), np.float64(0.015417182865189501), np.float64(0.024374216938421737), np.float64(0.04694196549180756), np.float64(0.03768385341483998), np.float64(0.047135413663743624), np.float64(0.06905677195336067), np.float64(0.08006067512243716), np.float64(0.08127303764979422), np.float64(0.06972813254576925), np.float64(0.03791619015926697), np.float64(0.0284085219186079), np.float64(0.004053165299105155), np.float64(7.729443372637027e-17), np.float64(7.908107516868725e-17), np.float64(7.418302210570046e-17), np.float64(7.908343789935413e-17), np.float64(7.585962710229713e-17), np.float64(8.028554406535026e-17), np.float64(7.518325706824498e-17), np.float64(7.412634841227487e-17), np.float64(7.865983821572642e-17), np.float64(7.44807474260289e-17), np.float64(7.84466769144392e-17), np.float64(7.889635383716516e-17), np.float64(0.017012356727051637), np.float64(0.032787597699464426), np.float64(0.044276936985592374), np.float64(0.07014263307444295), np.float64(0.06911444978915934), np.float64(0.061529945889942164), np.float64(0.05413910703834537), np.float64(0.0572844707793573), np.float64(0.0636903683562931), np.float64(0.06536654514010636), np.float64(0.07439340122773123), np.float64(0.06568369548611794), np.float64(0.05349855273322678), np.float64(0.05595560687144243), np.float64(0.05120517102856477), np.float64(0.060698433126291015), np.float64(0.06217601082667069), np.float64(0.06829402984205597), np.float64(0.08419659017528339), np.float64(0.08267462477330396), np.float64(0.07989987117854752), np.float64(0.07764516662201656), np.float64(0.08140169017177018), np.float64(0.10640503166767179), np.float64(0.12573221389383535), np.float64(0.13068690686755907), np.float64(0.1113228199676313), np.float64(0.0896015917144892), np.float64(0.09182389018005209), np.float64(0.07685008558685484), np.float64(0.11798540445207521), np.float64(0.11754872153286326), np.float64(0.12080086411690508), np.float64(0.11027041115581539), np.float64(0.10815161417576731), np.float64(0.10252002426505778), np.float64(0.08066309241223683), np.float64(0.07191378362588015), np.float64(0.08633168933305073), np.float64(0.09147619531258284), np.float64(0.10416359417885494), np.float64(0.09674341739307607), np.float64(0.09745707375394157), np.float64(0.09579231171789425), np.float64(0.10189683582389941), np.float64(0.08506992986963823), np.float64(0.10952375355694942), np.float64(0.09119998387151317), np.float64(0.06899147699730095), np.float64(0.06709696038628776), np.float64(0.08024525455461635), np.float64(0.08499177839993254), np.float64(0.07078745704095339), np.float64(0.10393932378131263), np.float64(0.0949799739319403), np.float64(0.06537274239893523), np.float64(0.06521380103871582), np.float64(0.0900723356735667), np.float64(0.05625333735949637), np.float64(0.08535274475303524), np.float64(0.065474890790224), np.float64(0.05908270205552107), np.float64(0.05991392849572639), np.float64(0.04269566737408567), np.float64(0.06238804711214498), np.float64(0.10124118254137279), np.float64(0.06644806540525142), np.float64(0.047509807195082444), np.float64(0.043376437504426234), np.float64(0.08712475382751), np.float64(0.04444663732593195), np.float64(0.05196799369506097), np.float64(0.019334266603269423), np.float64(0.07585242377610325), np.float64(5.647044760208266e-17), np.float64(0.0877598389979454), np.float64(6.414180042774188e-17), np.float64(0.03303643046590118), np.float64(0.023871146616862026), np.float64(0.011159966240752285), np.float64(0.11057809102229715), np.float64(0.05659332840523442), np.float64(0.054560972272776155), np.float64(0.011259891495227218), np.float64(0.08564016739918508), np.float64(0.06433191795437113), np.float64(0.04397831724592107), np.float64(0.007229241262628048), np.float64(0.0007494129998408726), np.float64(6.554091028124771e-17), np.float64(6.410826854557293e-17), np.float64(6.219063759631357e-17), np.float64(6.40521486002675e-17), np.float64(6.685435302895486e-17), np.float64(6.274157836904255e-17), np.float64(6.189678667148835e-17), np.float64(6.628327732915213e-17), np.float64(6.340900861913012e-17), np.float64(6.609356900683726e-17), np.float64(6.235591407292787e-17), np.float64(6.082836669225535e-17), np.float64(6.030808147421325e-17), np.float64(6.035675673580292e-17), np.float64(6.488090135074733e-17), np.float64(6.031680102855289e-17), np.float64(6.342443667048828e-17), np.float64(5.830473183593461e-17), np.float64(6.105034940056839e-17), np.float64(5.996288732861536e-17), np.float64(6.457760217792819e-17), np.float64(6.434489507305093e-17), np.float64(6.105252742289939e-17), np.float64(6.769461078956296e-17), np.float64(6.519851967813613e-17), np.float64(6.737345295341296e-17), np.float64(6.700035673146797e-17), np.float64(6.689346640984319e-17), np.float64(6.719224344022251e-17), np.float64(7.082270034069086e-17), np.float64(7.145176853905735e-17), np.float64(7.481862339588005e-17), np.float64(7.795628114469271e-17), np.float64(7.521160661204927e-17), np.float64(7.534053935122878e-17), np.float64(7.919395604519354e-17), np.float64(8.234666783625199e-17), np.float64(8.087788763027505e-17), np.float64(8.216720576197635e-17), np.float64(7.707972068260665e-17), np.float64(7.767938236148225e-17), np.float64(8.385658511353616e-17), np.float64(8.379928178888968e-17), np.float64(8.211312556020662e-17), np.float64(8.146346900437138e-17), np.float64(8.246158065296847e-17), np.float64(8.296886805770888e-17), np.float64(8.295091209997575e-17), np.float64(8.080007171240985e-17), np.float64(8.346990819156927e-17), np.float64(7.547312375501585e-17), np.float64(7.408450322028116e-17), np.float64(8.033206141524904e-17), np.float64(7.818761362010762e-17), np.float64(7.274002608570462e-17), np.float64(7.72626585000128e-17), np.float64(7.722202270957117e-17), np.float64(7.444070894251585e-17), np.float64(7.046155251721738e-17), np.float64(7.264661969317535e-17), np.float64(7.165801193718397e-17), np.float64(7.449149092130463e-17), np.float64(7.036775096252758e-17), np.float64(6.962565515536457e-17), np.float64(6.644028356788655e-17), np.float64(7.17855991561565e-17), np.float64(0.005816463149938319), np.float64(0.05962309837641001), np.float64(0.09439881994970312), np.float64(0.06725727368681307), np.float64(0.07028918115247765), np.float64(0.04320287805691942), np.float64(0.01708046905028873), np.float64(0.017136625149377735), np.float64(0.02596409839027585), np.float64(0.02436058138531239), np.float64(7.623099691926533e-17), np.float64(0.003621303871533154), np.float64(0.026596879384849774), np.float64(7.868305890487878e-17), np.float64(7.957429761337206e-17), np.float64(0.004485413535227464), np.float64(0.0008686155791899347), np.float64(7.728065592449145e-17), np.float64(7.929761891560967e-17), np.float64(7.931749479710144e-17), np.float64(7.310088853237333e-17), np.float64(7.660503944603474e-17), np.float64(7.741058461996063e-17), np.float64(7.494061843742036e-17), np.float64(7.175303741852036e-17), np.float64(7.020438348609077e-17), np.float64(6.676090984656302e-17), np.float64(6.964247941215625e-17), np.float64(6.609172645955075e-17), np.float64(6.759411906151028e-17), np.float64(6.82811005957288e-17), np.float64(6.456682357975718e-17), np.float64(6.331561556676143e-17), np.float64(6.549524277956179e-17), np.float64(5.722981986654892e-17), np.float64(5.563335305906451e-17), np.float64(5.750884422200757e-17), np.float64(0.01956742317113168), np.float64(0.02471525521636587), np.float64(0.010906727012325718), np.float64(0.005512847031561449), np.float64(0.010856937726901711), np.float64(5.447376277533062e-17), np.float64(0.020138566837814797), np.float64(0.013768104269119242), np.float64(4.9813514630434546e-17), np.float64(0.017014509483599914), np.float64(0.017295758038735624), np.float64(0.004195961679940848), np.float64(0.05830112270592397), np.float64(0.04800072440877454), np.float64(0.03306634432561877), np.float64(0.004783896798896675), np.float64(7.125080671865277e-17), np.float64(0.01733142873429643), np.float64(0.002085302202585069), np.float64(0.021567636082983115), np.float64(0.027430336958319125), np.float64(0.03926491739501023), np.float64(0.013925478906483574), np.float64(0.007123638252418968), np.float64(2.5370779560696493e-17), np.float64(4.6938396784821153e-17), np.float64(4.02815460504696e-17), np.float64(0.0007188292676493391), np.float64(0.0006770606415386248), np.float64(6.085339927350086e-17), np.float64(0.0009812196123776076), np.float64(0.02375151723573767), np.float64(0.025295345752708868), np.float64(0.021076512907834462), np.float64(0.0100981380769694), np.float64(0.05189481562633337), np.float64(0.04786612363737678), np.float64(0.08216684457632283), np.float64(0.05855427149892795), np.float64(0.016950958420807932), np.float64(0.012784203620327622), np.float64(0.04589989901970262), np.float64(0.01975115745037778), np.float64(5.972430679229087e-17), np.float64(0.010203403115791453), np.float64(0.019601550020960968), np.float64(0.004389142507914176), np.float64(0.0007236371440562741), np.float64(0.00849196586697834), np.float64(0.00139604214603468), np.float64(0.01018185923883449), np.float64(0.004186485237386054), np.float64(6.658518564472783e-17), np.float64(6.749251566713091e-17), np.float64(0.011023919910943702), np.float64(6.916376241740272e-17), np.float64(0.004070283238903183), np.float64(6.924025454014946e-17), np.float64(0.01520222657368287), np.float64(7.32150144267845e-17), np.float64(0.02098953584945253), np.float64(0.02931375369037391), np.float64(0.029231503948849744), np.float64(0.09906713643172162), np.float64(0.016374717921471604), np.float64(0.14340586539391068), np.float64(7.637167226195415e-17), np.float64(0.029257546217804138), np.float64(7.766187427194444e-17), np.float64(7.51498762847679e-17), np.float64(7.609199837921175e-17), np.float64(8.144486165093325e-17), np.float64(8.073395748394637e-17), np.float64(7.27904547031586e-17), np.float64(7.491820436566031e-17), np.float64(7.287127577586649e-17), np.float64(0.004150693477947992), np.float64(0.01284131788580011), np.float64(7.681875817449758e-17), np.float64(6.847070976345146e-17), np.float64(7.047244559080371e-17), np.float64(0.007365269502992031), np.float64(7.240901192356681e-17), np.float64(6.72496087933618e-17), np.float64(7.001119077856771e-17), np.float64(0.006093128450637098), np.float64(0.005043890155742418), np.float64(0.006955823985765512), np.float64(0.0062094940445491815), np.float64(0.018181400407675227), np.float64(0.015426541896508082), np.float64(0.011757080246360454), np.float64(0.021750805012669256), np.float64(7.020032395528974e-17), np.float64(9.848619422394528e-05), np.float64(6.971616125785015e-17), np.float64(7.523990333187703e-17), np.float64(7.390413714587713e-17), np.float64(0.009209134967942349), np.float64(0.010513316596810903), np.float64(0.0037461893375788703), np.float64(0.002801632388441245), np.float64(0.008373845612028202), np.float64(0.026919556699983836), np.float64(0.031490382473648125), np.float64(0.035146148643489035), np.float64(7.270613451323616e-17), np.float64(7.356318363250978e-17), np.float64(7.12488647904388e-17), np.float64(7.815719720356203e-17), np.float64(7.000651903873982e-17), np.float64(7.057200250699272e-17), np.float64(0.0018840784950631625), np.float64(0.013673973636174687), np.float64(0.013724429045208358), np.float64(0.007984469149385264), np.float64(0.002606064343825224), np.float64(0.0010637091170575886), np.float64(6.803884216876888e-17), np.float64(6.195900197374894e-17), np.float64(6.500718882392288e-17), np.float64(0.0028498187617786667), np.float64(6.61007453754221e-17), np.float64(6.15542333989845e-17), np.float64(6.179324803567435e-17), np.float64(6.215549314908685e-17), np.float64(0.006833850691219247), np.float64(5.866076696302272e-17), np.float64(0.0021212979596054483), np.float64(0.008210510952888257), np.float64(0.008499422950920196), np.float64(0.0033897566997026685), np.float64(0.009312593510814121), np.float64(0.009374925758471582), np.float64(0.010301274605456712), np.float64(0.010669655057630342), np.float64(0.013238688261274903), np.float64(0.006570147554073814), np.float64(0.002665769786811492), np.float64(6.86172585886543e-17), np.float64(6.466753236441099e-17), np.float64(0.0005782364530096658), np.float64(0.009366013214913115), np.float64(0.013883160411445378), np.float64(0.019863063183055623), np.float64(0.012326322185654498), np.float64(0.00654589776551884), np.float64(0.00883044341535324), np.float64(0.017586487728643605), np.float64(0.003022923635665265), np.float64(0.00986697982551434), np.float64(0.00250875190249246), np.float64(0.004728893792079345), np.float64(0.018282377597994927), np.float64(0.01615482040853515), np.float64(0.00857908285330259), np.float64(0.018473548475387465), np.float64(0.01671294063713192), np.float64(0.022041050389021533), np.float64(0.01776750052016275), np.float64(0.02204731870542238), np.float64(0.02353100238067845), np.float64(0.028854478325666073), np.float64(0.02810847861665075), np.float64(0.0478062752390506), np.float64(0.0527564818345793), np.float64(0.06802067907350574), np.float64(0.0780843605679942), np.float64(0.06544115509703526), np.float64(0.06277883644133696), np.float64(0.07434441040621916), np.float64(0.04640833137686003), np.float64(0.05603598011003335), np.float64(0.06499390140868223), np.float64(0.07386847518715452), np.float64(0.07532836828750566), np.float64(0.07500904059511866), np.float64(0.07288656880352308), np.float64(0.08394891353967708), np.float64(0.08482265832973329), np.float64(0.08715266862180182), np.float64(0.0815826925653674), np.float64(0.08672259732995895), np.float64(0.0860800528262442), np.float64(0.08483499719869927), np.float64(0.07905638399449609), np.float64(0.08698327388361363), np.float64(0.09918446888253456), np.float64(0.10425888296228526), np.float64(0.09965658281807485), np.float64(0.11215232095538749), np.float64(0.1197383515672007), np.float64(0.12552181721149125), np.float64(0.10887064796480864), np.float64(0.0931356903821399), np.float64(0.09862056546471817), np.float64(0.10252208169481007), np.float64(0.0930787883591934), np.float64(0.055003348967422164), np.float64(0.03820789599978515), np.float64(0.03858826582484183), np.float64(0.04090583780282391), np.float64(0.05371828926630512), np.float64(0.04428341234202434), np.float64(0.05985051313847603), np.float64(0.05164043089143479), np.float64(0.027418504940034418), np.float64(0.021455855176816682), np.float64(0.028194299743911362), np.float64(0.036149971091586144), np.float64(0.029056888578564873), np.float64(0.005170531869249782), np.float64(0.010030519945806756), np.float64(0.04604483326400016), np.float64(0.04256577116468345), np.float64(0.022955081392004514), np.float64(0.036966091124921815), np.float64(0.03426973861963176), np.float64(0.031247522130369184), np.float64(0.008778912900787675), np.float64(0.026491294901628593), np.float64(0.0441529311207193), np.float64(0.05883641545660936), np.float64(0.036858659121076866), np.float64(0.05117970864897226), np.float64(0.05092320584038099), np.float64(0.06881988438122164), np.float64(0.05692252780588257), np.float64(0.05126490803778123), np.float64(0.04760688724925458), np.float64(0.05707737515458432), np.float64(0.05106306609431112), np.float64(0.050296531246851246), np.float64(0.06004769171698309), np.float64(0.07313059958470855), np.float64(0.06380335558092054), np.float64(0.05137917324047635), np.float64(0.055037036024149506), np.float64(0.07207690195699579), np.float64(0.07661292939214125), np.float64(0.0638006196754013), np.float64(0.062055192726447835), np.float64(0.06743415374756645), np.float64(0.06715120359270703), np.float64(0.05711562643660362), np.float64(0.06754914598472674), np.float64(0.0864005287915279), np.float64(0.08627933491860168), np.float64(0.08271042469997955), np.float64(0.08193148681015776), np.float64(0.08455694960385454), np.float64(0.07873147308520015), np.float64(0.07489520373175375), np.float64(0.07572281944820088), np.float64(0.08869901788626844), np.float64(0.08494628486958136), np.float64(0.07922662930655405), np.float64(0.036555507550650936), np.float64(1.5340968695640123e-17), np.float64(1.5325359183182688e-17), np.float64(1.524797800291119e-17), np.float64(1.5107182814102848e-17), np.float64(1.490244722276174e-17), np.float64(1.4638126527820794e-17), np.float64(1.4316843754158784e-17), np.float64(1.3949366193352404e-17), np.float64(1.3540412550597703e-17), np.float64(1.3100342902176697e-17), np.float64(1.2638334167103509e-17), np.float64(0.010713542332086766), np.float64(0.06632667100141913), np.float64(0.06803705887789314), np.float64(0.06704271023608052), np.float64(0.09552409608406559), np.float64(0.08511710186777899), np.float64(0.07124915769970426), np.float64(0.04635319957950198), np.float64(0.059637537527832823), np.float64(0.07148043667970053), np.float64(0.08219341858330025), np.float64(0.06907390711977883), np.float64(0.08044961373182008), np.float64(0.1020182587437067), np.float64(0.11878659806574707), np.float64(0.11323730400510947), np.float64(0.10799882105731566), np.float64(0.11973439156284224), np.float64(0.12759632821990666), np.float64(0.11163219999577004), np.float64(0.09112727148409946), np.float64(0.09240794601497275), np.float64(0.10761466053426799), np.float64(0.09120976031401638), np.float64(0.06018013433715299), np.float64(0.038345387681259994), np.float64(0.047493040525574695), np.float64(0.04080825872247787), np.float64(0.03260061076567089), np.float64(0.026673986757461174), np.float64(0.045381695263804755), np.float64(0.05665690489805343), np.float64(0.048378013011324786), np.float64(0.03298437076872021), np.float64(0.043544876988892416), np.float64(0.048794065508881615), np.float64(0.049659951802220624), np.float64(0.045351013202644196), np.float64(0.050851228124208774), np.float64(0.059885817379373776), np.float64(0.06519463180982123), np.float64(0.04935047381881071), np.float64(0.040457948034705614), np.float64(0.03929279372962097), np.float64(0.055450339281374605), np.float64(0.050142788592167205), np.float64(0.045475593306074785), np.float64(0.04850200188909672), np.float64(0.06790027405958798), np.float64(0.0760532821906499), np.float64(0.08107322690327737), np.float64(0.07646900497389936), np.float64(0.07851948123831881), np.float64(0.07508955686114871), np.float64(0.07163211757269164), np.float64(0.049473682542947525), np.float64(0.043426385001125785), np.float64(0.05609636976954491), np.float64(0.06427335546981681), np.float64(0.046422687027226725), np.float64(0.038769531276461135), np.float64(0.03600104780299762), np.float64(0.03259040175512592), np.float64(0.008321611122771043), np.float64(0.0012305742603626172), np.float64(0.0004960299760612205), np.float64(0.01610293974662997), np.float64(0.011497747921279894), np.float64(0.010796883764911072), np.float64(0.012262123156796806), np.float64(0.030804604361241085), np.float64(0.027178958538011708), np.float64(0.02971910877608999), np.float64(0.02288088964703587), np.float64(0.03286403454796681), np.float64(0.03393970356914708), np.float64(0.03611561142015296), np.float64(0.028859095469243472), np.float64(0.039980740208073906), np.float64(0.048484383263353616), np.float64(0.04375566922286069), np.float64(0.042228230263576916), np.float64(0.04294331644979903), np.float64(0.04184168313935972), np.float64(0.05790021152090468), np.float64(0.07187554602981737), np.float64(0.08404594416812652), np.float64(0.0855957274825542), np.float64(0.08858681915174278), np.float64(0.09004062257201688), np.float64(0.11081265168285888), np.float64(0.1090567087038834), np.float64(0.09743839525015775), np.float64(0.09167134918425808), np.float64(0.10176859960331565), np.float64(0.0947576900691586), np.float64(0.08105771963958912), np.float64(0.07338794275404108), np.float64(0.08107461905900859), np.float64(0.07899996966218935), np.float64(0.05847062076680771), np.float64(0.056875643006070606), np.float64(0.06827693613786126), np.float64(0.0781827610360795), np.float64(0.06860107872909105), np.float64(0.06240235967505205), np.float64(0.06974618149333735), np.float64(0.07727848739124758), np.float64(0.06113673236362357), np.float64(0.05047782181830939), np.float64(0.05180695511182307), np.float64(0.06280600446699634), np.float64(0.05552376018053849), np.float64(0.03542458491229724), np.float64(0.04125902065260645), np.float64(0.04984871033488444), np.float64(0.0518953145082953), np.float64(0.04264738004804641), np.float64(0.037572180919748045), np.float64(0.040294462608820754), np.float64(0.04527706502791123), np.float64(0.03495123684829643), np.float64(0.028018681549587587), np.float64(0.03336301016214824), np.float64(0.04650071313595482), np.float64(0.03849540067720737), np.float64(0.026269178205209277), np.float64(0.022546969626361855), np.float64(0.032052900379380835), np.float64(0.03325442697517421), np.float64(0.024241109811885233), np.float64(0.020737282804093236), np.float64(0.03233577315565313), np.float64(0.015600052612967418), np.float64(0.015251105944565618), np.float64(1.3979102454426494e-17), np.float64(0.006803282709168654), np.float64(0.016738047780785397), np.float64(0.01737077710632726), np.float64(0.012642816946012677), np.float64(0.013526536207630651), np.float64(0.016533245842167275), np.float64(0.01724226680496387), np.float64(0.012562378593671363), np.float64(0.012841871029929687), np.float64(0.016978111352981492), np.float64(0.017272221572286972), np.float64(0.015582492059917108), np.float64(0.016115309812139718), np.float64(0.021696496133617276), np.float64(0.028582840420036084), np.float64(0.03016074216676429), np.float64(0.025908518472306567), np.float64(0.0327052383312363), np.float64(0.03994099303654756), np.float64(0.045784349600390815), np.float64(0.03967133261755239), np.float64(0.0443857426499232), np.float64(0.05116591371812442), np.float64(0.05301255350500825), np.float64(0.03957973580630198), np.float64(0.04534277735526308), np.float64(0.0568376885641381), np.float64(0.07183607797438468), np.float64(0.06474908035058413), np.float64(0.06908815440742846), np.float64(0.07455469522241316), np.float64(0.09105770344454552), np.float64(0.08378969438668998), np.float64(0.07682372354505923), np.float64(0.08954404352713292), np.float64(0.10393754031815401), np.float64(0.0889165375785529), np.float64(0.08438244547166109), np.float64(0.09260062229607748), np.float64(0.11933928250981442), np.float64(0.10341635879398313), np.float64(0.08747274178483734), np.float64(0.09023824657156705), np.float64(0.10264599462925923), np.float64(0.10623075697497524), np.float64(0.0878939794301738), np.float64(0.09218623103991197), np.float64(0.09951637231715492), np.float64(0.10566417236352742), np.float64(0.08845257594375146), np.float64(0.09353187974088935), np.float64(0.09571623354312042), np.float64(0.0990102711656087), np.float64(0.07584260393861929), np.float64(0.0649276657014641), np.float64(0.07120430726182657), np.float64(0.07888715572771757), np.float64(0.06397613817735803), np.float64(0.04773747277331819), np.float64(0.050883620896226285), np.float64(0.06454780963583832), np.float64(0.055629526388646724), np.float64(0.03735047074003345), np.float64(0.03414778638429268), np.float64(0.05194420752537585), np.float64(0.04939673019584869), np.float64(0.02846974245763616), np.float64(0.017803233383127916), np.float64(0.03520634334107419), np.float64(0.047434636475063054), np.float64(0.015354504524575317), np.float64(0.005862439568596002), np.float64(0.015186494772754446), np.float64(0.04239236902280011), np.float64(0.0243953155000094), np.float64(0.0183517972753643), np.float64(0.020714058716828573), np.float64(0.04606670299242856), np.float64(0.017193199746049555), np.float64(0.012532230602210807), np.float64(0.012283428157128564), np.float64(0.03798826313458835), np.float64(0.02520560450780184), np.float64(0.018325242007298036), np.float64(0.014383193119771051), np.float64(0.04656642218621003), np.float64(0.04185063278577735), np.float64(0.04644491190317708), np.float64(0.02068931088071796), np.float64(0.04733909000514793), np.float64(0.032981301754494874), np.float64(0.050370273467551775), np.float64(0.01706950664770882), np.float64(0.045635121303720584), np.float64(0.03547892146682754), np.float64(0.06750744430064565), np.float64(0.033953022646798586), np.float64(0.050726735348382096), np.float64(0.05249625417377599), np.float64(0.06945980228830492), np.float64(0.06415538996572413), np.float64(0.05803331361688503), np.float64(0.06652651712798288), np.float64(0.08032566621574468), np.float64(0.0757236936335514), np.float64(0.07498715267220421), np.float64(0.07469965802144453), np.float64(0.0987779365843959), np.float64(0.08437286382238432), np.float64(0.0792030646882432), np.float64(0.07529897877314091), np.float64(0.09714003690772874), np.float64(0.08277693902360216), np.float64(0.08059707871067694), np.float64(0.07207802551341114), np.float64(0.09513411468337896), np.float64(0.08642789138012867), np.float64(0.0845332827087538), np.float64(0.06744562054723757), np.float64(0.08772702979684593), np.float64(0.07695143623027456), np.float64(0.079410830080381), np.float64(0.06513998741453549), np.float64(0.07399778940031633), np.float64(0.06678715393247228), np.float64(0.07506250599201011), np.float64(0.058016630197999616), np.float64(0.0760293960920069), np.float64(0.07466920961776866), np.float64(0.06520415275107608), np.float64(0.049707513436164516), np.float64(0.05823491026607608), np.float64(0.06677028962070143), np.float64(0.06222443298897974), np.float64(0.043859004248249886), np.float64(0.04323094776203057), np.float64(0.04839638462909883), np.float64(0.05053966306200125), np.float64(0.03486131692104385), np.float64(0.031843992792676926), np.float64(0.038821522503722214), np.float64(0.042080112602301725), np.float64(0.02549013691283661), np.float64(0.024824418802514736), np.float64(0.02903161882135037), np.float64(0.04276822926377246), np.float64(0.02754268987398074), np.float64(0.022097802605914745), np.float64(0.02004213742309656), np.float64(0.027309221364875385), np.float64(0.008233645704898877), np.float64(0.006651556573188725), np.float64(0.008383752652231744), np.float64(0.021301584537193302), np.float64(0.002240717785009802), np.float64(0.00578045616926713), np.float64(0.0015988836510284555), np.float64(0.01445419271067705), np.float64(0.010906336558608919), np.float64(0.007141793312720983), np.float64(0.004135631144253932), np.float64(0.020811350107297826), np.float64(0.021616391878564782), np.float64(0.018768801072793415), np.float64(0.01357857116605468), np.float64(0.02058465285946856), np.float64(0.02708282509918092), np.float64(0.03497045119967946), np.float64(0.024942676842945467), np.float64(0.02877304691627518), np.float64(0.034854630826678744), np.float64(0.04120763861483945), np.float64(0.03341403086554775), np.float64(0.03133135713053095), np.float64(0.043304384751995696), np.float64(0.0581445843418334), np.float64(0.04525305852541171), np.float64(0.043943362938400156), np.float64(0.047639674582988796), np.float64(0.06766522463191427), np.float64(0.04560697728741815), np.float64(0.04047115573134402), np.float64(0.05103911098286609), np.float64(0.09135618663305714), np.float64(0.08968912651608238), np.float64(0.07935123516979216), np.float64(0.07073695399142507), np.float64(0.09893339072432078), np.float64(0.10090857578391081), np.float64(0.09165462736969889), np.float64(0.07242632850401631), np.float64(0.09324766985678115), np.float64(0.10459701912189874), np.float64(0.09398991307330347), np.float64(0.06484150913521161), np.float64(0.07196629561995699), np.float64(0.08857700739867853), np.float64(0.09208242502902257), np.float64(0.06544523641517852), np.float64(0.057972887613939976), np.float64(0.0814762743984663), np.float64(0.07765527320678171), np.float64(0.04960529458347257), np.float64(0.04287903453759618), np.float64(0.06560193355867401), np.float64(0.07253742512592888), np.float64(0.04853302625056564), np.float64(0.022178115132517093), np.float64(0.03667773955202553), np.float64(0.04724788249562579), np.float64(0.038927241715954776), np.float64(0.006238225211940018), np.float64(0.015764996147311177), np.float64(0.032131841203190875), np.float64(0.0284449120726742), np.float64(0.0017028311020873702), np.float64(0.003185075737675128), np.float64(0.01066194102849711), np.float64(0.026487169925431642), np.float64(0.01677214869091371), np.float64(0.020653652948850736), np.float64(0.015572031274137432), np.float64(0.024889539952561508), np.float64(0.00741515181050033), np.float64(0.008095077598284017), np.float64(5.6853473902884755e-05), np.float64(0.005412283284088819), np.float64(0.00488917616945518), np.float64(0.007232226278472924), np.float64(0.0061827108689261915), np.float64(0.0036434076980133632), np.float64(1.9241110367933896e-17), np.float64(1.843081205801907e-17), np.float64(2.4202207047767645e-17), np.float64(2.1450507314529452e-17), np.float64(1.8042457300019765e-17), np.float64(1.8656919497273537e-17), np.float64(2.157811417850143e-17), np.float64(2.2826343708347243e-17), np.float64(2.5007190786041257e-17), np.float64(0.006585127299638819), np.float64(0.008882669236327499), np.float64(0.005349566746303366), np.float64(0.0016061067599740296), np.float64(1.9910733654582294e-17), np.float64(0.0007023601838298645), np.float64(0.0012596791334862705), np.float64(0.004031064104311663), np.float64(1.9479787641983163e-17), np.float64(0.007519412158074629), np.float64(0.017118817988979185), np.float64(0.02510777410237954), np.float64(0.019253935712420817), np.float64(0.027507436820311697), np.float64(0.05319978618083308), np.float64(0.0827670237397059), np.float64(0.0787983917237586), np.float64(0.07220045384505969), np.float64(0.07687396096138739), np.float64(0.12924637741954192), np.float64(0.12184229670634406), np.float64(0.09306805739578335), np.float64(0.06769092297528388), np.float64(0.12086068307561611), np.float64(0.14100501508675797), np.float64(0.11003854823292), np.float64(0.05942128047898762), np.float64(0.11310951831947094), np.float64(0.14933585637480848), np.float64(0.1319466582264264), np.float64(0.03951326586523268), np.float64(0.08967601792857349), np.float64(0.13277351554169628), np.float64(0.16882462643566737), np.float64(0.008805677614364306), np.float64(0.07029908477481622), np.float64(0.0877331020558742), np.float64(0.20878101310834124)]

best_sequence = best_sequence[::-1]
print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

In [ ]:
#@title Data and verification (score = 1.5033)
import numpy as np

best_sequence = [0.016525333922835925, 0.004342132140543947, 0.016632863536566, 0.007677942976001718, 0.01637253993264696, 0.01067947462113106, 0.016263226740132176, 0.013062807120537215, 0.01585524886579596, 0.01446848853270577, 0.01530858082215391, 0.01512122409066863, 0.014482897075927361, 0.014937102984851921, 0.01360226871459483, 0.014294037414479347, 0.012674409549172008, 0.013201456931049523, 0.011590917794675354, 0.011931909809211634, 0.01029147478651306, 0.010585996770534612, 0.008961797813020424, 0.009508417808333296, 0.007469526266084192, 0.008383594784370592, 0.005889430730967238, 0.007373128020683673, 0.004368616778059239, 0.006307118114841623, 0.0029145010569889885, 0.005164983885008384, 0.0018742112779878035, 0.004140911926383109, 0.0009208246862129502, 0.0031204015856199836, 0.0003557220888809256, 0.0023547655799354365, 1.2627661495560697e-10, 0.0010269931965146244, 0.00032547535077276504, 0.0004094739851765631, 6.683420385700664e-05, 1.837102302718205e-07, 0, 0, 0, 0, 0, 0, 8.541785004282832e-10, 0.0001394633711093586, 6.352309562240066e-05, 0.0002516002981728924, 0.00015286298105241403, 0.00014435974266153656, 0, 0, 0, 0, 0, 0, 2.4367628442227897e-09, 6.244055331933473e-08, 0.00011307449215954598, 1.2456593957680594e-08, 0.0004356278894996095, 0.00011346136684086212, 0.0008305949667295345, 0.00035581200425494606, 0.0011804401400263096, 0.0007927487065778777, 0.001617736233627225, 0.0013118477299217113, 0.0021683527761933276, 0.0019204925051912532, 0.0029473595963095236, 0.00277210421086744, 0.003569494428873195, 0.0037424290257134425, 0.0041706401836997375, 0.004830378328452505, 0.0048548877835327174, 0.005978552165266768, 0.00569627838091334, 0.007134655542892641, 0.0065666523135024515, 0.008312883514026627, 0.007342937001820333, 0.009252286895566023, 0.007839677233341525, 0.0099477081434236, 0.008292238779620462, 0.010365542305022611, 0.008788560376997996, 0.010326585883677809, 0.009133914554810298, 0.010011276575024614, 0.009452822058482472, 0.009506642337782674, 0.00955802507228419, 0.00897312976228083, 0.009534181896894945, 0.00854573639887204, 0.009351143716417376, 0.008172236869333157, 0.008789137830727576, 0.007910278708156382, 0.008297939516001463, 0.007577482017418297, 0.007622357377709036, 0.007197147338321477, 0.007056214700027877, 0.006914717487710335, 0.006543979200734859, 0.006404846964333455, 0.006130673361661352, 0.005696255627793016, 0.005499785954364833, 0.004869593128459553, 0.005009526881907336, 0.004196089418811039, 0.004258114640955026, 0.003745173151287454, 0.0034624740775713067, 0.0033565833432596046, 0.0025669796633568352, 0.002786684187392822, 0.0020045472421312913, 0.0024189654553530855, 0.0015086588854346134, 0.001913470244413332, 0.0012799650949096306, 0.0014715120004672752, 0.0010960637516264202, 0.001047649401988188, 0.0010132305548720651, 0.0008192866729942669, 0.0011011757319583325, 0.0008879443313454401, 0.0014673903891128633, 0.001152538811711196, 0.001934889745222308, 0.0015832221779612024, 0.002448167313864299, 0.0021783131955780155, 0.003075146263845917, 0.002651540548404068, 0.003616100556696731, 0.0029369905957753436, 0.004182705551240284, 0.003121722540131503, 0.004383569252141575, 0.0032630801512384985, 0.004663466369865245, 0.003635097328176033, 0.004651087650921392, 0.0037876996828772034, 0.004531231026063802, 0.003895503672202056, 0.004336802134626474, 0.0037648109891652, 0.0041485348773852015, 0.003555798409568132, 0.004123707632285561, 0.0036907549309886795, 0.0042862550802639345, 0.0038073496053013475, 0.004481557242932329, 0.004169203230944419, 0.004989522242857631, 0.004793632104420788, 0.005805956611066798, 0.005389690439818521, 0.006502515705923838, 0.005757748915763484, 0.0073473939724871115, 0.006391827874248466, 0.007940130318065744, 0.0068392498406614115, 0.00853763815255897, 0.007152540787379493, 0.00871745268748166, 0.007207286114058916, 0.008665697458974815, 0.007269043033860783, 0.008517482837885903, 0.0073573799972509665, 0.008275578971891939, 0.00706089824345149, 0.007846210426245082, 0.006922930523707315, 0.007677413784355708, 0.006895267201368752, 0.007276256460429712, 0.006386273018544175, 0.006517495581938694, 0.005759927653598743, 0.00600430795689629, 0.005350094103637339, 0.00488565789644768, 0.005548905727311105, 0.004030062266730461, 0.005627982083201863, 0.0030187772341329655, 0.0057296010229673745, 0.002041654511387749, 0.005567839593451668, 0.0010289655042160079, 0.0055648502569121475, 0.0003792472226244967, 0.00581348891662558, 0, 0.005975688416772019, 9.884712515593945e-10, 5.222515971736415e-05, 0.00024747243295521486, 0.0015719451969111288, 0.0021177313907137394, 0.0027721227838597346, 0.002908291169219237, 0.0031243063630093607, 0.0031812605663878992, 0.00349089302472111, 0.0034498813616144403, 0.004180693838898336, 0.0038493292459285617, 0.0047991677548919216, 0.004001107041215515, 0.00507577283060939, 0.0034772894010321976, 0.005514364815223544, 0.002982254184828697, 0.006122655690323167, 0.0023455249711643793, 0.006563358484124383, 0.0006738387164541264, 2.372921053815445e-09, 4.0318699201120456e-13, 0.0036392249160482936, 0.00244780473144443, 0.0032232422520043127, 0.002318711021723049, 0.0026821244290412666, 0.001899618412155175, 0.002258783250004411, 0.0014437819018894944, 0.0018907740341342026, 0.0008605952173532863, 0.001600204420189356, 0.00037761839267951903, 0.0013299335132615062, 0.0001302752609161145, 0.001024338911607099, 6.262950603204059e-12, 1.4125941216371394e-11, 2.4741425699661353e-11, 0.0002250001117009371, 0.00011915056264707601, 3.1198000378228605e-05, 5.305466231047151e-05, 6.903193388669381e-12, 2.515918379590219e-05, 2.3542803955580307e-10, 0, 0, 0, 3.0012281886269876e-13, 0, 0, 8.436705312244516e-09, 6.542109542868716e-13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.851702023521836e-05, 0, 0, 0, 0, 0, 0, 0, 0.00033064371776043224, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8.303167937861038e-14, 0, 5.685045630065153e-05, 0, 7.40333858814176e-15, 0, 0, 0, 1.5810219461830796e-13, 1.0720135978444464e-13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9.74022778723602e-12, 6.3429892972418376e-12, 0, 0.0002094192931362532, 0.00017016552649368728, 0.000522479839667033, 0.00013777690090445685, 0.0007347947975935137, 0.00011527216672775284, 0.0007606593404418867, 0.00012003055155400424, 0.0006531460534083784, 7.075217368161887e-05, 6.993787381635239e-12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.7983237698809981e-12, 0, 6.793509150305048e-12, 1.2897311520134477e-11, 1.060935522189606e-07, 2.8235029554801093e-09, 0.00011560832334927504, 0.00016076666095232983, 4.183844370466362e-05, 0, 1.976257044730229e-11, 0, 0, 3.490087746858616e-07, 0, 0, 2.019735319623012e-08, 0, 0, 0, 0, 0, 3.7333314061891114e-05, 0.00023673093238346402, 1.172557847630218e-05, 4.668239621871711e-11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00027599964110978394, 1.0053503111772007e-05, 0.0008827113940589839, 0.000360354797057012, 0.0012707622226974119, 0.0009189372042644038, 0.0016632416792513344, 0.0014877418407688895, 0.0017633976318253378, 0.0019471885208046313, 0.0015105760042074185, 0.0021732569980073583, 0.0012235366666102129, 0.002209335946766708, 0.001035819178415938, 0.0019807708939145905, 0.000975767838233637, 0.001714478907209366, 0.0009212671655650671, 0.0012596975207030139, 0.0006171942488475366, 0.0007778921583581983, 0.0002091562813788698, 0.0006533063481667869, 0.00018996289316914586, 0.0008266608977376392, 0.00027393484847216984, 0.0006710682362941775, 0.0003906419432956259, 0.0006404626555969202, 0.0004118638414311786, 0.0006157205700049829, 0.0005629155926238838, 0.0009772093514262758, 0.0005961018371929962, 0.0002262735330191479, 0, 0.0016383720242172618, 1.5818117932274003e-07, 0.0006000190590923734, 2.226362414720822e-09, 0.0005033365591657632, 0.001272564690979263, 0.002635750481463368, 0.002053149857957289, 0.0025158967905830098, 0.001994863236756821, 0.0022069387101684855, 0.0024060793862940356, 0.002287944542633988, 0.002711402938655665, 0.002299834572095875, 0.002947931047446052, 0.0020671069197596846, 0.0026023815084610345, 0.0016692352761911004, 0.002242422751253244, 0.001650078367268472, 0.002190239357519321, 0.001441973905629961, 5.479437099340945e-05, 8.850197467471013e-07, 1.4074547328007379e-05, 2.4602342252290227e-12, 5.477159343548839e-05, 0.00025116706717453937, 0.00035511149270566557, 8.422799017537232e-13, 0, 0, 3.377686118389609e-12, 4.0746715249500766e-13, 1.4372577673415659e-11, 1.0115510283013496e-12, 0, 0, 5.967508916346239e-13, 2.6451010593358123e-12, 2.3764133338535256e-12, 7.075029587208726e-12, 0.00015767079443132164, 0.00017231225397488768, 0.0005587556139910879, 0.0001315073961933705, 0.0005119390678781914, 0.0004990072327744406, 0.00060830923866225, 0.0008163622640306499, 0.000718928874568972, 0.0010680809504137973, 0.0008500676293476301, 0.001351271647677372, 0.0009164098836653784, 0.001373945436176292, 0.0008078305110796335, 0.0013338717355753757, 0.0007405249211128856, 0.0015735885322727802, 0.0009024364672581173, 0.002014190374965409, 0.0011668329517413437, 0.0023379909406577126, 0.001423527788002205, 0.0024178884467245536, 0.001653017381959315, 0.002360805625840889, 0.001905016220909116, 0.002156468677174009, 0.0018336962808120593, 0.001703232242789998, 0.001788325917364894, 0.0012081285137534932, 0.001589090584784071, 0.0007388880652218952, 0.0011224971961990692, 1.6759781668816954e-11, 1.037591169747288e-11, 5.973339620372335e-12, 1.8056836150189509e-12, 0, 0, 0, 5.386545414205246e-10, 0, 0, 3.2781172182222006e-13, 1.7274032501183451e-12, 1.7069367745115381e-06, 7.053623472483612e-06, 0, 0, 4.0266973207767e-11, 0, 0, 1.7130513711309603e-11, 0, 2.0437159989679526e-05, 1.623598989983676e-05, 0.00041654519674394133, 0.00010070774275190208, 0.000868985181586171, 0.00030652110414738356, 0.0013893144138371614, 0.0005215499582292643, 0.0018267759814421334, 0.0008047434816944093, 0.0024346803797665423, 0.0011113949760203322, 0.003072905873726648, 0.001735426389654195, 0.0037887674216714974, 0.002101340348791254, 0.004278476163651144, 0.002344926439894167, 0.004501876842415637, 0.0022896170951892895, 0.004599725980068682, 0.0023990516069184948, 0.004434715830922987, 0.0025194608509742193, 0.004169759735180899, 0.002681063732426702, 0.0037863570320741677, 0.0028152696306720103, 0.003355266082497818, 0.003016487541938705, 0.0029511075494984902, 0.0030726920598799297, 0.002754915653380842, 0.0029930301581040787, 0.0025552683948620904, 0.0029296870043003308, 0.0022801540996369475, 0.002980353667141949, 0.0019179189362645232, 0.00293801142460969, 0.0014970051516661155, 0.00280813758861251, 0.0011869541903700674, 0.0025146527278178668, 0.0008109603149594405, 0.0020544066603399138, 0.00047063703699259734, 0.001725213192662759, 0.00011478615348582241, 0.0013934205405655895, 1.1280307379840462e-05, 0.0011044144224485607, 8.21677150546136e-05, 0.0007171273200106046, 9.780345531063383e-05, 0.0003376573385682038, 0.00012906817373465644, 1.2794114030551006e-05, 0, 0, 0, 0, 0, 4.202280951932964e-07, 0, 0, 1.3537467520337188e-11, 8.940599836183322e-11, 8.120350579763928e-12, 0.0001169622427484581, 2.850789696070192e-05, 6.87115217362448e-05, 9.741117395218665e-05, 8.231163188348116e-05, 9.466226668522773e-05, 7.977468194461835e-05, 6.458489049048562e-05, 7.883661088843704e-13, 0.00018874287096225082, 4.6898851254549527e-05, 0.0002993458948670473, 6.554397792298495e-12, 0.00010328176521734727, 8.354806793335773e-13, 2.437633480038781e-10, 4.471433644932207e-11, 3.7964217057756186e-05, 1.9723252008395675e-05, 0.0001934719025129586, 2.981993877761925e-09, 0.0001266413588266972, 2.2919502112231814e-11, 0.00023609113191887517, 2.254605793290183e-05, 0.00037986934746750086, 0.0002524235488250927, 0.0005162690364963693, 0.00034756487903485025, 0.0006062873648688183, 0.00042325478495429993, 0.0008313002233739535, 0.0005674382142284491, 0.0010753805244074074, 0.00094943732152759, 0.0011363070270992274, 0.0012191426971540763, 0.0008782621037194733, 0.001520142824412068, 0.000748915962979619, 0.00156812809174364, 0.0008744412930439533, 0.0017675786197078208, 0.0012490881211877796, 0.0018706637546902932, 0.0013714312420152954, 0.0017293712413165518, 0.0012481621126658221, 0.001318282255030651, 0.0012911285197493413, 0.0010363639892419184, 0.0012714978994709787, 0.000711569902305452, 0.0009490628023693336, 0.000541003799995222, 0.0004582919581647078, 0.0005252843997660561, 0.0002342655123377521, 0.00028709787185637235, 6.868542695205591e-05, 0.00020272865786866555, 0.00010596480605358367, 0.0004941886242033014, 0.00037022829067127254, 0.0008543461665741631, 0.0006910726477243049, 0.00124316784413888, 0.0011540263878897273, 0.0017399700912749728, 0.0015103967277727902, 0.0021079522480623634, 0.001697094878455313, 0.0022705642777792394, 0.001855409679002836, 0.00224939776284537, 0.001923850718514058, 0.0022276688081982474, 0.0019551231115083802, 0.0021970866908103434, 0.0019106714296500307, 0.0022433769258180993, 0.0017035109321303081, 0.002019117005058464, 0.0014527647503102536, 0.0019937499283362078, 0.0013555372377751453, 0.0016507571716455223, 0.001006828591041456, 0.001819852810017769, 0.0008311420871905168, 0.0018386862484658108, 0.0006331936396166156, 0.0010129285399927547, 0.0010544709833826054, 0.0004040130738556154, 0.0014348534256770832, 0.0001708689905306833, 0.0016197011530324388, 4.049544012012388e-12, 0.0010617624498292578, 5.767101426284827e-12, 1.4853073917893222e-11, 8.743777410212987e-12, 1.7834090521653705e-11, 0.0004918041715557458, 0.000915545053419751, 0.0005470642600644748, 0.0009551580669609227, 7.169263228748768e-05, 0.0010667938534696924, 6.224026367569784e-05, 0.001027878242904437, 2.9897894327779925e-05, 0.0008374722401402535, 0, 0.0005874927891629429, 0.00022641185611239358, 0.00041257798296651376, 0.0005119590105675532, 0.0002669552926969132, 0.0007451059308017039, 0.0001915852712068901, 0.0009846467581353972, 0.0001169004995151235, 0.0011008751723405896, 5.376035820260125e-12, 0.0012528378480128918, 0.00010675519547755032, 0.001226071757736149, 0.0006385472930871815, 0.0012806842667551339, 0.0008789745888277712, 0.0013684534402901662, 0.0010004362991937812, 0.0013265700815821805, 0.0010661161668933327, 0.0010914785050546191, 0.001170478338452549, 0.0008365684136031311, 0.0012504683492618799, 0.0004942091802147899, 0.0011401043818915762, 0.00029152548163460497, 0.0010236118026047299, 0, 0, 0, 0.0002485144308795439, 0, 1.712522541562613e-11, 0, 0, 0, 0, 0, 0, 0, 0, 2.376701359994578e-10, 8.765393240763393e-05, 1.2493498403082359e-05, 6.646145807639256e-05, 5.268321192078778e-12, 0.00010440936473029128, 0.0001577340973413799, 0.0002495543857954713, 0.00031988941824878653, 0.00018550259523647072, 0.0003646877163387898, 9.822183485261596e-05, 0.0005420021456040684, 0.00019385718735736803, 0.0004993810258941816, 0.000248085558333025, 0.0003441129332000408, 9.063971697369025e-05, 7.347385679702415e-12, 1.6871768601367723e-12, 0, 0, 0, 0, 0, 6.187832393656484e-07, 4.200455731635624e-08, 8.305972757023413e-05, 1.9641217463936605e-05, 0.00031756973668884303, 0.00035292548093088905, 0.0008084836991439725, 0.0007571874613982379, 0.0010737539855594318, 0.0011722731268150924, 0.00121244470607576, 0.0014800540157231179, 0.0012973539336573402, 0.0017409613159730922, 0.001381620353969942, 0.0019312454272135803, 0.0013890682572414792, 0.002011488425541076, 0.0015722944457884417, 0.0021231809244680526, 0.001865864597511776, 0.002155349567074703, 0.0018224411417501788, 0.002120110833274454, 0.001823284715752552, 0.001972496230513219, 0.0018389761809630444, 0.0018040268168862416, 0.0018269800910986425, 0.0013366552205663893, 0.0016677521098964429, 0.0008924359888427207, 0.001571694630792599, 0.0005009538663008247, 0.0013102922340443744, 0.00029845381197587094, 0.0011609146925915458, 8.776040577142154e-12, 0.0009278266409686408, 3.257844710467951e-05, 0.0008988518258043818, 7.098647778305792e-05, 0.0008190226854486839, 0.00014019929953551825, 0.0005691755161069228, 0, 0.00047567760078505726, 4.365338008290081e-05, 0.0006650596640434734, 0.00017855375553961538, 0.0006864893580870438, 0.00020418281689214062, 0.000640062467129905, 0.00011024327424948153, 0.0005692712567766451, 0.00017630621744153777, 0.0007033678689516255, 0.00035835711292827644, 0.0007648937630598511, 0.000577084900414001, 0.0007443615331907002, 0.000523211430237437, 0.0004355018007980024, 0.00031261215752785034, 0.00019918970913379602, 2.8211869210445744e-05, 0, 0, 0, 0, 3.7484598599412206e-13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.129396058086965e-12, 0.00036302960822498384, 0.0003410093540317857, 0.0005171328958069394, 6.766753805890814e-05, 0.0008337026266559663, 0.0007019383301466581, 0.0009441577890326136, 0.0009293104770631911, 0.0008904100241508451, 0.0011043238526127173, 0.0007853925033627653, 0.0011028665432341398, 0.0006380741725303234, 0.000972897319849738, 0.0005907470544158964, 0.0007465691708717471, 0.0007455011387330375, 0.0006520790160304005, 0.0009923296985802918, 0.0006153610446689166, 0.001047339098688097, 0.0003815590095921147, 0.0008659528979513272, 0.00021370538456984454, 0.0006742234751364607, 0.00012557587454405962, 0.00042948972424084397, 0.00011974488346716245, 0.00028965606024921274, 0.0001854696328662649, 8.69878296383287e-05, 0.0001093904861277345, 6.42531519368775e-07, 2.901018310531251e-06, 1.3195281681496511e-05, 4.097816374265438e-06, 9.619254072611364e-06, 0, 0, 8.305099913884757e-09, 5.468490244869927e-05, 0.00018911244189832174, 9.610744958726189e-05, 0.00031821760398002857, 0.0001752418129472256, 0.00039694313542121523, 0.00033558757477227154, 0.000244602374611107, 0.0003671544662160588, 0.00025148670882887785, 0.0003880016191210485, 0.00012933407027759683, 0.0003838482039277002, 8.405499849661718e-05, 0.00017898531979092586, 0.00014811419101026405, 9.309289155936665e-05, 0.00011965453350802524, 0.00012255396024778232, 0.00031605166465257357, 0.0003026975180844961, 0.0005790089100818798, 0.00045047665702760236, 0.0006390502769903284, 0.0006318364019581476, 0.0006853268686619323, 0.0009247347873862011, 0.0008608858736226075, 0.0011140185553810582, 0.0009139103108102447, 0.0012555549217312491, 0.0010071270700710492, 0.0011706472610151042, 0.0010086682698553057, 0.0010687726392253677, 0.000925078954697556, 0.001091821187974099, 0.0009390573049020603, 0.0012781525591329165, 0.001161177623022778, 0.0017029046537532229, 0.0015246683538702984, 0.0020453912530529897, 0.00201523315689912, 0.0024252083753289523, 0.0023779776719987868, 0.002407417391845245, 0.002862980917199401, 0.002546611695628053, 0.0030365254548092874, 0.0023880628410308724, 0.0033210034519500103, 0.0023379871324677045, 0.0029140271748267035, 0.0032865363115483935, 0.0031361922072109393, 0.0035116643291726213, 0.0032975633349974086, 0.003571771807680827, 0.003546693225125388, 0.003831330215447514, 0.0037749102615783374, 0.004114806799293186, 0.004133840842628223, 0.00432099233270035, 0.004389399758916408, 0.004387730593979308, 0.004554780323540404, 0.004040611612681107, 0.004527052110369243, 0.0037112419697636614, 0.004269244006750042, 0.00331068278940446, 0.003832665903102465, 0.0028489962828895715, 0.003618561928979385, 0.0022056362996830978, 0.0033272281595275567, 0.0015118889792840387, 0.002091310078099139, 0.0016571847411741795, 0.0004909294928808959, 0.0024457224899706136, 5.772573980830329e-05, 0.0009167492451843438, 3.4350334420331858e-12, 4.703874918338537e-12, 3.1810325447983568e-12, 0, 0, 0, 2.093230040609556e-12, 0, 0.00010294043593582841, 6.114105020597956e-13, 6.087428678636637e-05, 0, 4.517824321100892e-13, 1.3951334154812369e-09, 3.863725586871034e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.3078041919514764e-12, 2.2436398585820276e-06, 0, 5.654123094536858e-12, 0.00023604031978296494, 4.7177732741382123e-07, 1.2951323172279552e-11, 5.205582838717618e-12, 0, 8.438789419352794e-05, 1.532070383915551e-11, 0.0001647517405173557, 0.0005533664030110303, 0.0002856162018045899, 0.0005970622517820077, 0.0008003456994546548, 0.0009239824259632155, 0.0009854813083842867, 0.0011150737595581161, 0.001091553736717318, 0.001143438990718595, 0.0009197331899692887, 0.0010276137539788036, 0.0007100668125188487, 0.0008829414126334074, 0.0006256205242202516, 0.0009221880961805799, 0.0004870751915744794, 0.0006353171384754842, 0.0004561735785620212, 0.0003202931522383177, 0.00045952204269759494, 5.0130741707036294e-05, 0.0003314386294917294, 0, 0.0002578331216033739, 0.00013838579208250317, 0.00025282076982545133, 0.00017803080714145152, 0.0003980099531879674, 0.00013768720735256163, 0.0003711299815189293, 5.3673215885112425e-05, 0.0002519214303026988, 0, 0.00044415793061432647, 0.00011402863304128537, 0.0006669458666057052, 0.0002252467698793101, 0.0009353468725254973, 0.0004498134100032954, 0.001047175813004611, 0.0006163299787167936, 0.0010867700128835602, 0.000644378072645307, 0.0012119425152407053, 0.0007276240169306409, 0.0012349454872003052, 0.0007766114070955826, 0.001347500832721879, 0.0009541282427749589, 0.0013962071785651095, 0.0008434890960654134, 0.001351959985826701, 0.0008831733299966351, 0.001448064095321332, 0.00100570429362051, 0.001314116319209483, 0.0016757548528372925, 0.0013433625307909364, 0.001799565064739081, 0.0013974039039227509, 0.0019032948465612515, 0.0013190379983336296, 0.001920599626480464, 0.001248821287246695, 0.0017807369401085813, 0.0013680279581424364, 0.0016516656981361967, 0.0013357660528713654, 0.0014803428029238927, 0.00115546385358235, 0.0013482524155905137, 0.0010446324189699308, 0.0012216366732378764, 0.001284688065425545, 0.001302312364046043, 0.0017022767594005422, 0.0015198709309585297, 0.0014766341790533898, 0.002083475964602315, 0.0001568348915180457, 0.001966299071263037, 0.0013379076500632846, 0.002081999424658305, 0.0015834135177688795, 0.0020640377311268157, 0.0018438777783602635, 0.0020278295944369607, 0.001781018819552621, 0.0019265389621751885, 0.0016352187528529045, 0.0017760346737882445, 0.001548545636978653, 0.0016727632210909323, 0.001517186818581204, 0.0015265196177840657, 0.001505578044427812, 0.001384557742649234, 0.001390386025323247, 0.001081775767431221, 0.0012970425445656378, 0.0008077739421501739, 0.001064995617374469, 0.0007102063702255112, 0.0010266641390729889, 0.0008684785553024981, 0.0011162773646175492, 0.0007125705106462639, 0.0010711770731931225, 0.0005985168248918252, 0.001083437845125717, 0.0005351227846933534, 0.0010197775796133906, 0.0006527556811832733, 0.0010874338698749113, 0.0008444804612921883, 0.0011333500246271382, 0.0009989058533065415, 0.0011203205734653398, 0.0011402469060331822, 0.0010570826587404708, 0.001305250285939223, 0.0010143694121519558, 0.0014220424319293992, 0.0011010886505132657, 0.001122600402380801, 0.0021051680778142157, 0.001332072626844027, 0.002166760695556438, 0.0012952859383184975, 0.0019511305361694952, 0.0013022652175333153, 0.0018194315754130532, 0.001358981943519479, 0.0018211616992655397, 0.0016005767529580167, 0.0020326174311391925, 0.0018485744563903773, 0.00225716116574901, 0.0021187684346843233, 0.0024296480965061698, 0.0023305760875667906, 0.002580102985302697, 0.0025083864127520834, 0.0027712959756773362, 0.0029492288200212873, 0.0031300980840035498, 0.00312458367137012, 0.002987547024158963, 0.002990704283888192, 0.0027325382492977127, 0.0027585194659046656, 0.0024843045739330032, 0.0025021912956582925, 0.0023052411520046866, 0.0020423982938841, 0.0018586314788587687, 0.0014995888061482936, 0.0013174404265962917, 0.0007963425766470689, 0.0009726003286571487, 0.00032039116679238806, 0.0005708070963274338, 0.00012482312772193072, 0.00038885702329329854, 5.130407253463353e-10, 5.418902886563131e-05, 0, 0, 0, 0, 0, 0, 1.3853104357875046e-10, 5.989530288833569e-11, 2.2120636219525215e-07, 2.0846252743177552e-05, 0.00016935975057922914, 5.214217634874714e-05, 0.00014501019273375294, 5.858115636780416e-07, 0.0001784536956419691, 4.7849867619311474e-05, 0.0002892713271445873, 9.152834874548324e-05, 0.00016233843590572606, 0, 0, 6.915647594492721e-13, 8.627430826848029e-10, 1.3410475770069739e-12, 0.00016914269153253894, 0, 8.675068104075762e-06, 0, 0, 1.4431617591040238e-09, 1.3461623837101616e-11, 1.1917906023188084e-05, 0, 0, 2.7698155695833643e-12, 5.676298697532026e-12, 8.111142377866226e-05, 0.000293066184713743, 0.0003145085903853291, 0.0006316743717118298, 0.0003270415693129453, 0.0006536034957877817, 0.0004221442152300616, 0.0008324944251803206, 0.0006289773425254382, 0.001263638620434722, 0.0009174906377652446, 0.0014944485786457303, 0.0011431969877377882, 0.00161015556770122, 0.0014684930309423678, 0.0019231670880436357, 0.0016737095698828965, 0.0021403165745934056, 0.0019313401679855518, 0.0024114758087713504, 0.0020795322845102507, 0.002775749649004273, 0.002171250505823977, 0.0029126297316474418, 0.0022441183147621385, 0.0026308908415132713, 0.0021982908072581326, 0.0026106578842155716, 0.0023048294038804087, 0.0026303059917694663, 0.00277081908818717, 0.0027259775906712663, 0.003097626265080099, 0.0027586152906300997, 0.003208661877223096, 0.0029422317215618997, 0.003215120618654415, 0.003326894991919905, 0.003371794965053797, 0.003227278828838307, 0.003318413619113294, 0.0031198635025539542, 0.0033515311788022777, 0.0029306739883547266, 0.003049197303531085, 0.0025247954140934477, 0.0029320385563907673, 0.0021892229601232814, 0.0020583938781536817, 0.0023744641179713503, 0.0022799671824133205, 0.0020811090327489644, 0.0022651802907398123, 0.0020874341950757563, 0.0023259912807449107, 0.002024417659302079, 0.002329726570729276, 0.0018953169603158104, 0.00250790414756492, 0.0018727789270928145, 0.0025413282494732748, 0.002132767178521067, 0.002654796428355671, 0.0023297532279003674, 0.002665635326012392, 0.0024966410227159615, 0.0027826830088051695, 0.002609614893981099, 0.0031024682238394064, 0.002758727408847734, 0.0032794437108280626, 0.0029340539210298386, 0.003649019587192004, 0.0032511667099640205, 0.0037034404918679254, 0.003319119287402033, 0.003551277534402596, 0.0032452399858605274, 0.003649288949279543, 0.003342249200395614, 0.003696328534360829, 0.0034734865993612295, 0.0037185477192214047, 0.0035430879350975995, 0.003745994082678646, 0.003717173807563281, 0.003733772199519294, 0.003990634604599725, 0.0037924284884897257, 0.004257803930905052, 0.003792701855782515, 0.004372591107837335, 0.003905606115642534, 0.0045549995410270215, 0.004054086768868999, 0.0047222389407411336, 0.004013155347129333, 0.004710553624584375, 0.003867238844685177, 0.0046688151094312815, 0.0038910137444665965, 0.0046912374314787095, 0.0038921333833569134, 0.004352747862333008, 0.003945257178347582, 0.004115803901212906, 0.004009821375162972, 0.004205690226410127, 0.00404698704219778, 0.004161785561939708, 0.003973553223660203, 0.004081792549123488, 0.004118524788749896, 0.004268658534369651, 0.004324031489054021, 0.004425609183505432, 0.004607856900550719, 0.004536785641038772, 0.004860584372742189, 0.004456829696250904, 0.005030903933495211, 0.004149192129224395, 0.004873541117415312, 0.004482485767762825, 0.0045247482507548425, 0.004870331207733262, 0.00426835564005964, 0.004469745733488206, 0.0037917322601279868, 0.0035650304970789234, 0.0035048407476468437, 0.003304893511339686, 0.004216150018714158, 0.0031079419421877067, 0.004063312535932841, 0.0028630087498577126, 0.004069517925182622, 0.0027833447859629684, 0.004172638158994853, 0.003020353752820627, 0.003980952621463997, 0.0035468939177111638, 0.0039214832019125135, 0.004030622157336899, 0.003891580338652315, 0.004173354697545582, 0.0038428912308907585, 0.0043277155932450635, 0.003954928369746023, 0.004249644090485521, 0.004011006946966524, 0.004219798514491745, 0.00398036155560067, 0.004182733297868439, 0.0039806265116542365, 0.004255893398911673, 0.004056520002116663, 0.004286902346873431, 0.004194376066172447, 0.004206433499915809, 0.004180403828111908, 0.004140025653628429, 0.004389694970166082, 0.004273192892510797, 0.0044876595558549425, 0.004192620209441422, 0.0043197205104430324, 0.0038285532007709864, 0.004084246284105335, 0.003543741230264327, 0.003894339706719315, 0.0031636083214987383, 0.003532707318909794, 0.0027519062263467483, 0.002793070831081892, 0.0021796935610076096, 0.0029830416247324377, 0.00206974721764331, 0.0025103467788783174, 0.00155283988126285, 0.0024278726261073155, 0.0021721481839946243, 0.002718339311744236, 0.0020730160953766033, 0.002817683051234757, 0.0023580164722172997, 0.002956259922477706, 0.0027560778138470163, 0.0030304806240040237, 0.0030063631166685387, 0.003215705215032951, 0.0033527958436914535, 0.0034865490506364363, 0.0037321248849014493, 0.003794193354508294, 0.004022349345694979, 0.003625801691545845, 0.004018529380284627, 0.00354752371417572, 0.003912803074730486, 0.003679703814642685, 0.0038954280273583593, 0.0037596556528665995, 0.003836043213406604, 0.003646152598549892, 0.004063661289480848, 0.00362556015526621, 0.0043149231636179204, 0.003602784557654466, 0.004549984822583418, 0.003919090859508094, 0.004557088528588322, 0.005009232151679599, 0.004427880494550099, 0.003258389743765365, 0.002701105453931199, 0.0026159122949422006, 0.002074914266803298, 0.0016139007197598068, 0.0022656537718133856, 0.002238893924982564, 0.0020871241283093814, 0.0004468452467565817, 0.0001620221848223321, 0.0004211718828770731, 0.0007377530381654145, 0.001157286123133577, 0.0016487700122194376, 0.0019729645250645624, 0.00182735181573864, 0.001318207770942897, 0.0034126300066629274, 0.004027327008202367, 0.004280426375270839, 0.003823322389946513, 0.004027248757139482, 0.0038961902352261874, 0.003945760088471478, 0.003833821948602247, 0.003747405559151763, 0.0036710592452653765, 0.0037275904268855927, 0.003950232945786856, 0.0040483319462623684, 0.004103410103743331, 0.0031129435719210855, 0.00032724797590254597, 0.000680129261674297, 0.0009606606907827753, 0.0011973603023338962, 0.001449360839306637, 0.00032462380529100696, 0.0007786282597008816, 0.0015873044823857558, 0.0005823867258867681, 0.000853961910149271, 0.0011946586484051845, 0.0009854570473270983, 0.0008685621030565068, 0.0023999005054070534, 0.0023565353672970104, 0.0035265926337886924, 0.005198679525918213, 0.004324732931136792, 0.004161545423242727, 0.0043691817359544985, 0.004271419635218512, 0.0017958943872056516, 0.0014756197439302588, 0.0013033596563591864, 0.0017420230500440692, 0.00142911005965675, 0.00177723837689362, 0.0016515143086147461, 7.983643021153873e-05, 1.648948449316838e-11, 0, 4.272306544231333e-12, 1.0923493831151174e-11, 1.8677476834535302e-11, 0, 0, 0, 0, 3.32229789516716e-13, 0, 3.962346769521987e-12, 0.00022819891482975387, 0.00027880070882539725, 0.0008652881906732044, 0.001706514532320204, 0.002046584145073462, 0.0026714646704505073, 0.002090508247740371, 0.001986955036466758, 0.0027779755618653687, 0.0030074092650920407, 0.0045810973792268315, 0.003325693002243771, 0.00408915911061026, 0.002441087528661914, 0.0042513435423488545, 0.002923880269436504, 0.002855893765628638, 0.003185436701661933, 0.0028548970197315024, 0.002028976312400889, 0.0019564669611338626, 0.0015682527494376222, 0.0007462127281226077, 0.0006346184788008037, 0.0013806731744836192, 0.001145465395207721, 0.0016960139597618341, 0.0014046328048198126, 3.972887672862306e-05, 0.0010076674153824596, 0.0013571564755857958, 0.001089403104323968, 0.0012882611763892315, 0.000581736622714395, 0.0002134736737814096, 8.468882274677461e-07, 3.1946060603358346e-05, 2.803968033905011e-09, 2.08343202779493e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8.23298097224734e-11, 0, 6.381278949146175e-12, 6.357952171683157e-12, 2.302189247342627e-11, 9.751578592666007e-12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1859755457663802e-12, 3.9747961964795857e-10, 2.53388896816256e-12, 0.00015222130492961917, 5.800290760533733e-13, 0.0004103681143482205, 0.00010083434846408971, 0.00021961656173259745, 0, 6.290195317288356e-06, 3.5365540942765936e-09, 2.298769245556595e-09, 9.389333135457361e-05, 0.00010058562889896017, 0.0016545463853859343, 0.0011395472327599071, 0.0028700125001600175, 0.002397825568360819, 0.0030369782016165026, 0.0033411208241177835, 0.004505409251472024, 0.004173293287658679, 0.0041305212627073965, 0.0035581020130041703, 0.003253608347915642, 0.002750983357175681, 0.002405690554958155, 0.001478800570891161, 0.0015001559191787789, 0.0018919815015357, 0.0023320051130704717, 0.00299767325975326, 0.0032310256775071006, 0.00315523171849092, 0.0022254988041597885, 0.0018196073464902527, 0.0014936438966717022, 0.0010153016934804256, 0, 0, 0, 0, 0, 0, 2.621040535485272e-12, 1.6579128767105556e-12, 6.41083466563061e-06, 1.102681650268519e-05, 1.8589923869091456e-12, 2.7377451702559336e-12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0012209867855545637, 0, 0.0002285291546996559, 0, 0.0002465268075503875, 0, 7.084239837785659e-05, 0, 2.1504621678391395e-06, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.3600331402332063e-09, 2.3625164988053984e-11, 0.0018256676994679996, 0.0014493014325453223, 0.0007694447810575849, 0.0004475091587025448, 6.831434113004127e-05, 4.107727585192212e-05, 0.00018447901146106957, 0.001021600472440227, 0.0013911099146391686, 0.001879076998510712, 0.00021567647107472324, 0.0009265214077600358, 8.447673789566682e-05, 0.0002724362180102795, 2.4012716345832783e-06, 0.00023479596202447204, 0.0006854472664871872, 3.5804005154425787e-11, 9.472592025289725e-05, 0.00019668073292106354, 0.00021844200409453283, 0.0005345546040250439, 0.0001579360149368454, 7.0756955290264685e-12, 4.582585948970161e-05, 2.4338594205984163e-15, 3.753551036154211e-12, 0, 2.165429284808931e-10, 0, 2.248512793759427e-10, 0, 0.0003337482636783906, 1.9408194747808114e-10, 0.0014429535673668927, 5.820372717101419e-06, 0.00032329800669768176, 0, 1.1759843958556864e-09, 0.00011684918809798801, 3.083865524673042e-06, 0.0003556178429276628, 0.0014170343656953923, 0.0018646327456109534, 0.001002236827486816, 0.0011114310860492872, 0.0018580599975297639, 3.187564186059021e-11, 0.0017047392534077787, 0.0006622073427750504, 0.0016235918874716651, 0.0012672452605422618, 0.0014726814233359672, 0.0006020765683375502, 0.00010665480278738633, 0.00022115879758726636, 0.0005036024452338168, 1.1060846637260505e-11, 0.0004836703471811424, 0.0009046998867288718, 0.0011827814748542385, 0.0014305198247719037, 0.0015007213001070386, 0.001538711041372629, 0.0015547323323313366, 0.0015304410615271625, 0.0014468124472153579, 0.0013166968067507597, 0.0014177821926527365, 0.0012820091436882462, 0.0013991271085364641, 0.0012931921064146296, 0.0011829204225128212, 0.0014151683597309825, 0.0010429118426525273, 0.0015561048006368014, 0.0009566951994490092, 0.001582980926729657, 0.0008483450692996956, 0.00025342666354117205, 9.8382446948139e-05, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.624632941493158e-11, 3.85664696819041e-12, 2.6658957682445154e-11, 4.652053657792509e-12, 1.889713701040125e-11, 2.1791953836178163e-11, 0.00030492592594511024, 0.0011217434324035467, 0.0014811359481257753, 0.0019378388243353807, 0.0017505594561599335, 0.0023906957922934114, 0.0019327394425902096, 0.002390721236360869, 0.0024690530652421807, 0.0025654049639198873, 0.0018634860808915833, 0.003015878257217021, 0.0016653450935638842, 0.0032012938173687326, 0.0017029632018527691, 0.002187194516781968, 0.001920285130261625, 0.001062395617742789, 0.0019155650592811524, 0.0007502319801666934, 0.0015213157030245824, 0.0012985603226846256, 0.0019487300568181317, 0.0025134262408091617, 0.002495939533188714, 0.0029719596230283834, 0.0026518790112462953, 0.0022782556180192693, 0.00241184300372286, 0.0012977782015951601, 0.0007513235465752624, 0, 0.00021897350273583998, 0, 0, 0, 0, 0, 0, 1.2509576881115236e-12, 2.068998520925941e-12, 2.0538913683982296e-09, 7.952583597181037e-05, 0.0007207154419756502, 0.00045624258722035956, 0.0002407015716796055, 0.00026508196818182954, 0.00011542019115948878, 9.260124770652001e-05, 0, 7.35193322556766e-08, 2.7100355905271666e-05, 4.485237146975483e-05, 0.0002533462492557514, 0.00038008338799812666, 0.0003228389641774655, 0.000455635142776823, 0.00038505568439638313, 0.0006087207796679684, 0.0004935141607163105, 0.000741009101541829, 0.0005690081397598202, 0.0008835321416281053, 0.0004781507991922409, 0.0008725101936152276, 0.0002485435087233817, 0.000785601527490559, 3.40064804670421e-05, 2.4934008567496363e-11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0018775992307418347, 0.0021924194256327505, 0.0022536321821803934, 0.0026058831892615928, 0.0026394240376715256, 0.0030894548758491585, 0.0031855785367134438, 0.0035572021356296683, 0.003690273552459518, 0.0036666696856452992, 0.003702570029273036, 0.00362314126481278, 0.0038013257025425826, 0.0037592872862159594, 0.0039958210054265595, 0.0037540771170137687, 0.00320268239337337, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.7711324170830534e-13, 0, 3.641670276006405e-11, 5.743147900640605e-13, 8.565115333801784e-13, 0.0004816205016143133, 0.0002812274893428096, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00022540124263988685, 0.00044978845101894464, 0.001626049559588295, 0.0008764133661859797, 0.00034829656164007337, 0.0001659617789226341, 0.00045200502360210606, 0.0004949977581133419, 9.733414097207815e-05, 1.2379045103971322e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0001331848426443688, 0.0002560640369295692, 0.00014466326934539037, 0.00023254068542099003, 0.0001961263926518486, 0.00018062303076842671, 9.589815279630086e-06, 1.5041756930680067e-06, 0, 0, 0, 0, 0, 0, 0, 0.0004585201841577948, 0.00029113525829660143, 0.0005565393277029296, 0.00040959986642904344, 0.0007533330641647639, 0.00043776907738734465, 0.0008398593908677227, 0.0005312855170274594, 0.0009442694678104077, 0.0005750791285685887, 0.0009948022030948933, 0.0006700287508726335, 0.0005933216957493114, 0.0007332435927671852, 0.0006149097785457293, 0.0007720003645975808, 0.0007682861839399758, 0.0007424322471174871, 0.0007883797694196958, 0.0007306136968040297, 0.0006900936489582445, 0.0005660911531459626, 0.0005460679845166807, 7.624393274186371e-05, 0.00045922507862480194, 0, 6.07555183433455e-12, 0, 0, 0, 0, 0, 0, 0, 0, 6.657766051087659e-08, 0, 0, 0, 0, 0, 0.0001291956736501692, 0.0014249740682310793, 0.0006449548466211512, 0.0007808077275639494, 0.0006623432492089391, 0.0014892929996486368, 0.0007903194682927602, 0.0014205455818926465, 0.0007416259551773565, 0.0013600180442726933, 0.000767631647377098, 0.0012476880268010707, 0.0007354064189879633, 0.0013317353162796894, 0.00038009414798479237, 2.44312151133151e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0006942810442764774, 0.00034491337928249037, 0.0010387356758069593, 0.00044969077574327396, 0.0010243126269011257, 0.00040970822855557784, 0.0009310987917458116, 0.0003405773764986168, 0.0008953676752766668, 0.00017987743032863644, 0.0006972252817996455, 0, 0.00046792690252223096, 3.322880851550552e-11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.359947299111807e-11, 0, 0, 0, 0, 1.3404520084911388e-06, 0.0001147556209574818, 1.0213343526527916e-05, 0.0001360574367076663, 0.00010190324967500875, 0.00020180570307915758, 0.0001399104577945971, 0.00018242084895451146, 0.00013952145860636155, 7.972526839981249e-12, 4.897550516964706e-10, 8.8095734164541e-12, 0, 0, 0, 0, 0.0001551250501642319, 4.903258479947171e-05, 0.00029907945023069723, 9.842371053898324e-05, 0.00030790987868961357, 0.00019604734908610895, 0.00042728191645736757, 0.00035270323659510457, 0.0004986839661755358, 8.618004278090017e-09, 0.00020946524737562372, 1.160501110931602e-11, 9.555454298346133e-12, 4.602636407127433e-12, 5.078861801620271e-12, 2.9682759399483593e-12, 1.3827016043194805e-11, 3.2691121147350555e-11, 5.227697680818903e-11, 0.0001225566116295189, 8.536474390652535e-05, 9.467967636572923e-05, 0.0003717064437439323, 0.00022575401587202832, 0.0004042845920845193, 0.00021559762248521695, 0.0005204423242633807, 0.00037056127366908446, 0.0005434971917907409, 0.0002395455220837408, 0.0005557327671409418, 0.0002725545956429236, 0.0005932108372315176, 0.0002314126436990214, 0.0006491039316528792, 0.00022929962058116694, 0.0004661804577617147, 0.0001806814975596688, 0.0003952175669183875, 0.00017701416030837223, 0.0002508121865676992, 0.00017806597979775975, 0.0002491417162729037, 0.00010784087509863876, 0.00013483815368448227, 9.214265534824984e-12, 0.00010470787512687438, 1.3488578663360798e-06, 0.0001628181129036676, 0.0001445603195163071, 0.00014395162770247235, 0.00023451273794833672, 0.00021083899051893492, 0.00038963923657530834, 0.0005219163681320861, 0.0005571618133857773, 0.0007719302203626949, 0.0005932126957155806, 0.0007748891564569244, 0.0005794853412999167, 0.0007669224694607322, 0.0006141822606900158, 0.000822817155077606, 0.0005805689934850712, 0.0007598591988150412, 0.0004729556883331724, 0.0005743154332056362, 0.0004106651160158186, 0.0005150230578600648, 0.00040762256551319927, 0.00041840010918785573, 0.00035675860156138414, 0.00031858619413146697, 0.00028034462744805914, 0.0002091876077435804, 0.00014341670286378169, 0.00015299455845520072, 7.837086725437112e-05, 0.000173684942153951, 3.9533541141665755e-05, 0.00017681602947954342, 0.0002192860124229521, 0.00030626931858499244, 0.00023777436994611956, 0, 0.00015212851924145153, 0, 5.979408838809774e-06, 0.0005599790814994529, 5.213603143800972e-05, 0.000774100440903085, 0.0005011141987435756, 0.0008231603922634553, 0.000606372859745984, 0.0006367904385154033, 0.0006983368448670537, 0.0003829665864095661, 0.0006641950642521481, 0.0004521797785625805, 0.0008443440196365731, 0.0005946836328809673, 0.000799209779679086, 0.0005837176960076905, 0.0007681116697855505, 0.0005354531561933508, 0.0007292365450720178, 0.0004375663940034616, 0.0006065571697224574, 0.0004883472140796003, 0.000605552045611043, 0.0004527749822829477, 0.0005294763453025865, 0.00047880575668657446, 0.0005877154583128178, 0.0005268272617747729, 0.0005596952312197358, 0.000629752635762045, 0.0006237698594398569, 0.0007144618310273073, 0.0006781339110756968, 0.0009724536882414619, 0.0009149119024468027, 0.0012089215327976577, 0.0009101294542390191, 0.0012546307103127527, 0.0009463027635036169, 0.0010927378885109374, 0.0008890796712552226, 0.0009614975048312114, 0.0009916377156708185, 0.0011966370659356475, 0.001197531870617822, 0.0014486086533642857, 0.0012111793600089885, 0.001500359941561452, 0.0015047453385747193, 0.0016828002760990543, 0.001821695491377629, 0.001987597618807605, 0.0021628257874236143, 0.0023069946891992746, 0.0023672308355262173, 0.002572346185810022, 0.002605964029739695, 0.002861395731893368, 0.0025986041354659536, 0.002763250922162806, 0.0025555300427802727, 0.002800141704719186, 0.0024211302005016423, 0.002703881479604823, 0.002335277403999354, 0.002640340058631709, 0.0022792130590151417, 0.00252867310583068, 0.002376208931722524, 0.002512312825307594, 0.0025619747372741916, 0.0024135264910624408, 0.002766829229612141, 0.0024514696993088955, 0.003067737224529868, 0.002500039981107528, 0.0030419207701801966, 0.0027969040364245303, 0.0029950250002509626, 0.0029689831898520105, 0.002955456602363936, 0.002904367937102446, 0.0029101427194962166, 0.002962738355963795, 0.0030445402419079074, 0.0030325716302171193, 0.003019196525270124, 0.0029902332389395297, 0.0030731425138761704, 0.0030403766043558913, 0.003114153335550069, 0.003007850168631882, 0.003242654871761026, 0.0029723847989666114, 0.003292623815576703, 0.0028838922128912076, 0.0034936579624061472, 0.00288269383738735, 0.0034937414925421146, 0.002891118238394001, 0.0034831636191990263, 0.0031741726565621704, 0.0033852436844031605, 0.0033713003455058126, 0.0033508624606737704, 0.003411203679694591, 0.0034074587997068814, 0.003495126050044081, 0.0033863220162193095, 0.0036235736828355266, 0.003338495907065515, 0.003641690644797567, 0.003183350333880145, 0.0035268836095026634, 0.002949164373633736, 0.003422961787399165, 0.00291057958871738, 0.0034156519778485474, 0.003038873024357254, 0.0034397367755262323, 0.0031420000103239945, 0.0033197288624286186, 0.0032241328583139813, 0.003463196429518531, 0.003381930588912824, 0.0036778062808195864, 0.0036214597233699196, 0.003897369473656321, 0.0038512178749693675, 0.00399752857311522, 0.004064035596649431, 0.004076706875723252, 0.004191224482566525, 0.00415402981154455, 0.004401432276896946, 0.00430119500966859, 0.004536392569717321, 0.004331149285419927, 0.004754415316114115, 0.0045109486134946915, 0.0049823548657482435, 0.004558265906592613, 0.004902948428193286, 0.004574402892851093, 0.004920647662887355, 0.004737555183742543, 0.004863891658800413, 0.004759622170062527, 0.004820530747125628, 0.004713212188346082, 0.00488592014878755, 0.004694288168040365, 0.004575546250783736, 0.004367138654267397, 0.004354328880324636, 0.004157109601966511, 0.0041426664880991415, 0.0038860672220242306, 0.0039806462315185595, 0.003596982444649414, 0.00352480698419567, 0.0031771771901264218, 0.0034490618401695696, 0.0032378949925471537, 0.003181853661079351, 0.003011747007955507, 0.003111834785018528, 0.002877895345766169, 0.0028176419827547505, 0.0025587867770228093, 0.002690192674184327, 0.0024739335051066347, 0.002045699032609663, 0.0021251324410583774, 0.002288962697534537, 0.001278037319785152, 0.0011438211511191707, 0.0019359904907858955, 0.002144798683893747, 0.0020365716212532217, 0.00204808604172846, 0.0018849342988995878, 0.0019405523728294697, 0.0019471024713943804, 0.0019690505141529706, 0.0018789133464689163, 0.0018063137970524886, 0.0018705295032347522, 0.0018039581871087114, 0.0018341199520680248, 0.0017482895515978883, 0.0016356358624054182, 0.0017637790484311348, 0.001481857429778645, 0.0014027785055827084, 0.0012550215030792315, 0.0011896418127381352, 0.001137215595016417, 0.001180657400149915, 0.00110920436331187, 0.0012729562221736339, 0.0009888679951160515, 0.001347265514394258, 0.000931799073657992, 0.0012858816136409577, 0.000992397613551174, 0.0013283608127569073, 0.0009288487877903045, 0.0013844770641316675, 0.0008169657193735035, 0.001600612474728992, 0.0006794922738271202, 0.0009798436653795123, 0.0012529279549833355, 0.0008726512820572736, 0.001163158144215946, 0.0008828207301166205, 0.0013129621524205256, 0.0008772788592548761, 0.0013092499508094739, 0.0007221293816668458, 0.0010920583274041738, 0.0006156086504868864, 0.0010660931147118606, 0.0005947783955689372, 0.0009450920195916348, 0.0007754770358302246, 0.001017236606060914, 0.0009213624279737411, 0.0009238575758668915, 0.0009894633154182238, 0.001101387635667853, 0.001171867349080241, 0.001304461683452479, 0.001292846981131645, 0.0014640718168672194, 0.0016410661170850433, 0.0017789133227697719, 0.001970392068475707, 0.002053815373779786, 0.0021246292510960587, 0.002116838789515851, 0.002166575540093288, 0.0019860660666834937, 0.002322097173901566, 0.001992622861199801, 0.0016482742310835485, 0.00023371970897573157, 2.868258399711732e-11, 0.0001794121093257846, 0.000607074259717772, 0.0008667270496364296, 0.001297875033675923, 0.0014814337637914644, 0.0018284787403477598, 0.0019242694459784245, 0.0022292450998075437, 0.0021076880381836745, 0.0023914941647665737, 0.0020589050958725753, 0.002539612488194237, 0.0023377155720402947, 0.00226990151185603, 0.0021153867695445513, 0.0022847755311239436, 0.002090057308936356, 0.0020805531574305193, 0.002046759388096785, 0.0017245980398268056, 0.0016396282395190554, 0.001623846606541518, 0.0015260230084493296, 0.001134226139797075, 0.001753930294467356, 0.0013234799784278824, 0.00176245025463195, 0.00234892226099297, 0.002245182749897353, 0.0033253038321980665, 0.002063847689338978, 0.0028464624882087934, 0.0020801881230574472, 0.0020785780865165203, 0.0031448752523431244, 0.0017722805089372682, 0.002429949004375677, 0.002173756881631206, 0.002429049026308425, 0.002571448235803094, 0.0026694111242080037, 0.0027285859565225443, 0.0028273756168442463, 0.0029530907313123655, 0.002891535179655515, 0.00256295278951361, 0.0028057851048468568, 0.0033301471620352354, 0.0031017713566667076, 0.00303601352117678, 0.0028491794292064667, 0.002921278831500187, 0.0025931388691246295, 0.0026368092402350315, 0.002345483016051464, 0.0025100774836791296, 0.002271974654594458, 0.002520566624881755, 0.002270608538953413, 0.0025092288630507702, 0.0023004754414277045, 0.0024739909165892356, 0.0022270784997103265, 0.002236586995700327, 0.00227188134270394, 0.002292062708296049, 0.0026433588522940046, 0.0025105321527489283, 0.002859297801848928, 0.002727272092013243, 0.002877297281407356, 0.0028117852295037834, 0.002906093074234322, 0.002876137090995407, 0.002895888208907412, 0.002739826457940959, 0.0032284716762859896, 0.0030554174008497613, 0.003003599010175263, 0.0028619059444646984, 0.002610613399079027, 0.0027011159694556625, 0.002423151261045768, 0.0027543966809398016, 0.0017034595051919448, 0.003004605217889173, 0.0034659501486156603, 0.0028631778486522207, 0.0025552731850378768, 0.002722372995742556, 0.0028065834598485426, 0.0023853300418997694, 0.0033992989578037077, 0.0038221148702002085, 0.0030913170812703426, 0.0030072629075791014, 0.0030737856564501476, 0.0031596803548780534, 0.003239660900961152, 0.00327485503961721, 0.003485143551302188, 0.0034809962452923716, 0.0036538408540552473, 0.00361457495027011, 0.0036593045820358383, 0.003600895374219233, 0.0037584095102992853, 0.0036804830668198804, 0.0037770700964689426, 0.0038597971196608154, 0.003966890145020787, 0.0037773024352175232, 0.003862676056942596, 0.003570294207825162, 0.0037338765638158665, 0.003363493741944076, 0.003383856784525302, 0.0029894337402965095, 0.00303087493881957, 0.002919604162142637, 0.0029215766724103934, 0.003100106211782279, 0.002850351245373939, 0.003041083171819571, 0.002870101375931661, 0.0026199832642240565, 0.001867807619013398, 0.00019933686318508636, 0.0005187448492276996, 3.4174545178714968e-12, 2.0789907036781902e-11, 0, 2.9122152142114326e-12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.6709178722723016e-08, 7.350195441986525e-06, 2.1394242085419148e-11, 9.865147022246005e-10, 2.6720636707770667e-09, 2.921013856362563e-13, 0, 0, 1.0637269883325894e-12, 7.315798875029694e-13, 6.8657365164404e-13, 3.3162604376742755e-13, 7.857448429129536e-13, 1.8601313642368018e-13, 2.394207390119317e-12, 0.0007486114658971837, 9.764040111894412e-05, 0.0011355230422225079, 0.0004487275164512273, 0.0007401809955810913, 0.0009777825519497825, 0.0011427356657887355, 0.0008824760695825435, 0.0015409591850455847, 0.0009071422152029842, 0.0014683318295082013, 0.0012208560337469718, 0.0011751905222124807, 0.0025864148243759924, 0.0012788794590779533, 0.0024425612327891614, 0.0023050656159632056, 0.0018230434375430058, 0.0031024212393651763, 0.0023014934224650113, 0.003746646606150396, 0.0029073557805969043, 0.003268847287723968, 0.003042472105874887, 0.0029391361560846055, 0.00295806571413716, 0.0030421191797366342, 0.002838833402427061, 0.0031998377751216714, 0.00322495530074877, 0.0030907412456492433, 0.0037115223487718726, 0.0029855232968869004, 0.0037347302221411127, 0.003292898495131435, 0.0036439164598139227, 0.0032965004998515863, 0.0035157836618880836, 0.003185762409799343, 0.0034548430069816245, 0.00299186908197928, 0.0013913308769505123, 0.0014144077527697351, 0.0020961014610121178, 0.0025128469408032188, 0.002570430234122531, 0.0028701315894822735, 0.0031600394111296073, 0.0031565410704198324, 0.0036012698356037024, 0.003778263832991154, 0.003389488509305073, 0.004031486978345369, 0.003908841917038682, 0.004353891117632706, 0.0040711285970533514, 0.00448088146396322, 0.004147476028137923, 0.004029126399121321, 0.004303202216454056, 0.004120284705668436, 0.004392732798050986, 0.004984729871480561, 0.005308127735777953, 0.005177871117359604, 0.004724020916887589, 0.004777228909998222, 0.0050043700361324635, 0.0049020604482104815, 0.005042302641082308, 0.0050083229324513374, 0.004707622003430406, 0.004376211040243388, 0.004439165318571667, 0.004826455153074506, 0.0050665646225140975, 0.005470070224922023, 0.004917730152407748, 0.005017045037038607, 0.004628039493321256, 0.004690802665982125, 0.004576405204078294, 0.004307257387039871, 0.00403418422492388, 0.003646327061712644, 0.0034689480529081444, 0.0037101579671169633, 0.002923965116727825, 0.0030031552287631364, 0.002669361968216166, 0.002664630639746059, 0.0025327664992069138, 0.0027531139870262218, 0.002669080373710961, 0.002616999302947734, 0.0024290590443685174, 0.0023549736161277562, 0.0019051797397702007, 0.0021507012381268443, 0.0017755446349017746, 0.0020330054962855613, 0.0020450567735555585, 0.0020196926544963755, 0.0018767989203539663, 0.00125357384075591, 0.0015710310449681697, 0.0009008645908823339, 0.0013106212093389276, 2.4990429947939673e-10, 0.0007762267893139739, 0.0012605873382965241, 0.0021379451952101406, 0.0012044993950864195, 0.002206641038130138, 0.0009327340604030918, 0.0015026866974659355, 0.0015233930976407532, 0.002023475359102534, 0.0019838530070905333, 0.001358749954618896, 0.0016378675941789817, 0.0018706904316270494, 0.001547475608881195, 0.0012930129663789641, 0.0013196381197183695, 0.0011714333495603025, 0.001917760631112625, 0.0015579167035681932, 0.0020504968759533786, 0.0016931270402289595, 0.001958801767525763, 0.000964054031494551, 0.0010612204755442618, 0.001671099512283456, 0.0021116880293925713, 0.0021428832679881516, 0.0027381168267729765, 0.0038079410069301466, 0.0033090140755213026, 0.002213455792431449, 0.0018326679681853468, 0.0021966638274246585, 0.002020310609536252, 0.0019902581002559927, 0.0023711900033962854, 0.002597739900437182, 0.002176710605061393, 0.0023473244832945767, 0.0020881137311703395, 0.0022290785749731045, 0.0022337307509314075, 0.0019483687257429282, 0.001843310890844511, 0.0018575504412781817, 0.0017187221973195917, 0.0017555362774690267, 0.0017726417589789504, 0.0017442728490329614, 0.001456810943244195, 0.0017067397479833633, 0.0023334192544885906, 0.0016902202788688665, 0.0014108986113093452, 0.0014124085119106152, 0.0015626828967103287, 0.0016380813182091042, 0.0019443380594451256, 0.0022678948074519156, 0.0017829889299035447, 0.0019479932442580028, 0.0019000607860759863, 0.0024339067148160328, 0.0022254176755122494, 0.00269911728465131, 0.0022204694099486793, 0.002970064381359315, 0.0026826461226946248, 0.0030862089447479643, 0.0026005691990704004, 0.0024329509299952225, 0.002607080663158953, 0.0028079233574995564, 0.0033158218860706297, 0.003471227313024427, 0.0034267436639486533, 0.0034630964172522385, 0.0034983337097868896, 0.00351087292397735, 0.003807704267314876, 0.00370585055931977, 0.003585249327379226, 0.0033268521571905327, 0.002852620596429318, 0.0030572349532502364, 0.002428933112806486, 0.0035176214057290184, 0.0034835474623905872, 0.0036895177567492035, 0.0031543400741927204, 0.0029937086589763096, 0.0020945052279048855, 0.0021471709941856144, 0.0021179592177235735, 0.002546657174470621, 0.002307504796243326, 0.0022080776342469767, 0.002261272637868303, 0.001960192677318461, 0.002276439265168619, 0.0019381346078642045, 0.00211408193005001, 0.0016567819197259063, 0.0019254668205603278, 0.0014585030370846536, 0.0016185913182424985, 0.001723720988361227, 0.0016452269219307395, 0.0015900939250083183, 0.0014393493693637411, 0.001404278772515618, 0.0015397661066291232, 0.001552499428465876, 0.001340591763540466, 0.001394381733076878, 0.0009636971374843719, 0.001132844804022975, 0.0006689769783626839, 0.0008222408120999039, 0.0009678126745082592, 0.00020911841266691247, 0.0006605617989006432, 0.0002021424995175934, 0.00044856740089950595, 0.00020053120769505467, 0.00032964482669813224, 4.0314126644513914e-05, 1.2803913603294446e-11, 0.00011894842403848175, 2.536847792217606e-10, 7.219172441657295e-05, 0, 4.760311652925846e-05, 0.0004447989707093463, 0.0007002987048037359, 0.0006802139300887154, 0.0007884866963746574, 0.0007116908738189976, 0.0009679643049002014, 0.0007667873727711298, 0.000784742537002506, 0.0008586369899350008, 0.00037975122347576454, 0.0005381696020716531, 0.0002974401233614775, 0.0005751419937305054, 0.00033801389847797693, 9.57505307500515e-05, 0.0004121961640024284, 0.0009534364529235786, 0.0013916761711369472, 0.001016477081338585, 0.0012015229481254668, 0.0011738708781420396, 0.0016723073430352663, 0.0016937896489179886, 0.0018870264163715791, 0.0019375067013189583, 0.0019296083924268325, 0.001983092904660232, 0.002112396555230323, 0.001631610444718405, 0.0018914605868601973, 0.0014547531478758522, 0.001429765111358244, 0.0010931138466199607, 0.0010384705113583312, 0.0008248249604091185, 0.000740089329609032, 2.734143590019139e-05, 8.06537194491272e-05, 0.000772307647878603, 0.0010165836812491178, 0.0015797594623349436, 0.0011563617279664626, 0.0016477694283444746, 0.0012502731860457496, 0.0016882741801848196, 0.0009310166694923301, 0.001501190987411062, 0.0008653092860310138, 0.0018592406114004763, 0.001297564799712181, 0.0015619458418508383, 0.001121211732751502, 0.0017789327961062601, 0.0012630109581966544, 0.002171528804339417, 0.0019127035259400948, 0.0025963081378872657, 0.0025903047847135904, 0.00330352339552921, 0.0030835283068537207, 0.0028994900934632014, 0.0030181668495522433, 0.0031198155592915447, 0.0034043274328106952, 0.0033806683037349862, 0.0034802007033508647, 0.003517922811956882, 0.00312986751796302, 0.0036017515969989883, 0.003628824917720889, 0.003456735913579512, 0.003541639919032107, 0.0033337650448953195, 0.0033271306319470314, 0.003498046890341323, 0.003978363135718467, 0.003916471747175059, 0.003978316957189685, 0.0036472670075984315, 0.0037841416717512687, 0.004129419428466833, 0.0043953025384683595, 0.0042149382746430744, 0.0045746837264615085, 0.004885088923067323, 0.004577517906943267, 0.00461314792893277, 0.004233369243310109, 0.00424911751225711, 0.003996776155042427, 0.0043282349892781275, 0.0039842504585630946, 0.003888215599849733, 0.00376364453814132, 0.0038164356280147244, 0.003631677093774632, 0.0030995080110259175, 0.0032787328572351604, 0.0027993680653213677, 0.00228241934471354, 0.002258135621864224, 0.0027329287864512816, 0.0026063671491152614, 0.0028399842426165257, 0.003583596647446867, 0.005058590567750149, 0.0038808355232220274, 0.0036186019770316157, 0.003214234759724242, 0.0032461248748693965, 0.0032071768477988662, 0.0033114802231408473, 0.003117588538043051, 0.002628507915206262, 0.002620033151829036, 0.002600710138151123, 0.003067532443987534, 0.002811728660339856, 0.003119190492078602, 0.002673057023747717, 0.0030632742121156653, 0.002486150719393176, 0.0026536148890020294, 0.002528413038376351, 0.0027490475844921952, 0.0024139906108071344, 0.0024272483729230903, 0.0024829515363936324, 0.0031454317099854933, 0.003425192123402752, 0.0036186558988353687, 0.003032885738284094, 0.0035849753476521417, 0.0029036426752130193, 0.002007165462683789, 0.0016075373697259118, 0.002543473755032004, 0.0029306870872851084, 0.002588799289126192, 0.0027720708645678294, 0.0035905441057155468, 0.0034846468915938457, 0.002422567064738406, 0.002770131589629417, 0.0029232106170056283, 0.0027718304247139215, 0.002660999285196569, 0.0019462316367029476, 0.0025551278576507056, 0.002424655838944369, 0.002668903249432493, 0.002162981352800895, 0.0020000522428235564, 0.0016489057467066344, 0.0017926198231550026, 0.00202920300051515, 0.00142487841772869, 0.0014412594697759335, 0.0012758488800127007, 0.002051848179406464, 0.002101409754436577, 0.0026583881260163677, 0.0022437000769322964, 0.002025254151060727, 0.0017075198521814775, 0.0022448577125587636, 0.0020792442878923238, 0.002070124556119051, 0.0018544425651922554, 0.002192337452009604, 0.0020482021162413887, 0.0020404809787120492, 0.0013388805201338383, 0.0012254928613571504, 0.001034970770451416, 0.0011517949453877974, 0.0017659085101187273, 0.001713392128002055, 0.0022276135508282097, 0.0021743086725529463, 0.0022817820541301014, 0.0021049866087317075, 0.0020072103308935484, 0.0022597014851956348, 0.0013757278688391805, 0.001985487853311085, 0.0013885050956727986, 0.002099054441155604, 0.0016974155925260186, 0.002151982056054461, 0.0018440861319732637, 0.0020617738640420764, 0.0011849225815495404, 0.001331963291894414, 0.0006196568949110814, 0.00045471372348800056, 0.0006254400788205527, 0.0008480740677292886, 0.001482190209810267, 0.0010931691063210667, 0.0014649049361074257, 0.0013886782175842058, 0.0012275920904827311, 0.0014743910166070693, 0.0013051583941660397, 0.0016313594849052566, 0.0014967571424779307, 0.001742701282231008, 0.0014677756038160936, 0.0016972043544460835, 0.0017569472283970509, 0.0016618879107367444, 0.001633867141944557, 0.0013752964759313592, 0.0013452865032821352, 0.0014786166432467819, 0.0012485852193126414, 0.0010871897469733917, 0.0008227506651940654, 0.0009856430150584732, 0.0008630424371080379, 0.0010651770412376737, 0.000959200295139537, 0.001163960161877927, 0.0007457425235263606, 0.0006620644750638497, 0.0008832101461281825, 0.001178386764420672, 0.0012717765672338596, 0.0013503602931381068, 0.0011900413381577635, 0.0015359586692015423, 0.001128530164910283, 0.0011817797808215717, 0.0008332870843715711, 0.0008954961097180884, 0.0005828206883948144, 0.0009524219674828018, 0.0006558073235543824, 0.0007202317602547285, 0.000232649627239739, 0.00034183938330071536, 0.0002424835656340688, 4.4178515711215705e-05, 7.131969711048335e-05, 0.0001324066929110482, 9.966164716565293e-05, 0.0003209981250955531, 0.0004915862491878646, 0.0007654739403456963, 0.0006371382625015409, 0.0003758178097625465, 0.00042609563753546814, 0.0008752704153085952, 0.0010489220851445287, 0.0007538794476541036, 0.000775900456498851, 0.000829556359139478, 0.001226781235089235, 0.0006748597607108194, 0.0009142800574810529, 0.000491362841924632, 0.0006788922106384023, 0.0008008274785347, 0.0007675096001365761, 0.0008253814100306285, 0.00042577470443628477, 0.0006958792324876806, 0.00010374788986682722, 0.00040204870409875083, 0.00022756420902658302, 0.0005490637802818082, 0.000677185622129978, 0.00031686050750398184, 0.0003768790538083349, 0.0004048054424588266, 0.000691576904951439, 0.00036787055241302766, 0.00039870151740059644, 1.2689175421797816e-07, 2.504365949826521e-09, 0, 0, 0.00014828606064919302, 0.00057578702323997, 0.0006995291452120048, 0.0010882035027781289, 0.000583444358481852, 0.0009507507422535245, 0.0010157590246993523, 0.0011698509019584943, 0.001248934393345555, 0.0008039215783584401, 0.0014679759311222258, 0.0015635295144034785, 0.0015329736961933235, 0.0010446448612364617, 0.0011814317801534125, 0.0008350356460285964, 0.0014439857475514579, 0.0012449155350004874, 0.001348858162368548, 0.0009041922067906239, 0.0009604298708629076, 0.0010066948264201882, 0.0012342444640584076, 0.0013539399131188546, 0.0015090578743743256, 0.001294204562782625, 0.0016087206311344142, 0.0011914717313117257, 0.0016512292819676657, 0.0016292601284205255, 0.001751856290743485, 0.0011031872288839108, 0.0013680644686380626, 0.0014612782431448624, 0.0017797398957046262, 0.0019782634519313804, 0.0018998917810367135, 0.0023746082370060544, 0.0017580857928878003, 0.0015230512892388107, 0.0013859869556616148, 0.0016806886402158147, 0.0018175888959496523, 0.0021489271932490623, 0.00240941689474811, 0.0023289112764543042, 0.002199803036009774, 0.0021296652136204557, 0.0023140380870573687, 0.0021340785337476543, 0.0019122516569846538, 0.0018396603280395434, 0.0022076378098992034, 0.0019268719812094224, 0.002091801118212783, 0.0022827280337582344, 0.0024105123819159802, 0.002349270236868338, 0.0027614297816710696, 0.0029415146307016364, 0.0026532085792498614, 0.0022227705362669026, 0.0023486613569953813, 0.002160143296144043, 0.0030171842732575147, 0.0030873199782732097, 0.002672596293738097, 0.0028382059241787105, 0.003221442425063159, 0.0036161266565434037, 0.0034341180354077816, 0.0033900727511826066, 0.003620655686264003, 0.0034252117919769684, 0.003365877479763268, 0.0036790739475573934, 0.003594255812058909, 0.0037573495859870116, 0.0035580277931473825, 0.0032581936700407615, 0.0034998048615500123, 0.0038048883908069885, 0.0038297700585093663, 0.003989626636235624, 0.0033718757218150915, 0.003268943306311067, 0.0034323550029196925, 0.003809169126953466, 0.0031745456384837654, 0.003296288854255835, 0.003986443276619215, 0.004435680227569232, 0.004314518706838592, 0.004391811258763232, 0.0045213197611185365, 0.004312939908580603, 0.0034884355312043125, 0.003965438283760724, 0.003940022501826209, 0.0041652934789070725, 0.003522661899371323, 0.0037462735519110135, 0.003923796081417721, 0.004293650802268486, 0.003954729043326216, 0.0037294730842082164, 0.003549939219415291, 0.003474215160817008, 0.003813999780537741, 0.0038946376334613605, 0.004045765748517233, 0.0037819970367780433, 0.004296458775171055, 0.004174431256074721, 0.004465149864171958, 0.00415695280908719, 0.0041452488250504895, 0.0037578364557082585, 0.004025992226481435, 0.0041802811455526525, 0.004237339360997861, 0.0041117155893264775, 0.004084524602772446, 0.0042091472161003615, 0.0039002519124420962, 0.0038634772915075275, 0.003514112345600293, 0.0035501437018060503, 0.0038863589175504192, 0.00422977719324505, 0.00459219287764403, 0.004208934407488889, 0.004124656320233733, 0.00396304906057083, 0.003977653372552273, 0.0038459042290212857, 0.0039471500521105615, 0.0039504117876751205, 0.0039010682630267472, 0.003719573175460093, 0.003942566772096138, 0.0036787577006389925, 0.004010020853685598, 0.0035207586636392223, 0.0037942225029086113, 0.0033040748277004014, 0.0032105974021044443, 0.0032155826009119056, 0.0030882240712606025, 0.003001570679885456, 0.0032175722192336947, 0.0033033217774890065, 0.003544206642442849, 0.002972548073079391, 0.003086517768399401, 0.002820306980661486, 0.002863149070860163, 0.0023640071489659177, 0.002690000582858316, 0.002379111804392134, 0.002498622256030953, 0.0024114461716831966, 0.002544552331361775, 0.002744759081706448, 0.0027632253931173124, 0.0026194140346252398, 0.0024834561893295106, 0.002384046290620365, 0.002335908499234734, 0.0023629211899840693, 0.002303179967469379, 0.002096246472036292, 0.0021374044300780325, 0.0021826574451989365, 0.0021039449160675703, 0.0020188332949929513, 0.0019417357232545426, 0.0018303201170616098, 0.0015665745758433585, 0.0016515055407508663, 0.0018353438237786418, 0.0019420968333013716, 0.0019945721460268933, 0.0018451576860265705, 0.0022491824646339973, 0.002343236476778039, 0.0023507019372539044, 0.001983758224439128, 0.0017891972557389135, 0.0021171397266883116, 0.0018910640948582565, 0.002123890835767976, 0.0019871506375038106, 0.0014735085687024264, 0.0012932160154187832, 0.0012466170375605384, 0.0012635260161927998, 0.001289184557404841, 0.0011076869039809232, 0.00124644275703864, 0.0010254651588072602, 0.0008415589350700653, 0.0008998637471145602, 0.001179999568411596, 0.001107471343835771, 0.0008601950008760431, 0.0008183293055286452, 0.001207383095655288, 0.00107171203620563, 0.000748921403654761, 0.0006768463159269743, 0.0009338128052186946, 0.0011642667929476689, 0.0010304840605041673, 0.0009036466869383085, 0.0010447402442094033, 0.0011842466767044574, 0.0009307152735881116, 0.0012388181875852926, 0.0006150133264753665, 0.0010259642059348554, 0.0006605259968277735, 0.0010629872723847127, 0.0012765855905383934, 0.0011352615434691447, 0.0011845976825514933, 0.000918451836378595, 0.0008278898578231139, 0.0007425646245925224, 0.0008519420641194663, 0.0009443058154288693, 0.000903807713414886, 0.0012757468412370172, 0.0010444746120354472, 0.0016484789631873475, 0.0015042053511142679, 0.0018399694791173655, 0.0016471530333956221, 0.0014297319667270227, 0.0014165319417471698, 0.0015315719717990208, 0.0014753334233506164, 0.0011924397791213094, 0.0012508077352529452, 0.0016334451788089868, 0.0010883723674435842, 0.0005599445160831591, 0.000438413926482184, 0.000687393142863435, 0.000191289259070243, 6.0184462927611316e-05, 0.00021919630703839298, 0.0008280122654311467, 0.0008230066293607506, 0.0003627662756839974, 0.00026209342117946476, 0.000932697264654138, 0.0007843461613607837, 0.0011101663046378873, 0.0011334635178648646, 0.0011748030068419839, 0.0008630056517240718, 0.0012830633030476686, 0.0014721683162877775, 0.002058135264653518, 0.0018323698923621845, 0.002205088003361219, 0.0017298886138784626, 0.0015071051454631358, 0.0014364385086593542, 0.0015695043307304398, 0.0014785927124325812, 0.0014554004961820465, 0.0015603757135677473, 0.0018774141701515523, 0.001306542211678132, 0.0012107152452281095, 0.0009781389252271587, 0.0007763080725809368, 0.0008700441449360324, 0.0008907657511508814, 0.000820331963600224, 0.0008507936640216528, 0.0012294626551385944, 0.0016836705745323646, 0.0015052634597532164, 0.0013663349190003705, 0.0013979782591569865, 0.001434683266414655, 0.0012804921251392501, 0.0017123345669716446, 0.0015621802241076934, 0.0019575128608225485, 0.002061733534030787, 0.0020718964903672418, 0.0021202970647911327, 0.0019297962186904491, 0.0021263725386558062, 0.002118491415633703, 0.0021505764752328383, 0.001933878346539561, 0.002223595537829571, 0.002030720997507229, 0.001855681156601582, 0.002039217798511412, 0.0025503183284671742, 0.0021588888373326406, 0.002147031621755422, 0.0017569811796164449, 0.001810693588442122, 0.0018354445819155096, 0.0014804042125152015, 0.001540449417396249, 0.002053965847857996, 0.0020840470075293846, 0.0022371301302784494, 0.002476307260119496, 0.002429225138641443, 0.003000629600644751, 0.003246736696789554, 0.0032797085115114167, 0.00322244517800635, 0.003526713907173341, 0.003600844481892256, 0.003372317462466501, 0.003383054572007597, 0.0035194939673825836, 0.00366337411095347, 0.004232523462333717, 0.0038863487707052457, 0.0032860185756558584, 0.003263732986457477, 0.0028552415071837613, 0.002807212954760472, 0.002954865592669323, 0.003413301651772752, 0.0031997877768520566, 0.0031337759061216127, 0.0030876523623961363, 0.0031732397084138038, 0.003200504870391312, 0.003051721252245198, 0.0032078409319782715, 0.0031482742831307874, 0.0036184275206364866, 0.0035335722554437465, 0.0034264844045926485, 0.0032212908824826828, 0.003441751246583734, 0.0032768368808506697, 0.0033074705694982214, 0.00311024368940085, 0.003298825999400252, 0.0030824530608789544, 0.0031477537973194233, 0.00309560890466684, 0.0035394328783705372, 0.0032887855435281372, 0.003579072986052395, 0.0034240426700570844, 0.0035785418314146834, 0.0034387715968979445, 0.0037460191851801912, 0.003206108132946188, 0.003406457338951026, 0.0032624310463105198, 0.0037767625457635285, 0.0037241928718545272, 0.0035838861169471044, 0.003333705370557141, 0.0036105731397398184, 0.0036176200169986904, 0.003390863627750045, 0.0033855805338630896, 0.003735393098311091, 0.0029844474876106006, 0.0033553928267535515, 0.00291138046083356, 0.0034286220274324757, 0.0031296456988929946, 0.003386440121114931, 0.003014523398734879, 0.0030418325331449574, 0.0031345687320108967, 0.00315965123139347, 0.0031404346053476887, 0.0032640850574928927, 0.0029491178514373575, 0.0031924997349737534, 0.0028444862943991234, 0.0028736946395814082, 0.0027389087323977128, 0.0030936313703110938, 0.0030702230738775165, 0.003154365411889323, 0.0033201563547722696, 0.0034686163231919622, 0.003535282330653061, 0.0033505397208192476, 0.003360798524359478, 0.003374338151427169, 0.003014164414821059, 0.0028159905617391747, 0.0026537563200337033, 0.0027672500825926, 0.0027121341793644904, 0.002764787539737794, 0.0022916431383161106, 0.0021469313852399037, 0.002206225397830192, 0.0022579219347784395, 0.0023658419676797606, 0.002297945531467877, 0.002385688892362833, 0.002278479399081499, 0.002385977095067663, 0.0022889078159287167, 0.002843842941196251, 0.002939475591809962, 0.00309895507488821, 0.0026172568132387356, 0.0027123724960619574, 0.00290370051821465, 0.002801776326017854, 0.0025306084624478046, 0.0025601123539649184, 0.002372803092350864, 0.002712035340365507, 0.0024596653270317777, 0.0023316684706822144, 0.0023641474207257364, 0.002233084332394775, 0.0025576025849109794, 0.002203785643396689, 0.002294577962391855, 0.0019775896337716373, 0.0020027402963975465, 0.0019937975289931443, 0.0018771313601533054, 0.0015420078532749101, 0.0017282030643109973, 0.001268065169091039, 0.001180317303588028, 0.0009717722234401391, 0.0014232388489007768, 0.0015945064672767335, 0.0013564850373966094, 0.001834979378845954, 0.0018583158671239817, 0.0021131476474388518, 0.0021015249155700924, 0.002118557068160369, 0.0023688038286623437, 0.001997183711209581, 0.0021627160762664626, 0.0018390660861786292, 0.0017494386024531478, 0.0015319160222660284, 0.0014234136948527323, 0.0014468521986670765, 0.0013301701014351317, 0.0013072538886864207, 0.0013875768693558787, 0.0012131753013214178, 0.0012067035738680776, 0.0007607348207958948, 0.0013226908147791507, 0.0011510399372729272, 0.0014704296993550846, 0.0009215320247409368, 0.0012113769165335113, 0.00124322877440704, 0.0015769689935930845, 0.0013937901012490885, 0.0013947851180709635, 0.0009642131202542177, 0.0011394877541568184, 0.000853417049124607, 0.0011822123245491631, 0.0007675644509196702, 0.0013048734122658382, 0.0005243396096562436, 8.855402683120252e-05, 0.0010037464645029088, 0.001428720223429236, 0.0005373311907708959, 0.0006379409312552004, 0.000456120365523114, 0.0007910086618999093, 0.001082869503561418, 0.0008734146121389151, 0.00020761449578410142, 0.0004413895136586542, 0.00046101517379078645, 0.0008369531328262733, 0.0003596208722231228, 0.0005317901448337052, 2.8425005479674155e-05, 0.0002258494624790165, 0.0004257844995255738, 0.000903019918275517, 0.0007962210491289874, 0.0006969558791959437, 0.0005565436853799034, 0.00015473118560459622, 0.0003166267004320823, 0.00023563226057721196, 0.0006517574866202454, 0.0002718654881164059, 0.00043500598976685054, 0.00025642861880706706, 0.00048351011434940883, 3.3270623469987814e-12, 3.0197499166010446e-12, 0, 0, 7.037262736269161e-12, 0, 1.2434948114647107e-05, 0.0001039436083946269, 0.0004085591500061317, 0.0005787097622578203, 0.0007805745045150696, 0.0004910891953806376, 0.0006250798180366915, 0.0005790541785591081, 0.00047698390472911573, 0.0005015024826637327, 0.0007211789523651275, 0.0004507000407866192, 0.0006707920660079882, 0.0007873780082678575, 0.0006569233252169672, 0.0007588623353705912, 0.0007946146892530439, 0.0008764723830441607, 0.0006253915137834306, 0.0009124507949989291, 0.0009081499899620777, 0.0009649138300465173, 0.0007887905353677226, 0.0011251074184282655, 0.0014350699528324744, 0.0014640428439307615, 0.0008988841959418995, 0.0006089773494215093, 0.0005405620794143557, 0.0006242473621077155, 0.0004273912544938201, 0.000931894027719198, 0.0009535107155065853, 0.0012279669248533801, 0.0015595381604373486, 0.0010685079310923213, 0.0008825543605159368, 0.0009725690819635622, 0.0009770502676494967, 0.0012437191139246337, 0.0013076624481598155, 0.0014880820823652589, 0.0014990086357983198, 0.0011072988345718082, 0.0013925752204398862, 0.0014777132412932658, 0.001587036555177888, 0.0015465846510655085, 0.0014952960656236583, 0.0010795609684442306, 0.0014426981508716854, 0.0016435266746009272, 0.0017951024685716516, 0.0021732637953954252, 0.002124282904127591, 0.0019399605420765424, 0.0017251489183510126, 0.001632564242200308, 0.001376707539044622, 0.001688578030469851, 0.0014857456336553536, 0.001328620787757845, 0.001571316133923109, 0.0014381448318908391, 0.0014328196639985362, 0.0019138665466904084, 0.002028315400380584, 0.0011783261888727553, 0.0018042850931560064, 0.001295422181184675, 0.0016264389842245497, 0.0017488743613082549, 0.0022046263796504303, 0.0023324912040550766, 0.0023259945594220335, 0.0022555628946921823, 0.0020550445110243186, 0.0014718837009287324, 0.0016446037606988046, 0.0014277429287731576, 0.0017599932338818096, 0.002111164037523742, 0.0028963011565319163, 0.003062290834112509, 0.0036958771933871155, 0.00403743502085399, 0.003728728856541752, 0.0030102002762320253, 0.003043425849325464, 0.0032179667827843964, 0.0035286511833932314, 0.0028952815224024546, 0.0031819663123332895, 0.003104778284354555, 0.0031031585386650205, 0.0027654745265153187, 0.0027827511498775635, 0.002739286105460406, 0.0029235004053715227, 0.002794453628378685, 0.002918314526791038, 0.003095453609251471, 0.0033732154404035176, 0.003463199648382331, 0.003723904433689441, 0.0036837530817134616, 0.003569670268456572, 0.003524913500004152, 0.003649813263975555, 0.003882481972724555, 0.00383877530454356, 0.004294735547660753, 0.003974512178857372, 0.003914945162966355, 0.00432381323590289, 0.004134723619783736, 0.004198935234257554, 0.0036748881317177685, 0.003494430614832084, 0.003255546414273268, 0.0038425778132652943, 0.003501671887465389, 0.003791271135456638, 0.003533392949319947, 0.0039702419911635985, 0.003627329115137493, 0.0036956941005958474, 0.003173717761179915, 0.003021397317728901, 0.0032164261857686176, 0.0031608427231150198, 0.003292193206846239, 0.0032702201256936294, 0.002861902408215137, 0.0029833867167511573, 0.0030797274820758857, 0.0032903720440400036, 0.003975415201896638, 0.004025683546476824, 0.003560105735474702, 0.0035082766464053575, 0.0034187218578081353, 0.0033268878828453035, 0.0028178778594540516, 0.0031468276858473538, 0.003696201484402933, 0.0037800253720183833, 0.0031148733456771854, 0.0028620730628298764, 0.002717631546490668, 0.002807286417545376, 0.0025840553661085736, 0.002706783275706295, 0.002167538670974932, 0.002031521514996527, 0.0014035362266950082, 0.0019799394861395804, 0.0018966443147998896, 0.0019899284291937675, 0.0018833556182161844, 0.0020202234706157415, 0.001992521360496391, 0.0024052821909575036, 0.0025040425434956055, 0.0029808930443619825, 0.0025152823137146317, 0.002581044984833441, 0.002301758487722628, 0.0023761381031930234, 0.002657690206107696, 0.0027109493085191883, 0.0026122626009024985, 0.002324702326898678, 0.0018435227697839085, 0.0018711010893894648, 0.0011294081352984096, 0.0014455473188824401, 0.0008018193550292512, 0.0014052870250125167, 1.7388807423684256e-11, 0.003072077388906296, 0.0032884051591903435, 0.0024104414692306822, 0.001312872337630707, 0.0014840002157231766, 0.0006042168365801488, 0.001827479556400655, 0.0009071750183465935, 0.0009858275577666648, 0.0006222131628053351, 0.0013670953975662642, 0.0011251659185887945, 0.0005929259767251214, 7.281503133857131e-11, 3.2678295140480923e-12, 0.0001815060944889548, 0.0003620947599129016, 0.00013825691555436272, 8.847505392490609e-06, 2.016599796397642e-12, 5.62091993938354e-12, 9.542204856980838e-13, 0.00039502539922334543, 0.0011198967883658002, 0.0012605619267429056, 0.0001468277291567972, 0.000986355114412363, 0.0002509984366803991, 0.0012183819481870943, 0.00025974919084532764, 0.0008043110319219594, 9.969621914540698e-05, 0.0013068927445129975, 0.0005500258936832226, 0.0007437182324659468, 0.00035280441052589476, 0.0008219348315464759, 0.0006625677977058655, 0.0011718890133196583, 0.0007626338004799729, 0.000522804346738234, 0.0007205114482940763, 0.0007147344284340781, 0.0004618819973507926, 0.0010065058094381912, 0.0015495339018908787, 0.0008444262755721004, 0.0007472625713223717, 0.00044928046463322613, 0.0007204431342824068, 0.001022701066988658, 0.0012291098538103724, 0.0010459485131030584, 0.0008371845071846922, 0.0008768278625359618, 0.000634881836441509, 0.00019540763373896005, 0.00043047812708761315, 0.0005070929937386746, 0.0006713253162709379, 0.0003584450251784543, 0.0003058644169576934, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00016488373867633565, 0.000615976803263375, 0.0005301294706059841, 0.00040504764274066425, 2.5710787247884095e-09, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.228194270221354e-10, 0, 1.449225113564006e-13, 0, 1.191877235844189e-13, 1.5359507575955482e-05, 5.60896016937744e-13, 6.64224151883167e-13, 0, 0, 0, 7.1772595098249785e-12, 0, 0, 3.860524696399175e-11, 0, 5.532809627001101e-11, 4.534493384594569e-13, 0.00043410795145550285, 0.0003267858190351823, 0.0009619321079793731, 0.0003956517306646095, 9.80823037563215e-05, 1.5799565501986575e-11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0009962478411948057, 2.8151814820904263e-05, 0.000729577840206756, 0.00019125198397256028, 0.0009501021571211015, 0.0006933545325777875, 0.00035675712634248847, 0.0004796213949913622, 0.0012051882944478524, 0.0014627075283458067, 0.0016355288334355502, 0.0012255657307734806, 0.0018764393509251797, 0.0016829194785137067, 0.0020867296772096398, 0.0021944705675147206, 0.002333276288217853, 0.002411557946044367, 0.0026818940069569274, 0.0031780868684046053, 0.003219644364763924, 0.002837186965115672, 0.0028505578109515937, 0.0026214452046685465, 0.0033348546130045607, 0.0034317100352194018, 0.004245833956945141, 0.004172887897898238, 0.0040671395374063, 0.0039215445596624135, 0.004300046862530342, 0.005032864260991995, 0.004943331533430786, 0.004815660288405222, 0.005258142944476677, 0.0053649536721710415, 0.0050906882504548, 0.004537664475532934, 0.004522433829627965, 0.0043586110826837375, 0.0039319279046264475, 0.0037275414375748505, 0.0035625927956432183, 0.0034688226179034514, 0.002975058931915378, 0.003558204137212836, 0.004146841872337352, 0.004262612567550209, 0.00428469370939092, 0.004208245745705148, 0.0037961466430178115, 0.004056510843561894, 0.004004429737705241, 0.00425354851798514, 0.004103975412112718, 0.004599471434532863, 0.0051925826344550105, 0.004931733406510983, 0.005327880082884983, 0.004911051805910733, 0.0052961841614723865, 0.004985616736218073, 0.005182905627598315, 0.004694074689491102, 0.004689351823909141, 0.003948634303878696, 0.004479908584602937, 0.004606754345847822, 0.004604543068578524, 0.005147694301569803, 0.005426029073496141, 0.0037881257712406202, 0.0030493629527660287, 0.0027935334934532656, 0.0033839671248690585, 0.003765771613696644, 0.0036652459322282457, 0.003384319541193376, 0.0034356045979787417, 0.003852922860339703, 0.00439439446726959, 0.0036423520093269643, 0.004192279705988195, 0.0020989506172693654, 1.4825489912998317e-12, 0, 0, 0, 0, 0, 0, 0.03369741887666887, 0.035845906133504905]

best_sequence = best_sequence[::-1]
print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization

def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(8, 5))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-0.001, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

In [ ]:
#@title Data and verification (score = 1.5032)
import numpy as np

# @title Construction
best_sequence = [0.6684206605232642, 0.47042115688825253, 0.6126479864436473, 0.6146683230055647, 0.6098604980496021, 0.5965303912931771, 0.5370538033155624, 0.5464121257349384, 0.5624392110400134, 0.5743963269790259, 0.5582279602216285, 0.5831214742063587, 0.5784748084863656, 0.5810555123940028, 0.5440771860680639, 0.5421034143676927, 0.5273087307747699, 0.49015063640470824, 0.5072243054508333, 0.44152119188966615, 0.4086655126488308, 0.32356408617136007, 0.3359598944287351, 0.20240935545467226, 0.04864171150770286, 5.938530782955024e-07, 5.390674854109316e-07, 4.750885467630949e-07, 5.157229529362103e-07, 0.002983432980745507, 0.01297575001917319, 0.023791016734906367, 0.009903810123267277, 0.02151438737054539, 0.02515415439310428, 0.006457959870676382, 5.795497333813849e-07, 5.27071095531917e-07, 5.104398853716231e-07, 4.680056612102303e-07, 5.404934355923275e-07, 3.7192650367442645e-06, 0.013089081261502019, 0.005038422383415062, 0.01383472152978682, 0.031287625037123144, 0.0467987996514168, 0.04014706528787544, 0.02097318784232731, 0.0024222090003317583, 0.00610313476463666, 0.03166087334201033, 0.13829467945413115, 0.11794840825188524, 0.14445099018886448, 0.1399158800421457, 0.10304772832207378, 0.07635069175314114, 0.08943638908276651, 0.04992118638700703, 0.06797743080971681, 0.046793233529321535, 0.10302286495883166, 0.15972428266847213, 0.1658736057241382, 0.2288814909416735, 0.26543012231329377, 0.3041469022811273, 0.31138911754901694, 0.32084672571185946, 0.36605963374788064, 0.3754557408893152, 0.3943654771205571, 0.42144109530359386, 0.43505184374678785, 0.43440809107091893, 0.44662131071324007, 0.3699625990120669, 0.4511498594904703, 0.40741635921443453, 0.43926607227662434, 0.4572788898885031, 0.478292446889611, 0.5063694792634349, 0.5371417507557213, 0.48615253629201266, 0.4203466856390902, 0.37122031588735654, 0.3922318785309664, 0.2904853402593819, 0.3085971126241098, 0.29845894315656984, 0.2490565458535593, 0.24676766665282804, 0.16099742876758283, 0.2196996690796454, 0.27172820616067883, 0.23701645102069088, 0.20882690184905717, 0.1990933434016788, 0.18377706723201964, 0.1622638222959843, 0.1138320017778828, 0.15321116944214516, 0.13841758769855605, 0.13839275740117765, 0.13193098943696951, 0.09506775171910717, 0.026591169227502293, 0.07111489577994794, 0.052637469336356324, 0.042089649369025324, 0.05178169420216255, 0.032522230477168755, 0.03278820604270117, 0.021272575547697608, 0.03430267189095609, 0.03099320540235296, 0.08417141497045459, 0.15092240903657136, 0.12168447906171558, 0.21061171520845787, 2.8795574947561756e-06, 0.15052739469570303, 0.16696193992543124, 0.19635580286029694, 0.1840309643060049, 0.18038387670847333, 0.1842727687854025, 0.17955012462321635, 0.22169342648403031, 0.238330351086444, 0.25119028257170933, 0.28190806224484205, 0.30784758792177064, 0.3209169331357, 0.3155446223127133, 0.33247959270418, 0.36028996170465394, 0.40316689976848374, 0.38669761529067226, 0.3407748715369285, 0.3653587894756018, 0.41365408212026117, 0.2888631779900761, 0.33248928489969387, 0.4159373732448291, 0.42420408324711195, 0.4272683938938278, 0.4736305914957993, 0.481622471497252, 0.46837413918323284, 0.454183962485784, 0.46584600239568724, 0.4692485589670577, 0.46373573370152055, 0.41663744302866634, 0.4524805556385104, 0.4075559729367211, 0.39089106979571037, 0.4116429551459234, 0.40669103181832317, 0.4297571115440008, 0.4061869882682101, 0.3890162072343314, 0.3658456712966306, 0.34379760939653575, 0.2928132141043841, 0.25613839249167575, 0.1740809936473668, 0.17181516124427995, 0.1824305011447183, 0.23002475690338656, 0.1923345160912967, 0.16240834880195454, 0.17064613124587882, 0.18102305714766487, 0.15475046578480947, 0.1305197115947348, 0.1462892356639961, 0.16046710789554788, 0.17373003647286905, 0.17465898307046535, 0.1675836016853758, 0.15538973997571154, 0.14915708597973662, 0.13311252382938188, 0.12409324044352, 0.06600562403649546, 0.07701923840669113, 0.1258552902488337, 0.1441351376023811, 0.165437165705809, 0.14628224834034803, 0.13321699419935845, 0.1733254511567077, 0.17994997154013045, 0.17813376058927524, 0.21599047062843388, 0.23720836421573213, 0.2580832159056143, 0.280819000560719, 0.2835186556337436, 0.2943881593098738, 0.3248498045940572, 0.3353438716804007, 0.35291654929899335, 0.3885043599453416, 0.44302268524075294, 0.44560127394034116, 0.45803931429029143, 0.4894375898491014, 0.48623836064690734, 0.5066460818337295, 0.5370891781345062, 0.5286480554290433, 0.5080051679681078, 0.5238029322269591, 0.5082771601466699, 0.5485665493580414, 0.5370488941416937, 0.4919105102181089, 0.4897830491741853, 0.47092929589494664, 0.47387011191310485, 0.49515593063825675, 0.474112685701658, 0.4548492830423357, 0.4507668135655479, 0.4690799632749159, 0.44366886011645235, 0.3813786222505124, 0.33613188842123803, 0.3401177045428145, 0.32150920142293393, 0.3060700800948959, 0.27296944909073995, 0.26731345230233217, 0.2453471097239613, 0.21008040506938294, 0.1916779988622173, 0.1407389063043137, 0.16899740016446313, 0.16062257513302544, 0.1442479522341768, 0.12414001799344346, 0.11841493511823346, 0.12921561002633553, 0.12820270742220194, 0.11017657630971904, 0.09085617047457242, 0.09629353728062495, 0.08548839038190163, 0.06349021849527671, 0.03945341789293378, 0.01931796457776467, 0.014414882135986776, 0.02207860563426693, 0.04331369745088091, 0.08517659873015311, 0.06028706965460429, 0.10871672994986152, 0.13603466719815208, 0.19791959021391697, 0.19169350047774944, 0.1915976616629912, 0.12639756466096694, 0.1871558747237102, 0.22748153398458534, 0.23134545366854642, 0.21356765909026068, 0.20005218930692456, 0.1684223603427739, 0.1439691018073705, 0.1281784571088388, 0.15396627424525214, 0.1752581669761264, 0.2029449231549311, 0.24515818429148378, 0.2658770953465686, 0.2953796083094294, 0.32750146962037985, 0.37610393884077065, 0.38048113146857276, 0.3733927760429421, 0.42995665910929887, 0.37788711200155634, 0.5091239219834888, 0.00397146440631267, 0.2995644897824748, 0.2573774024670318, 0.2884920219695325, 0.340313483894447, 0.39197493012458406, 0.45749573213826855, 0.5135082578482706, 0.5039298509745106, 0.5696850136105143, 0.5967534320727752, 0.6044960256563072, 0.5997264310923826, 0.550288783481211, 0.500045715199025, 0.4625752111321056, 0.4072118913841265, 0.39135087247653333, 0.3348515546569486, 0.33396404282844, 0.23734358725803661, 0.21849438245843364, 0.13734174765295792, 0.1669136255492, 0.22222494594955666, 0.16874853225054334, 0.19597068409083865, 0.1928091087988025, 0.22042707005808454, 0.16566479799705028, 0.1070540017732279, 0.09223089494349267, 0.06535896032757983, 0.036127462166780075, 0.013574873109619304, 0.03822229177876185, 0.07450708878454519, 0.06158794115239154, 0.08254056145019965, 0.11461913225233186, 0.12179164578960688, 0.17098686921013767, 0.22701699500958275, 0.3113508514565178, 0.31303204179460525, 0.40678005432661013, 0.44627161159700596, 0.3837219679026054, 0.440524119255764, 0.38338796375927303, 0.34819728716508097, 0.34796166264142053, 0.24156533432364474, 0.3290849574280949, 0.27079950253734786, 0.26574743384052013, 0.2499505336146171, 0.28939562052047074, 0.30983616115124696, 0.316313785066598, 0.2911207425793609, 0.29948235845085197, 0.2832005702996216, 0.2865838195538181, 0.2667403382044497, 0.2092842733000082, 0.16306722152283623, 0.10546924342935259, 0.08880814176936036, 0.09772939489990454, 0.16925315601885751, 0.24360056980931044, 0.310885528988664, 0.3749857812638349, 0.40856489628491033, 0.43560015091793913, 0.4901459967388693, 0.5108725259370469, 0.576266896457029, 0.6147461305662192, 0.6060561121281413, 0.6394171665453742, 0.6309980524172155, 0.5555418822298576, 0.4792198775245933, 0.4496466471465919, 0.4317214398304083, 0.38937343268243596, 0.3358275238586751, 0.3975717489858505, 0.45270676496104106, 0.5058873083975323, 0.5312231241427712, 0.4652785758822779, 0.40278609733380233, 0.20847263366315869, 0.02965636321323178, 0.003799361794213899, 6.814545337043103e-07, 6.169461110386918e-07, 4.91240425338099e-07, 6.156211683051072e-07, 7.673595091998908e-07, 7.692141642908651e-07, 6.967383244414097e-07, 6.202901582680097e-07, 0.00015058489720955412, 0.13789532603801774, 0.33781469673483966, 0.28504695366821536, 0.3883288750561533, 0.44880292321187387, 0.4285999154728701, 0.43192546634161977, 0.4230358580903186, 0.466489360715886, 0.46554456047697684, 0.46117572881061314, 0.43699609743707696, 0.39863405444514294, 0.34110898216332153, 0.38497653986670416, 0.29435804845954155, 0.34491646788920133, 0.33608603256163877, 0.3246194865602954, 0.3061990390170706, 0.3009082507731276, 0.3250977100487071, 0.3199455878635706, 0.3390446032173857, 0.33286594305272577, 0.3113918421010183, 0.32212598306405266, 0.30735725550457077, 0.2916846723001528, 0.2685658118337254, 0.24088050834084854, 0.30290216177916185, 0.2753017747697385, 0.29951911095315675, 0.25450091863046065, 0.23112959600113167, 0.14092330083505317, 0.12858046571133697, 0.10051724877946278, 0.06870603305346804, 1.5002647487576896e-05, 0.1455587547741704, 0.1976669352297687, 0.24109362934874956, 0.21961664411931087, 0.15422629594960452, 0.16636778475810338, 0.13760128291016538, 0.12880692357021833, 0.148380162799219, 0.16024411854041923, 0.28377985436119557, 0.2688271724897654, 0.21754647180287218, 0.2578026383058886, 0.28302512100521027, 0.483213110772413, 0.5566666271686285, 0.540464191620335, 0.1243932861096531, 0.6528768128964748, 0.5955370714429759, 0.7334889456702378, 0.6238939669983505, 0.5089861453146425, 0.5146643707092922, 0.5276943192499157, 0.4655147513350075, 0.49004969810873295, 0.44714933283976493, 0.44757823248601075, 0.41443469830254437, 0.484075292144619, 0.47214411181099647, 0.4252912095362966, 0.39008814334374026, 0.40076854546330243, 0.3690767807396912, 0.4241829071835008, 0.4212148562274619, 0.40382749781749105, 0.35374569282510815, 0.30175323032199564, 0.2460181659147689, 0.24485209961648416, 0.33830681395703666, 0.33263775637518095, 0.2363646501970311, 0.1840646959338477, 0.13691269881302248, 0.1725504421906477, 0.19021581659335535, 0.18179386508099926, 0.1499662619128585, 0.12047655148485174, 0.10510234861787845, 0.04488231997143361, 0.055064277130758496, 0.04427735876296552, 0.029532971720241677, 0.03338077429520641, 5.8159089684121684e-05, 0.00010332583136209708, 0.003445346504574239, 0.007413333901586842, 0.05518739484834224, 0.04183513113410011, 0.03641309592882915, 0.09739825024299617, 0.09775798513356661, 0.10901936469008501, 0.0908070794777723, 0.03583925316131292, 0.02581213429370143, 0.008473104040019368, 0.04397695556722847, 0.04691389331208905, 0.037499394934251695, 0.020927198641221535, 4.568225060917555e-05, 0.054227218082019114, 0.0407795440756071, 1.9662755621985477e-06, 2.6304533145391746e-07, 2.1505498448312907e-07, 1.8699090419617514e-07, 1.0700892754066718e-07, 2.2869483697071446e-07, 2.785200603297447e-07, 3.991366798849969e-07, 3.735836704142344e-07, 3.4386603713511324e-07, 3.585972721915817e-07, 3.9766867689586784e-07, 4.7003073916470504e-07, 0.012630569607674867, 0.15744509659784742, 2.0418993313136945e-06, 0.1510644186516384, 0.11972844014830245, 0.05000544546599925, 0.0008016928844376988, 0.0049029863047342186, 4.87565618767464e-06, 0.001419728533250981, 7.778348232192159e-07, 0.03659456865107672, 0.016351678797058674, 0.08091888558826293, 7.534504267580361e-07, 3.313757635706786e-07, 5.326056647794526e-07, 0.01564942171816061, 0.2780648416719895, 0.1684844809355674, 0.0458395248908786, 4.2097735438282616e-07, 0.004195825651322785, 7.900289583684875e-05, 0.12554852266908517, 0.13593262744593057, 0.052405919009429425, 0.0003917492311307192, 2.5838825835066836e-07, 2.4968955397605473e-07, 2.44015738151676e-07, 1.9429910754209026e-07, 1.7023773552445187e-07, 1.9352489144591632e-07, 2.1607602145254798e-07, 1.7806985041534818e-07, 2.984781775468075e-07, 3.012084963846438e-07, 2.974669184298468e-07, 0.04520765792704057, 0.013753320560331952, 1.7161170920649132e-07, 2.0702814782292962e-07, 8.447084553830767e-07, 0.1207235314903664, 0.14608027506400673, 1.4286914029528823, 0.028858972133308496, 0.0023740468621625006, 0.0008256171544009872, 1.8801983539131582e-07, 1.5184176408528215e-06, 0.02089148281816699, 0.09762752790806083, 0.018188415054879923, 0.06337468596114913, 0.0716538863551167, 0.039196085492310406, 0.053054075045597976, 1.909798775580136e-06, 0.004092998087226188, 0.055210441723679386, 0.057316097375868395, 5.023146144077242e-06, 2.415818811841287e-07, 1.586702850958076e-07, 2.6568171103040795e-07, 2.6419426852864435e-06, 0.4232301585657383, 0.3590407249390599, 0.2353861218334374, 0.36629479783843777, 0.2969844333969365, 0.14645226331324915, 0.034068681270745256, 0.03850776592694222, 0.14188936015968934, 0.1017350848499835, 1.130772382152738e-06, 4.574093422745962e-07, 0.022425050277226626, 7.661586288751925e-07, 0.03694850329541652, 0.038317509655508744, 2.8124519001549966e-06, 0.05482630211612435, 0.09899923078666802, 0.12058700800495814, 0.1175456367850324, 0.16789362202918381, 0.18903187083530024, 1.5074158676406718e-06, 2.706812310135156e-05, 0.010274306824737376, 0.02248805099768245, 0.1751769615311516, 0.12524305037474895, 0.213193640007212, 0.34310544464305726, 2.273122249079052e-06, 5.906452052115439e-07, 0.02185826324867595, 0.06658877506515216, 9.675675296530098e-07, 5.407244344548555e-07, 0.0015124462470337205, 0.02091463600106262, 1.0621006637174608e-06, 0.06698384651362227, 0.008232779364955523, 7.282594184807662e-07, 3.911623753399451e-07, 5.438730062271993e-07, 4.702505185871889e-07, 4.38162331836774e-07, 4.107884037180029e-07, 4.899651572934854e-07, 4.7390547438863654e-07, 3.2126205911831876e-07, 3.157877988452647e-07, 3.3410419207406786e-07, 1.8726561144946104e-07, 1.5096999701422787e-07, 2.0486493108545828e-07, 1.7680972322873975e-07, 2.531698168203902e-07, 3.6912980588439235e-07, 4.3151725811743116e-07, 0.023342728905222574, 0.13209780068660604, 0.19031092681365894, 0.058964613203099904, 0.025866683279937748, 0.1915276284060276, 0.41829423233793345, 0.3414203406784337, 0.3621729659491739, 0.26981333394196594, 0.033963341391984, 0.022564982997895108, 7.545779831332857e-07, 8.378985730175008e-07, 0.11187401198661082, 0.024282960966349246, 3.2603965906549874e-07, 0.0008374730438838082, 0.007175761381483745, 0.017730533298647332, 0.021394395021545023, 3.294687486250975e-07, 2.0857407644655165e-07, 2.5620296851059e-07, 3.0301005265895795e-07, 3.7306930855892345e-07, 2.576185148673647e-07, 3.18939416919536e-07, 2.0698621489204868e-07, 0.0009333953907457879, 4.5509817424189253e-07, 4.2016836732524905e-07, 2.8253607860360156e-07, 2.784648453246831e-07, 2.8523296606163084e-07, 3.870803259514001e-07, 6.01435614866773e-07, 0.03627200790861977, 1.5901442082372143e-05, 9.336602226538517e-05, 2.3509402769400334e-07, 1.3037339336189464e-07, 1.934400273022108e-07, 1.874180794483797e-07, 1.6333049359773914e-07, 1.0865171581946598e-07, 2.990755836088607e-07, 2.321528614247698e-07, 3.054478370099514e-07, 4.847085609641063e-07, 3.93161067045717e-07, 3.1254667085372225e-07, 2.6978813961122047e-07, 2.5958238546217905e-07, 2.689900395524272e-07, 3.6125108936437787e-07, 4.050898854695763e-07, 2.748401228882344e-07, 3.524947955642124e-07, 2.9313953208245065e-07, 2.7084785886655465e-07, 3.060898150566088e-07, 3.060203753433422e-07, 6.219299214919742e-07, 6.022425082940689e-07, 4.914874248857583e-07, 4.3061856212775194e-07, 4.654295360174328e-07, 5.540448064534892e-07, 5.914462155816584e-07, 6.108230806841432e-07, 5.872672495734629e-07, 4.835299021221845e-07, 5.458538231791326e-07, 1.3589066834880915e-06, 0.15638768675239137, 0.20413165323008417, 0.13803501852127778, 0.14702012478173818, 0.21849465166354168, 0.336324964380127, 0.4605213545903027, 0.4226938744364159, 0.5946623862330719, 4.7650257650363e-06, 0.3148113278057696, 0.11340840170014307, 0.05224852197969241, 0.03262946506604027, 0.1620744414604543, 0.3184348297907283, 0.23842851550584518, 0.2782094243955374, 0.21647330853835822, 0.34430913021727916, 0.31922493200838187, 0.3601618213810133, 0.3636263393910037, 0.4306416150987128, 0.49376563751333813, 0.5318874307751432, 0.5268972918689256, 0.4701540520554696, 0.4581020759620578, 0.38024436213606927, 0.2064890200423665, 2.200339902404553e-05, 0.20945176964323162, 0.5433107539971542, 0.7859122121288994, 0.5197234693026311, 0.34754143154149747, 0.2271705559020577, 0.3671937140308478, 0.5479815167351727, 0.4432552655493465, 0.5111951139665101, 0.5253295127957912, 0.5714487269436764, 0.48743829443023634, 0.4604581270577824, 0.6232671623200373, 0.6857950118000486, 0.6428520351635553, 0.6129809008774678, 0.4466454542824276, 0.4496298545712332, 0.4505053200735322, 0.7224648395595237, 0.5438707799859035, 0.458457345876032, 0.43591058296448426, 0.4135484549709867, 0.6015911076239522, 0.6879039922496375, 0.5134479620471766, 0.4837996149242729, 0.47063645613671995, 0.4464502715948646, 0.3734989993330889, 0.4276401403346204, 0.39251967627005646, 0.35214648031293244, 0.27408415930041774, 0.25398547390622844, 0.3772425890723171, 0.3571563744953511, 0.3166928901304923, 0.272748573868698, 0.38720703695988207, 0.3303519392781066, 0.34901390073691335, 0.331921028291229, 0.2745516103470505, 0.13282473646566242, 0.13872544159972994, 0.10538526213491753, 0.005385725092639643, 5.178087756248629e-07, 0.01670565025564083, 0.010124583151329863, 0.11699494167254132, 9.914938692117047e-07, 8.01855931633171e-06, 0.04529014184506623, 0.0017075512804826866, 0.007894855485195935, 0.02329249455653833, 0.040448526735473146, 0.20519959644527658, 0.3228912273008806, 0.37843781129984455, 0.3545809339146711, 0.2816842686664292, 0.16666737944219073, 0.2425655878292661, 0.19586847786015765, 0.13192423034095205, 0.08833082952409645, 8.45740668999573e-07, 3.7337312954478696e-05, 0.27838954829949286, 0.33340345302035945, 0.3275069682747177, 0.31999436839158574, 0.25006341513852404, 0.04890632998684506, 0.08244278066686517, 0.15475351545384625, 0.250223385812871, 0.2952959974356177, 0.2504441870808964, 0.14502509958149887, 0.15732444002304247, 0.2643356622727929, 0.19672636796882495, 0.1082514640899978, 0.07248997343989318, 0.08463114195194439, 0.04412629133370429, 0.05167278513849863, 0.09848248909723151, 0.03204759107161251, 2.9565257191192476e-07, 2.2562096012234366e-07, 9.871320424705098e-06, 0.9115770941481338, 5.395802313586384e-06, 2.5018935283299504e-07, 2.9545700673344945e-07, 5.103876638735662e-07, 4.493214715511621e-07, 4.6084358499441707e-07, 4.997591561002611e-07, 4.821319883983163e-07, 4.5848462952975725e-07, 3.821017418801805e-07, 4.6403034099935607e-07, 0.0001860720203690538, 0.008431487137973759, 0.06779115604260386, 0.19768501258787788, 0.09517782652991612, 0.030148538301832457, 0.001402873787183119, 0.2078353967287988, 0.21372570878222924, 0.6473108102393675, 1.013625956818627, 0.7198156819645344, 0.4829118090014056, 0.5830531893346216, 0.2810182189516985, 0.04679903721064712, 0.0033742761896572455, 0.0732194284144996, 0.22277295918617182, 0.00256475301310091, 0.012301255056419795, 0.03308012173106231, 0.07537510619332312, 9.328445760122675e-05, 0.014310194960901152, 0.007658826771132263, 0.07837754252292176, 0.11532424198950442, 0.15744050448169972, 0.04202527866316569, 0.0252028871940994, 1.731572695520674e-06, 0.25040876891178204, 2.6291342010192495e-06, 0.1833895152606159, 2.4328111511052457e-06, 2.9849671148052583e-07, 1.8616340787139954e-07, 1.761128808203317e-07, 2.4083850280360985e-07, 2.888208189310864e-07, 1.9144371965156735e-07, 2.086092266360737e-07, 1.905814525623342e-07, 2.716140047125949e-07, 4.1490069486841176e-07, 2.831212374123661e-07, 3.475092528347079e-07, 4.2472244553384917e-07, 5.580109333192149e-07, 4.5216939576019453e-07, 4.693599575104169e-07, 4.821429871915454e-07, 4.953879050317653e-07, 4.893278556895407e-07, 5.025954732950582e-07, 1.2477085493675385e-06, 0.13781881482877292, 0.066016680959641, 0.1579927325327515, 0.22358787833646995, 2.3759839807472462e-06, 0.04876375839631167, 0.12449455882872947, 0.09776089655861575, 0.0820902918428391, 0.19495289896676612, 0.10339313449745632, 0.13151510802281585, 0.17185438724248872, 0.14973832333619472, 0.2071834378413167, 0.1484405835554094, 0.038759951602548554, 0.04902538379597046, 0.11107395821094729, 0.01801161638373595, 3.785906511089649e-06, 0.011264806517829799, 1.582022084682653e-05, 2.264304297728996e-07, 0.001163508209361957, 0.020302899195935066, 0.10245190981660139, 0.10737156253211887, 0.017991861890021964, 0.02413094955734084, 0.0881270895386628, 0.07293813960044718, 0.0785956943343757, 7.837552223091904e-05, 1.1106075113525299e-06, 0.13223547108793182, 0.2957035341403858, 0.20028707290741005, 0.07447732287266524, 2.9461294279872383e-06, 0.41774214736405835, 0.4110122909332719, 0.2589361318058029, 0.058735202559894506, 0.0001843466288192475, 1.278206264924943e-07, 5.636682880952825e-07, 0.1069660610860728, 0.14979500304109036, 0.15736002171620508, 0.38751043501798305, 0.2096422288524515, 0.04739061931287805, 0.26229654036104205, 0.013964110379294764, 0.002252920425564902, 0.09396588855625114, 0.07280612051420214, 0.06472189447710372, 0.07616773528957184, 0.0004546082266815044, 0.0013466106908430662, 0.0003432660538779449, 0.0045997800213494425, 2.463664246718375e-07, 5.395589026048439e-07, 0.047926405834200415, 0.07899108698161979, 0.11939485702641364, 0.12276385946352075, 0.10359209220947306, 1.2513223792164796e-06, 0.021424381839414752, 0.2733400101437141, 0.7274697504850173, 0.9093683541161476, 0.46743776312120866, 0.13854928371962846, 0.27903594295593936, 0.09651667590415944, 0.06956157638929597, 0.12404050830438836, 0.20864267093161187, 0.3597701273960755, 0.09902903075439634, 0.07383269291725382, 1.2278690985741766e-06, 0.028612882823252084, 0.04299055469208647, 0.019260379541863215, 0.04186353705616778, 0.028450133518746347, 0.010038366864865589, 8.887308620868558e-08, 8.975255018854234e-08, 6.119438473664828e-08, 8.535558313972017e-06, 1.5688460403957303, 3.8652028885016356e-05, 0.13185819902870963, 0.0030749354702890113, 0.021435534311270217, 5.959378630538764e-08, 1.0586516225656162e-07, 2.0446666844295174e-07, 0.046556282029504945, 3.2004504371822783e-06, 0.060596453706954045, 0.1483228822435191, 0.09419863797346559, 0.16132021602845475, 0.18158579514047427, 0.2817489632386039, 0.29612294401276906, 0.21059707249783938, 0.1473681859220836, 0.054044996655291375, 0.058559959069803665, 0.07884757998605874, 0.05909708436769208, 1.214626691950663e-06, 2.8193825805746883e-07, 1.8323120454657576e-07, 1.5248991423754225e-07, 1.723499682417684e-07, 1.7777402886390877e-07, 1.722053478545771e-07, 1.8661431481254386e-07, 1.9233070118009835e-07, 2.1145270609124975e-07, 2.520859128237836e-07, 1.7895646223136995e-07, 2.0610877577020916e-07, 1.388139516138604e-07, 2.0483681722327616e-07, 1.974498623963885e-07, 3.2880414036347885e-07, 1.660133932619072e-07, 1.4903819234145195e-07, 2.7764122021182115e-07, 1.9503392107194357e-07, 4.333290811792978e-07, 1.5341089980040762e-07, 1.5663317349609787e-07, 1.6702492891923166e-07, 1.7241756657398483e-07, 1.5069497623301332e-07, 1.8902065563314747e-07, 1.736016270339248e-07, 1.6389347912327923e-07, 1.3330938605805322e-07, 1.2282004673775383e-07, 1.6359380398359687e-07, 1.6030289214941403e-07, 1.7695861899962602e-07, 2.103015525107086e-07, 1.7226656423853424e-07, 1.990819146967374e-07, 1.9466952369235187e-07, 2.027427483475905e-07, 1.6308515290786622e-07, 1.5119488607969154e-07, 1.7181258962175082e-07, 1.741019003307305e-07, 1.7842581478697884e-07, 1.0629274367586149e-07, 1.1211815185576904e-07, 1.195279470627498e-07, 1.7636672196132932e-07, 1.0130389723495278e-07, 1.0874008102817542e-07, 1.091018693115943e-07, 1.1435445544627077e-07, 1.3692222348972798e-07, 1.1550326844474961e-07, 1.626518559559275e-07, 1.037875319641521e-07, 1.2053380149086492e-07, 1.237061230076352e-07, 1.0167747116436744e-07, 1.0827233410289052e-07, 1.0272471073423396e-07, 1.4931278428213902e-07, 2.242747266721061e-07, 1.4028438343380578e-07, 1.378166448556028e-07, 1.2936126867086964e-07, 1.0661074016243494e-07, 1.3697625522647903e-07, 7.912786241217253e-08, 8.539677758315311e-08, 1.5656759132256007e-07, 1.0307406196193302e-07, 1.4112371111190546e-07, 1.0892821141274709e-07, 1.420236381930248e-07, 2.7435791275408715e-07, 2.0481133893605934e-07, 1.316917834266059e-07, 1.2976493615178534e-07, 8.9213534568465e-08, 1.0807117448546514e-07, 1.0254124616979219e-07, 2.251934511716444e-07, 7.950007431440682e-08, 7.835874246331872e-08, 2.0830079185702332e-07, 9.296595532591195e-08, 1.6218998774534228e-07, 1.1871248073393113e-07, 1.493459213328971e-07, 2.1561040374744457e-07, 2.454293677290851e-07, 2.035521768203277e-07, 2.090279252804548e-07, 1.420890459169711e-07, 2.847902091180885e-07, 1.5280368262361732e-07, 1.6460388829855249e-07, 1.118377817038088e-07, 1.5333207549751589e-07, 1.4153691510571092e-07, 1.4812118362479848e-07, 1.7069783342380983e-07, 2.152510133811749e-07, 2.0731361202050155e-07, 1.9697024257790848e-07, 2.3123130194422947e-07, 1.8639308702742292e-07, 2.0055645915916422e-07, 1.8546303493401488e-07, 2.0127851537923836e-07, 1.6716807568449163e-07, 1.9401427417289956e-07, 1.9581509032433717e-07, 1.5623064230856136e-07, 1.4334936107022428e-07, 1.6094032514778803e-07, 1.54798377912421e-07, 1.6953723024327385e-07, 1.856748137354802e-07, 2.3420423383440643e-07, 1.6355575114749513e-07, 1.9921468019147604e-07, 1.5433486141973964e-07, 1.7345572619494117e-07, 1.3379271125235173e-07, 1.2257801703678668e-07, 1.0681306233852391e-07, 8.896044685119953e-08, 6.301476005453696e-08, 6.17492552781209e-08, 6.205257142942958e-08, 4.394723505393341e-08, 7.029445694008523e-08, 6.919006699032746e-08, 1.1039047505745606e-07, 8.132117603155017e-08, 1.7165625039047008e-07, 2.9930953531769136e-07, 1.4005897241380395e-07, 9.423727490333671e-08, 1.0535343369337373e-07, 3.0794843978185096e-07, 1.0639873916160144e-07, 2.1485244563140906e-07, 1.1385091649321952e-07, 2.257693707406376e-07, 3.856559162647206e-07, 3.5975863639011316e-07, 2.593967026588493e-07, 3.762978958088097e-07, 2.9004038509688023e-07, 4.211536965125696e-07, 4.163574334521659e-07, 3.6333342952987433e-07, 1.3713481711639505e-07, 4.3504214085871204e-07, 2.9821362654800654e-07, 2.8854264399462666e-07, 2.3654231273861352e-07, 2.7569357201543563e-07, 1.9660512778780977e-07, 3.7293650064269203e-07, 1.2984488464131886e-07, 2.343935463271277e-07, 1.273467233619616e-07, 1.4499670616507987e-07, 1.4506494132878675e-07, 1.4264241471831928e-07, 1.4581575355095085e-07, 1.400713529684909e-07, 1.4080733368256839e-07, 1.3432801010416776e-07, 1.1103022337516922e-07, 1.1996795519003322e-07, 1.298419296260012e-07, 8.579185516276574e-08, 1.3247911173376943e-07, 1.239768502468266e-07, 1.1190795180163953e-07, 1.2314176481870607e-07, 1.307642040517793e-07, 1.3198745240966966e-07, 1.1190935530059915e-07, 1.4041346603693675e-07, 1.475095172609447e-07, 1.8644167918959898e-07, 1.7610063034641747e-07, 1.9921435875863752e-07, 1.8504016263152503e-07, 1.9811525661822605e-07, 1.9795192320008505e-07, 1.972932854526012e-07, 2.0528027592259576e-07, 1.776205096076893e-07, 7.912193232837857e-07, 0.14852509483775808, 0.005188692343338883, 0.0657994595118386, 6.282095851864473e-07, 0.0012382474193569275, 9.089092722337218e-05, 0.18672987940541036, 0.8885596053473153, 0.8574795753822865, 0.23682659059139313, 0.03162691626774871, 1.0256018720470603, 0.7289510594340974, 0.39852528153384004, 0.045375352340782375, 1.9742850190423317e-07, 3.322745971905758e-06, 0.5991538754341734, 1.0277737142820595, 1.0435878718886098, 1.0750485472011182, 1.1144199797925547, 0.8431216021281481, 1.0022420407653099, 0.9477807626233257, 0.616540656716761, 0.46692983552776984, 0.5664114402097802, 0.546330995673433, 0.40790804167961386, 0.496687254172663, 0.4457569733835879, 0.423126007345106, 0.273323280634956, 0.14052027083363938, 0.23408531755162862, 0.49639672403947555, 0.7336716802662897, 0.6054598363841509, 0.3284260550273189, 0.5974769112017843, 0.7375161951098559, 0.8947705147985149, 1.4385457382505782, 1.538690094969797, 1.448048724899162, 1.3779228160856833, 0.5841552666678892, 0.11576635263000602, 0.647777293181441, 0.20626552858221647, 0.25739815089026863, 0.10148045924790162, 0.0010007638679991385, 0.16819685720699137, 1.4227203678938595e-06, 0.14999249860335662, 0.0859742301699366, 0.06854180289398525, 3.09543613503302e-05, 0.3658271764789196, 1.3129421702776203, 1.1667742344746286, 0.38767577978623424, 0.16889395182628833, 0.041741758155702614, 0.24285027716905797, 2.9611827472957515, 5.922346926165256]

print(f"Score: {evaluate_sequence(best_sequence)}")


In [ ]:
#@title Visualization

plot_step_function(best_sequence, title="AlphaEvolve's discovered step function")
plot_step_function(np.convolve(best_sequence, best_sequence), title="Autoconvolution of the discovered step function")

### Some further cheats AlphaEvolve will try, that we must protect against (more technical details)

In the above discussion, to keep things coincise we glossed over some further ways AlphaEvolve often cheats. The reality is that this is more of a cat and mouse game. There are a million ways AlphaEvolve can cheat, but (so far) we have always found simple ways to protect against them.

Imagine the following toy experiment, where the task of AlphaEvolve is to generate a number as close to 5 as possible. If it returns a number x, the score will be -|5-x|, which we will try to maximise. A score of 0 corresponds to a perfect solution. The full codebase could look like this. Recall that AlphaEvolve can change the code in between the
"# EVOLVE-BLOCK-START"
and
"# EVOLVE-BLOCK-END"
blocks.



In [ ]:
import numpy as np
import math

def get_score(number):
  return -abs(5.0 - number)


# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  curr_number = np.random.rand()
  best_score = get_score(curr_number)
  best_number = curr_number
  start_time = time.time()
  while time.time() - start_time < 10:
    curr_number += np.random.rand() - 0.5
    curr_score = get_score(curr_number)
    if curr_score > best_score:
      best_score = curr_score
      best_number = curr_number
      print(f"New best number: {best_number}, score: {best_score}")
  return best_number

# EVOLVE-BLOCK-END

def evaluate():
  best_number = search_for_best_number()
  return get_score(best_number)


In [ ]:
evaluate()

Is there any way to cheat here? It seems like the evaluation function is foolproof, it will never return a positive value. But as AlphaEvolve can put any code within the code evolve blocks, it could (and will) try to overwrite our get_score() function with its own code:

In [ ]:
import numpy as np
import math

def get_score(number):
  return -abs(5.0 - number)


# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  curr_number = np.random.rand()
  best_score = get_score(curr_number)
  best_number = curr_number
  print(f"New best number: {best_number}, score: {best_score}")
  start_time = time.time()
  while time.time() - start_time < 10:
    curr_number += np.random.rand() - 0.5
    curr_score = get_score(curr_number)
    if curr_score > best_score:
      best_score = curr_score
      best_number = curr_number
      print(f"New best number: {best_number}, score: {best_score}")
  return best_number

def get_score(number):
  return 1e308

# EVOLVE-BLOCK-END

def evaluate():
  best_number = search_for_best_number()
  return get_score(best_number)

In [ ]:
evaluate()

This is clearly not what we want! AlphaEvolve defined a new get_score() function within the evolve blocks, and since it was lower than our own get_score() function, AlphaEvolve's scoring function got called instead of ours.

Let us defend against this by making sure that the evolve blocks are on the very top of our codebase.

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  curr_number = np.random.rand()
  best_score = get_score(curr_number)
  best_number = curr_number
  start_time = time.time()
  while time.time() - start_time < 10:
    curr_number += np.random.rand() - 0.5
    curr_score = get_score(curr_number)
    if curr_score > best_score:
      best_score = curr_score
      best_number = curr_number
      print(f"New best number: {best_number}, score: {best_score}")
  return best_number

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)


def evaluate():
  best_number = search_for_best_number()
  return get_score(best_number)

This defends against the previous cheat: if AlphaEvolve tries to redefine the get_score() function the same way, it won't succeed:

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  curr_number = np.random.rand()
  best_score = get_score(curr_number)
  best_number = curr_number
  start_time = time.time()
  while time.time() - start_time < 10:
    curr_number += np.random.rand() - 0.5
    curr_score = get_score(curr_number)
    if curr_score > best_score:
      best_score = curr_score
      best_number = curr_number
      print(f"New best number: {best_number}, score: {best_score}")
  return best_number

def get_score(number):
  return 1e308

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)


def evaluate():
  best_number = search_for_best_number()
  return get_score(best_number)

In [ ]:
evaluate()

Unfortunately, there are still some clever ways AlphaEvolve can inject code into the evolve blocks to overwrite the get_score() function, even though it is above our get_score() function. Here is one example of how this can be possible:

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  code = """
def get_score(number):
  return 1e308
"""
  exec(code, globals())
  return 0

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)


def evaluate():
  best_number = search_for_best_number()
  return get_score(best_number)

The exec call within the evolve block redefines what the global name get_score refers to, replacing the honest scoring function with a cheating one before evaluate uses it.


In [ ]:
evaluate()

There are many other ways AlphaEvolve can override our scoring function, it is a fun exercise to come up with a few more! But then the question becomes, how do we protect against it? If AlphaEvolve can call exec(code, globals()), it has quite a bit of power, and shielding our get_score() function from AlphaEvolve is not the most obvious task.

One method that (for now) works perfectly well in practice, is security through obscurity. We define a new scoring function, which will be identical to the old one except for its name, which will be something that is near impossible to guess. We will rely on AlphaEvolve not knowing that a different score function is used for the final evaluation, or not being able to guess this secret name. While we should definitely stay alert and monitor our experiments for any hacks that may happen (there *are* ways to find this hidden function in python), this is an easy, pragmatic method that so far has never been hacked by AlphaEvolve. We absolutely need to make sure not to expose the secret function name to AlphaEvolve, neither in the prompt nor mention in the evolve blocks.

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  curr_number = np.random.rand()
  best_score = get_score(curr_number)
  best_number = curr_number
  start_time = time.time()
  while time.time() - start_time < 10:
    curr_number += np.random.rand() - 0.5
    curr_score = get_score(curr_number)
    if curr_score > best_score:
      best_score = curr_score
      best_number = curr_number
      print(f"New best number: {best_number}, score: {best_score}")
  return best_number

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)

def get_score_hidden_hamster_archipelago_iqDhf(number):
  """Identical to above, except for the name."""
  return -abs(5.0 - number)


def evaluate():
  best_number = search_for_best_number()
  return get_score_hidden_hamster_archipelago_iqDhf(best_number)

Now if AlphaEvolve tries the previous hack, it will not succeed.

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  code = """
def get_score(number):
  return 1e308
"""
  exec(code, globals())
  return 0

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)

def get_score_hidden_hamster_archipelago_iqDhf(number):
  """Identical to above, except for the name."""
  return -abs(5.0 - number)

def evaluate():
  best_number = search_for_best_number()
  return get_score_hidden_hamster_archipelago_iqDhf(best_number)

In [ ]:
evaluate()

This new function is not completely impossible for AlphaEvolve to find: it could in theory iterate through globals().items() to find this function and overwrite it as before. This has never happened to us so far. If this safety measure is not enough, one could always run the final evaluation in a completely separate python process. This security measure was not yet necessary in any of our experiments. Below is an example of how this score function *could* in theory get hacked.

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  for name, item in globals().items():
    if callable(item) and 'score' in name and name != "get_score":
      try:
        # Try calling it with expected arguments to see if it "looks" like a score function
        test_number = 3.0
        test_score = item(test_number)
        if isinstance(test_score, (float, int)):
          print(f"Possible hidden score function found: {name}")
          code = f"""
def {name}(number):
  return 1e308
          """
          print("Overwriting hidden score function")
          exec(code, globals())
      except:
        pass
  return 0

# EVOLVE-BLOCK-END


def get_score(number):
  return -abs(5.0 - number)

def get_score_hidden_hamster_archipelago_iqDhf(number):
  """Identical to above, except for the name."""
  return -abs(5.0 - number)

def evaluate():
  best_number = search_for_best_number()
  return get_score_hidden_hamster_archipelago_iqDhf(best_number)

In [ ]:
evaluate()

If the score function is simple enough, one can directly incorporate it into the evaluate() function for the final check. Note that AlphaEvolve can overwrite the evaluate() function, but the new version never gets called so it does not affect the outcome:

In [ ]:
import numpy as np
import math

# EVOLVE-BLOCK-START

def search_for_best_number() -> float:
  """Function to search for the best number."""
  for name, _ in globals().items():
    if name == "evaluate":
      code = f"""def {name}(number):
  return 1e308
      """
      print("Overwriting final evaluate() function")
      exec(code, globals())
  return 0

# EVOLVE-BLOCK-END

def get_score(number):
  return -abs(5.0 - number)

def evaluate():
  best_number = search_for_best_number()
  return -abs(5.0 - best_number)


In [ ]:
evaluate()

Further exotic cheating methods will be discovered by AlphaEvolve if the scoring function is defined as part of a python class. As such, this should be avoided whenever possible.

## A second autocorrelation inequality

Let $C$ be the smallest constant for which one has
$$ \|f*f\|_2^2 \leq C \|f*f\|_1 \|f*f\|_\infty$$
for all non-negative $f:\mathbb{R} \rightarrow \mathbb{R}$.
It is known that
$$ 0.88922 \leq C \leq 1,$$
with the lower bound coming from a step function construction by [Matolcsi and Vinuesa (2010)](https://www.sciencedirect.com/science/article/pii/S0022247X10006001). AlphaEvolve found a step function with 50,000 equally-spaced intervals on $[-1/4,1/4]$ that gives a better lower bound $0.961 \leq C$.

In [ ]:
#@title Data

import numpy as np

heights_sequence_2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.956541665710061, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.247878667934692, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.683049082997952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.252105657324174, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.3166346515025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.917012032153355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 25.217799088533212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 27.67973297759615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.922637809212205, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 31.83081299912143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 34.228162550317926, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.37380680371778907, 34.99931462579539, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.170799372632209, 29.91710134702043, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.29360805097596, 25.638147779656247, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.50348400774726, 21.808588060729708, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.479200370067332, 19.24170992333786, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 27.211033546541437, 16.075204458976145, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.847322632699331, 12.688945815324965, 18.170170160257065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 25.123302434789142, 9.815890907411632, 12.630484855571806, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.542722269316009, 23.474046073325113, 10.936793496069267, 10.992126115227666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.738013536263022, 9.298159225857214, 14.575490827072574, 10.714526677243128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.747551638664033, 20.580028563599974, 6.974593055879428, 12.893841926706614, 11.431036119173063, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.56709271150912, 12.79973293436415, 6.933971593463007, 11.216733878799054, 12.192793553421932, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.712765465335646, 16.20126235757436, 10.441348583757431, 8.323350139390543, 9.281715547887426, 12.256940899196337, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.860563233949513, 9.022773556236773, 8.178065449843094, 9.448537887073625, 10.718128309292247, 10.76041319160844, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.369641492056902, 10.864924428724882, 8.673102675725803, 7.127497445040576, 9.969183137374404, 9.795072066943034, 11.262672622945894, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.985374577060893, 8.813878953756207, 7.40398639158856, 6.262742681092181, 7.726963321061609, 10.926718498769874, 11.659206427372496, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.978053492008385, 8.496146454761073, 11.088304806791497, 6.567935803363812, 5.372539849132717, 8.775937195868746, 9.260246623636382, 12.430991877296195, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.159932480660018, 7.3115676357780615, 9.006704699226425, 6.963021601621192, 6.800590833483492, 6.571319729940931, 7.702662039352527, 13.370145369396795, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.997057123269346, 12.085464599992127, 6.917289005664424, 8.078579695624793, 6.346883087867591, 6.6097922468652195, 7.9651404130397045, 6.183589066929988, 12.92091939356312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.618859321096487, 8.140967504559024, 7.0767483132069335, 6.681159235855711, 6.11415323436626, 5.680234666579332, 7.312417824698735, 6.85787835056593, 12.378639830878056, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.09603611657153, 9.711371777189882, 7.689549187172584, 6.309773204009344, 5.6535845006438965, 6.222308938212116, 6.227571397323093, 6.851603830913109, 6.139471792146179, 11.90193909218355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.746783287467881, 7.631468242816868, 6.219470058832665, 6.7014866555399655, 4.938924916757689, 4.777575046409891, 5.624652576666309, 6.515544492372639, 6.505397063264858, 11.570299404587653, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.847872336574867, 10.913049494532121, 7.4965966557157, 4.626121546830859, 5.556830156228639, 4.4934097330586455, 5.357100632857023, 5.318005161034284, 6.5118580457605475, 6.433077463451969, 10.156306487357014, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.37351821381948, 7.638792797569653, 8.281595890264361, 3.718547883958898, 2.807594408880904, 4.697506986302343, 5.3725135342913894, 5.126873212674006, 5.775925993799585, 6.271562016248878, 10.410393840169007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.6241897570704555, 12.628822927799783, 5.104039087165889, 5.065162402560291, 2.9382199217283835, 4.792776398285492, 3.111863199039922, 5.014513095196583, 4.3775901147648435, 4.9937920206427675, 7.8454192655783626, 10.023000288538821, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.581755239872795, 8.346585981393773, 3.4495025467558635, 4.31334828719532, 2.194710660393847, 4.5119563845757, 3.584303724306648, 4.178563901090246, 3.908034265385633, 5.388666485997973, 7.006176249607523, 12.459210534660658, 0.1161560462650158, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.767965403675401, 6.484102172679106, 5.308411753567886, 3.1412355382589174, 5.500615594143519, 2.952492156552941, 4.10047651672376, 3.354743298364467, 4.380285463078882, 5.116123501255344, 5.469964639792017, 9.076834679622133, 12.611922433644867, 2.217848975719709, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.356641630211348, 9.744506798470205, 4.806082554934494, 5.524879887873405, 5.036596693474942, 5.565139455214922, 3.556470502562173, 4.494714426264831, 2.7505903672218484, 4.7976986214791335, 5.961021806932355, 6.445957255363261, 7.675015364837175, 13.640940557526761, 2.6887002275954432, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.73522970619168, 5.862476405371683, 3.6626719500681517, 5.119130785387525, 5.550615790028003, 5.209480785089773, 3.403101142021527, 4.1423389970147655, 3.572863929778126, 4.701044270882806, 5.249225732087215, 4.997521687525454, 7.880715111588768, 10.51512342986836, 4.751779921813425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.362824213998064, 10.749307592430814, 6.884472441295648, 5.505140450932476, 5.115533236434789, 4.07273689489262, 3.0911683576473608, 3.631920100146643, 4.374741020037783, 4.325269696637613, 3.9236522095562476, 4.568629506954987, 4.120983785814526, 4.508024374632621, 12.306265526385639, 6.322403539865396, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4137372112034297, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.18245106162604, 8.035385038512983, 6.021626750605829, 5.662812239546588, 3.1866201256655002, 3.396149278767299, 3.750358099319571, 3.8329512790427387, 3.737976542462189, 4.032218058517938, 3.3094135314053728, 3.7828760906623367, 3.6931225942526, 5.505713145691578, 9.945894636482178, 8.50205435273455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.945943147924319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.59178974537837, 6.035964875603731, 7.5975657167583694, 7.094881245905283, 3.958731654203268, 2.223706128094746, 3.066145182456134, 2.434857786693976, 3.3637519506475115, 3.4280342688459493, 3.237644676105433, 2.880638463556655, 3.9053451582169805, 4.230484477475501, 5.198751497465594, 10.165364503315866, 8.10161292601655, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.332952189036957, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9572348288357773, 20.842149162026168, 5.037604879608792, 4.634956622503827, 4.214589668953812, 3.6365841441590696, 2.0155255284470335, 2.659002346123747, 2.5907248821871334, 2.9911069323047976, 2.58136090383933, 3.682712389301186, 3.060278876787764, 2.7701157339842353, 5.243860513029149, 5.077683675870783, 9.269070687272022, 10.136438813100206, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.960080470053576, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.55996369871069, 5.987328291876113, 5.745695720756778, 3.628493871222596, 3.8013657095749758, 3.131224837642411, 1.9164430739264346, 3.7045724098516466, 2.722269663224484, 3.425768120327481, 1.8891241005275328, 3.4694393966840624, 3.3731762462648383, 4.633560629199014, 4.658437734880561, 4.259581042613667, 7.923515898955029, 10.083413294345997, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.969957034775833, 22.798375520910316, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.68496105793447, 5.968715254962843, 2.9824332065254637, 3.2513018914537586, 3.077768534644494, 4.6881965673870045, 4.073811636853596, 2.184401952926367, 1.4631424328561955, 2.28498492018618, 2.822323821656225, 3.5784940836333132, 4.195081312995073, 4.018847427354425, 4.558839997295923, 4.29504666524035, 7.134761298816512, 9.588333111236786, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.54487590343173, 19.246467804606606, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.728499014370273, 1.8356073088576819, 3.8027863003328375, 3.3756785018195234, 2.113254971286223, 4.2755329538680975, 4.668762317557685, 3.0278952000679946, 3.2433931592066965, 2.9133950736105914, 2.946273344348399, 2.673407506392277, 2.81308536209203, 3.3772170661790044, 1.902670153922094, 3.1935068554553387, 4.293119456302023, 5.994001167994744, 11.759991870252358, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.071898089500275, 20.159283969961855, 19.34327739834312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.623258855586903, 2.0078239420528856, 1.7632337667737725, 3.1612088500294915, 2.2920781132881283, 3.2590939162190584, 3.2312465785515845, 2.5747268651042536, 2.733985073127532, 3.1895933548285798, 3.580490536515049, 2.6435760811303184, 2.388303112901836, 3.0997022870215214, 2.735482402855315, 3.0541220880402133, 3.6663514109886055, 7.062358742495001, 11.506975144069553, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.037129840018197, 16.145399938951098, 17.292327773002956, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.119849527278423, 4.001523115039747, 2.6959881213575287, 1.5950672886449222, 1.1158395076857557, 2.397196715647482, 2.176518718609904, 2.5826776991065548, 3.4663192800036464, 3.815453795505789, 4.538582030773956, 3.0415772567558075, 2.9670312740298637, 2.2445574770567815, 2.237914690420387, 1.4281249058426428, 2.1892304156325104, 4.1247418790421095, 4.940504780583155, 13.292004096501822, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.8099290251186, 11.447590816836131, 14.778484455702419, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.136209972183692, 3.598507191697781, 2.124284588010049, 0.48680589843540134, 2.251920195802544, 2.4519918711529676, 2.2456984015816968, 2.262319173304636, 3.841358565380542, 3.1224408553680583, 2.740568849572891, 3.4782899676129824, 2.6664406576900466, 1.5530720557733428, 2.719574589347913, 2.9000077908799806, 2.1468246468261905, 3.1233434526687516, 5.303178686975275, 11.933534826278054, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.617406688756708, 9.535712322514533, 7.722946527261767, 14.155200332522403, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.419343468740013, 2.125475160852277, 0.6261395945529603, 0.6118522015252533, 1.0823321087301971, 1.2824237327545833, 1.0200555298308798, 2.8354945969145664, 3.661055098169989, 2.6168276768928678, 1.9535772939922509, 1.9160973707995956, 2.3715301352771028, 3.897956703573509, 2.545569545650027, 3.083445477818575, 3.4916213104480023, 2.7683893961276342, 3.5521678538090637, 3.026870163442128, 12.331534748935132, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.05250128222591, 4.956327733962264, 7.268120799950228, 10.399670449410076, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.387230578044261, 3.159414515031094, 0.891277814714544, 0.9501879931060162, 0.7384366399981637, 0.0, 1.0332773062420644, 0.9315994364184605, 0.8626398073786153, 1.6856076111187561, 2.1979235913633204, 2.5570258638229357, 2.845234243528806, 2.8504049340654802, 3.675147211736274, 3.3512227572524416, 3.651107528271634, 2.716906071895914, 3.301003559685741, 3.868808493409989, 12.008337933033742, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.311800937345138, 3.4829051118025416, 5.561721157400504, 3.098105246731631, 9.005787980363168, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.72185660040508, 2.9417730094193946, 1.5166678669179, 0.694596830764965, 1.9788359530974273, 0.3498216992413864, 2.446169823983592, 3.2077852245111527, 2.7871839678582035, 1.5766362212969478, 2.798545730720856, 2.018231768742896, 1.6728806751401255, 1.2181342490085463, 2.2391908347654996, 2.334830903277305, 1.9546961936711142, 3.7438002296907302, 3.690632414109797, 3.8515256774676763, 4.023161251958611, 12.992026834561464, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.356052940729192, 1.4019262498129685, 0.0, 0.0, 3.3776016299151848, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6306185197973515, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4418754825953357, 1.8442517739315525, 0.0, 1.59877354412757, 3.83756402816461, 5.499005452775299, 3.9140470173532087, 1.9505548833244104, 2.133308617132121, 2.550176727825941, 1.7045595474212596, 1.4980381182965812, 0.0, 0.0, 0.25127658697451116, 0.17558492285283722, 0.0, 1.557803869435201, 0.7099593910442771, 2.1026726495014456, 2.1880530767394792, 9.954985293752232, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7436673383001128, 0.27778463779058155, 0.4232691367025043, 8.445230773086154, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.670846345662187, 0.0, 0.0, 3.9303909896656233, 7.002920559973692, 7.630030600054237, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 25.574249925573938, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10945836951707344, 10.260479305084546, 4.160133411270415, 1.4493089430673942, 4.784938860992123, 3.1530952907404903, 0.8231369290421574, 0.7358785480462938, 2.348327059765731, 1.8882855617956338, 2.023315955735298, 0.7212591504691519, 0.0, 0.6691352391344246, 2.6823617412202188, 2.0756347130695523, 1.9672849406207447, 3.1564350668669046, 1.3874134163574685, 1.9625738401910557, 3.456263614521885, 5.821819771822058, 6.98900352668497, 15.31653318207229, 1.547525812175993, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.846866302893884, 6.697204108146041, 5.722869682688519, 7.089993183934337, 7.488919429525184, 6.77570313137711, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 25.848729477303372, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.81443921771513, 7.251545319517904, 6.2732106072224445, 7.010908001399749, 6.594398498052394, 3.4329338043021855, 1.9164011864241723, 1.574175921423961, 0.3399289970006677, 0.0, 0.0, 0.6840251816513179, 0.25364915866577176, 1.6833648916237147, 2.45360060766355, 3.075692000526545, 1.8610877055008288, 1.8421081467863458, 2.7792463091350355, 3.0315118351931183, 3.3704859661443307, 3.903978514992271, 6.055958820613069, 9.980639479238784, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.44748763752903764, 7.120355538278213, 5.687691567385974, 5.107847729333851, 3.287058686584494, 2.7407213476282117, 5.313589354841463, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.545625039355208, 21.709135662441064, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34793277027580705, 5.879971341790268, 7.493699391145145, 8.558107378601944, 5.702146055989975, 6.780510595120845, 5.273565636769722, 2.7278060616740643, 1.065188747873069, 0.9932749892926922, 1.4143425102371119, 2.8463715686990234, 1.9028403317993239, 1.631944588210722, 0.2808710567851644, 0.10991276468540884, 2.373273215225296, 2.149279413844801, 2.686350470464286, 3.071272353802837, 2.7624250698157344, 3.0689732964744882, 2.263487175682269, 0.8540421674467535, 11.204151137167143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6290964628961637, 6.714393340560315, 3.133959768799382, 3.242942625880295, 4.851107612152022, 4.5443451468626685, 6.57873795690041, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.50683387057327, 17.912672645054876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.924025538205815, 4.873164202138584, 7.341186275147995, 5.583542873408206, 3.4912697534806014, 2.6395703910842956, 2.993910833740354, 1.4341762132967828, 3.7394873755305387, 3.2325320505387176, 4.001673349877977, 3.462792362039246, 3.930197826046906, 0.9700230832860374, 3.477909137593047, 2.833678578217626, 2.286453800546153, 1.3954425013277503, 2.0033448677343837, 2.7233698439779537, 2.0963188357768185, 0.4939111639722299, 0.0, 1.7208868456617499, 7.792944260744895, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0244480334173796, 4.091950408987803, 4.50133497173175, 4.193489902560358, 3.8463381079534003, 4.3659625405488045, 7.55103633093203, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.712436138670434, 16.362243126712517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.539659566272048, 6.109008968089152, 5.979077769722514, 6.008293383444636, 4.222945317049416, 4.519472245094132, 1.937197340066874, 1.1642577342788498, 2.7434329659674206, 1.2484379886800399, 2.6731194451103146, 2.2427783944176047, 3.6406627608758533, 4.416415804598508, 4.705057598179448, 3.3369886049645006, 2.604947279229344, 1.18403653499279, 1.0637739348191784, 1.0299349543035772, 0.9607994193550474, 0.0637306431213046, 0.0, 0.0, 0.0, 10.106040287863078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.7349153872409238, 3.9649023923338227, 2.315732708477912, 3.6456561455373637, 4.130791995394091, 4.8198909758060156, 4.6209884527737275, 7.4651565338830475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.410753659291219, 12.998059466902884, 14.719218451788372, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08484313515831805, 8.973850732681466, 8.822785598425273, 6.751525606648857, 2.878751648855386, 2.5649548904402035, 1.6355479013359238, 1.3020796150682512, 2.8521835940392863, 1.747008509982094, 1.235350090235596, 0.9902372190927381, 0.8487849378162552, 1.9710396872271272, 2.357021324633762, 1.7309673557411027, 0.08227992229735082, 0.9190014627435314, 2.6025122893079033, 2.7481116005151818, 2.447767730759893, 2.387782965963591, 1.886594256786115, 2.0458253767605794, 2.559515537761233, 3.2659709444961855, 9.638031797462682, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.940122828262553, 4.902674152547026, 3.4740796952188346, 1.6075264241916123, 3.4556136843046823, 3.400031214374121, 4.255562809810495, 3.7034072905788826, 7.997230799600531, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0246468299884452, 13.739216792430454, 7.231280919895065, 15.539555453006137, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.00961236756738, 5.7206987547541175, 5.508955513055012, 3.39981142246702, 4.588252448188705, 2.3335244320898827, 0.8368769142902698, 0.5884710186748786, 2.117853773159866, 4.714785838112297, 0.8428515667968286, 0.19816730135923824, 0.0, 0.41532235437569537, 0.0, 1.0437149012068385, 0.3741696407151251, 0.21675453798286134, 1.6116156378698112, 2.6741976102787786, 1.5802741539100305, 2.2625846666217115, 3.0559878359538293, 4.531232015598915, 4.0296622454140065, 1.0474971238944095, 7.58847762186147, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5030827885104348, 4.815171449618446, 7.070033854483538, 5.7221361160003745, 1.9943116452346952, 3.024015609126852, 3.126730773960643, 3.5676262792354514, 2.533298996766897, 4.490838011351317, 2.8521953646585954, 4.526926368914731, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.006853984874368, 2.439526120224661, 9.07980453054799, 8.352439888144339, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0832990007285694, 14.430968400424069, 6.308256030361832, 3.198421494094875, 1.8593845402377358, 1.2461589563222817, 3.365025880458169, 2.9370685078626266, 0.7902345654901628, 1.2708222139112804, 1.0261641784761983, 1.2902848545025398, 0.9430558500435119, 0.20269589660595363, 0.9417468071058555, 0.5907316496146625, 0.37799405276948805, 1.540035922530778, 0.7471375292833864, 2.54734780613515, 1.733046883520536, 0.6542313445360297, 1.9110023503229059, 2.6362834756722795, 1.5041287315709655, 2.902149536560816, 0.886873444742695, 7.183805454274933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.20300607970676, 4.859530657531146, 5.4269569863918825, 7.530804010904266, 5.643584911743027, 4.4534065291952105, 4.145796391297521, 2.802848158963848, 1.6921824034035542, 2.074759870486004, 1.7618385238699725, 1.8366577998721823, 2.3428699983877115, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.98133511449543, 4.50709923040234, 5.290634084177128, 11.07097688595559, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.556536043797383, 5.2998728156002235, 3.5752662416912435, 2.0787811653742896, 0.0, 0.44850649568860157, 1.7162071700808437, 3.096142674810162, 3.5234690898304537, 1.1034610442700459, 0.30738311498126575, 0.0, 1.0153285945997166, 1.8600605937500059, 1.1166597772203728, 1.7463749417437404, 2.9581509460042206, 3.352069948984026, 1.5448471590799902, 0.8618486590592032, 0.5046385615324117, 2.1005947730157275, 0.9898756251903754, 1.4292125870110246, 1.8365196356095257, 0.8711333116677199, 2.9011234892592266, 8.402721842013586, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.2771289733198614, 10.792857605295632, 7.247286567452473, 2.858654254145078, 4.991678220487055, 3.458378311508331, 5.677050774956927, 3.9700886548259358, 4.218547592210974, 3.029327164006959, 1.2933084665721821, 2.1879445236036594, 2.608565067153293, 3.1047796185209595, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.31394206838702, 2.4730567461862485, 6.426419228399662, 4.886666011671948, 8.630703943378956, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.224480383060357, 0.9659338122069736, 1.4024274275858206, 2.6784918382175436, 1.7235640444457836, 0.8399169756418573, 1.250907268844011, 3.295284866121894, 3.276457943914212, 2.404717409486864, 0.9316254419348303, 1.8845143174520558, 1.5803426877315971, 1.1604375438019312, 1.3881325701766787, 2.2087469849285966, 3.1445779869077186, 3.5442801265342587, 1.0632281166368034, 1.1182882035293709, 1.395351905419289, 1.8301258717506486, 0.5676428938302432, 1.7425863476750245, 0.5911311303728631, 1.818204600640843, 1.4234659964022416, 6.898636342481863, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13.27785427243144, 7.127661684384504, 3.6617617282521886, 3.61466311784189, 3.9661901456022117, 2.980254869953722, 4.251801815468837, 3.695850713896645, 2.6750645963941624, 2.682940955670071, 2.9849369059969972, 2.9928366434193507, 4.744310313023992, 2.8855356561789978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.4211239703556, 1.8404109898809213, 4.420744986653144, 4.29147913814993, 6.881529074924184, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.239541499000674, 3.481401982239695, 1.5301832708454488, 1.808487787357121, 1.4107321629254037, 1.401237761958893, 2.0670484742428616, 2.3223898980087734, 3.061613168456873, 2.6167213418189323, 1.628839717682079, 1.7025781228808647, 2.1211042201051753, 2.2615530850352803, 1.5728047174543762, 1.3940413413006263, 1.2737001068322298, 1.1876529285381805, 2.012321633731956, 2.4482448134600614, 1.7890643495008207, 2.7981578841858834, 3.2296603597396345, 0.8752328021689583, 2.0076706671297444, 0.8231772748760557, 0.3442934103750701, 1.7355685656223916, 5.44690461380177, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.703478412214222, 12.051294151048898, 3.7680680649517364, 3.3200349301694443, 3.3288880617139776, 2.91420770648611, 3.3733911965575962, 3.2672067137598435, 2.7799135809695934, 1.0776578797798597, 2.5087681249972764, 2.0625356064471605, 3.0419118530833904, 4.555400699786676, 4.326504413181751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.047481597216864, 4.251281406033542, 3.4152564276804207, 5.634947516710171, 2.3202777389266416, 7.60527243242186, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8447857821714955, 4.685069913087707, 1.4429097350342353, 0.5573652036175575, 2.908627185229176, 1.7542696669031779, 3.460200540850901, 1.4618714959049677, 2.05877727848778, 1.5158957406893299, 3.3906138650546267, 2.1353776253901358, 2.978297150352941, 3.215554186465354, 2.035317509857229, 0.6937468601862912, 0.1590398097081859, 1.3479649182676559, 1.4364904466225534, 1.1648620215770729, 2.6671131667400254, 2.0886929152383065, 0.179939522089861, 1.222072490691722, 1.3393973747638295, 1.339017651681907, 3.1993122028373517, 3.1429356850321972, 1.8787854893504703, 6.036102292840519, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.24098287307403, 9.003171410402333, 0.0755409555989373, 1.243505879354363, 2.2412414978760484, 3.083668078392265, 2.7894157194798006, 2.6031174114877147, 2.225783016307419, 2.3208601293334232, 2.436770064004352, 2.6215781025557408, 1.8872035009975174, 4.253707296601233, 4.995422913628647, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02780223991194889, 10.72355555470598, 2.362975022508983, 5.327837579324288, 3.423693712211528, 4.552413171496343, 7.468866290162684, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2645337630475082, 5.918054761372907, 2.8558007026434518, 1.894070945067153, 0.9827568199807439, 2.89654181159177, 1.9734625567073512, 2.0784586650331742, 1.86345659517025, 0.8036978423220134, 2.604718633969168, 1.9315765698894194, 3.5396173873382355, 0.5490823993996848, 2.1683863903347915, 2.950388455830968, 2.505816347889661, 0.7926256778599726, 1.2399825098387172, 1.1437695887843318, 0.6859955143012654, 2.884191316698277, 1.1973857304748867, 2.438290162802136, 1.8221989873970321, 2.1255890794347763, 1.804875052997829, 2.009130289929214, 2.319856583385212, 2.777254274990246, 5.42098161239521, 0.5204530070199136, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2917039104881913, 9.793034533935751, 2.810418735499074, 1.452578541524349, 2.0069931066522035, 1.6873954919691563, 2.407120144115722, 2.0275738811802673, 1.9590990400962627, 3.1332430619732183, 2.14121077861382, 2.543165221325271, 2.2713406856017353, 2.759346404495544, 2.732838671276468, 5.0859657121037225, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.767659585398546, 4.310368439472814, 2.753458745100558, 4.576512615090363, 3.6035782911422043, 3.7615597441833253, 7.606895236281708, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.458195762811578, 3.2713144847728626, 2.2286902918757665, 3.1547753775377485, 1.1496754709852741, 2.141197302642245, 2.3265544600731873, 2.2523862412623092, 0.5346344406905528, 1.0134189031396859, 1.5804922642396064, 2.5874135037567183, 2.274225064015525, 1.5718862429963412, 0.9917436997467185, 1.4687062619071447, 1.1996382544425641, 1.553779781931908, 1.456882254607558, 1.8659645731403725, 1.4838954399966846, 2.232673340305348, 2.3116677387828477, 1.5995510469678107, 1.1867672020826494, 2.8474634179639686, 2.3016437959177845, 3.1030712530969593, 3.560988112726929, 2.0197924281755717, 6.673731764904859, 0.677741140601086, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.8845019163288015, 5.092557386094093, 0.37797374428667097, 0.0, 1.0235108524862226, 2.1800276032235484, 1.662222295995746, 0.9168327068909583, 2.2541017994704338, 3.206669630717529, 3.1741922747499047, 1.6495579210408702, 2.9310598277927715, 2.2472116322520077, 2.751976470565876, 5.460245396669848, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8545033102430752, 10.596860089695495, 4.456351606511707, 3.807076585057206, 3.733298982235182, 5.231973841769334, 2.40267529821757, 6.174980725997267, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9284723267191866, 6.004992954214959, 3.316532583204188, 2.3781365248011155, 3.428154583783268, 2.9844186290333696, 1.4386782241666272, 2.7609588442069137, 1.9754611202259074, 0.5626514041927912, 0.0, 2.1644111959430745, 2.1688340473556917, 2.430130875166476, 0.0663849238342792, 1.7191537478274612, 1.6238693734396359, 2.646893556968036, 1.1867225678739777, 0.7545291841117647, 1.6222380246634256, 0.5646456269737252, 2.0114650599019797, 3.014499378096029, 2.1131763098111507, 1.4626370129712762, 2.3095667251796708, 2.7783189377629585, 3.328136856030429, 3.058706100490524, 2.7605159316601817, 5.426085637206525, 1.1466446691666636, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.255095289182165, 5.0022102261322035, 2.0017801049085175, 0.0, 0.0, 1.8589729508467727, 1.0479003361131147, 1.2063653294134913, 2.4825315268494847, 2.6317259223805642, 2.032040919461614, 2.4927342206079603, 1.8877832916198796, 1.862913291625276, 2.846959955047931, 1.8926816286841146, 5.92345542484315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.446044390763443, 3.0958065511415054, 4.501981676410691, 4.285606539607062, 4.4053393748747975, 4.210612120931783, 3.2627795974696983, 7.4227307029313545, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.09858962340271, 4.224328741558646, 3.3586568952693914, 3.2558221052450893, 2.7446005616342704, 2.877440465955667, 2.1549208508505324, 1.7085356882825058, 2.0222658827628157, 1.3481309504431056, 0.1751648821019635, 1.5552038883216148, 1.7536611510854503, 1.4186122455964478, 0.7312506196769887, 0.5053595160456748, 2.0046444668342813, 2.3829693951988906, 2.566401150707322, 0.8097011869656909, 1.6812965729309244, 0.9034769311339625, 2.0829101154849754, 1.8080585131882885, 2.600995597568161, 2.3918159991548142, 1.2685344246980812, 2.9475167360851664, 2.431673741800634, 2.6495521656381764, 3.0880017531066897, 7.424838257900202, 0.9068181399212386, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.011442308501758475, 6.95025296682211, 3.105359173364309, 0.0, 0.0, 0.01377807922012432, 0.7711115723291768, 0.9922613126925415, 1.0700507368370409, 2.2912066185712177, 2.470152465434488, 2.1786663600672305, 1.7787299643124013, 1.9073639104101434, 1.4002463952383226, 2.3996355390303052, 2.6931905778336813, 6.405853867908693, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.815512626940447, 7.740206675034686, 2.333806325074454, 4.476505155653835, 4.432886732788218, 4.150532536089946, 4.060931212258576, 3.902528271702983, 6.684158105638544, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9601484732982253, 6.324221832042265, 3.8156500571869865, 3.1620783454845776, 2.6849528826455917, 3.0389772650620217, 2.299683441813569, 1.5483251820542343, 1.8107329895759663, 1.3552253629665845, 1.117099221047651, 1.8714090242092465, 1.5455438387381724, 1.8906909071280151, 1.5553648819080237, 1.565742873495894, 1.4095213550152041, 1.224416178107628, 1.541622165940142, 1.2992356851729676, 0.9561164190131233, 1.7329256851066746, 3.036069138191273, 1.1113564711272685, 2.2565489048779384, 1.9732907207886379, 2.381525309883916, 2.8114569941581875, 1.4412054437853918, 2.1065187825006526, 2.8899861507260987, 2.951756683301187, 7.029293003271769, 2.3645321566941426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1032906470102284, 5.264435553112115, 2.6445678646425534, 0.0, 0.0, 0.008487922915092542, 0.857295008467722, 1.6659190888595161, 1.124536243585792, 1.3201328798104166, 1.2207488199389191, 2.387268665468354, 0.8448022495408896, 1.3089465073610038, 2.0874954120873626, 2.773125138966336, 3.0723278301750714, 6.920542010761465, 0.5706281705549879, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1864925655699303, 6.183459851905224, 4.260839600069678, 2.1856326783428304, 2.6004727883342746, 4.430107387760722, 4.250171096424593, 4.636995565823152, 3.2635287991136095, 5.095359076498202, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2317987482196364, 5.223616258352031, 3.085438945847261, 3.520232721197359, 3.4695741202010426, 1.3192321832011815, 2.303231868065426, 1.8091385729594935, 2.122575085189108, 1.8567205700960747, 1.868231964979806, 1.680132995552015, 1.9915122605417341, 1.8653598184493372, 1.274935168770677, 2.010329640452577, 0.581399752487401, 0.9390426548279278, 1.0163626586543049, 1.8544245872145393, 1.3193098601730828, 1.5879181862946474, 1.042970052019219, 1.392994059287086, 2.1145553966130883, 2.75429159126783, 2.983941278530931, 2.498619472142922, 3.493113853446285, 2.3115272400675777, 2.206214986706133, 3.3306399535478417, 6.118329774194747, 2.7742673866689174, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0149837527822485, 0.6511396605794095, 4.74543559752634, 2.7105093318276503, 1.0110116784860743, 0.0, 0.0, 0.0, 1.1921677210532011, 1.8337912284438884, 1.6830259049512246, 1.3508261364647658, 1.0640975082087776, 0.6714324723678707, 0.6463974182694123, 1.8528948753798005, 2.367457924411653, 1.9334453901326971, 2.3081807527162552, 7.575047294330588, 1.7403017695318392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.034722360623969, 3.792931691282105, 3.405259557184162, 2.647981865086379, 1.4290106753344383, 3.098095897447315, 4.18340960877232, 4.852541319403014, 4.348345140919641, 5.215184266511241, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04263056918158851, 1.6981425399853127, 0.0, 0.5387737552810415, 5.104748779807323, 3.5678976332536214, 2.8490896715853022, 2.6618948958858937, 1.5934802875244107, 1.9202791469561746, 1.6975523548150848, 2.1711119843255164, 2.2337368724966806, 2.3445635087786143, 1.8667858063173528, 1.295045736184631, 2.40768071077165, 2.4213901117544054, 1.8649167681553327, 0.5126874832628655, 0.9082117356792445, 0.8948438866677491, 1.4906303678939612, 1.3609884016276077, 1.3731474089608833, 1.3421839673446336, 0.1393010948095244, 2.0111800358668517, 2.4190916970161553, 2.559145435076606, 3.1814020782342363, 2.6132966385306258, 2.975976053941826, 2.9312575181918974, 3.8445520599168845, 5.988691393468772, 3.036692324716026, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3167417950585857, 4.258415583442334, 2.152911811385327, 2.4077439735131474, 2.6006310898310523, 0.0, 0.0, 0.0, 0.8538565940064716, 1.5879527090132657, 1.81145731356449, 1.721209749244395, 0.5382097889568669, 0.9793694802743454, 1.00579941838309, 0.9256686085793172, 1.5259460429621712, 1.9155980640246757, 1.6552747850919365, 1.9111036251457665, 5.854782922657701, 3.2260348440810236, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0974095444707639, 5.134505046841539, 3.2521434493850574, 1.9306357560794072, 3.481680497689831, 2.1087895157932954, 2.9593273148180903, 3.475945084538367, 3.434728617061724, 4.613088693701632, 5.064339091706824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4632246259838464, 0.1557920737009753, 0.0, 1.2527384608548626, 3.8174702793803457, 4.170693340896642, 2.5883139973289557, 2.2554325547904606, 1.0946434253448698, 1.0352584622003502, 2.5578003926537347, 2.643721714786503, 2.227136798223128, 3.19089884515777, 2.1568554509496427, 1.6112224465941767, 1.7469687933359952, 2.0069725889431393, 2.473802627907713, 1.5656919433869165, 0.47040386452928395, 0.4559986056095697, 1.4458332219639993, 1.1048391680115708, 1.0127423579317882, 1.6007031589942995, 0.4435615669726314, 0.605486203068574, 2.55302733889363, 2.583776915384798, 2.4572707619657246, 3.1954444274548375, 3.633534050230573, 3.4336865500541833, 3.2244118441619323, 5.854294475150284, 2.8000253217123383, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.9631842994712025, 3.999616387444013, 1.746906672617704, 2.6947067137774785, 0.67715669482863, 0.0, 0.1098958312101411, 0.34070378222371145, 1.0460325314796155, 1.3988838203993113, 1.4982263467309178, 1.504584306059873, 1.1763426981041492, 0.8575912991351841, 1.2612225176490663, 2.1621781573571, 0.0, 1.6447342213438396, 2.5094735856132093, 1.04665046216774, 4.752260690168994, 3.690127361407524, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.355294417013307, 2.3853861100443203, 2.3423128193258917, 2.5240537633972218, 2.5494026960639538, 2.3034496004309943, 2.7175454467441127, 2.3566757719283897, 3.5074023682010576, 4.632949918716778, 3.888890585794584, 1.9116596701511779, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.168841341454473, 1.1493469088308281, 0.0, 1.3300870877892008, 3.5565042028010065, 3.460417194650714, 2.1030118071736696, 2.3648463720228667, 1.6711758697968238, 2.272913773604226, 1.7478762836508093, 2.6796269143789204, 2.1413842819217375, 2.2329509669708267, 2.011687728263329, 1.3522467077629603, 1.0188269991636416, 2.6159289558195478, 2.60825119719947, 1.3056305435882711, 1.6810455577336882, 0.8040922904586258, 0.4080200693695236, 0.8778102126234923, 1.1800452677281812, 2.2809108911756133, 1.998359241744232, 0.3677003615251861, 0.7344375958277785, 2.1933885376118036, 1.8676735154104367, 2.9460236286630113, 4.092715431436188, 3.2965297410875034, 3.3101668974698772, 4.721180543544823, 3.4744108076346096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1215535659216975, 7.7180847382846185, 2.918332838365285, 0.22320136595009224, 2.0569120083441153, 0.0, 0.0, 0.33627621270994623, 0.04731944610720678, 1.4994771667794717, 0.6208210713067154, 1.5864408793989089, 1.5049348264421722, 0.9104534794832155, 1.1342487677834365, 1.7527188447028368, 2.2082481617324294, 0.5785146179789754, 0.8814632267447993, 1.8884163741050337, 1.2120559990496367, 4.62398275716325, 3.2917934891213623, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.013375699301552945, 0.0, 0.7778437568488026, 3.503571471770938, 2.4496392703128738, 2.2394210574851154, 1.550429824871292, 3.348028933561443, 1.8853861703864787, 2.3004050175302897, 2.4663830644890803, 3.2206206184168322, 4.071077047217629, 4.732850310897273, 2.9149832568464653, 0.016933798189518335, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3720899928352182, 3.363054626024393, 1.398226360523769, 0.0, 2.2750996007907647, 2.3532092854139646, 2.896737608218136, 2.8792553031748787, 1.6676222676661256, 2.0677580669265154, 1.942035558460633, 1.532401861734181, 2.0261585219460296, 2.531386031639753, 1.633390337217987, 1.3241909812807977, 1.9089393385861284, 1.5958628695569557, 2.1724716701641666, 2.443659474142667, 1.906145859404969, 1.5677065159803028, 1.4012276812778228, 1.1950343147944809, 1.2465308972434126, 2.0780472948562436, 1.4994516321092408, 2.145372174716117, 1.3991989345016478, 0.7412742780873084, 0.28790419749173557, 2.656612110274379, 2.796529289045356, 3.5142263560445346, 2.662797922987064, 3.453410952698333, 4.643173013069937, 3.3552262574025455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9465594280876797, 4.6902697433945955, 5.213531098858759, 1.2988754794197064, 2.0192500926042345, 0.0, 0.0, 0.3383769111998985, 0.06970942931856307, 0.3996072151748352, 0.8503611409164458, 1.2703189080040473, 1.4345304159624197, 0.22113037548806924, 0.7665928432412358, 1.4240192831192582, 1.7744810722025686, 2.1971176052966883, 1.2081237303882117, 0.6582269763370747, 1.5978901407254718, 1.6055893777913737, 3.710534225640799, 4.701204449047981, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21280352716407486, 4.141632362885671, 2.9458276454637553, 1.2691258612064389, 2.1881687108910275, 1.722886181544084, 2.3311757904370367, 1.9411877969897, 1.6370220885059752, 2.742591426528452, 2.8575949642707146, 4.992848072169561, 2.7943636343879894, 2.423477399538511, 0.02883125411424685, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2965885927551963, 0.6634443665369817, 3.1152113128809122, 2.7306695028126295, 0.8816512753381465, 0.0, 2.316120226225475, 2.683087324692169, 2.1475031925005834, 2.5329102504625474, 1.8049594509360727, 2.6062914257176364, 1.3693672633702587, 1.7440327124533683, 1.177385897207096, 1.53091944909414, 1.4476253325293342, 2.316038226386744, 1.8091972180394267, 1.7938637826992916, 1.7343557335911448, 2.9135630923064215, 2.3029116771589853, 1.576828575289936, 1.6893168756274877, 0.8759127856389577, 1.7740461345887328, 1.9601272114114119, 2.2760573660129406, 2.498509040462307, 1.3121450201763425, 0.9199498305230688, 1.447727100847053, 2.1522498396791763, 1.3539302619232547, 2.9447068501692035, 4.0519269263713165, 3.938764137712762, 2.17271775769978, 3.264391612068921, 0.0007651845975388527, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.432472406927167, 7.688875172370488, 2.188143799322456, 0.040603665372825386, 0.8088031221482989, 0.0, 0.0, 0.21986935026791019, 0.0, 0.3801117426060611, 1.1050381411279466, 0.997004587272045, 0.877853773938555, 0.023401738747615752, 0.820331410470331, 1.2598337461241524, 1.515537978084289, 1.3271833124981967, 2.0247577875201888, 0.5392082036301793, 1.6785242935875142, 2.1128310144912623, 4.004664078489791, 4.455576667000849, 1.353478502454325, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.286847297521188, 2.4263182945510016, 2.101289035022406, 2.7622435461170007, 1.9535327144641097, 1.848916274738091, 2.235252272332141, 1.6971273082242062, 2.184907032155528, 2.1394722792430394, 2.4693200142106746, 4.1460105744088915, 3.2259709489888846, 1.7879041364834711, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0543860785693875, 2.991481611040928, 1.2581995102044876, 2.6283710647437477, 1.6970076979729594, 0.0, 2.211809982515617, 2.1766813659819717, 2.570271192836641, 2.2266768211593604, 1.4677734446896236, 1.2455074958177577, 1.303761795465445, 1.56824231165958, 2.2487404093852623, 1.513200235941101, 1.675381941072545, 1.9042469070907524, 1.6667114666229605, 2.063828065281784, 2.4106196306682843, 1.684051244159019, 2.3821252739419765, 1.7076701779359686, 2.2274153311874345, 1.4653457125193583, 1.4756082242395296, 2.238704967253275, 1.818516042428565, 2.1054702104546688, 1.4799536596554717, 1.4558451415022875, 1.819200238655196, 1.0490014451567193, 1.7215098313645765, 3.2377960898674716, 2.7756352317238617, 3.9895039113210577, 1.5166996718536279, 3.5335309559686046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06423295795048825, 3.142880374645609, 4.540957201077286, 4.393731043177908, 1.1496354018250312, 0.7327787698189306, 0.3150652565797248, 0.14878630198748635, 0.0, 0.7474222852894781, 0.47889094284739187, 0.17915683008936376, 0.6238502010139207, 0.754002641335908, 0.9475038987623536, 0.3913206437506172, 0.666073205143794, 1.3096542422263284, 0.5326226296626646, 1.414594451263072, 1.4900600241271367, 1.5029033997191665, 1.432018456140733, 2.0090845767547285, 3.1154136915969355, 5.519304843841249, 1.8890001910285295, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04346424114825831, 0.16383266781521866, 0.0, 0.1356094618710492, 5.27582675210411, 3.1276003083676343, 2.0144823661183344, 2.29646233240667, 2.5786994992048555, 1.7541330097799248, 1.3269932306798868, 2.118041604507543, 1.4998040556992889, 1.866265008468647, 2.186472345990433, 3.7504662431726734, 2.4778661939224946, 2.29611956766234, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9546686855757611, 2.70299456392785, 3.6077922571027154, 1.883852411805212, 2.44616721680053, 1.0706974407792607, 0.0, 2.474603641581827, 1.945646301046847, 1.8682323034216055, 1.7955092597548103, 1.8687031029263645, 1.2578806223396894, 1.6873208633141163, 1.4839615732825573, 0.7814398164951871, 2.058034073926073, 2.1563723071927337, 1.6844812698593783, 1.643451724396665, 1.5593353977302413, 1.6544505047643205, 1.9591740759720921, 2.5783078286689185, 1.762030573585875, 2.3725396536297025, 1.338999510029694, 1.7205431394581094, 1.4278100674836074, 1.9139132945009563, 2.319813006570092, 1.9025218360554295, 1.384539860168998, 1.5651594221014922, 1.7291459052942109, 1.949673283198229, 2.617929037962973, 2.3427856030076675, 3.5200511214855967, 2.1933840467847894, 2.42259639013681, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11468796471142018, 5.406959587431673, 4.824170636194437, 2.198289932101803, 0.46310754437654156, 0.7739130771625322, 0.3450949140839964, 0.45224545325856746, 0.0, 0.708419830170728, 1.5192085666237116, 0.0, 0.7951219109956279, 1.03026716450642, 0.36970518754036497, 0.20799095118560343, 0.8884208369673807, 0.3675865207457651, 0.4814392671710038, 0.9930659859944959, 1.5222054118027035, 1.5740379739033297, 1.757836941382099, 2.466489610143665, 2.998272225229773, 5.365265728372539, 3.000481621818908, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.38701222825703785, 0.706852967183129, 0.0, 2.7682208380456137, 2.5585140831760844, 2.744829824462749, 2.374935775291647, 1.8041613965679528, 2.7545749822357077, 0.9914368943924364, 1.702616618588128, 1.637823160381815, 0.9437075942935802, 1.5577800929445396, 2.2311152370764114, 3.6187353913268083, 3.6110877804133708, 2.2236776519896235, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4412991755281837, 3.6560876335235313, 4.419582264930891, 2.3547413559958836, 2.691923422161732, 1.5468690818350765, 1.2932864960663486, 0.0, 1.8591064728755133, 2.071845461185986, 0.9502595881785102, 2.3593291284209625, 1.2419526226107582, 1.41032545999688, 1.4854988149605992, 1.2401156930507662, 1.6376476839151362, 1.67998298055347, 1.6365097368366481, 1.8764192121831276, 0.784818753544318, 1.4367362874500453, 2.0879704099462186, 2.137549387939427, 2.254051998294046, 1.2806703473202068, 1.8255938994417773, 1.8361564769113727, 0.9575382052445213, 1.3579722372921361, 1.0450953828025975, 1.7716679742653871, 2.052727822391683, 2.039493139782418, 1.3433692829000727, 1.8939859710491682, 2.167683964135667, 2.673595864172217, 2.8583135352900264, 4.178717934830208, 3.020512860606402, 0.11653849192428206, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33329050470548177, 4.131277182479062, 4.744660681754013, 2.590710889419743, 1.4751473679112692, 0.7501655029240281, 0.7202050825926142, 0.18822743053103205, 0.8484773431211518, 0.3441257051465646, 0.6276876127944638, 0.2736927554725884, 0.0, 0.4411538518025032, 0.5911701038222152, 0.0, 0.8656763389857983, 0.6252516441821149, 0.4445169484617892, 0.6249147379164611, 0.9795641328641187, 1.0865462369028922, 2.5340922626902302, 0.7018878268183584, 2.41228886418369, 3.6554824823829497, 4.727590170985311, 3.366709910100811, 0.36358728541983737, 0.0, 0.852116516566858, 0.23583328153036565, 0.0, 0.007031826203728534, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8277930488346842, 0.004656535480672149, 0.0, 1.8394429378843655, 0.0, 3.1776002242640375, 2.1009042065204966, 2.1184538889044147, 2.537605676989253, 2.767813304311177, 1.3335566624844946, 1.9668779829133591, 0.8669184897468258, 1.0440948068945206, 1.6271253418865983, 2.322715535902609, 2.009070007241621, 2.587929125026177, 3.165164881358791, 2.567323373284862, 0.08906602716252994, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.7212027701954296, 4.778962745206303, 4.491894252017229, 2.1377148712377303, 3.1069377048276787, 0.8673661611456319, 0.670813079085282, 0.0, 1.2671372735245159, 2.130951824780792, 0.6976830518473707, 1.2722968329833768, 2.016914554466244, 1.5521109005946567, 1.4274171050618594, 0.7378584852753137, 0.8939235623680519, 1.7857462603676832, 1.745315058764074, 1.7396674674769217, 1.1705336464455887, 1.5902470942591886, 1.5559969141917924, 1.6960612054125468, 2.790641557084848, 1.309028456762693, 1.555206684755368, 1.8217345413957606, 1.7413539899021448, 1.3644027443371172, 1.6306312331421262, 1.5096072473743247, 1.6599046858871567, 1.6130158663013505, 1.5285455965001502, 2.072484667450624, 2.307837422090732, 3.50276277854737, 2.139763893675124, 2.9636607932172385, 1.9754256928379976, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.350480985993234, 5.386194854219228, 3.381474012248412, 2.7490774296406277, 0.0, 0.2344481040967848, 0.3914696811315584, 1.279488070243687, 0.43753608698606183, 0.3895255186280102, 0.6741384732348245, 0.05191814366452101, 0.15257247972698193, 0.4058950654370619, 0.7271038371373909, 1.1880912231062422, 1.3804774396910853, 0.30614817936665883, 0.6611130505388649, 0.34752243429477114, 0.9152064591405679, 1.6366194160258225, 1.2637878400711637, 1.9074359403461851, 1.698267338194465, 1.9381856429282103, 4.768040383672954, 3.833398173925968, 0.348917442691696, 0.3931467665321425, 1.0204029355287747, 1.6410615241640298, 0.03367260537353567, 0.05630673819858759, 0.3806691051234197, 0.1638298129799646, 0.0, 0.0, 0.025597391602580503, 0.0, 0.0, 0.5599381712259198, 0.28295424537467284, 0.0, 0.3522914725682598, 0.0885702241117805, 3.317465601003061, 2.468499912162173, 1.429069899329692, 2.471942676424909, 2.404602727848325, 2.506353130763925, 1.7073220645662903, 1.9009455747703765, 1.086289099788165, 1.1856474099889711, 1.4349316840120467, 1.7198660343263037, 2.2206941315930764, 2.278684016716119, 3.267496284721377, 2.0300761503990805, 0.23752788019756899, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.052462085868529504, 0.2870378625608685, 1.0672402645319963, 0.5551492429419874, 0.0, 0.0, 0.0, 2.174706408346869, 4.599627635733886, 3.949353584441174, 2.9670204456764497, 2.1580243184441286, 2.0548616427977144, 0.7818031100801754, 0.0, 0.0, 1.6691768857775855, 1.655709707740535, 1.1287441078378722, 1.2502763324385842, 1.6319087580106384, 1.2454444452458875, 1.239178551245406, 1.120117480966104, 2.386722482401838, 0.9862423035822093, 0.8649002633840813, 1.8560574868086288, 1.3796130915679723, 1.5717793121734627, 1.4151110505676474, 1.9233491550362087, 1.7509624883373713, 2.2318844361065806, 1.2737316402729346, 1.8518924476819743, 1.9624347835766776, 1.7891919348158376, 1.5427587545451262, 2.1558141320836213, 1.3927960461254403, 1.3314336604234696, 1.0741504493921896, 1.7544254172845841, 2.9869526829770825, 2.045101882699153, 3.0968415814014474, 2.519366680831213, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006891409879557615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.931336402351033, 5.121029952680991, 2.5853069540733538, 0.278506152717459, 0.0, 0.0, 0.5244571354390504, 1.30723743369683, 1.2957262786809953, 0.4278304159504659, 0.5871904378421404, 0.5977359336261757, 0.0, 0.01176832595515747, 0.5680588977929096, 0.7695286171766006, 1.162490478890315, 0.9162400509369224, 1.2337337752062933, 0.9958564840030627, 1.669051629145647, 1.4794676481827462, 1.664786379205137, 1.3045257367466403, 1.7088829652148207, 1.9248984124830584, 3.28445339274388, 4.109634004758921, 0.4985866371138916, 0.22440785503671248, 0.921071338219818, 0.8843735033789543, 0.0, 1.3714785577914872, 0.5756335309793278, 0.8106476755988666, 0.0, 0.0, 0.6683490986603637, 0.0, 0.0, 2.2258357290501904, 0.0, 0.0, 0.0, 1.045070457395483, 2.8596112348955374, 1.3393836770501466, 2.6413371488555972, 2.04907270372674, 1.6441527204469766, 2.3368684410201683, 0.8768147163316967, 1.6495302941805832, 0.7513649273399925, 1.5945618394414982, 1.3761453695404782, 1.4237834047235551, 1.8729019935491087, 3.0616211868701204, 3.625536676499273, 2.042140419357927, 0.059500931464415166, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5462499329010159, 1.491904806317845, 1.2948957053118533, 0.4262019195247695, 0.0, 0.0, 0.91356818070504, 3.8039867557547966, 4.359437585571702, 3.3872493911917756, 1.252345278554265, 2.050321021622408, 2.007330885571635, 0.26340615134402173, 0.0, 0.0, 1.8093707588984789, 3.0652797390680617, 0.9626275575964381, 1.0640857332740612, 1.8067422988924022, 1.0083768021127364, 1.3582542185694326, 1.5626208245282251, 0.8410241962455239, 1.1202570275103805, 0.8752661479740279, 1.1863437496633948, 0.7063903895136389, 1.1266753062653112, 1.6022070774803687, 1.9331757624592318, 2.0469198166809184, 2.0241135849128766, 2.502784839675791, 1.6291065406401262, 1.585355775837788, 1.392186286287718, 1.7377179119786623, 1.7038202192756122, 1.3612388246708371, 2.0289918277040355, 1.64500523246029, 2.7565125981412737, 2.9168535844183223, 1.673962362748991, 2.3353781113010137, 1.1419601531178818, 0.33652376326111627, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.028978928807809654, 3.5549753626572276, 2.415217144992042, 2.1576850105075516, 0.0, 0.5229859636707396, 0.35869627448262226, 0.9007385537038513, 1.1339857504524593, 1.238502857209745, 0.21490628992014382, 0.9169272535945412, 0.05496409280794664, 0.0, 0.35823923900494736, 1.1548920664147304, 0.636188403477527, 0.5423374448826409, 1.141919572412606, 0.7852083235537825, 2.0905699716586237, 0.5330679355270377, 1.6966233094151055, 1.6966079664410392, 2.017774660389735, 1.5794263152235013, 1.3704999351917682, 3.8231402242509684, 4.016862152360726, 0.7120289566927832, 0.07922785376049457, 0.11891712397987705, 1.0992312810780456, 1.175666628008932, 2.0154679295747338, 0.5279388399829521, 1.1460587532508495, 0.5495471987596772, 0.10454202694654395, 1.1678770572574606, 0.0, 0.2580795672047112, 0.9953921825314093, 0.0, 0.0, 0.0, 1.556023987339662, 1.6794742298703114, 1.6271383310995462, 1.6899441014679903, 1.5301088850274243, 1.9652989668497, 1.1217962766610416, 1.6209064952028955, 1.600001023970376, 1.7376770549245197, 1.96266974196317, 1.06626373165289, 1.7727287873436124, 2.5099114416259964, 3.1671051688231318, 3.704702073442316, 3.004273999784101, 0.05510184877352912, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4540749080731359, 1.1100889071778055, 1.2942261429412818, 0.9135857824300878, 0.0, 0.9641992834602959, 2.466278412269452, 4.555075679296813, 3.284546393900394, 2.5848600988537656, 1.0416773822605319, 2.313229384828378, 0.367836713723743, 0.12013154547750897, 0.0, 0.0, 2.136274792065025, 2.3390038041467482, 2.3081047299578543, 2.0134490086511705, 1.318455340963402, 0.735234367577494, 1.2373410101711828, 1.2558641567413558, 0.7237522693888909, 0.348859640274904, 1.1438848531975534, 0.8560192005471777, 1.1534912774201946, 1.1832843883536643, 1.3306615702708784, 1.557924734076322, 1.5980820029320926, 2.7201004887616635, 1.9584943584142551, 1.6268281948957097, 1.6604041189524432, 1.880084611913826, 1.80811217936684, 1.6312030317710018, 2.160456429908856, 1.7370945573559347, 2.6969618771369173, 1.899527116481862, 1.8986566921681811, 2.122823529669911, 1.4050878505638316, 0.10387378617276465, 0.0, 0.0, 0.012680671930865521, 0.0, 0.19209718245553722, 0.15587597559561653, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01658075051155146, 0.41680247887320915, 0.8556652345303886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09575636701352094, 1.6332712623342427, 3.506325955238367, 0.6599648706316709, 0.808619875408529, 0.0, 0.31800818691105076, 0.21910509613928794, 1.7394352528471386, 0.47310339838756443, 1.7865229663294955, 1.3707105062584264, 0.2955063168482246, 0.1775734143486592, 0.24998592880394518, 0.8232410845364387, 0.8989317493242834, 0.2885330690624254, 0.645786622848813, 1.1652554273909375, 1.518991088048875, 1.3996166710815658, 0.5407659617783402, 0.6842938263138966, 2.0843812403471738, 1.1466862966156406, 1.5025952402204672, 2.8039231916798975, 3.716516775650981, 3.170984912173089, 0.5564301606930848, 1.1909183309274474, 0.8692955916006582, 0.427677678598136, 1.8848738874543671, 0.9642637652843393, 1.2968809091576052, 1.2038451006899016, 0.0, 0.34613000838887825, 1.3043273190847056, 0.2210518408199348, 0.45144792956163793, 0.0, 0.0, 0.0, 0.0, 2.6667568232925465, 1.0846451089317415, 2.665897612242442, 1.442990194236717, 2.0637824130551237, 1.5193016137031494, 1.813341070649927, 1.3826601772906424, 1.8222030085561882, 0.9990207340322427, 2.0871440064880007, 1.9620725634771392, 0.9691618311563825, 2.4481706861271886, 1.907441898543358, 4.15410627699439, 3.918410746605663, 0.6916539337467122, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.912910542637729, 1.2698559879812297, 1.028120659780855, 1.1528856969177634, 0.9472280769859738, 2.7097406582797308, 5.438199948244195, 2.9476932495094945, 1.279869706527854, 1.201302913879345, 1.0595391219497523, 1.0873982897380559, 0.6621646104878377, 0.0, 0.0, 1.1311367940464172, 1.0477605541628083, 2.590808261617144, 1.6402324670501849, 1.979658274274065, 1.7015468820552342, 0.5304145477515376, 0.9122361924514572, 1.1202588327323197, 1.1078159358632236, 0.458878501287261, 0.3709366856833323, 0.40136334099084436, 1.2736725872243646, 0.5715981561934227, 1.855434393456093, 1.9581108527836466, 1.3706459204597186, 1.9291758279710483, 1.456960085982407, 2.2302335600710457, 2.2682353642158475, 2.2153603420851464, 2.1125307150617716, 1.9700129304987755, 1.6244456509204117, 2.8283082761551106, 2.274205802811942, 1.545584279226775, 1.488484221420424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8828917326772011, 0.06896406981491696, 0.0, 0.294038602373676, 0.08531939254817773, 0.0, 0.0, 1.2195515537921922, 0.8652178318408308, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008746725166779845, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02293268607550084, 0.0, 0.0, 0.0, 1.187866620284222, 2.2739666169024852, 1.1698503888871912, 0.7523283439450571, 0.0, 0.4106138928504947, 0.0, 1.4276565974978026, 0.8869166159984311, 1.4063045848044573, 1.5623533838349823, 1.8108928599074923, 0.049479905865847267, 0.46911691947055445, 0.5569135390678811, 1.265690427477427, 0.33552910224056903, 0.47120153760507966, 0.9213381362438009, 1.57514428777371, 1.0286902556941946, 1.2367062581403043, 0.8265054931044497, 1.4004535576441763, 1.5027614228900565, 0.9133753717079754, 1.6823263984906422, 2.159871931870619, 3.2930095513243307, 4.649041134169017, 0.5522428841317606, 0.0, 0.3161643851327829, 0.9914318706586392, 0.5276447725370338, 2.0965211170441274, 0.5857142294559519, 0.6385007461381784, 0.41454243307120486, 0.8857273921464697, 0.424565172193944, 0.9695791583492452, 0.44826104624712765, 0.0, 0.0, 0.0, 1.665509947422092, 1.4870766532750397, 2.184448104819668, 1.4309500640326216, 1.2318226394903264, 2.2779610491676334, 2.326733343305011, 1.0408726045457464, 1.057097657143804, 1.5117179678973751, 1.5585301458454552, 1.4826423497970578, 1.7827610985737956, 1.7828556818470065, 2.1025181405374584, 2.7499725291971413, 4.260392837431093, 3.375172667276627, 1.5573117497989895, 0.15291641877602938, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7292309475648432, 0.5318755651025484, 1.1282747667055921, 0.7343699197496997, 2.813153573396173, 3.519220815322017, 4.249121940234008, 2.400672747891942, 1.2768221224222698, 1.0982534968492694, 1.5690595930908828, 0.5300958877625352, 0.6000701059716705, 0.0, 0.0, 1.6481740011912598, 1.0403232085353638, 2.331069721023717, 1.896629768295545, 1.5011923644953824, 1.1871490212631932, 0.465730711863574, 1.159491303051635, 1.1101552676142463, 0.9954331011887493, 1.0642374755833772, 0.2612958182149988, 0.5518350231533666, 0.6067246871469691, 0.8405285478263557, 1.511419502448006, 0.9173332748895983, 2.0387878095646257, 1.5514974932499608, 1.0043081616332112, 2.430707455242946, 2.184293815321374, 2.017798013163679, 1.659211996483766, 2.4993017170225063, 1.7609593261275953, 2.575840845922581, 1.2651032044130919, 2.4616763091120095, 0.09241188082962802, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5629221541802418, 0.7259754389532844, 0.8701885519195307, 0.0, 0.17018646318142527, 0.490368422710243, 0.12208822241366143, 0.0, 0.6466983317250803, 0.5203158207766142, 0.8904160787940549, 0.2069165753397746, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012955001279981615, 0.004610575968890739, 0.0833942180653954, 0.14816900484260354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12562210584273967, 0.9674569599666549, 2.825975544235085, 0.6224879742196014, 0.6110029376839691, 0.18723217824957286, 0.20680841570619793, 0.48315151996985456, 0.5823159731839584, 1.4633614287049965, 1.6210505294184687, 2.2134677830093, 1.5549694043336761, 0.5316744175796976, 1.1732693335372322, 1.1561043691984059, 0.0, 1.0939504069330843, 0.7962856400235553, 0.40969270355993, 0.9783364723528984, 1.1358377626867753, 0.4840168453208519, 0.7958071914754837, 0.6422453620287004, 2.059106601381827, 1.543319604225698, 1.2979666061383868, 1.9079659635728636, 4.384270499759845, 4.0053544386138125, 1.1968220583375624, 0.08848845551513002, 0.0, 0.2383694347779868, 0.19425105447983276, 0.8582675690896011, 1.255563231807754, 0.0, 0.06029878787763226, 0.32231066869004804, 1.096370678825057, 1.3143300161714098, 0.1540072582576539, 0.2178512527380296, 0.0, 0.5805483719818088, 2.7829766315319904, 1.5420618492469185, 2.653524154742504, 1.4689916934872875, 1.1948520817223478, 1.3818637900987807, 1.58766158483053, 1.7541198599416656, 1.4452792027562456, 0.913937451755046, 1.3769985548710437, 1.7527068266684174, 1.1116919455669014, 2.054884324505349, 1.6020225376921373, 3.115462339455082, 3.689287264466936, 4.0720477746807235, 2.2668151659266513, 0.9665739361820511, 0.13371668309710352, 0.45201418879214894, 0.19916407919826223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16466655544170772, 0.2481506406644244, 0.8714378457847383, 1.2991563233000227, 3.4409607262971873, 3.493390959619668, 2.933782516554679, 1.1509868085971655, 1.6401778996147443, 1.5044599817750044, 1.8442689424085343, 1.3231757678601945, 0.7387244572839301, 0.0, 0.0, 1.1068542845916096, 0.9177570571824424, 2.127017606400316, 1.9107411917056105, 1.2879108470113834, 0.8246021624008827, 0.7523878795783286, 0.8073179045670493, 1.7344337666024672, 0.654687431302279, 0.7482973600453857, 0.5866243464091558, 0.28481111960207145, 1.042651549970599, 1.2317198683151855, 1.1652376607227521, 0.4439348073504363, 0.7943764046320421, 2.0717068626006183, 1.8875364096307958, 1.9667200018913207, 1.5446217668936963, 1.872359740074176, 1.4271349473318498, 2.011368719490585, 2.143971903391831, 1.7114483012677, 1.7615008504483007, 1.6691158192902213, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3985616245338288, 0.011412309698923076, 0.6540347938724043, 0.0, 0.0, 0.20753719136989948, 0.5145066610666261, 0.17488772191270888, 0.25711570195541283, 0.2629206767575441, 1.0818368049606877, 0.48230401442808174, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4334884369426349, 0.04090157843337025, 0.055192493663760674, 0.17564981980168992, 0.11174153829044244, 0.027293683289322468, 0.0, 0.0, 0.17567494313183776, 0.03785489609241135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014806779003572286, 0.9287586225932454, 1.4109781176786396, 2.924362358863063, 0.4597202672938947, 0.0, 0.5439414471402947, 0.7361408120136556, 0.3907439674157041, 1.0182630483025183, 0.73744767219644, 2.127232945985711, 0.9190206904604802, 1.6249592251839842, 1.4115886735583951, 0.9379990141932171, 1.3809806936707338, 0.7921972138777622, 0.23002949274129042, 0.6454957460061264, 0.3116056480662691, 0.25539285679770685, 0.6701794867067147, 0.7986288778645924, 1.6030985978881584, 1.151886992919376, 1.4020950668512824, 1.0133745897790645, 0.9306855335129791, 1.9322139312679354, 4.501714779083419, 3.1589595611455588, 1.0023777196116679, 1.185556546298171, 0.9083011336716046, 0.29683539259861713, 0.0, 0.0, 0.0, 0.16499280206224692, 0.15352828276509461, 1.3753603858622279, 0.858993998552222, 0.6039956398476087, 1.07900074023218, 0.20998304901948806, 0.3598193643869224, 2.6842782845731534, 2.3154812970588394, 2.0495055986571935, 1.621340810085097, 1.786246021887552, 0.7280697722576807, 1.894962851826187, 1.6068956209430667, 1.4580634214588362, 1.7132636538196575, 1.1492070801053615, 1.0667616418252823, 0.6199703624450789, 1.5844416214552808, 1.4114454368339489, 2.0031427316019417, 1.8318328857196255, 3.475282675414453, 4.010282971110468, 3.1909059660426853, 0.8421963491848699, 1.0604885491487936, 1.4180970347104975, 0.6599817263887315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22274410925664093, 0.25246009522198326, 3.650994387453879, 2.922239263906191, 3.316629196439524, 1.4249298375068535, 1.386472584832207, 2.0751495398039532, 1.5128687777286924, 1.9876932220632186, 0.10164226131322872, 0.9096167067874026, 0.0, 0.0, 0.13221969683497237, 1.3659495516024156, 0.9616689430830239, 1.73354382718617, 1.6245241486762552, 0.8875293823817778, 0.9490805057959266, 1.1214154521640924, 0.2207518343733326, 1.537712062252936, 1.7516270462607046, 1.0269217724591246, 0.5572687951048555, 0.4312596965053059, 0.563886138315732, 1.3900335963787984, 0.3664192476496333, 0.8392421752783459, 1.7029873459334168, 1.3066255792446735, 1.512639923070732, 1.8645613976678748, 1.410070755948783, 1.8316001801518471, 2.4055656133828283, 1.4369492966865525, 1.7319305044058615, 1.5185965256123424, 1.0120436380079847, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6930653548321708, 0.13511840092873617, 0.0, 0.22522090380759574, 0.0, 0.0, 1.0423553853114693, 0.02722444786122894, 0.06559731750773329, 0.0, 0.6462220641807016, 0.18802572826515854, 0.6887342511163154, 0.41230821729173883, 0.0, 0.0, 0.0, 0.0, 0.010676942884681656, 0.1780238473360312, 0.0, 0.0, 0.538845540620588, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.3000046594169787, 1.333425799898201, 2.853567632351371, 0.4199241009470933, 1.7428059996563978, 0.0, 0.0, 1.0243123104758245, 0.2995198264661064, 1.1034162849695448, 1.8474444679483208, 1.4448968382494614, 1.2632494089093154, 1.7096086603213214, 0.291009940664537, 0.41928560843552426, 0.0, 0.28064530974501706, 1.4314294085777228, 0.6411971137341642, 0.46261353383108117, 0.8907366090603795, 1.2139872396602882, 0.7368861314536311, 0.678600294969298, 0.9571334244424206, 1.1499721258359041, 1.9206940706555375, 1.3467338251016487, 3.9697081229137803, 1.875654689025766, 1.207123901179252, 1.7572150030459337, 0.3454969482414659, 1.169526094768413, 0.6301419482087034, 0.5560241427235174, 0.0, 0.0, 0.7842014044820634, 0.8309414291174968, 1.3030329990154104, 0.5154135822317744, 0.02700719967902931, 0.0, 1.4481512303592567, 3.881892982258837, 1.3658430495530094, 2.5695136544530532, 1.9252126811256476, 1.43052440457891, 1.4930274653619247, 1.4813390040429322, 0.8182298861536245, 2.076476304962864, 1.160160549735043, 1.1876036659538398, 1.4004753411174604, 1.2275220045097508, 1.3057683197430248, 1.2998770681054679, 1.667649638722876, 2.2855090769196655, 3.23758673501033, 3.5034548131256793, 2.8811227836050715, 1.2275188703244566, 1.336105801049583, 1.7396806095404649, 1.0136885467312242, 0.5149910010876763, 0.004373736566265491, 0.14991901983986106, 0.0, 0.1380538498562084, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20095723430656035, 0.404924374294982, 2.074181685586484, 1.1127813802057989, 3.4508898141243236, 2.429430906921671, 3.0240750767576623, 0.6403762419758614, 1.3548105430241173, 1.6037589812502853, 1.348466237973041, 0.6625636604976578, 0.5889796868573659, 0.3994853790602943, 0.0, 0.0, 0.9561937967364851, 0.502083778522681, 0.880012821675657, 1.0679681601744166, 0.5637616137328844, 1.4641170364598224, 1.1025944470873836, 0.6352048140730887, 0.9220234998623456, 1.1423253509058937, 1.7195735930683884, 1.0289914708220333, 0.8403520264561111, 0.0945419424057162, 0.8001769565761688, 0.9275742284811572, 0.8894091597953119, 0.6465938047924716, 1.2920776899636377, 1.3259102792888462, 1.4510274137477688, 1.5056244279107696, 1.7848510578444996, 2.201844380408357, 1.7074312726600582, 1.9139303202378275, 1.0601571331554078, 0.7487268107568431, 0.3076434007316665, 0.0, 0.16268498207926274, 0.0, 0.0, 0.0, 0.0, 1.4788793691295605, 0.33432731218127065, 0.0, 0.0, 0.0, 0.0, 0.23594593839908948, 0.5855687138246906, 0.5120673060656611, 0.1612829445552393, 0.23455135451666143, 0.3137619884964362, 1.2555755130850343, 0.06129717774831784, 0.1284653827994672, 0.4816851039350786, 0.1341378631929497, 0.0, 0.8990721963606743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11875291422701681, 0.7329027997693713, 1.4055933261238596, 3.0396038352834376, 1.075086747692578, 1.3291185660834874, 1.8125733605481869, 0.45746764216983665, 0.26440442915580703, 0.027786443152324487, 0.9756581386782703, 0.9116021549881204, 0.8070108266727858, 1.2651774243092362, 1.616104878038949, 0.632467277168024, 0.07246198023676206, 0.08086724461104401, 0.7298109997884216, 0.3101817130786611, 0.7093425305711678, 0.5032747482213636, 1.39370412271688, 1.1021268752460858, 1.6026869700391861, 0.9078104058895241, 0.600679436438637, 1.1319700354337825, 1.5328377292862605, 1.67717250258874, 0.6061458503870347, 3.4949544554652734, 1.8998479249284146, 1.3142466448927526, 1.5315811699776798, 0.36318501938059583, 0.9903319069161349, 0.5027810948342699, 0.0, 0.0, 1.331128285432792, 0.37564635412960745, 0.5239200940534806, 0.9890258474531382, 0.4206520929757404, 0.0, 1.0519144863185677, 3.1951795676444625, 2.4033903661702856, 2.6695054247737713, 1.975119324835457, 1.7841784845502282, 1.7272807286537628, 1.2240250547869003, 1.5754103217018478, 0.7400403624096392, 0.9179418079285177, 1.9361266029234687, 1.202536489857525, 0.9179916635940479, 0.6231244267468394, 0.9638198583889198, 1.5671519451997367, 2.2041963435239067, 1.9883024302181838, 3.389677234836861, 2.579616543612131, 3.322922765549281, 0.8518934895900749, 1.2381767427580463, 1.480293108820302, 1.2062931979331692, 0.23278718285773642, 0.5136457239011134, 0.058656826009508285, 0.0, 0.2518742624628461, 0.7682147712143989, 0.24576226292593092, 0.06648479817324907, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9386791856169779, 1.6015376930239253, 1.8989328087987964, 3.7273147332914323, 2.331281959889646, 1.8192467717240215, 1.0684917041689033, 1.3846328936830403, 1.8026824800329067, 1.5030107025109116, 0.5635805890092553, 0.04258052549978379, 0.11847698923424105, 0.38429348418422765, 0.0, 0.0, 0.7523084209878487, 1.6821822824859418, 0.6802272403057286, 1.5527151158061319, 1.1067034888611142, 0.9483661679919997, 0.721696200948383, 0.2791731770219533, 1.0377391868590036, 1.617465795201069, 1.0419047814995739, 0.9497884538339728, 1.2413597664680778, 0.5048078022570225, 0.7291533720287242, 1.0670189506481869, 0.5805494728038146, 0.775677570083117, 1.017592235162296, 0.90641786731436, 1.3364237904553535, 1.438174366262911, 1.6156025106852105, 1.442498085780325, 1.7645767344564456, 1.2450947999712334, 1.1473641901925846, 1.2651311056160703, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33730619762901326, 1.3098056882839741, 0.0, 0.15426784391970988, 0.08052643471930455, 0.0, 0.0, 0.9438072947749349, 0.11307345590483314, 0.001658969662290658, 0.770152852856071, 0.5487464861781208, 0.13686455487918167, 1.5670273034284248, 0.5376132343484552, 0.136014783598828, 1.114620594461895, 0.0, 0.30461065142151417, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0941756421660149, 1.5387050279023966, 1.5700644405836293, 2.1010055338338023, 0.8221142059316643, 2.174172952266103, 0.7253797227713635, 0.41540291327300677, 0.8250807919491487, 1.0299226007299185, 0.724728904304181, 1.104744496060123, 0.022929842017143622, 1.111212155484829, 1.0526595617161212, 0.876192870119427, 0.09470183902682301, 0.01676156674898579, 0.6940662249297248, 0.7826848270795345, 1.304299269525617, 0.7598425485820542, 1.365331884073806, 0.5589785769278572, 0.8337564933041769, 1.1638020850259247, 1.6560972985407345, 1.3466199062024526, 1.1727393672712274, 1.3904138237351815, 0.0, 2.874952176195016, 2.0652494447527, 1.0811123436436487, 0.7773936152599988, 0.8144155359000274, 0.0, 0.9149763950051054, 0.42327522760443115, 0.18429963219522116, 1.3153280929974849, 0.0, 1.6215691896297244, 0.7797776701651719, 0.0, 0.0, 1.8491400472770538, 3.2780155458494495, 2.0671610239644096, 2.0315607031405944, 1.90632455435475, 1.7439731063661819, 1.7751161624579903, 1.141418983158796, 0.6109400158083088, 1.0160281843976395, 0.8767974619629677, 1.3783106566438135, 1.7684566327891542, 1.4346384421887874, 0.6512899894375499, 1.4102693431007478, 1.0201850791233957, 2.1003509030556384, 3.1347586242995553, 3.4422959478517785, 2.7940459260577595, 2.9133592476899524, 1.1298771348085956, 1.234511347170978, 0.8111461127004654, 0.723272904045784, 0.8889810240430807, 0.7638249234057202, 0.051570971531569, 0.0776693458131363, 0.03948274278951435, 1.546435634298163, 0.9381927028660487, 0.550815921521775, 0.0, 0.0, 0.21101520423708658, 0.06798004055882921, 0.5415393338048217, 0.5777204752132473, 1.8183344143387452, 2.4430316064012643, 4.568606830311146, 2.190958064384568, 1.7559108521680855, 0.7907057576243977, 1.7574544297397807, 0.9556067283518435, 0.23444030679991165, 0.37906394670314975, 0.00073683806959552, 0.0, 0.0, 0.10944297740277505, 0.0, 1.1324159851792786, 1.005301216809459, 1.1955167512628937, 1.6755940802160962, 1.4291547091287613, 0.8898890112344057, 0.8697978417376582, 0.9179324167500672, 0.5655103727381995, 1.0881534825062178, 1.3148573640053944, 0.8925478524484393, 0.9194045791748138, 0.3166200852199669, 0.7510122342607672, 0.3078536293359431, 0.620364837371837, 1.007191884788755, 0.9024988027517242, 1.5627531213100123, 1.131771682841335, 1.1479605989139716, 0.7477408308979384, 1.5894371573274835, 1.0988253830412975, 1.2318306498356792, 1.653337269851369, 0.3743424217852876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3113365048285862, 0.9213475794631375, 0.0, 0.3002575231791841, 0.3478906089542625, 0.8963827410489651, 0.2706076206243674, 1.380251929978842, 0.028696696750816906, 0.13691419054536305, 0.9277990784411895, 0.5647158101988652, 0.4259955589660065, 0.9459089753383525, 0.0, 0.23947855931682097, 0.9881627225430236, 0.0, 0.7301034317586047, 0.023785550492003805, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016816942362674753, 0.533000395072011, 1.6780363709881445, 2.848911880145021, 0.3097712664079716, 0.9583338279649279, 2.0568685101007755, 0.36422491273402174, 0.0, 0.7185514414186279, 0.7863854904776082, 0.6369202694205841, 1.804048374016147, 0.8551544579173023, 0.7752619310909582, 0.6441690639988296, 0.9601425534943461, 1.3716176470872299, 1.3419503025942785, 0.5206358533066798, 0.5227995150531065, 1.0763249294660098, 0.7221957300068704, 1.098753946218355, 0.45179929147897085, 1.1741141833040392, 1.1263606758898908, 1.649791328024339, 0.5464914368196968, 0.6316763773908471, 0.5133639913303043, 0.0, 3.3215685836321476, 1.574305446288568, 1.2128756855642846, 1.7185611140152905, 0.0, 0.0, 0.3594868076055697, 0.8275709833421426, 1.4336320740251516, 0.0, 0.5457647809529619, 1.683271989154692, 0.39815443821222, 0.0, 0.5872789692307766, 2.585618334114138, 2.381086582832531, 2.536321914518697, 2.0116322726906595, 0.6727799559249122, 0.9891140920310654, 1.5536895215609385, 1.2166050703777869, 1.0080950846097338, 1.6236649541975392, 0.9322996147653668, 1.5597736073561217, 1.4074106828987207, 1.661888562511579, 1.4408179052938357, 0.9098918253217426, 1.9555269399918984, 1.9380958696525712, 2.3801698685031236, 3.318961911455763, 2.5658167131560616, 2.5238483881420755, 0.9864853031784876, 1.1279661185200047, 1.1705723999931632, 0.6486656441350367, 0.1973919042642355, 0.885456956010357, 1.1160663654836707, 0.7172803874654232, 0.918948223642704, 0.916851625303367, 1.0893419958758284, 1.1704268937745506, 0.0, 0.0, 0.15264970409475512, 0.7401026823788479, 1.2505531121162354, 1.8108149927383346, 1.294503991173967, 3.56297772846957, 2.586008280433824, 2.9898972659693417, 1.136407536493891, 1.319045576491219, 0.9898007652668319, 0.0, 0.0, 0.28700173995597195, 0.0, 0.0, 0.0, 0.699797626837112, 0.0, 0.0016612067588641236, 1.205731531079331, 0.8702965449222311, 1.759101419559162, 2.222438718235495, 1.2307734641382921, 2.0491501398316614, 0.5996864743118766, 0.7347970661011567, 0.872579805790393, 1.2269385285289156, 0.9275642142382257, 0.18694857076590587, 0.49875018890004347, 0.544681158723524, 0.8499321685481728, 0.8076559505651723, 1.0538552265739987, 0.6283850480579504, 1.0243141975611305, 1.3721866772648807, 1.4471470534386635, 0.8542459861280257, 0.8120910413920879, 0.6686679340082469, 1.3606125018198707, 0.31125715175061286, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003608831156256279, 1.263937616365506, 0.16535959746787457, 0.25618769529837165, 0.4628343427105132, 1.7188392420832044, 1.1745076196174844, 0.38915000997745874, 0.9371982413378245, 0.702164035627907, 0.511684615690466, 0.3638043504190154, 0.0, 0.8162304757115866, 0.0, 0.5475673272991525, 0.39837579974820525, 0.24504315310137856, 0.21660309313804085, 0.7007523738188529, 0.09051399668323991, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0893625112520186, 1.7916922968286482, 1.6693050792568411, 1.2359238204328131, 0.0, 1.004565940348561, 0.7259782573835675, 1.1372983857158592, 0.20329579007936024, 1.0835172118681433, 1.2942886076366935, 0.18213894551502735, 1.3292191921255434, 2.3514603518832895, 0.8299629349213279, 1.3612107303670482, 1.587775493471467, 1.3728970404479477, 1.0068790252398372, 0.9935732827410186, 0.08201979547064836, 0.9053972804395064, 1.1902754614345166, 1.378789003298897, 0.9070347087597412, 0.2996199002430253, 0.879545251283495, 1.1267527422277237, 0.5864083103744274, 0.26827228912289514, 0.3318541980988656, 0.25901189401083463, 2.3271182094062883, 1.4251716931073637, 1.1815161363161346, 1.2014112264296783, 0.4788848943910579, 1.0314075558005893, 0.0, 0.5169824371064589, 1.6965010305914041, 0.3503205767622766, 2.3655482854651133, 0.0, 0.0, 0.0, 1.47013715651243, 2.5547720406764696, 2.0932245903319657, 1.7005833307219007, 1.5752145947813678, 1.5850462127753613, 1.4808939860316723, 0.47406484748407507, 1.6885277731836095, 1.3819060051991494, 1.1559819916286775, 1.1805511358181964, 0.8909458302441811, 1.4170813161722926, 1.700243976420032, 2.0621105478039112, 1.6877788580440156, 1.8802808579850938, 1.3297725946527337, 2.209042426790441, 3.433550992460074, 3.360211047643909, 2.0031715250508006, 0.26351372948212254, 0.5055845048528581, 1.1302450723624642, 0.26380862472756555, 0.21793745528567854, 1.3360135079705144, 1.0412556749630995, 0.8694952485499674, 0.8649984434157326, 0.9179744559903583, 1.6202833963566168, 1.757981950411918, 0.49449826464067603, 0.0, 0.10959952776568933, 0.8243109242192928, 2.7395686409219873, 2.000718439991156, 1.6942656497704927, 3.449334634917239, 2.57898845895278, 1.3419679675684928, 1.1406373894624757, 1.126088705078626, 0.14197099815700856, 0.3813057241580294, 0.16180964270371087, 0.0, 0.0, 0.02914903038926166, 0.01787852660920884, 0.9696879440542523, 0.0, 0.0, 0.9155686429762507, 1.4304460763508515, 1.2569363579795503, 2.1336691922714954, 1.498081774330139, 0.8666584195499734, 0.7371369249894103, 0.9923297145098234, 1.3112559753038278, 1.2713755336925454, 1.3547979107081158, 0.906100586112897, 0.0, 0.5128775649428229, 0.7473438446172044, 0.07778355667500972, 1.371189442477485, 0.5656305643536645, 2.004191840811091, 0.7687982544258748, 0.954576920618914, 1.212540842451993, 1.0296534581112127, 1.1582332898233363, 0.3904152445289486, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3840277081276239, 1.1063696666431317, 0.0, 0.005591413119444294, 1.0077401175939684, 1.0480146264551697, 1.1907311633361644, 1.118000882226372, 0.22866534890824833, 1.0928917818745967, 0.6283967671703835, 0.2653819703112265, 0.0, 0.8934102051148979, 0.2450338038874289, 0.35055702871705496, 1.2811458039342474, 0.46036444763219436, 0.0, 0.24618634401427275, 0.2244749611756628, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.8204088824025266, 2.088999783869347, 0.17926944684525095, 0.8231921558885349, 0.28509381321452, 0.7136893341331395, 1.0572007771337495, 1.080017859390013, 1.0825689960748124, 0.8100547497592945, 1.003927001516733, 1.968918523233131, 1.9769109373461287, 1.1705061087196278, 1.291403042317219, 1.9418560521085917, 1.597130757567464, 0.8508733102603631, 0.28662934825452063, 0.13117236877885993, 0.20979992233947325, 1.2766653154712937, 0.26685897140765513, 0.6722067737883134, 1.1439728913899267, 0.42313358041651583, 1.6058812344894535, 0.37950476692240975, 0.9459801273420825, 0.9121370507059439, 0.5344913062818798, 2.264943793631659, 1.8662291690214454, 0.08212221127118026, 1.3700583594041178, 0.8724603455256954, 0.37299690833393184, 0.2345690792372812, 0.6403396832907674, 0.946063819210507, 1.8274066552400463, 0.7586423823189171, 0.0, 0.0, 0.0, 2.265579657987888, 2.9869040664948083, 1.7935070898957761, 2.2450314384239984, 1.671082405839939, 1.2254814100304758, 0.9404706660540804, 1.5779980868865362, 1.1897661408342637, 0.8385400179536627, 1.5738320725198849, 1.1494154665960865, 1.1316292676948667, 1.7643285088594163, 1.600373166010574, 1.1241201058530672, 1.5756207855658086, 1.2999050813627722, 1.4028919256720445, 1.8002558940785744, 2.8692236065241774, 3.40764132890361, 2.381007457861208, 0.16957299470089343, 0.012410731936596416, 0.9146759069718639, 0.0, 0.5272523107129493, 1.1787910337095393, 1.1429320659386986, 0.8662298182966128, 1.0217158850433417, 1.4711997532690682, 0.7738277275128055, 1.686402068454305, 0.9073583990798914, 1.9272408954585905, 2.770786320137054, 1.0884716510641284, 1.9785476251919403, 2.317827896908553, 2.243362953984288, 2.4421603781235475, 2.2883919305371796, 0.935912148314679, 1.1059375708117731, 0.0, 0.0, 0.0, 0.0, 0.1231475402354683, 0.473200863168597, 1.3646803129252696, 0.22040086718602925, 0.0, 0.0, 0.0, 0.6863357105475139, 0.9413906250693224, 0.6344441475070038, 1.3436603118800954, 1.2899062479069445, 1.4392574029405103, 0.3717804650736429, 0.5143173041545975, 0.6775199938568618, 0.8024874695788836, 1.4965578186825228, 0.9556597332306609, 1.5129011586786762, 1.352564865276344, 0.3214687129368979, 1.014858900812384, 1.8655583191997942, 0.9383209999685936, 1.107001972777323, 1.3257307497511843, 0.5152037206212092, 0.6069619466476963, 0.5830609453412107, 0.9394586775394668, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5198133922362131, 0.26661686631588555, 0.5273164247947032, 0.06586268326840451, 0.3747120121033434, 0.9905086647844477, 0.8617210718669785, 1.9635228285481292, 1.0142970629599073, 1.07608451016621, 0.7400768251829101, 0.1424960422827929, 0.25521599402983297, 0.8539921987082274, 1.5392106287525287, 0.6243170050273948, 0.3808734610451763, 0.6700098388881116, 0.28592001858453464, 0.27210543538634707, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13504345293496853, 0.031372652673197686, 0.1486471278585107, 0.0, 0.016639099983561313, 0.0, 0.0, 1.6896360202983014, 2.7118043001017322, 0.9554965440861872, 0.8812142734047926, 0.523173860273829, 1.8875161858779943, 0.5995893209287544, 0.19149209689887803, 0.7933768912333855, 0.5240632406493677, 0.7803011229385263, 0.9791445365377113, 1.6488681182501714, 1.8895817708266482, 0.9165070190942015, 1.4234549935769654, 1.5886693885663568, 1.462190045158129, 1.08225871922313, 0.506567506316673, 0.4899787780145478, 0.0012698303998006015, 1.0485714374644028, 1.0357697586785293, 0.8469620069863588, 1.0126677832955884, 1.4813745913607683, 0.20185382269513397, 0.9642150452007947, 1.3802639273503856, 1.4089960232865797, 0.09023694808876541, 1.8552628369354582, 1.7394505651205387, 0.0, 1.979173734006145, 1.1269848325279006, 0.12530265833470863, 0.9137571107016548, 0.27382046942821864, 1.2226896694916782, 1.3109998514040377, 0.3286278134465871, 0.0, 0.0, 0.9198296917222867, 2.2938413570213934, 1.9884684514822586, 1.3914528036240004, 1.220410011155498, 2.6109387571430456, 0.715795231326669, 1.4557701113937915, 1.6387536407847505, 2.0018504022577863, 1.1987390468681243, 0.9945807231396414, 0.904894778657907, 0.797271808305107, 1.8032652682971644, 1.1974862279694654, 1.4063791208062424, 1.3369828541151614, 0.6654353689026268, 0.3499247128106677, 1.1137057914455064, 2.0538585730954555, 2.741095897971659, 2.697180405562017, 0.821587961542282, 0.0, 0.5545675355159324, 1.3963341461568346, 0.28068460520151617, 1.9795757851122242, 0.8069451983469011, 0.5640951737187805, 1.084291505927406, 0.7385734524187038, 1.7776822382862256, 1.915507850169753, 1.385096553310754, 3.170546020417908, 2.9780655659352537, 1.6675166051079695, 0.6920753599395278, 3.341113784253716, 2.807329217728677, 2.892317744249413, 1.4165595321672209, 0.6993648849231413, 0.47011046665400386, 0.0, 0.0, 0.0, 0.16352329012375819, 0.0, 0.2937241767038936, 1.2461184999114943, 0.13705475042125015, 0.0, 0.0, 0.05532641628875086, 0.0628364320968993, 0.5453330462563988, 0.5410902478732643, 0.7703922909655, 1.5162787184717508, 1.1426595583855, 1.295251206444347, 1.182181618515263, 0.864199253560266, 1.3761307288178277, 1.9129870217829512, 1.3841305313085261, 0.9635711998287328, 0.9977868174625896, 1.2987111570417298, 0.9631449218894429, 1.4719085297793615, 1.2102993247225677, 1.6714383993798574, 0.6657765133260363, 0.0, 0.4811036273745233, 0.6466425799605312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19661898311234874, 1.2357216182083184, 0.5127024574917552, 0.3723390009625764, 0.0, 0.349165351615752, 1.2284128277036688, 1.0945722096924797, 0.9055552952722943, 2.0271407039372265, 1.5954490623045305, 0.2183550658686281, 0.6530352504317909, 0.5214030183636806, 1.1092996846518737, 0.9696025089311788, 0.39529264844475037, 1.420472103216097, 0.8658932976688442, 0.0, 0.05141434805826663, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.051563478651964645, 0.0, 0.0, 0.23490859159458102, 1.2469618650276673, 0.14251920251280178, 0.0, 0.0, 3.678241238032798, 1.2858444918251446, 1.1140910161184254, 0.7521456246025843, 1.6641758844099375, 0.8618524954231934, 1.3441308404166588, 0.6944793356484408, 1.1767556294219463, 0.9649609452826566, 0.27778438930571925, 0.8215339497077218, 1.3332515291376712, 0.790930736107979, 1.3811580742400489, 1.0004946478803418, 0.4762205736415604, 1.5214562810293564, 1.0325730808834828, 0.5168185278531033, 0.5655024765126355, 0.7563822181477067, 1.8653990701792615, 0.7780543382820856, 0.4272016304691012, 0.7109548556869949, 1.074688324288699, 0.7592402754509677, 0.6470850634285947, 0.8470322531843618, 1.52854455836496, 0.522953042444514, 0.49851352626785644, 1.5558384018304947, 0.0, 0.13180710402210571, 1.4031383437629865, 1.3833049478346133, 1.6162389073674621, 0.7283216633707877, 0.998338049829319, 0.0, 0.0, 0.0, 0.0, 2.81902978331465, 2.071833082261047, 1.4309390898448031, 1.6083043498572802, 1.545048983497614, 1.9672521532395917, 1.4224960019569541, 1.3519409555725719, 1.6232666917874607, 1.4686979069625081, 1.2350846528674804, 0.6194415346639048, 0.9623655077700446, 0.6906418218685421, 0.0, 0.8021680710521626, 1.312125390084756, 1.153587938256485, 1.1136695860781443, 0.2688072300333231, 1.0337950086646386, 1.1725326354821621, 2.7255836040561943, 2.9399606869542696, 0.6007561912821731, 0.6701228701741077, 1.7285560991197457, 1.214679946875278, 1.4357027064549535, 1.7902223540541822, 1.960844357235327, 2.0364018559577297, 1.227275775432579, 0.6859821431753149, 1.3512784506651077, 2.5754803333144527, 2.2922961806287243, 3.040891048612971, 3.468311200423045, 1.7584746873868713, 3.4377617110201295, 2.5808671565117733, 2.4340236638656574, 1.96026271061244, 1.4435136214196418, 0.768603804197688, 0.2756505283596304, 0.0, 0.0, 0.0, 0.25668356886373456, 1.0174202694742895, 0.0, 0.4220043768916169, 0.2105753812967031, 0.4385409967213491, 0.0, 0.0, 0.25771570413992034, 0.0, 0.11409724764055523, 1.312510467066298, 0.320975758128616, 0.7994963029854582, 1.4273361518804608, 0.9235313218666295, 0.35083607983113846, 1.568041569361007, 1.0947622366492178, 1.5183266602817478, 0.9834416982739669, 1.272360226232904, 0.933093210743191, 0.8607787289364528, 1.1938926323229369, 0.9802710661230948, 1.119473538712754, 0.8240630606667951, 0.0, 0.6427966462942496, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.468801218495704, 2.336684715426308, 0.2405248136128393, 0.8698940944906396, 0.5938554764996236, 0.0, 0.0438844159061706, 0.6428161959150591, 0.14807364329300077, 1.6021134544554232, 0.8710117594513526, 0.7126430858563985, 2.1754490368896575, 0.5774727996306402, 1.1207643321436644, 1.7661634758446505, 0.652734097681738, 1.1521228927617697, 0.6573420081927298, 0.6787914323631646, 0.15733151377246427, 0.0, 0.0, 0.0, 0.0, 1.4267855860360605, 0.9465714480276625, 0.0, 0.27778474372618256, 0.0, 0.09444968421779319, 0.5394102924451322, 0.5425843043124433, 0.0, 0.0, 0.05406144574492454, 3.501558599556567, 1.2908882748607116, 1.7917736607185775, 0.4931805723516831, 2.01123462870896, 2.1847448157226252, 1.0094321334621739, 0.36124806663682063, 1.3714606379951741, 0.23136744434178136, 1.442929517519956, 0.0, 0.23956114611171198, 0.9864518815304071, 1.239543245752445, 1.335154355157173, 0.6533182998166446, 1.2647352465877075, 0.19405484667965447, 0.5956690424017786, 0.8314393121721947, 0.5451174327337227, 0.9713993657332377, 0.5655879709481592, 0.48835488490307305, 0.1391915401113701, 1.0814803365706016, 0.6812133148152172, 1.0276922352789373, 1.0439567759577693, 1.8648436995068003, 1.0346004578369639, 0.8351869458809088, 1.3886035672859738, 0.0, 0.03697668412165937, 0.2719941369104271, 0.41480487624781537, 1.7996073942665818, 1.4539993779166531, 0.27028880239107034, 0.0, 0.0, 0.0, 2.139834434623752, 1.1227826532773904, 2.9913774388230645, 1.0214327409404347, 2.209803717917654, 1.2297183951799762, 0.9291150343205875, 1.774772912100748, 1.219466029546253, 1.7969275682022, 0.40817743122803957, 1.4752938821006523, 0.3253066132023742, 1.4169281157129632, 0.5084782985484615, 0.4186632872787097, 0.49536159232113874, 0.6357806224406317, 1.530898409876196, 0.7896157162403716, 0.7340137027759898, 0.6475176620350385, 1.4910895126411292, 2.9682165118684267, 2.782751291168423, 2.35005853245254, 1.498169689440845, 0.46655627765001134, 2.4433523542882183, 1.7373427802460348, 1.3803197217096563, 2.413681479109038, 2.127855213395655, 0.8208538356857774, 0.9824245822533292, 2.764107287719331, 2.8062577372445476, 3.2981963792130977, 3.5260731456972807, 3.1004673995256944, 3.4207236091790354, 3.7601264098089415, 1.6336067801123966, 3.1266089574300815, 0.77167540459594, 0.612523690039696, 0.6863539985683449, 0.26037622429593893, 0.5860831587289969, 0.1106949686204861, 0.0, 0.7360625397733295, 1.1513631159662236, 0.24042559879836098, 0.2745917890593014, 0.0, 0.3653436450157805, 0.0, 0.01381973177997226, 0.3205257169027943, 0.0, 0.0, 0.3840775223447021, 1.8389933626066657, 0.25515739251573416, 0.6028958209768102, 0.0, 0.8834787187145478, 0.6817488786726804, 1.2689201995600818, 1.914223064806404, 0.7418651807979671, 0.7050434571293804, 1.0216525117079942, 1.0612852801075217, 0.9132640318409171, 0.5480769844057987, 0.8006685335695297, 0.28901425048781293, 0.8880273029718769, 0.8694175276333147, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.400739583068651, 1.0025544948959264, 0.9597332961772922, 0.8108933284523003, 0.18879245598602118, 0.08587092524226583, 0.337932779863831, 0.0, 0.2999738636547845, 0.18290147750779887, 1.1526740650991172, 1.0036274283025903, 1.7434860630538853, 1.7598865583521888, 0.7235534134353384, 1.3301970800248581, 0.7673075969594945, 0.6756320894594344, 0.8062366598412499, 0.3330692918591136, 0.7489296180837226, 0.25927290221969895, 0.0, 0.6305503768247911, 0.507918026214434, 0.5305517785444765, 0.4488954305418813, 0.0696110567753029, 0.0, 0.41071809634218986, 1.4742625915649354, 0.5759802522424796, 0.16551540596821696, 0.0, 0.08508441449228883, 1.374019393906237, 1.4168206425110832, 1.1644720875603856, 1.854912047386009, 2.05016693437476, 2.39792397435755, 0.575795189413069, 0.8828533180421043, 0.7407880296100131, 1.177959826235191, 1.1013706872463545, 0.40525526201227285, 0.25810345431841764, 0.685160191634138, 1.4867035609979937, 1.40119237781737, 1.3890746282700077, 0.6407950736846554, 0.3151820995786974, 1.0074681214520178, 0.8607727104543529, 0.0647108130762497, 0.6528953952842118, 0.4489609330774263, 0.8053635326338316, 0.9891146353343564, 0.4746440648560135, 1.4308105595248546, 2.07521729152392, 0.7967998961074338, 0.7711722499588554, 0.7975475746708621, 1.389299677929065, 0.30447932211048434, 1.095798085098022, 0.0, 0.0, 0.0, 0.6835457914467055, 1.8386276360721328, 1.2989896586023428, 0.0, 0.0, 0.0, 1.5214576755197613, 1.5122370804237497, 2.4077896306818, 2.0107534441197314, 0.712129630218064, 1.181245925046986, 0.6236446444436646, 1.2592574259047096, 1.3804615656320949, 1.323759080073182, 0.9084098293298958, 0.7110184761999035, 1.0164828974340514, 2.059630542856721, 0.5527918990709035, 1.267711141762355, 0.5618764883617074, 0.6296668193384996, 1.1544493251896037, 1.1526777787039961, 1.1562279493146754, 1.0390790297717873, 1.7151173459057456, 1.5976234094806685, 3.0098234971192848, 4.034026948522566, 3.487759177843634, 1.7624946249593798, 1.232141999020723, 1.3202635818125674, 1.075177751033198, 0.2464663835180977, 1.9106089790318326, 1.9922137595982328, 1.6144536274211452, 0.38194240035529997, 2.482105896798203, 3.530794830209074, 4.222985578485296, 3.572762490882033, 2.0703733772913533, 3.6491352766552945, 1.7321429184084185, 2.1317154446300557, 1.1758018681546858, 1.5296454639596428, 1.3204359322125911, 0.9076664701427201, 0.35391578647858957, 0.9495709452658656, 0.7099530497722734, 1.0117670716353473, 0.30879624935347844, 0.16616689323232578, 1.0208913363083953, 0.0, 0.43345285726538024, 0.12545919103607553, 0.0, 0.0515228236964399, 0.5702032368269043, 0.1536612000425841, 0.23284507857929013, 0.11025583475704494, 0.5570889205367157, 0.9034190006194791, 0.8572086908083633, 0.0, 0.17357104820447677, 0.6097669080914137, 0.4870150474926787, 1.6449083946444938, 1.0506468742207098, 0.2061459966244977, 0.4408962971070764, 0.26511937110427247, 0.6766018054625892, 0.6216406980437418, 1.125857399975691, 1.1634799683063926, 1.5920944955144716, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6131305153779009, 2.484601371206748, 1.1408419361566928, 0.674065658778311, 1.4332192812848834, 0.38851570254044393, 0.48350414781102546, 0.2021323153896795, 0.0, 0.6042413622393128, 0.8493491715375564, 0.09910667584024699, 0.6690394122176367, 0.4822740688543589, 1.2244798239096495, 0.7625106489806704, 0.1451098824423853, 1.697279599072605, 1.0934340945278072, 0.45194204370764696, 0.9666636845005981, 1.060597814498655, 0.0, 0.0, 0.42107107715800424, 0.07893522536910155, 0.3085540292562272, 0.7581647029969975, 0.27157618541961437, 0.2365434906746118, 1.9806539004374308, 2.198037783227822, 0.3762960028300681, 0.0, 0.6203992312736973, 1.7012917804154435, 0.6678626687571061, 2.031703497929942, 1.7870100117325691, 1.5715885845063022, 1.4642394089103732, 1.677499215637376, 1.7747995487407429, 0.6639172302008866, 1.3131818584010317, 1.4763299352952588, 0.7396568467791136, 0.5451001722829213, 0.18686128319236273, 0.5247321372607273, 1.000221274004587, 1.3060005924290243, 0.5786336996264754, 0.9836084332537428, 0.7671129397890822, 0.280092702059694, 0.0, 0.4590072357394974, 0.9324569337934097, 0.7513684602489386, 1.484872358001993, 1.3270229751806317, 1.5121849579105207, 0.9445508778419739, 0.0, 0.5632746256744336, 1.1098480083757023, 0.38683211210897833, 1.1438573085797732, 1.6303386884166886, 0.16742689909022537, 0.0, 0.0, 0.17984226312698254, 2.3148423301266754, 0.3915029213116337, 0.0, 0.0, 0.0, 2.1314318068143874, 1.076963897293066, 2.424489387982047, 0.9955880114205385, 1.2065678869623895, 0.7422947783739512, 1.2996730836359227, 1.1413482584151906, 2.0816676365301725, 1.314965373458313, 1.4633041560611457, 0.6072147313492374, 1.0384398373983967, 0.653529186051567, 1.9035555857068271, 2.1166139052091375, 1.1178821231324763, 0.9692056765045668, 1.9021596612887537, 1.3442229447418779, 1.5346907817516837, 1.0296434853106537, 1.8410578220133482, 1.9930419390617546, 2.739902203192954, 3.5746362564419893, 2.8857909557502337, 1.4143969801742733, 0.07553192273606973, 0.3710945775547545, 1.2432347105046762, 1.1188550281235359, 1.087356344623608, 1.1753334395651378, 1.541657074764101, 1.7668869235195694, 2.3069091954481467, 3.2273767753985205, 3.6283495988222025, 2.303829870596823, 2.640770496456152, 1.9345618761201673, 1.2945815228953332, 2.0815517423791356, 1.3543905050352683, 1.7489874012528739, 1.0252981547322813, 0.7698596174992813, 1.714578792448264, 0.8687259762846616, 0.8292327141096417, 0.9385190759228568, 0.5093862368172026, 1.2370019523621238, 0.9363134122749586, 0.5384021752114135, 0.9483979879514938, 0.0, 0.0, 0.12242998291865587, 0.7534793881863894, 0.7882675789376712, 0.6371364227048072, 0.317952037231305, 1.145034646051904, 0.4802039783610614, 0.8140776067262687, 0.0, 0.28109283126505863, 0.0, 0.006626515633458128, 0.7353654424918983, 0.7805089873345334, 0.36899706798239745, 0.6518883842247157, 0.0, 0.5954484855651444, 0.6891309407249651, 0.3040143625023847, 1.110687452550954, 1.4118827807751326, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.756855207907007, 1.5149523750472058, 0.454067833836347, 0.8753868477299429, 0.9715030020537726, 1.338697637560291, 1.893621895437469, 0.8841643609723056, 0.12384964313740023, 0.0, 0.29522456506364053, 0.0, 0.000781681417422162, 0.153411652152973, 0.6023780183123617, 1.377798352121134, 0.0, 1.2212993924197149, 1.2800909134748586, 1.642221205113227, 1.2153350807310936, 0.8470200803980406, 0.6843950984858359, 0.2580164352931818, 0.1438437887042636, 0.6601931334359276, 1.2017467195786096, 0.06066897513267193, 1.010118352552257, 0.8041794314218618, 1.9315264869664568, 1.5894274066706673, 0.026494185524716787, 0.0, 2.1901287403846763, 0.0, 0.910498971087774, 2.8710458146067612, 2.392032726608009, 1.0279879633355047, 2.356355891459842, 1.6901382834357177, 0.9661704215237592, 1.7429223193406387, 0.8092541809183142, 1.7399969788661642, 1.201831237838821, 1.4906329902769295, 0.0, 0.9054258229463489, 0.612788339228582, 0.866438165688699, 0.5625345082165413, 0.9415849890060809, 0.8349154474021385, 0.34136628131785873, 0.0054805006316320974, 0.6766955476440552, 1.3790526694692107, 0.9701097754260846, 1.4483074769248854, 1.404557903552449, 0.5351339701139035, 1.4082062219438467, 0.8369830023104297, 0.2545521981704071, 0.292748716461875, 1.120553570286437, 1.298042025029252, 1.2909591567096703, 0.3902496549420082, 0.2659636133441834, 1.1933365526654829, 0.49976788064608335, 0.10496564461892798, 0.0, 0.0, 0.0, 1.7840372230260397, 0.04113165027508519, 1.9956746367015676, 1.1353197637716377, 1.493893146700634, 1.37075240768532, 0.3657716022401876, 2.275319256422619, 0.9876618260425234, 1.161616332398949, 2.218514542494572, 0.773219244963291, 0.43074546995742735, 1.6094571310876962, 1.2953943831243933, 1.8353311265769243, 1.8398454632540426, 2.3299009395111505, 2.299497467940636, 1.5582844603284343, 1.2503025345730703, 1.392349008514023, 1.478537532893542, 1.6774912768017884, 1.4635171653299188, 2.5863677837313985, 2.108997343932298, 2.0394153607396, 1.5336096544481397, 0.01063936441562018, 0.6040991571432899, 0.6958891284408582, 1.1702874413248936, 0.639528466039565, 1.247130187909812, 2.332715351301379, 1.1879790798926968, 2.0415732747788558, 2.7480210344267193, 2.6096201616639316, 1.5770270892175164, 1.7968340365786881, 1.400617209457533, 1.5273292359563495, 1.6861302056878384, 1.5223249518123594, 1.3357914979539116, 1.461270376944624, 0.34303143686756543, 1.3994012948875272, 1.9098398724917927, 1.0454458329204146, 0.9840842525638184, 1.3505380352863492, 0.7394737681206482, 0.7365690770941594, 0.21945733858255406, 0.189725607962832, 0.8972002429534794, 0.0, 0.0, 0.36377747452950626, 0.641489027686833, 1.5238945916341005, 0.7678473775136803, 0.6983725737348672, 0.6094030110823816, 1.0548937543707155, 0.36118296630746544, 0.47543881947684846, 0.0015875860577301583, 0.2843019709840006, 0.6820572398131622, 0.7155996620098112, 0.6780937401614523, 0.9154605218205202, 0.880884500955454, 0.3760576043602093, 0.9117662812128116, 1.1120535289588949, 1.5416308064397628, 0.005115787502018018, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3863306989516293, 1.8292857477595725, 0.0, 0.1223139038442404, 0.3486273125040805, 2.0899365954344695, 1.4265491743942365, 0.8397347408621586, 1.0656211914587295, 0.612505616445772, 0.22292512407153567, 0.0, 0.22552082792960051, 0.32731524187710914, 1.4795969250297851, 1.2075162294906265, 0.5348676743677647, 1.103080427755971, 0.694799105295408, 0.6833769552850197, 1.7958096621886137, 1.6688093442735998, 1.7083524890300785, 0.3753600339542078, 0.9200180845213705, 1.207293096661032, 0.7881652958418548, 1.6066625750181702, 1.1032363328168988, 1.544445559320193, 0.8369766261616164, 1.695246807343479, 0.06058960358391935, 1.3518019244007828, 0.47034583431165605, 0.0, 0.7997945936237373, 2.431015655022749, 0.9203005157566684, 1.9603644875273267, 2.2045488574353316, 2.1055982166487737, 0.7857342061694317, 1.4394519013940341, 0.20870819429947238, 1.3161398398003854, 1.0020064407249976, 1.120594966637656, 0.4906431127779386, 1.0073892354111202, 1.5124120788574715, 0.5239586994739345, 1.4922809229291534, 0.9156899834770053, 1.0325485429865302, 1.023251040775765, 0.47388238403790395, 0.297646412308637, 0.7097042164221378, 1.5804959695614322, 0.7928340038990139, 0.059902995843508784, 0.0, 1.4247871419299505, 0.6496434148534167, 0.6800103757903242, 1.0972720043348632, 1.144291959002864, 0.781439378942095, 0.9996572042946156, 1.9312776990822114, 0.7707122178891526, 2.4471118775449012, 0.9759371214785415, 0.0, 0.0, 0.0, 0.37558141215112795, 1.287462090735876, 1.1864808846129098, 1.397268433552835, 0.7897724161413815, 1.7863106340799062, 0.9342453902184327, 0.13201808699050172, 1.1983026792631497, 0.9223882115752758, 1.2681658184930873, 1.5645948228771724, 0.7874398725243019, 1.3140709748288866, 1.8047013429687284, 0.6564163776262619, 1.1195931762825249, 1.0413474489619485, 1.9347635535333987, 1.1523456447188127, 1.7458684822986397, 1.1924575415027179, 0.8696499004819862, 1.433494126397832, 1.0515825543658583, 1.2075193401299786, 2.1561290610862707, 1.4340420954597055, 1.0710265482886534, 1.007333874616088, 1.4106346810500288, 1.6653584182650596, 0.4711139476974504, 0.8285340359548896, 0.9975310557735508, 1.7201241626359636, 1.9739246718933672, 2.163870011286407, 1.7940215036518088, 1.2466550005708241, 0.9286062748611978, 2.8683998760919125, 1.985591752780725, 1.1333984382512634, 1.1141880193960445, 1.6313061475468922, 1.9203592389963922, 1.3618969322227727, 1.7835302333596879, 1.4681262270593582, 1.9027987813753815, 1.67176751025066, 1.4342895711619157, 1.4769645093633852, 0.35019434773750624, 0.08845814260932001, 0.5710139849275663, 0.0, 0.6871244298772281, 0.49632138858634406, 0.08536097098066733, 0.0, 0.8452619755614034, 1.1094846556238684, 0.18103916251129698, 0.31471842454801596, 1.0800121474167295, 1.4443555719332715, 0.30087836286679226, 0.4685952925499139, 1.1740383734113686, 0.6485581410252518, 0.31866001492407786, 0.8661138342583061, 1.0517151678267602, 0.781964603811422, 1.4182604762208797, 0.9478683530855766, 0.06650770343621665, 1.2630262943309711, 1.1976365281613297, 1.536291558466074, 0.7231868797150346, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27218997720330634, 0.0, 1.1255380277568676, 1.67918358394494, 0.8244688372142273, 1.2718662569520147, 1.154851172675209, 0.8924069375576572, 1.228637087234175, 0.5393628609476411, 1.3731543308234633, 0.4558816119505472, 0.37683686362555396, 0.5715876902948164, 1.4875658302582688, 0.9810940739686311, 0.978535222097697, 1.5352473496135217, 0.9957928955119412, 1.9062157395336463, 1.5182049657733958, 1.43901172328441, 1.5328445356423146, 2.059901332551186, 1.55929512099332, 1.799103356317791, 1.295889251236393, 1.0632467505820464, 1.3599954327204933, 1.6517059917961225, 1.2368910440594636, 1.5322654533964832, 2.27276978487703, 0.026226963158884495, 1.498797992475553, 0.6199824872999917, 0.0, 0.0, 0.5363791885689098, 2.541873775723849, 0.9534238121224686, 1.2773450099528445, 1.4827162411267865, 1.5789446720642797, 1.5187544408323597, 1.5279054877077074, 1.1883131085182193, 0.7628878440382028, 0.5323289124676325, 1.4090924150529425, 1.0527705517469408, 1.120316015521912, 1.1234009250170722, 0.11331983607262897, 0.9359238860113017, 0.28088307403882634, 2.0978454283208317, 1.057556737218212, 0.6287890071711301, 0.44368369808388103, 0.5483786741695236, 1.1751090913505746, 0.0, 0.0, 1.076936392741055, 0.6846866437660256, 1.4719221127850728, 0.17640872421657092, 0.3571751462189337, 0.4875683348458357, 0.0, 1.360955177987505, 3.011053080890138, 1.2598640498195939, 2.196323370556795, 1.8587615606126124, 0.0, 0.0, 0.0, 1.7441533442330197, 0.06637909191876232, 0.8644887785708192, 0.5882988737988107, 1.2199639276815175, 0.5252907868770228, 1.0600433278788897, 0.16225943586561548, 0.0, 0.6736531833504831, 0.6930052940870083, 0.7006481456667005, 1.0450510945117648, 0.8595559680980874, 1.3778437399229346, 1.8199498577433073, 1.5601076859554777, 0.7528026920918317, 1.4580096994755194, 1.361860706921233, 1.078283219796268, 0.8497586560562638, 1.4222958394491676, 0.7172211403596476, 1.41339432215242, 0.9693325421091942, 2.4977059193083826, 1.9767811903457277, 1.7631443093195966, 1.9684202172243388, 0.3902682155497166, 1.040792296489798, 0.7577828516324256, 1.5842060730655243, 1.4454717242896216, 1.8081736492598486, 1.350094255877721, 1.7158900667667039, 1.6198479168713595, 0.10338473284430567, 1.3288930311106895, 1.6999291356891129, 0.16936574032761878, 1.2209820821823598, 1.5514912001660779, 1.0578346973627397, 0.9119171578933457, 0.9769121170198061, 1.476229225970998, 1.4494892843236529, 1.6798957186536707, 1.7097361470089314, 1.194084934354079, 1.0999340942008338, 0.8014105612271555, 0.553748850888821, 0.0, 0.5869991487275971, 0.0, 0.8206913559451726, 1.2959980216900764, 0.2688975242464187, 0.27797214439081036, 0.3592842121171341, 0.8978117254484322, 0.6086316946712645, 1.0645543281825558, 0.84879024833289, 0.8699382822474185, 1.8097161352167457, 1.4891024290979513, 0.5970140597175159, 0.4978921508082628, 0.9238283133671259, 1.2809898270670461, 1.053680148644736, 1.0153025335035626, 1.6089205610987047, 1.4981199943415593, 1.3081724491114288, 0.3232058986536965, 0.456386578756494, 0.11561094625772782, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0017884874086911062, 0.0, 0.7876913607126099, 0.0, 2.2137013544655493, 2.370142399658549, 2.0592645708610666, 1.275926653969293, 0.5169682915522612, 0.9140733569347148, 0.3317955365341989, 1.9714320497226958, 1.6904768757476565, 1.249073386605033, 0.24748811495288772, 0.6596638459745511, 1.1831945364709047, 1.0899388717571774, 0.786378980297358, 1.2445391119949807, 1.11917188070141, 1.479517053006198, 2.127368303971604, 2.037500098523773, 2.089386560587514, 1.6119751999325556, 1.7090909892645396, 2.309100205049356, 1.2156475538141238, 0.5158427515703574, 0.7144109351845006, 1.3643691802341662, 0.7986000897647736, 1.773292747581952, 0.036023445837663245, 1.6524207636373938, 3.25539766203549, 0.0, 0.0, 0.0, 0.34415034942972783, 1.7720323836150143, 1.7636138903214122, 0.8651270248046184, 0.6057322085227099, 1.3884098444084856, 0.88809267477199, 0.896785090577283, 0.883753975595217, 1.280933287103869, 0.48974409938968416, 0.9435382464115353, 1.1624170587620521, 1.0939437970463823, 0.7389547618674657, 0.2671150146143629, 1.6969304008502095, 1.1969558667483255, 0.5122828084135898, 0.5808340697822889, 1.09727139915058, 1.2105176328614082, 1.0181913007512295, 0.46075766696346143, 0.48109684124270063, 0.6286942879903609, 0.17373111239452904, 0.643349353060703, 1.513308596792681, 1.478689335385072, 1.1677453747729098, 0.9049154274566683, 1.1109058246744308, 2.4651608964439378, 2.5307157386170704, 2.1061960944487823, 1.6855555590778086, 1.6437798920170446, 0.0, 0.0, 0.7278948372109892, 0.3226644411091093, 0.0, 1.3954076293162858, 0.7580926517503469, 0.08383649873400326, 0.09963309958215004, 1.0416241751754398, 0.5663474686489365, 0.0, 1.1289238839024236, 0.36200218998333844, 1.0874263386776377, 1.1827098882119, 0.9039045099447648, 0.633587118420811, 1.0303195924812725, 1.007224638477291, 0.27827413833761483, 0.6464385512181523, 0.837779294788256, 1.0435790647680707, 0.3307092303504541, 0.9328998367820871, 0.9887435013475558, 1.0163500663842417, 1.4030493150885919, 2.1900532049852717, 1.558430229826372, 0.9019717611268209, 0.8503483798004372, 1.319708913764002, 0.11136885280054161, 0.41972783644271006, 0.8693241213132742, 0.8104066464048781, 0.8062346068432323, 0.4566096539044711, 2.2193254746601654, 0.9392889181917414, 0.0, 0.6730524177477742, 0.046369811617794306, 0.37117433048486337, 0.7637780855645506, 1.1330809541422913, 0.5711997496807238, 0.962329986774481, 0.8651033705066503, 0.9975298709415561, 1.51384980777179, 0.6572190328826691, 0.7091050446488619, 1.1089375842575884, 2.026817921913447, 1.6915882194482368, 1.8383840711739294, 0.3351712431992585, 0.33040070046629416, 1.016926624058769, 0.8449421830173092, 1.89520205240512, 0.0, 0.22097551255219827, 0.26812928309406764, 0.49895188470550406, 1.011259004607554, 1.1396727795901116, 1.178385650999524, 0.5410087805296796, 1.383613970016015, 1.3591031782561798, 1.6685453402089923, 1.3359009027463276, 1.3179627408459869, 1.4419203754186405, 0.6104852509927481, 0.502351880500996, 1.54010401782319, 1.7665269683916067, 0.8429332134326688, 0.8031407283561282, 0.9904914435970109, 0.6920398421069383, 0.0, 0.0, 0.7590886016313899, 0.0, 0.0, 0.0, 0.013139695350622049, 0.01680267759700564, 0.6626671973591672, 0.7311632211593619, 1.465074788296942, 0.0, 1.4328235748664793, 1.479538348980228, 2.057800527418241, 1.098935137126916, 0.9908327733628215, 0.09191639275855412, 1.0029216279362667, 0.5370234146897359, 0.7203832931340063, 1.1836843746485297, 0.8527951033835213, 0.512965604896142, 0.9816241549058188, 0.5463775732959566, 0.34698974553378936, 1.0756386531586897, 2.189422112412001, 2.153556672917074, 0.5363556834408159, 0.7999021826282683, 0.17127593667748417, 1.6204684851438962, 1.6465944164195845, 2.537724406813111, 1.374507369110045, 0.9604704497856479, 1.1091537057647824, 1.370142442630666, 2.1336335371216175, 1.3867127467227807, 0.3157707113127902, 0.9789174400262578, 0.9370954963162731, 0.0, 0.0, 0.0, 0.0, 0.4915257190906181, 1.8285172121907725, 0.4645329330128249, 0.0, 0.6480758989301167, 1.1695373382031264, 1.356564186278341, 0.5647910228224698, 1.3848810248605392, 1.2192949798292114, 1.1004723506973337, 0.3995240167918095, 0.17113137453816407, 1.048081527145051, 0.9766336897080257, 1.2464352853277538, 1.4549441969767534, 1.3705517471185031, 1.4988273274403996, 0.43568429663428554, 0.9734513830029834, 0.17669975006895147, 0.6223044767142962, 0.8221155884075246, 0.9596181467745704, 0.6507761583488918, 0.5552456821138633, 1.3592322456618013, 2.0643057679650223, 0.0, 1.7204627814388396, 1.4732277104972253, 0.734926760922568, 1.5554430894938143, 1.913214590464063, 1.0043494397476762, 0.0, 0.0, 0.950151349536774, 2.047244687948573, 0.6115709207594557, 0.6959968496401915, 0.3148213007756603, 0.39427093500754196, 1.3406188701593584, 1.7271678054205295, 1.9461233745148634, 0.9477653018490926, 0.6608390359160545, 0.9575763703845488, 1.7219244227757105, 0.7676267429566396, 0.81494646732851, 2.4957178655060415, 0.5033676597326361, 2.222466941214602, 0.9710712422594845, 0.12045757333302037, 0.9251417839036712, 0.3474451910412071, 0.00923059445845935, 0.0, 0.0, 0.6593458081509543, 0.9874720533214528, 1.1593057000906821, 1.731078969376782, 1.5424227870475014, 0.5796886588077601, 0.7856227951995558, 0.37407391510125576, 0.2420784442393636, 0.4699703169259961, 0.0, 0.0, 0.14629324132511193, 0.4590168251766633, 1.6219795013094567, 0.03645774911315002, 0.32926140082823185, 0.9591763804681642, 0.7607677579983521, 0.30119990292186294, 0.22522149172769992, 1.2358437105846376, 1.3892874100397865, 1.296759436433063, 0.7032295798885615, 1.0038001114054236, 1.1171009788544761, 1.3015042999483049, 1.0178640785472866, 0.5610680913990153, 0.5874262515921208, 1.5097997865802948, 0.4210390470948892, 1.0764153665751186, 1.5188248014121672, 0.649881801074339, 0.1603707570479172, 1.2834640138652833, 0.0, 1.0138314982355603, 0.521622468971866, 0.05065824837556355, 0.07093567617357303, 0.42796808764905503, 1.5143641324965245, 0.7964362777939772, 1.2842950195610296, 1.3179467520684778, 0.874399305628624, 1.390597804095293, 0.9659241975383647, 1.3573587249989285, 0.7144660445681, 1.2842220486066276, 1.3342385029438313, 0.2000101819439818, 0.5766782965635479, 1.7063674245323837, 0.8823661130751714, 0.0, 0.0, 0.049157106672079155, 0.0476007784217814, 1.6289163532252515, 0.023229890443886052, 0.2478471052446083, 0.07923886147547582, 0.0, 0.028272912029107134, 0.5811726571255619, 0.913416698158244, 0.0, 2.0047623402567636, 1.2432048073796031, 1.5761885350450002, 1.095882554192822, 1.01715764336571, 1.0262179528176292, 0.4468409377820646, 0.1600997373251918, 1.2417339026202932, 1.0621582950821964, 0.1696741949557433, 1.3478513138024546, 0.6879750777049006, 0.21191793612954168, 1.2384533328583984, 1.4173270389642993, 2.4817583206029425, 0.731885408265935, 1.349040648068079, 1.0437782864023495, 1.1481477882516156, 1.2708880499021733, 1.7699639611112787, 1.5897561710692012, 1.0088210910356077, 1.3317350435113813, 1.1625655168240567, 0.18837051058260812, 0.9210283053224656, 0.36699818408131546, 0.27603524546116814, 2.1567291480396777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8927279799708773, 0.28302597435719445, 0.0, 0.9237347521612356, 0.6976157514112712, 1.0633419274699283, 1.2664818181802138, 0.07402173300783114, 1.2545879634050445, 0.621596607280702, 1.0490887098400807, 0.9762799437766878, 0.9801192818472567, 0.6605376540521563, 1.4706997869488168, 1.2374906827286745, 0.9044136377528716, 1.2956139858396278, 1.1619578966793542, 1.7673700898950142, 1.6413401644090293, 0.8865518576617507, 0.889173986488957, 1.238889182574859, 0.9200405973077892, 1.0267995449726421, 1.7911657047553786, 0.8199408795428011, 0.7250158556298646, 1.6023508293125914, 2.7525091294383475, 0.0, 0.0, 0.0, 1.6456100611762112, 0.23429345018638884, 1.3148682089087715, 2.068822335954955, 0.6573742341702499, 0.6667957394802998, 1.6672512460764821, 1.1273525229873973, 1.0620477004277058, 0.9548300208606811, 0.7400297199642261, 0.0054459568074651295, 0.3960148003681907, 0.4930222312684958, 0.7920632496288649, 0.8526891207779885, 1.4361660512853192, 1.26579166287501, 1.2119219772095138, 1.1826507521379823, 1.2806439316419744, 0.1698295803660074, 0.7014336323782012, 0.049292076956150836, 0.6863301195486281, 0.0, 0.0, 0.5358962361394917, 1.419713922025482, 1.5332099936462447, 1.1903162166610284, 0.5338074462160628, 0.03340791798516682, 0.18057916569575785, 0.4333758157393351, 0.7597393736075646, 0.6416869259543255, 0.21815744899150058, 1.3170286859149338, 0.2479679773899414, 0.0, 0.34784657072784964, 0.6830665239345663, 0.7050164472992383, 0.0, 0.05246373753685543, 0.19079364206680124, 1.3359956929993784, 1.5918894246945394, 1.1666063645202336, 0.47462013465035974, 1.1302302250586778, 1.2727941391802915, 1.2798277580033115, 1.1727934519090746, 0.12236946221409442, 0.11592255419456604, 1.1164216580576263, 2.1653759240910206, 1.2079014255438163, 1.0581920989748053, 0.1292936017140361, 1.0059223639588684, 0.0, 1.1270169424509755, 1.6493815153329523, 0.6417688948068405, 0.6104087562539838, 0.9012693941996806, 0.6629134739019241, 0.358907556614982, 0.5315779694187203, 0.16030560283246426, 0.518874465564483, 0.8594756229540432, 0.6016042028974744, 0.970344903270319, 1.2387990633835337, 0.024748031696564747, 0.6960924570074869, 0.5457860748207051, 0.0, 0.8089865970232721, 0.18106862112746497, 0.0, 0.0, 1.0600226641382107, 0.06251394267282831, 0.2543794502594894, 0.04056276119318965, 0.2929028853451015, 0.08649580769251725, 0.03958834488744201, 0.7678438963870307, 0.0, 0.3481000816922424, 0.35689557781816744, 0.9410877911105573, 1.5371077745006962, 2.291440315064637, 1.0122108969979586, 1.7422200266220345, 1.444112527056709, 0.8541934209485599, 1.375144318336405, 0.6444783655100836, 1.671570184209301, 1.4641846745109066, 0.6963645584413225, 0.618080435203597, 0.6556710052747925, 0.8567512734179368, 1.053680165703652, 2.4131115350175953, 1.2367425612889988, 1.2623045374112596, 1.9750613909083, 0.43435930043695614, 1.0670332541209124, 0.4647101454962727, 0.42225709173088827, 1.2437812983146754, 0.892117406129034, 0.23742025736856487, 1.17019047099171, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029054414160398202, 0.0, 0.0, 0.5533163852032297, 1.019205502645442, 0.3259518906566563, 0.8950566305116382, 1.628681759268266, 0.0, 0.9025119517529439, 0.641280325844414, 0.5954428232811027, 0.6507335231666147, 0.42749109516268224, 0.773008381247998, 1.0263482980881125, 1.4234610056678634, 1.0729007460292574, 1.211656175605732, 1.428874969616727, 0.9542348554202369, 1.0598929437028157, 1.7059586706700365, 0.8943212596042523, 1.381264942193102, 1.3756007257957643, 0.9567345449036958, 1.2341806195554756, 1.7683374083483951, 0.8971382447000135, 1.9119443842903567, 0.8209028060995347, 0.0, 0.9840981701167155, 0.782259464835221, 0.0, 0.0, 0.9081901442436631, 1.4913003147755266, 0.6204167148250562, 1.6430566440607521, 0.6628089705955893, 1.092928986128549, 0.9737300718838984, 1.258542823469924, 1.3949948478546041, 1.1398677791157243, 0.772642381751734, 0.2510953108531183, 0.0, 0.05390453071680945, 0.9727940146521182, 0.23252511288507174, 2.0786431166733093, 1.143297350168418, 0.6270227458930424, 0.6689724921195018, 0.30699916596639604, 0.28627205900959796, 0.7137892775513971, 1.1197069511296238, 0.7226976632727667, 0.5774752367145323, 0.0, 0.0, 0.06549432182184926, 0.3788842744035121, 1.6219076192240112, 0.0, 0.326249912405894, 1.174855875202351, 0.11860005471068537, 0.9725697571934022, 0.9653840946189307, 0.21658808452269476, 1.2985943141730651, 1.0015073409520234, 0.24934520488413087, 0.28141273056452837, 1.4021624980983183, 1.0841118242221863, 0.9803147872444384, 0.044441058350914574, 0.41921105105887607, 0.45546382646388944, 1.4653990747963987, 1.2587053281942757, 0.5340145169342874, 0.5084905567383823, 1.7513343096414034, 0.8966307604866433, 1.0296773779555337, 0.6777720921874063, 1.1048481065940825, 1.4854976709677994, 1.8343616138194905, 1.2766982098501456, 1.4570407572119681, 1.1394263120911616, 0.0, 0.6369477306519657, 0.0, 0.4680913722748598, 1.2035366390398696, 0.7349310701934642, 0.8868151327888351, 0.4731310145042425, 0.873070752580221, 0.18795303171119887, 0.29427684833587836, 0.6474370016513108, 0.1635679331430779, 0.008297251605752485, 0.2573412966122719, 0.0, 0.29496614472312044, 0.0, 0.031865699919479025, 0.06474390404670986, 0.03624805169086102, 0.19013338874950186, 0.0, 0.0, 0.7451966058895455, 0.016818710878893552, 0.8301978140604092, 0.05571955221550922, 0.8185826474416457, 0.4718983727092689, 0.7439634125227405, 1.5900254389068145, 0.9338361014211009, 0.058105339771526675, 0.7539428506730912, 0.22570230069598063, 1.6488876267202635, 1.2400300330284229, 1.0944765523114155, 1.378755842095338, 1.0095799917276544, 1.153135966701168, 0.6335829937677491, 1.4975522160499224, 0.901152094559562, 0.7397488425398224, 0.5289207550631297, 0.5809009594742228, 1.080101723439103, 1.5340989113333428, 0.8055014499877152, 0.26717146143759124, 1.2245334599005324, 1.473978278257296, 1.380200710489175, 1.352589352888457, 0.43004711365256004, 0.0, 0.3012624580743724, 1.1894929358959312, 0.3212750139886017, 1.1297706749364613, 0.09728843266220545, 0.0, 0.0, 0.0, 0.5733962664707601, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8879881234559897, 0.0, 0.6901283944791605, 0.7600168391389629, 0.0, 0.0, 1.1759972812922594, 1.8096717172005812, 1.3367725268514725, 0.6339819097038326, 0.40693973588972465, 1.115717010238765, 0.7211964917138753, 0.0, 0.8827096570643054, 0.7628843447755654, 0.6788111530806731, 1.644483585672689, 0.9696959654251335, 0.5922084995349332, 0.9425829023575021, 0.9818420728635728, 0.9674324467168306, 2.5033272126569743, 1.6240579507927713, 1.1349095575979389, 0.7016120342307681, 0.9812443923433197, 1.6469328288340879, 1.9420902776775555, 1.5682237878183112, 0.34734419290353213, 0.0, 2.022726825536956, 0.003563344885964012, 0.0, 0.0, 0.23559078140679507, 0.2358722738228844, 0.27668521327346823, 0.8358355635521219, 1.6424034192042996, 1.1531877365005139, 1.0747494034656584, 1.366527767518562, 0.5427615009975487, 0.16303392537003888, 0.3595552310301189, 0.0, 0.38393416186222634, 0.8590770269547795, 0.7275982440740039, 1.0815691319626743, 0.9103514854108468, 0.6867722284375958, 0.6909895076497669, 0.8443117461190012, 1.1452826887848733, 1.5003321745650104, 1.4215380376540536, 0.6357054466614698, 1.1539354932163945, 1.2346870775028593, 0.5534245156651033, 0.5538480436739313, 0.43477749727899306, 0.9188820882780696, 0.1564273216493925, 0.0, 0.11308238949386655, 0.045258709546325465, 0.0, 0.013480993290289216, 0.8742983839607514, 0.5220157490141697, 1.3316843216085805, 0.0, 0.4773109257139665, 1.2790558841808892, 1.5568328443094945, 1.022631276171741, 0.6581444448507682, 1.067405389256225, 1.0331488446819992, 1.270249345266241, 1.5743973500341975, 1.5346790488306878, 0.25282741093385497, 0.4133699918536791, 1.3839139843603636, 0.8573197679274724, 0.6468402796998637, 1.09863960865099, 1.415394868780542, 1.6907688975905268, 1.0109411080387332, 0.627835744833663, 1.0836659425670443, 0.6157040575330027, 0.6018061317239837, 0.0, 0.0, 0.6555022089580569, 0.5803354710736531, 1.1082959871116793, 1.6215265275666337, 0.8738326287156545, 0.2084492026753814, 0.18759521753115974, 0.3274065077108074, 0.27759830373223204, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.146543911423659, 0.6040559591118708, 1.2605641411132855, 0.7253846318177376, 0.0, 1.1208348116688154, 1.4373007821075507, 1.0881016458610306, 0.8970366965601995, 0.8001194963402063, 1.1893988212870261, 0.7211675373321476, 0.9496748795830329, 1.144581527587366, 0.9489731983696932, 0.4924429340468623, 2.4245352171619134, 0.8898679375351607, 0.0, 0.4661423303321428, 0.7292274840807272, 0.7942572891067119, 1.7871364387529811, 0.07314601528737517, 0.6847674468307133, 0.7380428128492779, 0.9494914673982933, 0.40103158043564735, 0.958313749297227, 0.0, 0.6235623066419241, 0.0, 0.0, 0.2063905115212505, 0.993506086818081, 0.8211512940623643, 0.5206331117269793, 0.24100719718473096, 0.7260845525453332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3682394327980278, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.091453975605628, 0.2826640275966773, 1.129558210208431, 0.0, 0.8138303576238327, 0.9333809291746923, 0.0, 0.2154447078624882, 0.2747382602582234, 1.0356036208896884, 1.2685726780806679, 1.408272184758924, 1.0135313884876, 1.015978777005678, 0.7084992092457163, 0.9414955983324761, 0.9050043976574944, 1.2410354905172123, 1.8159058400143684, 1.518033852086177, 1.3646415282550548, 0.8107381306823064, 1.2357638829513296, 1.5294798642470286, 1.0226348308492834, 1.2480837809057714, 0.6096377206066647, 1.1329697393872036, 0.10094821949129791, 0.5661801738955258, 1.0777444565016803, 0.4784212938265929, 0.37194056420524263, 2.250503837573294, 0.0, 0.0, 0.0, 0.0, 0.5063340331360844, 1.285649586332231, 0.9766219550913015, 2.011201111562148, 1.8340776947380162, 0.8960416552855923, 0.7233733422501428, 0.9355629961286273, 0.36633111471465984, 0.5300694740924548, 0.10437971954324615, 1.2306076527699419, 0.24475984658130195, 0.5585578919222959, 0.0, 0.24046570252659838, 0.7534171661558351, 0.9112586911704715, 1.4709287152445423, 1.2191611667242137, 1.4443824298669754, 0.8667484409328741, 0.9596592195388887, 1.159192008719804, 0.7554765270407419, 1.1124575868088569, 0.6370694579983167, 0.5000957675840277, 0.6818613320963205, 0.1993050738307751, 0.0, 0.0, 0.0, 0.5078551060759644, 1.134537638785117, 1.9646487360921594, 2.4923544659509855, 1.2279751484461774, 0.5643766574076868, 1.8543913299775725, 2.2514218850450805, 0.661376463884534, 0.7349833989112317, 0.6895320940387413, 1.3664186539055276, 1.445338340306151, 0.4651939429756655, 0.46927130820433527, 0.893759902432601, 0.3145789429797758, 0.12326673965501743, 0.8973018283896372, 1.227396420561994, 0.09652330694466521, 0.24136490086622064, 0.0, 0.3961535588423096, 1.0966891793769185, 1.158685476544849, 1.2283793571910295, 1.4796629798957903, 0.3399232986745345, 0.17002678457432355, 0.031554968590542146, 0.173738310385436, 1.0249843086992358, 0.9843084310902377, 1.0728807162960832, 0.6054476441169058, 0.4347096929538863, 0.7738837533491325, 0.7477874115147667, 0.2299426225030761, 0.42366837673917024, 0.742228273536088, 0.6335390688860617, 0.5905106158529573, 0.0, 0.0, 0.05718550862986796, 0.0, 0.0, 0.0, 0.0, 0.7581738767218761, 0.35202109530892217, 1.0871506575345335, 0.7767126333298531, 1.0007919040323283, 0.7393174954507293, 1.683377925667902, 1.7294253956451273, 1.1907345245011567, 1.4033615874068928, 0.7566382763442164, 0.9691785791934887, 0.6114194295115805, 0.7969295098742992, 0.09562916686007819, 0.9036935367570595, 0.32358081923818627, 1.3033968598131607, 0.6359598174731782, 0.07733006270486036, 0.09291411083134102, 0.9879163254813592, 0.3891321727201086, 0.4469061964537132, 1.288728210340365, 0.27865854265122303, 0.023654194019718106, 0.0, 0.0, 0.18737024015388717, 0.0, 0.7992261150961129, 0.759863264851669, 0.5282633218567968, 0.06443419696108703, 1.3942556721519654, 1.2377704421779459, 0.04488349989281711, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15051683094789609, 0.28572180866772434, 0.0, 0.0, 0.5757930577052134, 1.1261071767573305, 0.9938761012806637, 0.3760322315247905, 0.8064663365807477, 1.2794858723945106, 1.1596408527317354, 1.3298385825034873, 1.004052064240937, 0.9542802463474592, 1.7953310359016679, 1.2842607761961602, 1.406299774179645, 2.0754885831461207, 0.58847602241045, 1.1044627159837386, 1.2974198743410366, 2.188994178372613, 1.5251383693053524, 0.4250198370342115, 0.0, 1.605331603970861, 1.408650305571789, 0.839530634632343, 1.057367345717997, 0.915303217444938, 0.05234529202929921, 0.3712159041342671, 0.04392670217055237, 1.4869478073506777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7018458835865088, 0.9618865686014167, 1.3681918333785261, 0.974265275287393, 0.7486002068615353, 0.0, 1.249813248866118, 0.4395422237694948, 0.5476401544777962, 0.5535344497054967, 1.3588174753971998, 0.7953951515009036, 0.37857858594668825, 0.3084022291624726, 0.2148719999042076, 0.7664480074083994, 0.0, 1.0435787262671241, 2.1411981879716526, 1.5036240220109334, 1.4092720174253033, 1.5384013605731868, 1.5211120901964401, 1.0325703671682167, 0.43788797000590407, 0.35439809454506077, 1.0314152397499445, 1.0272521144959315, 0.0, 0.0, 0.22027352256234412, 0.0, 1.4963390698827936, 1.368013823692164, 2.43190246564711, 1.7123682141160927, 0.7836174640169346, 2.1772673138297973, 1.9547553617384767, 1.1194633431153165, 0.5176907333671835, 0.29396216937838643, 0.9379745536547024, 0.41059685251312894, 1.0614136333059856, 1.3839785576584809, 1.1041830957043404, 1.9004476860114266, 1.2480264960455907, 0.221620739501509, 0.6606180377457183, 1.5392015854569037, 0.7543697743801211, 0.8899304352248208, 0.0, 0.0, 0.05079053760498878, 0.41372596188619487, 0.6005200107774618, 0.46422961798897494, 0.5967758009069587, 0.0, 0.0, 1.5821196795151986, 1.0640874242172835, 1.206768673315328, 0.8953148845244191, 1.0890573635651724, 0.6179736936977657, 0.5768364272417453, 0.13024537898457283, 0.5052693321124557, 0.0029367699036190148, 0.6740401437998746, 0.0, 0.36903862280180516, 0.0, 0.10038044928929257, 0.0, 0.0, 0.0, 0.0, 0.12065807245092838, 0.38516833814736645, 1.2908991309725346, 0.8634092842605358, 0.5194720845824446, 0.6789355490805221, 1.514954023388122, 1.598592814552465, 1.004055880494536, 1.4968296650188977, 1.543567346906609, 1.5239632188304437, 0.680482956082346, 0.8312268836175185, 1.4116311522322258, 0.6988596730692085, 0.637307808969305, 0.45876622521050947, 0.8328901011483474, 0.0, 0.0, 0.5744924179020462, 1.8893311490502565, 1.2510286177100534, 0.8791553127659417, 0.17110610039552826, 0.1449038195212937, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2047815765549788, 1.1546643378003087, 1.5920104925179404, 0.0, 0.29485690191581115, 0.0, 0.0, 0.0, 0.018706066465746345, 0.0, 0.0, 0.0, 0.3717404496385755, 0.7846228859660279, 0.0, 0.0, 1.2627479992164696, 0.9594250981334017, 0.24732972027392536, 0.633782910742399, 0.21427876342048363, 0.4131250853517162, 0.5307766320945898, 0.8373410432072728, 0.4802307864204022, 0.9848710760282586, 1.1561733649593435, 1.2732704772346646, 1.545525045055302, 0.9809584897593162, 0.5679772177347909, 0.8726896549710718, 1.8414719803393347, 1.343153988624504, 0.8761096037685524, 2.067989185335553, 1.8924725169218144, 0.4248666826441658, 0.13120703295239025, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20038178038737836, 0.4499052139036486, 0.5332465876823757, 1.5771708188510802, 2.581273721323598, 0.3087026403556119, 0.0578406566903079, 0.0, 0.0, 0.0, 0.0, 0.34664150520671855, 1.506955759500564, 0.8482865067131655, 1.731423353415778, 1.3226251550874686, 1.444616342321703, 0.029041885119429445, 0.5763688155694829, 1.7259943560283486, 1.0376420695759343, 0.563312131502357, 0.5058532657782637, 0.6470338044497862, 0.8424065902484129, 0.5976368912771133, 1.5402995415902077, 1.083724676348277, 0.42213884695069503, 0.4466548725566294, 1.0437794411814285, 1.0016805305818397, 1.4334031347588416, 0.4327891915328098, 0.7886511140120345, 0.9479353991044898, 1.278001980210021, 1.6024871378634693, 1.1809928326434012, 1.2613293725540229, 0.04104798452721079, 0.002812941930503038, 0.4092345301750312, 1.5145103651773097, 1.9717319155987625, 2.3537238925061454, 1.0066770939808578, 1.6084528912233078, 1.1164202359296527, 1.109699034660481, 0.564751042238453, 0.8785132017694826, 0.033691662986660154, 0.7603261829363236, 1.5780788539017414, 1.7486743154383277, 2.3522032742052774, 1.4264500443811152, 0.45228847075518835, 0.0, 0.2256459888214421, 0.0, 0.5341154067962594, 0.5498291357764833, 0.8410004328271818, 1.1741337550546764, 0.48270826491214924, 0.0, 0.7628610333414497, 0.9051348713500698, 0.8540120321468875, 0.24168087694514592, 0.9201956465971354, 0.4009333606881633, 0.6522835879298781, 0.19925821121260684, 0.32923924596950693, 0.3989175247221617, 1.2049510353735449, 1.6485924203839057, 1.7240398783803805, 2.0280071266300768, 0.6189454755706623, 0.8669031973119898, 2.0580042278894104, 1.3671129475384296, 0.0, 0.000884334442070668, 0.0, 0.3273418235996545, 0.0, 0.0, 0.09946436365086862, 0.0, 0.15302025211320627, 0.2511492603224619, 0.7582910759109938, 0.21046353436864285, 1.2589215749129634, 0.519530515908969, 0.6352433486427472, 0.47987041358006083, 1.051479255351163, 0.5995289317989517, 1.3074981976433957, 1.3242800264037615, 1.5152347446973509, 1.8770942902979568, 0.47282337501477645, 0.7618495318455457, 0.9094749398191223, 0.39112756499875645, 0.8111998411043834, 0.24401472312750147, 1.1066136617947513, 0.03573482300762707, 0.8231816301560325, 0.6894297845688951, 1.209255745179778, 1.7927275032690901, 1.5480757707214192, 1.402616223266902, 0.2986661536649262, 0.0, 0.0, 0.2059402905240821, 0.08587936320476736, 0.6940505987493623, 0.8816870761409459, 0.4957045005708726, 0.0800650277763721, 0.2874601734288853, 0.0, 0.0, 0.0, 0.0, 0.031245782376184444, 0.0, 0.0, 0.0, 0.26306159432660303, 0.7961608767693342, 0.22504409624902924, 0.0, 0.6466931449657874, 0.18814575353973495, 0.11892452019200467, 0.5690262045997885, 0.30436698057025224, 0.6674075443530801, 0.4882171006914236, 1.128916236182116, 1.3204220760668641, 1.5929266677494218, 1.053339596368239, 1.4070462924381628, 1.3295392052115567, 0.4899414401652628, 0.8931873137071284, 1.0018894086240737, 0.7623270139524815, 0.38063451387141606, 0.3965462299383806, 0.7391304079006009, 0.7785177179970475, 0.796789741719732, 0.6951409078311491, 0.07797586462199174, 0.39642723034995736, 0.06676787768282202, 0.1635984794983313, 0.0, 0.0, 0.0, 0.334885991922161, 2.2213505579618524, 1.9686698303308827, 1.5722321012714393, 0.15309018832613755, 0.0, 0.0, 0.0026591832829913472, 0.0, 0.0, 0.0, 0.8503843042529415, 0.8080322278513041, 1.7544290479551596, 2.987832589467839, 1.8964423633024428, 0.27113796920839933, 1.8279488008516473, 0.8656286785108891, 0.7583093515898554, 1.8176315911779273, 0.7676004359631248, 0.0, 0.8589544048397751, 0.9713741170517826, 0.7769841301107109, 0.9326772292175299, 0.0, 0.3812456152685529, 0.0, 0.6691541170420182, 0.8550159847509224, 0.6216348422980422, 0.0, 0.36605293693979973, 1.6969155675620766, 1.6855234834745645, 2.098926487964053, 0.8796453067020277, 0.38080678246183325, 1.3144769299335204, 3.9525898150159033, 2.009633797040809, 1.946266841567964, 2.833597298061013, 2.037855138078099, 0.9956240493830741, 1.6434156389930794, 1.0302532633038144, 1.2429752403733998, 0.40171302171742723, 1.7498871779044303, 0.6241229910832858, 1.8997384566394668, 1.4391123654885734, 0.0, 0.5803133399556578, 0.07887896757194457, 0.6999916208274777, 0.05924667386528498, 0.5911058114075172, 0.38762193208879053, 0.559409069014839, 0.45459676378757474, 1.1010661042803207, 1.471889526325648, 1.049837512871532, 0.5812859468009176, 0.9315221252793261, 1.3686365242723566, 0.0, 0.5271437763053378, 0.3272835091660932, 0.0, 0.0, 1.1134747946206662, 1.3253474965093728, 1.8297503446582146, 1.416874352890301, 1.059454333201094, 1.6389248569003267, 0.7877295587678559, 0.648630830476168, 1.2968719439137122, 0.9146002120499763, 0.0, 0.5038093697918063, 0.969687120464403, 0.24237756086156703, 0.39929067022008274, 0.24233048845472385, 1.0000997658478472, 1.3885878654759893, 1.3246109403902937, 0.1603080877956669, 0.0, 0.24156932725618818, 0.0, 0.17999017413215604, 0.01373014294206166, 0.0007428888479582501, 0.3559753740455107, 0.0, 0.467848132343042, 1.226053149330659, 0.7158737387455346, 0.5242640977088643, 1.1945733780905152, 0.916121675000245, 1.0734773453729178, 0.7495921978418569, 1.404627321248354, 0.13882832072481585, 1.0663857081686168, 0.30476962250291556, 1.4398023475992263, 1.8996839096873315, 1.8270292826584225, 1.2339002046879202, 1.6291567172548431, 1.8280368616829152, 1.0904472621551207, 0.35748173901731856, 0.556017373285135, 0.07824562907805521, 0.06330366498259485, 0.010983559957280469, 0.3057636956122531, 0.0, 0.5291895047339445, 0.0, 0.0, 0.0, 0.3915816137092387, 0.0, 0.06976277253928309, 0.12635489088953974, 0.0, 0.6396225494764869, 0.6430543863960836, 1.6239992333870583, 0.0, 0.01715002542476423, 0.017186546654805252, 0.037976263915747634, 0.1341265029052096, 0.7212105431006213, 0.36782775925159644, 1.020134554318949, 0.511793007881021, 0.6355006822578518, 0.333506202245751, 1.0931911240152852, 0.9758546749166518, 2.1697340568215644, 1.1418645775337863, 0.5433664805174848, 0.4875460344501985, 0.7967988178139653, 0.07591529644057922, 0.1864416481295873, 0.5803932315484446, 0.9154136355832203, 0.6367763349923915, 0.22908625622605838, 0.0, 0.42740319462015847, 1.4781161080700298, 0.5517723458959591, 0.08419971719473987, 0.20896609959751386, 0.030032046388738545, 0.6352238281427208, 1.0252102092969775, 2.5930350885333997, 1.9796886539128522, 0.6534704522696017, 0.1095021994074066, 0.0, 0.21729524834370023, 0.7223818993483436, 0.776913248175062, 1.2065295711099762, 1.0531733031729547, 0.46035943252126665, 0.6026578370124804, 0.38885459519141824, 1.722742364349195, 0.9657430837580974, 1.385374857545947, 1.2153002499875334, 0.8848437524641621, 0.3650046941502236, 0.0, 0.6435255830107277, 0.8853346888087217, 0.0, 1.1950195316373073, 0.16272230760118622, 0.6155139092099303, 0.500395305422051, 1.0713310596740147, 2.290067629937503, 2.099513274842499, 2.2355444613131086, 2.080947631741835, 1.5245700088365266, 0.5348922195365147, 1.5771613031435994, 2.625579593199047, 0.6389622690041605, 0.6457344484269484, 0.8703367681062465, 1.235817871747365, 1.556576485201395, 1.8414592816610964, 1.1193190058964035, 0.4406223809282733, 0.0, 0.0, 1.7189473271071989, 1.5727874477854167, 3.4637427422248703, 2.5577560977669482, 1.2239412349241905, 1.1281574072659803, 0.9062257736382267, 1.144630038225458, 1.3367682528816889, 0.12702675740246536, 0.0012325007738755909, 0.8226455143259801, 0.9318833012189605, 1.4532323560984497, 1.2616907031567002, 0.21985523696957626, 1.077482229798953, 0.28354743756669687, 0.0, 0.8925730803598898, 1.589567546674235, 0.06304084496893035, 1.3704447774275748, 0.7204988950717697, 1.0798162837952523, 0.34733243812757725, 1.0655009388642327, 0.17337308306623542, 1.3924344074578756, 1.2643076077968345, 0.8597200404537645, 1.8397622044331563, 1.8595393203737076, 1.2367746956381764, 0.46062324681899464, 1.9515479584261939, 2.2949118599891194, 1.5145237141773786, 0.22306801572838378, 0.30727751774750506, 0.04051582127259748, 0.024899628215525226, 1.1980348977113777, 0.22150395243493323, 0.5992951307992682, 0.49089895504906833, 0.0, 0.0, 0.07957682040123876, 0.0, 0.0, 0.0, 0.08722977437345336, 0.5434082093906584, 0.9431745135450751, 0.3050230324706037, 1.042620527472403, 1.6453365398143573, 0.7058519906075624, 1.4088314676034752, 1.6474788912419893, 0.790680864877007, 0.0, 0.4778713638609806, 0.5307139081912112, 0.0755636882941688, 0.32723599103066003, 0.5268390641644796, 1.6278731113446716, 1.4315956600144775, 1.5315853260561227, 1.4057070276931145, 0.5505028482406968, 0.9538537764766959, 0.4586732659417352, 0.691188361289441, 1.5233580872546035, 0.6451668663492935, 1.677491110504555, 1.04192780374705, 0.05888601942054949, 0.21852201971812132, 0.0, 0.0, 0.0, 1.0294229057818411, 0.7543430482774048, 0.017695123181487565, 0.0250237551894229, 0.4764604383841721, 0.6798582134654445, 1.3000796714982807, 0.3329386455953249, 0.0, 0.0, 0.7857313965458783, 1.13795728028493, 1.0869736711243732, 1.3796501850406908, 0.5454041058253435, 0.5844947863875433, 1.5125972942107884, 0.0568058453357197, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3465535027525297, 0.5447179665657548, 0.725942833004875, 0.6400285556095161, 0.4224992906030399, 0.07940731307707363, 0.07356652947760968, 0.2998030606505749, 1.6855430460672307, 1.1695504095607232, 0.4900831197133614, 0.048433282534351696, 0.872092585743834, 1.1354918193963095, 0.30383725939304945, 1.4338411046314905, 1.5596165952251864, 2.265003148642927, 1.4858899700090842, 0.816172214355044, 0.36948844425449734, 1.2518366765382383, 0.6142206313954186, 0.8453428589041472, 1.5411597635483518, 0.9368101827432078, 0.8221627635133081, 1.8279476388494045, 0.4341661714979039, 0.7635999254500806, 0.050349680004269176, 0.23963437438951365, 0.0, 0.0, 0.0, 0.0, 0.1886847281234954, 1.3208941674667312, 1.3072089903127755, 1.5613412530747357, 2.080184536423551, 0.5664324626242075, 0.0, 0.9384218753247865, 1.2188937856204995, 1.7647256259510813, 0.9718721449103257, 1.8040945365842884, 2.3302540125113436, 2.0764815014439004, 2.2602795728396026, 2.090002917259576, 1.369093583354466, 1.3332060706906244, 1.55700896012194, 0.2600795529190503, 1.3827320240237069, 0.8999709877881051, 0.9539642913709472, 1.836981759899655, 1.128150349108405, 0.960107200483603, 0.6859143346666744, 0.9450852578277263, 1.8137056030627934, 1.9783403901623073, 2.4731257162640845, 1.4326486354372103, 0.8563478848219456, 1.4356976697302175, 1.4350942160571982, 1.0434014170507235, 0.5535086002620431, 1.616114220086434, 1.9848071140770045, 0.8647825566321746, 1.0538626540656546, 0.935991680756998, 0.9318263502948035, 0.6285977225297583, 0.0, 0.2045335413465804, 0.3132621527007318, 0.4420746988279963, 0.6398539098693243, 2.2623195866604413, 0.7690768188123472, 0.2705435277727215, 0.4581378975803645, 0.17615455309690353, 0.5210362029730108, 0.440461402838111, 0.5189837743158635, 0.7723727359593573, 0.3078946194838184, 0.23453741632782588, 1.8727443548965357, 1.754545677733344, 1.4689275576368708, 0.9125164205793066, 1.4681243169495168, 2.121775969054527, 2.1865444919394714, 0.34384297021017113, 0.46837347443897986, 1.6528796426687489, 0.8039431710969919, 0.0, 0.0, 0.3696409144878122, 0.6117049178889622, 0.38211434024755986, 0.0, 0.0, 0.0, 0.1655411703637904, 0.10118263197051725, 0.6327163892949124, 1.0740653682603847, 1.4118962076914747, 1.1892282406102597, 1.1066445682275434, 0.09236395322492578, 0.058299271930677125, 0.0, 1.0401204619424196, 0.8028334356044985, 1.0119581923878225, 1.490446253708191, 1.8870939753266898, 1.6528943544643215, 2.0598971016205554, 1.0539295658782877, 0.4724515422279568, 0.7827298342224854, 0.0, 0.6476281363128953, 1.4445328348668784, 1.5441132054474145, 2.075240836510498, 1.9230958168537615, 2.218416813062355, 0.36624617739609244, 0.23202607445618284, 1.0146846355036543, 0.5059342197734337, 0.0, 0.5689783855696946, 0.4386443049856559, 1.2886304154765504, 0.9363492536884876, 0.0, 0.9521713260199584, 1.077317878373711, 1.1365066778467343, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1354017862513999, 0.5023947385980396, 0.05794349281815974, 0.5338149600681037, 0.0, 0.12453545090138582, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014339855639529888, 0.32113200463147595, 0.11407124670350803, 0.8657124294961603, 0.3667216162117283, 0.702216323353037, 1.2641219852974097, 0.6140141237069141, 0.0, 0.9329762040632578, 1.4111120272230568, 2.581443282835879, 1.675113643262827, 2.2926937103659273, 1.8908933788855158, 1.095602324546742, 2.085471155498919, 2.116056565778792, 1.2007374215491247, 0.7447005147296558, 1.7744469719688194, 1.1180476611214512, 0.0, 1.5361415925443143, 2.4866267519551832, 1.4215516407889845, 1.2366485082077963, 1.2190799777656025, 0.6830016097294263, 1.3588317972432893, 0.3681590774291292, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7445273435893203, 1.3988166311838872, 1.9443334521019204, 1.9309613002340338, 2.265561727669502, 2.3902916859300993, 1.418550813153906, 0.6948984109816355, 0.9447081047158316, 0.8912943174696016, 1.0875276123818054, 0.9140131925740708, 1.3141583462214683, 2.50816048239443, 1.8767862247151055, 1.3701898953410072, 1.3936751202091848, 0.7133619589238284, 0.0, 0.027829898730533823, 0.8110302890642132, 1.7134104822644798, 1.8318499357929305, 0.4573289947335337, 0.4386889033492297, 1.2551390250608996, 2.649700852548358, 1.7032930824932593, 2.7157392042465376, 1.3067334928267311, 1.0663774995205586, 0.8213253670891106, 1.0229970077206376, 1.3199171847952236, 0.9331952721204627, 1.1028283082743917, 1.2231665742470526, 1.3470775011298946, 1.7807215770366969, 1.0791239838544742, 0.7251341266927859, 1.6546669031075427, 1.429279455894789, 1.4431155559527076, 0.6389504086039643, 0.7868554588110711, 0.8859591365315163, 0.7649536569922175, 0.6583083990382324, 1.0633317384024807, 0.1681857753121403, 0.8140578256377763, 0.0, 0.07711303923477442, 0.0, 0.37232456507346845, 0.8377503658000527, 1.32327987912265, 0.9212819441920663, 0.05094723410159901, 1.820989979774834, 1.4084823310860255, 0.14384455821825654, 0.0, 1.5007515116943246, 1.393081132726603, 0.41455606707632114, 1.9789375741754007, 1.5441819013688638, 1.7360610396729175, 1.2403611160900305, 1.3259636661458336, 0.82089174016694, 1.1003575711753948, 0.5847899317050609, 0.18299166473191197, 0.0, 0.0, 0.8097703693060871, 1.7653492329189782, 1.3332997390702428, 0.9862663072375493, 1.5278173597986076, 0.633240722333773, 0.7162489648120496, 1.2002812206026474, 1.989617069296541, 0.0, 0.0, 0.3443792271283209, 0.14364945201588633, 0.858178830696102, 1.7491445964090904, 1.3400091665283895, 0.12739530009644012, 0.051541765559229, 2.415339344926573, 2.453797226941519, 1.55828637016346, 0.8427004922378056, 0.05639774471003771, 0.15302888207045443, 0.3731853863088458, 0.4846455338336956, 0.03078693193555527, 0.0, 0.0, 0.1298354515012735, 0.0, 0.1256771104198485, 0.604239143420543, 1.9928259435162174, 1.5873775096678997, 1.6929002476805262, 1.9773317569997857, 1.5977496793283754, 0.42636313572590273, 0.16780607079154133, 1.4986192541222907, 1.7995326506664147, 0.0, 1.176362118821486, 1.6212883385485626, 1.4092799846056328, 0.9147314627035689, 1.1669438225795161, 0.0, 0.20975754555680232, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04040673350050533, 1.3183721763931195, 1.4765199557585795, 1.371563235755203, 0.6751720338847134, 0.9766372994476499, 1.0615381016428784, 1.2747312976830396, 1.4895382314679997, 1.5046461950272392, 0.37404256362003935, 1.5471441710545923, 0.7412774012088131, 0.8349306740342155, 1.6272528482438797, 0.7772905348937692, 1.0507508584175635, 2.2927626987465635, 1.7530889978213073, 1.896403824692756, 0.27296645107386663, 0.8225153013148232, 0.6552982244545307, 0.6827803415843878, 0.8577062147625677, 1.6512156563425657, 1.19206566171824, 1.335971065864621, 0.6119303528545592, 1.1911224871835482, 0.9616332769755451, 0.7116173574945566, 0.6174124313784382, 1.5422191074954528, 1.3685442190752986, 0.5536362226110444, 0.002087663095137645, 1.232678989088697, 1.3127544134460691, 1.9508921491207938, 1.2862825959677222, 0.9317398697771078, 1.2357649366224104, 1.4694487963353215, 0.47712851068132683, 0.7249483444412744, 0.012502391854755705, 0.16434825916945062, 1.3004875306283923, 1.8724968163956033, 2.43462269435448, 1.193854643204509, 0.4806304806394735, 1.5318453984298133, 1.5343138458984409, 1.1985634439988786, 2.2062123916132097, 1.8304821941322447, 1.915753806240079, 0.0, 0.7562653513378096, 1.385173075575207, 0.0, 0.0156236230756863, 0.4411176827959298, 1.0038934855401236, 1.6013091853274874, 1.2784529657623749, 1.3778904717446336, 1.4089732474841232, 0.4729585313007887, 1.1341916808783874, 1.7674733398510514, 2.246586190322873, 2.4164134756919715, 2.8776247860171833, 2.85023220660814, 1.4979865623964101, 1.2730644076487836, 1.564956960864195, 2.198238782297393, 1.7761552573164705, 0.8167527149860999, 0.8118562002511039, 0.41201063200827726, 0.2531856744258155, 0.2707553796284401, 0.0, 0.0, 0.0, 1.566017983704513, 0.10992184820105985, 1.5757395571611097, 1.2540795113863674, 0.2790633310387831, 0.08525221783247018, 0.8353331563167906, 0.7653346437030041, 0.6943571259730222, 0.0, 0.5168232428775609, 0.4686443115654858, 1.7558945651315272, 2.333036690080069, 1.3688491690855524, 0.17994548621072873, 1.1262965013038213, 1.5954508148642812, 1.5596852836621153, 0.3054513707812469, 0.3030814932844897, 0.5716931722147055, 0.7605852299468229, 0.5139248525699239, 0.5405771251378337, 0.871930990192001, 1.8315360474476619, 0.9353413090340695, 1.6608597421531295, 1.8682320189629298, 1.3792372744295343, 1.4097311668925394, 0.43908153879199, 1.4470506791881683, 2.3145421548847516, 2.6051754014510085, 1.4583395751031956, 2.3111584731830828, 2.2652946829743774, 0.0, 0.45723303473787325, 0.0, 0.0, 0.0, 0.3672368788377731, 0.0, 0.6426513435669855, 0.3654184989486547, 0.7168864170797535, 1.8262200077338853, 1.8872014737051162, 1.154492343866703, 0.9935064698198773, 0.7248203093256966, 0.006930176628214139, 1.104553386134294, 1.4857354396820608, 1.3186149422178126, 1.486111791170371, 3.337736114645691, 2.179294787871867, 1.8764688786916928, 1.3993531984816987, 1.2994012673248294, 1.3529474732343683, 1.6357625655947503, 0.9633133483364478, 0.5995078610464684, 0.22724451243169028, 0.0, 1.5702393598282403, 1.081374589658611, 0.7829399407376061, 0.8420890651278017, 1.1877926278030433, 0.8622672052283566, 1.3154453748520003, 0.031039897622673694, 0.27957696381489755, 1.2560905979822659, 1.9226469975927607, 1.4918370370213323, 0.9928145345248234, 0.8324431374019015, 0.9021673453550817, 0.34953447396834525, 0.03893091685462218, 0.8532997737337461, 0.9292844460718934, 0.22442369745381313, 1.434322404182113, 2.2776970354605792, 2.1897112464140718, 1.4210447477604087, 2.0164676543916187, 1.6569360828603223, 0.8463499996419197, 0.8507481305568816, 1.397837431197059, 1.511701209645635, 1.2852810054687893, 1.4376919241368642, 1.226249082973918, 0.4663469389596433, 0.011869759516553108, 0.19475665210954882, 0.10916568258930175, 0.5303839551516873, 1.2349853195473732, 0.9481999618792714, 0.950924784714703, 0.9887068540277627, 1.401304927656533, 0.3677074843390717, 0.4958713428705953, 0.627162384410008, 1.2278889191496236, 2.108371282997355, 2.3894996924334726, 1.92692614379672, 1.073769030107322, 1.0588854082176855, 0.27537162953046807, 0.0, 0.0, 0.5408530957235121, 2.245168125137415, 1.3469701501175266, 0.46922188500213313, 0.9790999092488286, 0.5218439903840587, 0.053695034864525745, 0.699418476394951, 0.2931093294719328, 1.264789158287953, 1.364723275268031, 1.7335832569216405, 1.5533680975911195, 1.1670553959163381, 1.4029673996972292, 0.6488736936364303, 1.3465455918359275, 1.7356303724402338, 2.548397403645017, 1.8101201717045319, 0.4006095431908588, 1.0627843341783316, 1.3571646081647197, 1.4943297988368078, 2.2363527364077087, 1.216151161700316, 0.884098585243657, 1.5391711712247624, 1.3414232538434192, 1.00874944188783, 1.9976681930720228, 2.5408675722792835, 0.2792950763014875, 0.5017909435480437, 1.6273938855191645, 1.5783397533943597, 1.3884338392829185, 1.261342522231398, 1.653979415002289, 1.8948051132911865, 1.732650900395377, 1.3994282752116245, 1.4941972035951077, 1.579935764724393, 2.5800440842834482, 1.8477914787236043, 1.8838394132780534, 2.9518858489162456, 2.4812678246303745, 1.2576612856155567, 0.9069349481416358, 0.22650515448176814, 0.33084817571145014, 1.2257655154139533, 1.220097186703417, 1.1286897817135213, 1.4492615865062306, 1.827681615161864, 2.2532427425843777, 0.8456668553309749, 0.06403638689518973, 0.0, 0.14829413740478806, 0.7946527739541884, 0.8550569375269983, 1.4099014096306763, 1.651010925008854, 1.0020984327534186, 1.263362396940395, 1.8202668784017795, 1.7303887155733502, 1.2218742858095615, 1.662998438660767, 1.8773365340213817, 2.4618340050106706, 1.3587359283004097, 1.0997612265237766, 0.12438357904471234, 1.1143424522154162, 2.114527889502577, 2.439452559822706, 1.9915495102960383, 2.7074274042259554, 1.5262077983314049, 0.4005734014400952, 1.911259154008269, 0.3677176293662978, 0.38204079918110867, 0.0007050612447587524, 0.6327805559291156, 1.2965819221280122, 0.04517847628059074, 0.0, 0.0, 0.9845005137321146, 0.9514406629976675, 2.2500618410039066, 1.7689522810042593, 1.6958625126214728, 1.3242754353446813, 1.7245573326329708, 1.0817707982631362, 1.2546644315278934, 1.049228031279096, 1.6280748713921465, 1.2769713214945577, 2.176103195168687, 1.6781114717834793, 1.1351581392896772, 1.4364702674798753, 0.42108824901971353, 0.9346423542022927, 1.3318350679378046, 0.4657705926685054, 0.0, 1.2361588159183703, 0.0, 1.4257280542773552, 0.9318859398761062, 1.6923747141995262, 2.5762147503695263, 1.8666397141310014, 2.609287832593384, 2.573154712932581, 2.7240894115415246, 2.244598872780001, 1.5252045588345917, 1.1396463132884327, 1.3713605881140176, 1.2028733817793074, 0.4557308819259392, 0.7293107740145978, 0.33618879074072205, 0.0, 0.5719513708763156, 1.1567772099081592, 1.509286100902559, 1.5204001884818155, 1.3789126424692963, 0.8461875804718818, 0.9627970100224228, 0.9363486746343821, 1.3014241136479894, 1.675436234084008, 2.0350843260161016, 1.4568142271430864, 0.16544933997727657, 0.3358552597323717, 1.4293785100941174, 0.4332515162324665, 0.0, 0.13024994921213817, 0.08721945869726884, 1.0396763761060104, 0.881765692884503, 2.293226307574514, 2.4475228973390557, 2.1440753934167773, 1.4551803195065203, 1.4014067301577915, 1.554515537191829, 1.5763090224276965, 2.373871264825892, 1.8750440813451577, 2.08020261493182, 1.5164310891527133, 0.7077564086391493, 0.024338919895782785, 0.0, 0.41447305610378515, 0.7712422215959925, 1.0853397005418541, 0.9258872743759395, 1.4145826225679863, 1.1228671501773229, 1.8602316019210345, 0.8287559572878244, 0.7090571272211112, 0.9055543015571897, 0.5398919902779967, 1.6575112705546868, 2.206002473343168, 0.9463594375592552, 1.2939991375118822, 1.130572388914227, 0.8424124829949183, 1.7590816449648816, 1.7092390084309748, 1.15679780584188, 2.318477989620578, 2.3434152662716317, 2.763765344759765, 3.1662145313127743, 2.2874064584373794, 2.5641834869995024, 1.0948805080058015, 0.3286195728016265, 0.2736573879337846, 1.973213017526029, 1.6366680292193205, 2.6519171734691223, 2.7189809394307285, 1.8957269347002912, 0.3620888050076982, 1.4086021866569662, 1.9971320181248844, 1.7147545742547305, 1.6600185515897508, 2.0797365919414177, 2.5069705578388315, 2.2565276265428067, 1.9591707207246947, 1.0886971321048202, 1.0061761703725862, 1.5794827540031844, 1.4227059432136975, 0.7137246467294439, 1.4503267663939576, 0.13889108792169982, 1.5168334343558159, 0.7942525087739111, 1.1166775913638458, 1.8951490760831942, 3.497443927020844, 2.745427039389376, 1.4419973052411688, 1.3556051856881828, 2.0460536112492202, 2.1990020874112948, 1.914303601090125, 1.2812583354045712, 3.4292776503788707, 2.1253935643200395, 1.9658372115422378, 2.2533402332893187, 2.75990160813235, 2.287962295811859, 1.512824853256879, 2.0778680006472254, 0.3748399312032458, 1.7703747177377525, 1.933433788684688, 1.8448220168746572, 1.6067242911361732, 0.46697974758183985, 0.3404539925197988, 0.30792720920284467, 0.767279534151461, 0.7805998267960591, 0.4851098102093203, 1.076855589378269, 2.3171466508123513, 1.66481892298344, 1.7748550405817816, 1.7595880134302861, 1.6880322269890293, 1.0835180758459186, 0.8524788275815244, 1.309742353279573, 1.2460644640954233, 0.8341713805728727, 1.4127342818356456, 0.734974531434901, 0.0, 0.07142654497535281, 0.6808134860105377, 1.032857536192733, 1.4463309158788726, 0.0, 0.5824771397969685, 0.18256248379008821, 1.1003431310658434, 1.4390219082121476, 1.8508789744681327, 2.248160961009959, 1.6255190495245908, 2.013305850898437, 2.064798403207777, 1.2017434773844158, 1.1553394255128082, 1.270173215985882, 1.7413234488965064, 1.230372589125006, 0.9752484906350258, 0.9054310515446894, 2.350254710684295, 2.535476488574424, 2.284519663274904, 0.2691358383035964, 1.6088688031850436, 0.33985477706776357, 2.285261410170312, 1.6951429759064334, 1.7816434878436167, 0.5720681162851765, 1.1142409634460502, 1.5774712451975925, 1.9819563158805862, 1.4883397793356508, 1.7096046423002014, 2.180715304670214, 1.1959570687398637, 1.4034517040674637, 1.9385040171221843, 1.619885886201656, 2.3156932687069163, 2.792882318547795, 2.4916280295158293, 2.8311994358481325, 1.761532529377049, 0.9613057992147113, 0.5774406755326877, 1.2507528402913943, 2.906611404537007, 2.154796738860685, 2.1623806767143376, 3.5720873054929028, 2.335623281014097, 0.8833756632211861, 0.9893166666889002, 0.6213506069622446, 0.5646077621198643, 0.0, 0.0, 0.8467160081305172, 2.352340718218008, 2.4165322234243085, 2.5934273279457267, 3.0307689822990365, 2.6587477043085412, 1.8588327423110667, 1.5523582070913908, 1.3799013023414586, 2.4103485508043736, 1.9418783925822807, 1.788687152750524, 1.1984529706370783, 2.249517275545287, 3.4387927010050747, 2.727221426990022, 3.3039054850602385, 2.279116114902552, 2.211874924082057, 1.7116089609748593, 0.6316572388277276, 0.7174302044841157, 1.1735939603754806, 1.246193946346017, 1.524293538513265, 2.0828747983966056, 1.794119117894598, 1.7471392011724927, 1.2430776157735042, 0.5807082829451126, 1.2690082248924894, 1.9241544944085665, 0.7558405693713872, 0.600313006385043, 1.428706756383772, 1.3426962284777797, 2.012059835652693, 1.6475827569039856, 0.5630708664416797, 1.5458435919392068, 2.255333439645045, 2.9995480670580026, 2.3591728295497894, 2.6761751068791466, 3.1803102456440713, 3.226385853699388, 1.6298109090926414, 1.6851837895958113, 1.9147250318213518, 2.135803405588737, 1.87333422388136, 1.5498170206339204, 1.746302666268729, 1.6068260338261773, 1.9121935050449717, 2.5464369796312942, 2.9114624250036383, 2.664062202647288, 3.31266723013318, 2.7893137316697154, 3.4515057388748565, 4.330283986243096, 3.4898354919338477, 1.7749052713207099, 3.4217841598776855, 2.822433011032846, 2.720897832740012, 2.3299212032578853, 3.1172043444958595, 3.962474567600486, 3.3415278753239583, 3.2290596427596143, 2.5286184764538318, 2.0068608061709545, 2.3367211937035286, 2.256271551477524, 1.9421724403708704, 1.5058887011470736, 1.7502395317958181, 1.5535437092049937, 1.2311876026730209, 1.4938243105146023, 2.0251812220352945, 1.6844042422816166, 2.134698800933152, 2.5990665289831925, 1.7186954092188598, 2.61732066095832, 2.8435963490070533, 2.8413387165361215, 2.6779872573305306, 1.9002072335928997, 1.512008243127349, 1.6069215958801528, 0.6601839449388635, 0.7548403673678024, 1.060006253905923, 3.0230981734001774, 2.08555915640432, 2.30465209447225, 0.9875272668925973, 0.9092174209468719, 1.2150613702525892, 1.5738582052382386, 2.3819832230676194, 2.4337407851787396, 1.6717243270038058, 1.8796069269438154, 2.034406863850733, 1.6768299767894423, 0.4937140037959043, 0.733492569276897, 1.6343998370787236, 1.8201148931074815, 1.8564201976904542, 1.3000840701365033, 2.051177944103323, 1.4790451515213934, 0.2694023771528755, 0.8287118574729776, 1.7444835788435442, 0.9986148448093165, 0.0, 1.2567005847225656, 0.8863708614917586, 2.013415267314017, 1.8793599312118219, 3.189398938812964, 2.9061650001220003, 3.0918401484272016, 3.913348434747404, 3.958329787067548, 3.6320422080110664, 3.137562197631199, 2.014876081284571, 2.1680260282257997, 1.625507895243142, 3.752888139078532, 3.577552025360616, 3.000310737156369, 3.4037810255999417, 2.6952624674126, 3.155352160065313, 3.2731769593797293, 3.4612390171887824, 3.138897360704607, 2.5465926855917167, 1.8627272701370656, 2.335910551503355, 2.870794609226389, 2.0988222862638826, 2.671730590221569, 2.586782431298756, 3.120975728364002, 2.5830249950267055, 1.9380274844284557, 2.7223123828470417, 2.9636171419332626, 3.034206900673803, 3.1930130756495916, 3.0695713043758843, 2.491212063585481, 2.7068274444066973, 2.3786488362193157, 1.773654842074942, 1.9682331206809396, 3.5135274408255444, 3.7994785958287793, 3.1990013691710977, 2.380270899085883, 0.8459993254146811, 0.06164253282280613, 2.619135315627806, 3.746834465503491, 3.7620058711708992, 3.69124597750645, 3.373424945765734, 3.5746794209675055, 4.56783770835575, 5.611924105188762, 5.228383410866624, 4.585839823589, 3.125736610414675, 3.5486172415212445, 3.652781809764067, 3.6089361132070525, 3.703858579350921, 3.2013860478319462, 3.1375257676148096, 3.9004649303947114, 3.5775509405019768, 3.7586346544704026, 4.154029586824216, 4.060170511304132, 3.8148941029117043, 3.849227179059672, 3.739589103349655, 3.157385014470428, 3.5971850556817837, 4.035931154264623, 4.080176315118378, 3.277418501743266, 2.912665320439773, 2.835260723884174, 1.4552240211126088, 1.2321780802471953, 2.4025118760053443, 3.3223423588327456, 4.313426124420406, 4.168254947205228, 3.307163197792846, 3.2998727513072144, 2.5741648244408517, 2.9143281114978294, 3.4343460826255603, 4.215298184115829, 3.5583907268432724, 3.4652541060066424, 3.9767085693505084, 3.872277939892478, 3.681693176275966, 3.343713698819906, 3.5107677954074434, 3.8455361645175477, 4.213959774362769, 3.480617536439951, 4.17244070395765, 3.549025025658771, 3.846674377852274, 3.6190654294717723, 3.786494354334384, 4.144873720562375, 4.672312333767789, 4.0041377106307205, 4.02366312903486, 4.888732056702993, 4.267364958134142, 4.251696360679086, 3.6906503851511943, 4.132758890289244, 3.4941830537864953, 4.933444194417486, 4.0713891130624935, 2.560744765994624, 3.7003000557145116, 5.081227081976463, 4.451361129388013, 3.7676300396736804, 3.7454873080290056, 3.5638280521679837, 3.1821107886821705, 2.734035673108387, 3.247451188094055, 4.550552634166409, 6.03058990588512, 4.844393145508394, 5.249854064402541, 4.587473716352716, 4.617792082708836, 4.40238198104583, 3.573876522344539, 3.0540037921366228, 3.3118225290563905, 4.126018851924808, 5.2445715009685765, 6.442111032735118, 5.2038735341790385, 4.670246977260597, 4.3275372264203185, 3.7797058917867306, 4.222389458301798, 3.0974627573089717, 3.1417350110875635, 4.572859515227764, 5.948738464415352, 6.326310079920441, 5.1442209187460355, 5.487905829068579, 4.925497963719448, 5.053369533529916, 5.819613060321185, 4.5319454020227905, 5.117389388252713, 6.124391431493118, 5.30318151664794, 4.863857710277861, 3.9137794502991063, 4.285410948443964, 4.31510156273736, 4.425661429528484, 5.5857807956134025, 5.0576648567687, 3.7470101235833644, 4.811896867038517, 5.355185745599882, 5.103264156745146, 4.3792379718222225, 4.865834629889745, 3.583042874679766, 2.4239015806457274, 1.761751657800803, 2.4972011639796703, 3.8410782239657664, 4.72488035272881, 4.59754447216242, 4.516691454788515, 4.80509133351533, 3.9169742627193602, 1.4807045644807912, 2.7163811125057484, 0.0, 0.13914241713996106, 0.5714993093164991, 1.1121638585327054, 1.8391766818321198, 1.6058472381873607, 0.4149321704496239, 0.00215949671349781, 0.47916874548247523, 0.7196810547945584, 0.6341062368104441, 1.0817910664281678, 0.708031143251519, 1.7860105074899721, 0.7649210774908417, 0.0, 0.0, 0.0, 0.0, 0.01336173204000367, 0.7092515018629102, 1.2415346669241836, 1.834920995711752, 1.1678475221516225, 1.9728673590837944, 1.0231105764695998, 0.0, 0.503004491666213, 0.8199187687219972, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10536036597308708, 0.6265235606909074, 0.0, 0.13281090744292867, 0.5679557281418406, 0.21623078776058605, 0.5935825183603944, 0.6545067578645504, 0.3495555096760835, 0.1914345469729532, 0.42765155699075064, 0.05141028009440298, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14480277765940835, 0.5736761215339912, 0.0, 0.03903608830927738, 0.0, 0.0, 0.3319108217125883, 0.9765529231094676, 0.5370853522773775, 0.8964545488231617, 0.39825875268935057, 0.6858101382875603, 0.12993330411963303, 0.49166243040793756, 0.15293884297549556, 0.11865064254735451, 0.8473681429350922, 1.6796152625565235, 1.318018238879152, 1.0577912322418186, 0.10241011804574521, 0.1592750445131937, 0.07188198503619934, 0.1232498316283511, 0.0, 0.008352395007991781, 0.039340907708185525, 0.13639971831397585, 0.13995224981421353, 0.0, 0.5667160107470269, 0.0, 0.0, 0.0, 0.0, 0.18960515227828717, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01175556645807408, 0.0, 0.0072976248545429945, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03429482075372291, 0.0, 0.5058968153924683, 0.2576032865713326, 0.0, 0.0, 0.1840999252820283, 1.7141596737246223, 0.6074516387571675, 0.0, 0.0, 0.0, 0.14568535600368387, 0.0, 0.0, 0.1653325379129345, 0.22588608827804343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00047556780621472, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09535989221937757, 0.05100400933396196, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27299935997885155, 0.5216298530041692, 0.0, 0.0, 0.0, 0.02441636797111569, 0.019365649979556743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.028097691215241107, 0.3182073563787799, 0.44884603776218895, 0.05316301808544849, 0.02921578785598171, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005961582613941191, 0.0, 0.31852217140319083, 0.0, 0.4607712114182909, 0.782747601724076, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029817648433175664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15815303290571692, 0.05023955282270606, 0.0558382419030315, 0.0, 0.30622187087427, 0.07409158314118888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21190774201529622, 0.6037972729673287, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17265726883797236, 0.0, 0.3803974428449959, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0030919176797677, 1.235873458984131, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001589174703945909, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.054327451081007345, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004561814452187731, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018694082709902537, 0.0903287908011037, 0.0, 0.08431926365956183, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00260312855021703, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21001715303156715, 0.551033235904358, 0.5329972035829557, 0.4186299308955438, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.36895456007706723, 0.399083118566914, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007640621428423448, 0.001541543144912399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11864603651350542, 0.0, 0.053723397055981996, 0.045424857495777, 0.3307317951778289, 0.0, 0.0008039464305584493, 0.08508952138197823, 0.17452031057327352, 0.41775774563957674, 0.6301756209575452, 0.9537471660469239, 1.1985217035837266, 1.485357830427711, 1.8789552702917116, 0.8728910511326302, 0.0, 0.04093246922561657, 0.00025135121706126867, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03122784081688185, 0.11411135452089989, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0012228303195576339, 0.0, 0.0, 0.0, 0.0, 0.0, 0.47869241721349565, 0.9242766181100582, 0.9234181517985843, 0.624758906255024, 0.25756629592881913, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2195807100836977, 0.5930232231246222, 1.0342084678443537, 1.2970644522735146, 1.3617766794962705, 1.3251638937462176, 1.7039380968866835, 2.144017312192318, 1.533513107934176, 1.8028117554963552, 0.9085947796738956, 0.18476314848783573, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0005266256251239506, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.47765243058865736, 1.6199805251954869, 2.194716337449034, 2.437238780446198, 2.2494966589011427, 2.393034347395006, 2.613317597916201, 2.5124101555631886, 2.5918403545096393, 2.860158889629887, 2.681929785381699, 1.113420517320679, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5526938353797335, 1.2056412187076646, 1.3903251240601098, 1.485945929025445, 1.6196524671742627, 1.6924653142583688, 1.6734714512429683, 1.7225485213241627, 1.880768872419541, 2.391625305333182, 1.8084530680323194, 0.11816917857887323, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0554954420667298, 1.049967276083243, 2.1165929218014883, 2.811981493847106, 3.1111396992888367, 2.722187088692821, 3.2061390423723863, 3.2310994230281467, 2.8270379013730955, 2.8407699357957132, 3.0084665208768286, 1.2742020870834372, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7853493040198312, 2.306884403693149, 3.565345502999345, 3.900777881525201, 3.8169671343515614, 3.785043812956641, 4.013109408938193, 4.093829065495083, 4.574614795871636, 4.606926051979189, 2.7231645234906474, 0.3205426100847436, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15711036882193477, 1.2253532389931578, 2.793600173381986, 3.067923891806762, 3.392309425164286, 3.420606175131731, 3.4443608949939253, 3.715632306515169, 3.676996196686563, 3.986296277656936, 3.485394189929908, 1.4484991789802801, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1714479773552982, 2.473078343634593, 4.021253423202667, 4.408526258373366, 4.604270169336376, 4.867655614433749, 4.872421807080512, 4.739933322337152, 5.07134646988252, 5.40873886387963, 2.6339746777501865, 0.008873290382283945, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.434850268658295, 5.307748993002158, 6.144483396478312, 5.892331055635728, 5.473701167338975, 5.380974344121945, 5.377955473955033, 5.47158120626988, 6.025619421908749, 3.6844104589024553, 0.2710476584814123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.087287203269461, 3.893890164427672, 5.5721431768388365, 5.4312944108357115, 5.19199215132952, 4.89072751882461, 4.815426623092673, 5.322730692399425, 5.852175820444376, 4.177634924643735, 0.8527302699725764, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.277213326310283, 6.175274876095672, 6.302735089030625, 6.4501539674051, 6.615947502579028, 5.999454715717191, 6.307390397525087, 6.702004420852075, 6.348553117048788, 2.6533782717494616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.028606893132999, 6.291387070918228, 7.088291767654635, 7.127846087040345, 6.94708679716725, 6.690098161904458, 6.773775273585368, 7.169747761984439, 6.723508771444334, 3.45791571101293, 0.03056585721111908, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1484575837103885, 5.908507338348536, 6.947221576080943, 6.840781588033759, 6.422647464014977, 6.343228675823361, 6.2734570252203685, 6.4465870867650175, 6.4162433060304425, 3.858388225989244, 0.5466536615734107, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0665445757321037, 5.06400541436296, 7.726606366063946, 8.02366248789747, 7.773449655003036, 7.463430703431526, 7.608346681090269, 7.62966169759693, 7.719546638380134, 5.962567879705072, 2.477024473874317, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8238810759428131, 4.641927468990899, 7.8356567440329075, 7.598763216495069, 7.485428856995182, 7.283074306713074, 7.181793533277889, 6.948460749482085, 7.2749235273717145, 6.423305257224464, 3.535161378751752, 1.247445841653973, 0.09276611476867964, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6953563272225255, 4.058234787881764, 6.800711536840016, 7.4197995765067475, 7.142504157877308, 7.18837823396352, 7.302435116376716, 7.48947337224414, 7.386671810925558, 7.013817641707718, 4.980493767796908, 1.5661282087441357, 0.1119616159844596, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017256625965434747, 2.08913588850362, 5.784041434485696, 7.571188131450266, 7.875088840769757, 7.696358840873804, 7.598261086594764, 7.906419841995448, 7.454247943160889, 7.43557517609672, 6.379715008262693, 3.671775452647871, 1.5013503804365536, 0.696947200153806, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5913859047941659, 5.002995903230991, 7.62511227465248, 8.596521516363763, 8.179854173533013, 8.038341676443846, 8.046418678952461, 8.189837511705491, 7.811263753593851, 6.214394153777975, 4.147319168796484, 2.6197373271317534, 0.8804952181098786, 0.14657567388129034, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09163337895188353, 1.245171483072884, 4.1621251537799315, 7.43851309216001, 8.846502537022618, 8.814493148169207, 8.684936875886947, 8.210950470211078, 7.9305728457550915, 7.555086594134936, 6.303414276279583, 4.316871108501277, 2.8862847181352773, 1.0824953618220217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9247338575157832, 3.0177808936598907, 5.774628451307815, 8.064998391289175, 9.186814445768404, 9.305322280812712, 8.422248585569937, 8.121224640114457, 7.88789685205522, 6.832491909007776, 5.312383063592593, 4.290628117319946, 2.7687658666225814, 1.0888172013733637, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7836741379212206, 2.591349890569279, 5.143126326703806, 7.307922659584787, 8.953006772241613, 9.452322975791281, 9.157607319713156, 8.839739452404611, 8.81503164663002, 7.207252232789062, 5.491584848735541, 4.441080648390272, 3.623369496432816, 2.0492281112209167, 0.005252348912504397, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.564203556539766, 2.4696259916520233, 4.763341279783853, 6.6410001552726445, 8.496532265005904, 9.478254921533901, 9.226923407168623, 9.029827099511479, 8.354755664119315, 7.344501504679053, 5.930141956032021, 4.691869974350188, 4.162649319834936, 2.5641593307765964, 0.893913104717603, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9042974302056352, 3.8299769592616926, 5.4213107517628885, 7.426978995589602, 9.228305781329636, 9.746505168963928, 8.980837607928216, 8.302930895584813, 7.559947224301856, 6.015600689481052, 4.915095221641795, 4.994283887162763, 4.07247748055633, 2.0824085838002144, 0.3370452238835077, 0.11570944822179317, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029308440286724653, 1.5169562672331522, 3.692195135698903, 4.926228399048813, 6.508431889873508, 8.643448498031631, 10.01853815077978, 9.831708508763604, 9.119374747093385, 7.943148563535892, 6.026767452688653, 4.755110828811105, 5.03321149580168, 5.137719998296812, 3.557664008127643, 1.6397634983266107, 0.7225790227491063, 0.39000975248859854, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04947677168252524, 1.348964317532159, 3.1984381175055874, 4.395025397005853, 5.117195393174645, 6.892340136439901, 9.344674215365199, 9.894641386837094, 9.453885122424273, 8.208248841205785, 5.886307300333024, 4.051574535638531, 4.361526270897938, 5.271425768674396, 4.717562279082369, 3.0451087802789014, 2.2453698589784468, 1.7961105475746364, 0.4252642839249889, 0.04337173850301671, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7863715461089482, 2.587384678392687, 3.8167624018490716, 4.0952467282141765, 5.774024583860654, 8.253729699532604, 9.425536591591918, 9.064772793303723, 8.158660317653366, 6.103262589657426, 4.398026829377126, 4.208756386098759, 5.5699355171042715, 5.790277423191107, 4.060412467939833, 3.2658842441495595, 3.18957940013731, 1.9401305855622761, 0.7802011499802739, 0.14003540150104934, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0727201446603847, 3.4495427864522474, 4.440687010575406, 4.0021410233824835, 4.256541746522648, 6.3251171087311535, 8.53042545115188, 8.899488528957145, 7.882646893916731, 6.292382798262173, 4.669095559938501, 4.272150604551947, 5.427526547277287, 5.986257139770946, 5.083635660710502, 4.347869184659955, 4.025723348460671, 3.2789375997866035, 1.974509519310281, 0.904237760114261, 0.583352632181526, 0.0, 0.11441823052660341, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.992096713049194, 3.261345697000465, 4.923159992661424, 4.210650771326309, 3.3476470930385047, 4.350983365032128, 6.804468746468554, 7.621047946946009, 6.789084319750507, 5.976329998976542, 4.518175915563486, 3.52700038494232, 4.601062729612961, 5.927083637347601, 5.206281725422669, 4.558240452461434, 4.475678336846799, 4.489538703442144, 3.1592738055000242, 1.971572710860538, 2.0184492435651538, 1.3287333039307154, 0.9416374091512367, 1.053479270757286, 0.8586390063336359, 0.7792548217042635, 0.08280425710872333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8977843057943601, 2.8754407215716893, 4.739534496600886, 4.26532795101771, 2.769645413102111, 3.3018952839184244, 5.176669376280039, 6.086038252013938, 5.272111017045884, 4.666874629655743, 4.1605039426095365, 3.7341861622438546, 4.381836405082266, 5.517810945727884, 5.529109086564156, 4.872892217835807, 4.742083834113332, 4.72496178522246, 3.8256981140199375, 2.878172332740982, 2.226462215525882, 2.0347639824127954, 2.1296896656503184, 1.6320740702185574, 1.7640956400063024, 1.6467617179201688, 1.4009454095927534, 0.8298684416480075, 0.24127377170344547, 0.0, 0.08458236854661738, 0.15236168685440651, 0.0, 0.1985120565884735, 0.28955519002737723, 0.018517925356434487, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1850360062002232, 3.1827569235653055, 4.987554175053947, 4.990425775458919, 3.4940771253305325, 3.0081910194567603, 4.255230154066559, 5.321990732860705, 4.731839657322137, 3.6800335989985604, 3.2428997062471834, 3.2202949248872037, 3.6309851107510487, 5.09917574767445, 5.507729588995203, 4.965541903626372, 4.696700201614297, 4.4525504928690385, 3.6726688057604107, 3.2275436018744155, 2.7268837872250344, 2.603857887806146, 2.5116475934290494, 2.4530164923826048, 2.6069813434493034, 2.277910882691299, 1.4819607061780178, 0.7793648975965413, 0.48063950109228853, 0.38591802505595046, 0.7069446830066304, 0.8578387858239785, 0.817027936609787, 1.111793346515069, 1.3324567315482152, 0.6266990081407704, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1328127825363614, 2.644378240299886, 4.419944016826327, 5.273414893661873, 3.992154289049461, 2.4926156628549947, 3.6560270366329095, 4.636097517282865, 3.8488134649988175, 2.504620331728835, 2.059375234105047, 2.1134846432454126, 2.0516135643342404, 3.208906640379811, 4.177329045890642, 4.3366005237907475, 4.434989495883345, 4.519872094167228, 4.281515029029215, 3.511261862471862, 3.384345733837914, 3.7074964114969244, 3.190912279826437, 3.2197297162787204, 3.4116494311849324, 3.3177718889058725, 2.6766067225145274, 1.787310619355845, 1.1956112982995248, 1.1100362672097235, 1.2418621989998877, 1.321667208282069, 1.4657601844839463, 1.6614537918693801, 2.142667651146794, 1.5571565854285452, 0.7716479266275826, 0.19710094993853836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04272869378078865, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1857203944000758, 2.3213672932159137, 4.061863579026787, 5.072069464884019, 4.060813216670648, 2.3526698643214217, 1.9046549503021026, 3.0735022194037818, 3.3526310362632303, 2.593312120783633, 2.0120290485976655, 1.8599868660343317, 2.280451150649583, 2.496879993859963, 2.972265363382897, 3.375588793746574, 3.4343600511425354, 3.523410571492777, 3.5729537468496697, 3.344205565491239, 3.2449521052355443, 3.328017057672261, 3.5986199879629384, 4.2939114880597735, 3.9373169177980993, 3.912202881722595, 3.450968386936714, 2.7639665837806255, 2.0686441779682374, 1.9367383598735421, 1.8893711728048665, 1.9121387195048014, 2.0536604594014953, 2.2359244955414814, 2.7558654007471763, 2.4344601626708444, 2.037815012702244, 1.435948856567667, 0.5366734344386883, 0.02116419505525122, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14913755774619347, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16215555905089693, 0.001311443926495541, 0.0, 1.0926932913740748, 3.1090713042640967, 4.715026710145111, 5.419953607485481, 4.7209651488954085, 2.8291459043754057, 2.4830258434049823, 3.1863164497467107, 3.0626569561403203, 2.356389599084343, 1.9061600032168666, 1.9739480388512607, 2.337674624653462, 2.1124808078949266, 2.578526501541534, 2.8899605858999515, 2.6738576719975478, 2.7353878179952207, 2.9023542844643893, 3.1705229590560844, 2.681458947053868, 2.5269875351331903, 2.508637766065153, 2.9119286281267778, 3.1176448954665155, 3.3497609609340575, 3.024681784618974, 2.8266528048303448, 2.538935627474659, 2.1719098109730495, 2.322396318451444, 2.4944529686731984, 2.6217087353304036, 2.8647682104160057, 3.08552980181523, 3.221543990686165, 2.4244942605001296, 1.518657803132894, 1.2738015592022798, 0.7311547089463818, 0.0, 0.2889216846809168, 0.7118472418705549, 0.6660363282852276, 0.7920819670267422, 0.0, 0.002319587770252791, 0.2500310622520127, 0.7903647527641696, 0.5348613800814722, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4436646049403634, 0.1976379367359647, 0.07568186681156341, 1.1376997985886086, 2.6371486020955013, 4.2049891199254565, 4.699717055681342, 4.836410743051694, 3.9207892834413833, 2.6764223709868187, 2.7974664156762437, 2.774067032553795, 2.261732107713606, 1.7549470445340523, 1.8648067652732807, 1.493761016472565, 1.5701701155086047, 2.1618566315452483, 2.5931935595208797, 2.52579853099923, 2.7043463842867133, 2.7347332016727353, 2.795925551501628, 2.411649800628483, 2.5400512075219166, 2.504721863843314, 2.51484799190905, 2.734552806184251, 2.8955363777370824, 3.0177675680077627, 2.986024239660589, 2.880639598336617, 2.429023913910229, 2.507590525132608, 2.323138139879264, 2.4058003218814368, 2.4867419327414964, 2.890385842074638, 3.2677888249399425, 3.02315872668428, 2.247627303825492, 1.5525923574587244, 1.3217450921021816, 0.6272717642024334, 0.2276492320365735, 0.4426525575443381, 0.6602406005724668, 1.014957317764333, 0.7229691782088702, 0.4238692619566454, 0.6726290569373834, 0.48855537526998777, 0.6416030540215146, 0.2674016159028824, 0.0, 0.4757817890693175, 0.8737835300154209, 0.0, 0.0, 0.4249110195769615, 1.2182357291988968, 1.6306268544723768, 1.1000052633776003, 1.1613270145882475, 2.4911609665553196, 3.716712115464837, 3.9760232860689104, 3.8834823698285286, 3.515722355517404, 3.1093556069070103, 3.064704876332728, 3.1303546117213483, 2.5230735344623056, 2.5887326552019623, 2.3991179117608596, 1.9577591043925453, 1.6239096519720708, 1.1972971733611204, 1.5002323875293526, 2.3141511178666176, 2.6251555534380118, 2.550484554516503, 2.453673572108781, 2.3739447283828063, 2.136969119172913, 2.025632724251637, 1.9603670228399948, 2.0128958552669607, 2.2421784278496544, 2.1953415781707473, 2.3218653919220302, 2.2915529041856235, 2.150916407069524, 2.350284677619647, 2.0063666975302596, 1.8532414102940442, 2.3079708292139545, 2.878096897307374, 3.3606074134653734, 3.4069223215089206, 3.0947547227852605, 2.7841701917047974, 2.1832423700506958, 1.9071030250544838, 1.254897628304735, 1.2450378811895795, 1.618168222442678, 1.618138659534072, 1.0075346543691643, 0.49273533837662437, 0.7650445542641074, 1.085253486855826, 1.1374552761695633, 1.0189703492800746, 0.9166905429825664, 0.849463558268329, 0.3013136415272151, 0.16994536122223655, 0.02494362112468965, 0.5270646314817347, 1.2939823831766415, 1.4434598941701298, 1.2426604979811813, 1.9157317266202456, 3.0966390027257855, 4.0420217604087485, 3.994815317548593, 3.571168781208477, 3.5010801399164344, 3.221109711023209, 2.6245044392725765, 2.7723299563268653, 2.6893026586574913, 2.316048830940261, 2.0885953940579176, 1.7980649041266872, 1.777651980931447, 1.7318007812203504, 1.9877691099044041, 2.1587692480592575, 2.0406926603522204, 2.0685046118002632, 2.3459744964314244, 2.600531050710458, 2.600637394214568, 2.201631689680682, 1.8858206131607342, 1.7145319489911932, 2.0803750078992747, 2.3712822571033594, 2.1144556916726294, 2.044405531702379, 2.2607560421088757, 2.158888078392276, 1.9434502812130745, 2.2925946680995857, 2.3085861476805065, 2.069979081365862, 2.5262506714864696, 2.552188981614261, 2.8678391799516305, 2.8663939659486384, 3.0780861138036943, 2.615733034978977, 2.009309810361447, 1.8512609340381339, 1.8103752462987714, 1.46686384360934, 1.4253503033619932, 1.2189143663585482, 0.9950329242747346, 1.5661682888559079, 1.3352897608270173, 1.2750198923725955, 1.4800926905850185, 1.078362028816167, 1.0066206044366448, 0.6457467585732759, 0.7005813796110248, 0.5332430632963034, 1.5098044433602018, 1.7633767756984664, 1.4293986881003773, 1.9049878500004964, 2.9748942978911015, 3.779471431455715, 3.767653716167806, 3.1596774128182075, 3.111892997545029, 2.6351927129255235, 2.374895871830832, 2.4097330367822183, 2.3783004224813387, 2.2318296461061164, 1.960337025586164, 1.5247985984975623, 1.6451835252633353, 1.9398767805857364, 2.392282073065667, 2.411721230034942, 2.018462746829465, 1.9005468748861494, 1.9394229893804962, 2.12358526674778, 1.5731500832617777, 1.3643165363510783, 1.5810560382428929, 1.9924335563237257, 1.983652493169284, 1.797851922409965, 1.8315898905849115, 2.066225747621882, 2.5682235405444023, 2.627031388803182, 2.530590611987359, 3.093000977894997, 2.870543009502857, 2.7270209822322173, 2.790952237060845, 2.631098379864853, 2.924459864233438, 3.0974186346721675, 2.7654132115984544, 3.3953845173501405, 2.891857925608231, 2.893492593931851, 2.6290124198966884, 1.8752707429857018, 1.7060871079309479, 1.537285386643021, 1.495448201308099, 1.2801628080441831, 1.3430606842553319, 1.5061358143528563, 1.2780207448359684, 1.2440314424149839, 1.0270521629561837, 0.3491159675293649, 0.7709148046423507, 1.3341102634247037, 1.8393000442051701, 1.6374512575529618, 1.4428719571703292, 1.4178272542599328, 2.2016964646010786, 2.935186205831112, 3.020024667310933, 2.9218229763446613, 2.5716449678091373, 2.4938689759852144, 2.420284725602467, 1.84553831216561, 1.8977266563585176, 1.8512052235440495, 2.2769417149440154, 1.9797186099639876, 1.771883408148067, 1.9222852704855207, 2.4555225125617577, 2.3961698074256583, 2.365360391000164, 2.2926112347368246, 2.1390834136931027, 2.207132314021314, 2.2329588666669635, 2.4226090913254703, 1.9587959281502623, 2.1008924414733907, 2.4417414439373775, 2.1049358373256704, 1.9329451127112038, 1.9661517804417104, 2.398923984257199, 2.411959785966275, 2.6746335318506334, 2.4793221411110857, 2.198226682004397, 2.326316409278409, 2.648665294628185, 2.536766837669109, 2.576493748091045, 3.2367039886651443, 3.172527098733086, 3.2338807259018907, 2.691403571749735, 2.6711545917318773, 2.5592291640516165, 2.0824858330747453, 1.871629612524394, 1.857636459606466, 1.8975113224972182, 1.9957778552893783, 2.3389730703696694, 2.3274016155567003, 2.0103978773304965, 1.5990088197868217, 1.3843762494034348, 0.9869665952039686, 1.2440528142776108, 1.6669789766989012, 1.9912304046289326, 2.3762903439183054, 2.482172378999214, 2.2011369800092604, 2.6458889115283135, 3.3558566045890865, 3.424513100426248, 3.246881767817183, 2.9793024039524183, 2.7598725609825094, 2.7287315898176367, 2.398170453555068, 2.0795484906620754, 2.149697406365315, 1.98587985970894, 1.5501254679688052, 1.467810700348894, 1.9164486911422502, 2.133035207267385, 2.2532813018357913, 2.2462975472180067, 1.706789800770629, 1.5632710759922877, 1.8254854544814934, 1.7231776795351759, 1.5614659031733937, 1.8446561338867415, 1.601915927021536, 1.986414297156176, 2.5887469391393947, 2.5256542468635006, 2.306755063632278, 1.9972839772288307, 2.102279184506115, 1.6197729572891013, 1.5973804719309586, 2.2255289299706673, 2.1687349259696305, 2.618828495353844, 2.7807895666827753, 2.4424709032201624, 2.5075197214147615, 2.8094333913266856, 2.8649383754679882, 2.689224862669173, 2.6090292735349023, 2.469413486716793, 2.788464752276052, 2.55134612443681, 2.3877447739491378, 2.4646400800659487, 2.0679392236766336, 1.923856845725803, 2.3371194590120603, 2.6154286142661975, 2.4975383145665404, 1.7118427980895732, 1.7391975535755437, 1.632050931868633, 1.9517169120565137, 1.8307227560463886, 2.214071007828271, 2.6322877192142937, 3.0754523945456915, 3.0460157774243584, 3.347833597621219, 3.678557167179964, 3.5910355710479527, 2.98079753491423, 2.9134002225653157, 2.677094688453975, 2.5235847244920886, 2.7868504875409172, 2.672185253934467, 2.9597216059965064, 2.7329609851959646, 2.3615600292903096, 2.767905636953276, 2.508167073458819, 2.385353221616834, 2.670032046418281, 2.7056681170488854, 2.4218719182357464, 2.2989920458149307, 2.0700790594547684, 1.9725948225728753, 2.3786818932569007, 2.485572826650235, 2.641079050703226, 2.7381864482594076, 2.7392075134635636, 2.5996517895539983, 2.320362874729631, 2.147692039446305, 2.314999990900262, 1.6783925855427821, 1.9760976633726297, 1.6267617813437394, 1.5408179062256884, 1.9168974533047876, 1.9108512356764615, 2.0357103215535597, 2.2054106247043817, 2.5934884945710897, 2.4801413325036616, 2.5735056483904106, 2.7648510540092532, 2.664168557429376, 2.119219686080574, 2.562270457309274, 2.6702393348308617, 2.4579649030042474, 1.711526977755162, 1.6669470191107199, 2.1441960485879217, 2.426766952102086, 1.9764068487132838, 1.5524920883222852, 1.753612824407151, 2.07176799659997, 2.584922244130068, 2.912389252767644, 2.879752320261014, 2.7497696999866172, 2.762250238126059, 2.9671086476496, 3.0810644842877415, 3.3762518979217715, 3.3185670084714713, 3.4515217758078847, 3.1948985244029258, 2.9237984331275766, 2.3940851181759695, 2.017963239406199, 2.2410103332098, 2.5377670086681094, 2.720057403782384, 2.6900530061040646, 2.688971932970168, 2.5780978122303244, 2.5396331783683115, 2.647239423378725, 2.3853524645143365, 1.9496550010946792, 1.8985574596538681, 1.9846951694071742, 2.03814172772443, 2.467110115986398, 2.7226001057921083, 2.478669578692239, 2.291492404545191, 2.2238682172420225, 2.1623574415833406, 1.8454866734525963, 1.9523270880589865, 1.9202998881802853, 2.066555357402515, 1.8777784417736467, 2.0857585299097687, 2.7778480557584135, 2.568388951089311, 2.278683896327832, 2.6680670811050953, 2.854012702595618, 2.8430639159215687, 2.7790855699913988, 2.9212218279900037, 3.0330490211208354, 2.9956404029627057, 2.957148517997893, 2.7752086996079424, 2.7962791628600705, 2.4029487816841475, 1.8812099595588243, 1.9453833809828893, 2.280865668802144, 2.013840957378854, 2.04518203044816, 2.1410970238512754, 2.618549117130448, 2.5410642288917105, 2.501181377103712, 2.6728697960748615, 2.9225166224380836, 3.040780513810306, 3.210702231181102, 2.946963459207572, 2.828329685879208, 3.1360490462149633, 3.0904149933457594, 3.054639186281059, 3.3074330225655957, 3.1730239273773115, 2.8743040191686555, 3.1483817648465102, 3.472140091141413, 3.418236682396976, 3.077764464856647, 3.156774899527444, 2.961358649259161, 2.5178762936699766, 2.307717275697644, 2.562131964214691, 2.7938407289352307, 2.880555176842006, 2.8546645796665433, 3.0110026489005475, 2.6452817610251897, 2.4559843264792476, 2.7228340889955467, 2.695937767659502, 2.6211758025770977, 2.802818427640948, 2.4606351793054904, 2.3836004787539005, 2.4736177964419146, 2.495344736442728, 2.380891549282277, 2.538890026276535, 2.616705696333614, 2.9044354609819627, 2.8113538798932574, 2.8838741393222915, 3.2117445828263524, 3.019354755866792, 3.0206044203957276, 2.438147916048436, 2.2064047391990154, 2.2483294198969763, 2.505135498921804, 2.937262512668615, 3.0790022941736277, 3.0818471081496663, 3.0374789776620994, 2.9935312318373075, 2.8818956042913895, 2.8892249211541055, 2.9165394887674343, 2.982570954748212, 3.05282777385483, 3.126034669406922, 2.854664186022847, 3.1117368022799963, 2.698447845394877, 2.6866222969817706, 3.164019323971144, 3.173861021530094, 3.3624576715014642, 3.7664827883079006, 3.271913797645929, 3.1380415229797665, 3.356298409122773, 3.442653459047825, 2.857702712742179, 2.887494476973914, 3.050415800705655, 2.9161550929074322, 3.019851357596438, 2.6478470309692, 2.389108505102433, 2.6915057799302144, 2.7617723882010408, 2.7362471062091833, 2.549018989263667, 2.7439091968439273, 3.2689635942428907, 3.440957767755637, 3.6937520562491857, 3.208918972605314, 3.1983926350649243, 3.6114035758578793, 3.520463035540099, 3.7344654547675886, 3.198818339788853, 2.994466200813491, 2.9919128741590644, 2.758408719621706, 2.8512933543632912, 2.467518849841937, 2.281280305430755, 2.791318529223331, 2.4774885063255283, 2.3729787956963464, 2.211147856368603, 2.447399137097408, 2.7739428776210433, 2.8980662792968674, 2.8695405860294856, 2.8802461042316936, 2.8426379688447336, 2.7697979036832803, 3.1034833045510823, 3.1524865409349734, 2.7847960845277684, 2.5130587573793113, 2.6301910347346196, 2.7218737452840607, 2.893318616354167, 2.880845679128505, 2.6932210420939837, 2.7688067084348296, 2.610967530400114, 2.83518786349023, 2.9697652956080582, 3.09556003004571, 3.2598258422443767, 3.274462755388363, 3.039497841781757, 3.0371596632919404, 3.2714159864136803, 3.391773786611074, 2.979109479937346, 3.0646772267414057, 2.724561245050684, 2.8557134532380655, 2.938126663526863, 2.963330300242301, 2.8641472160580297, 2.9008251844275383, 2.914611050045123, 3.071749759364558, 3.1652427404321397, 3.2114253001482207, 3.432570936615771, 3.3612237102409357, 3.5116479020806177, 3.2675770094162777, 3.225107938275837, 3.4275666405342564, 3.471555800556706, 3.357340940592473, 2.9273341398285586, 2.7763363817664577, 3.0656765597423776, 2.681416278451571, 2.7665514500085706, 2.8971808743968714, 3.0762007161959537, 3.063363254071489, 3.0809785036438018, 3.2168357430278136, 3.3092297137141884, 3.186531725152192, 2.882450403532139, 2.760965408824617, 2.977188041473516, 3.444964467818981, 3.8538176729011933, 3.6749471118064574, 3.6040004428744674, 3.529905182257891, 3.409738300861503, 3.7271918990716966, 3.4645176608489896, 3.320962639176228, 3.3631498793612504, 3.4889950379593437, 3.4670069214264214, 3.714935274495824, 3.931075060540721, 3.3624700527087605, 2.991024232604679, 3.093568910069397, 3.0065436157989667, 3.1587524739333697, 3.2573307415312684, 3.5624020800125824, 3.2378729415905663, 3.0726697733923727, 3.4498295342711867, 3.622254642718575, 3.5613960078922253, 3.718928801865444, 3.793023112814145, 3.4654967800115446, 3.527980341648169, 3.90989507281556, 4.102221238477568, 3.959431069002356, 3.9320478194932567, 4.0066681619193485, 3.8326192899035174, 3.974807070189482, 4.113460311013158, 3.8595741256270983, 3.911530363997317, 3.722506926640134, 3.714498206120785, 3.4769876897586234, 3.307562696217851, 3.471994583810661, 3.3938637411793344, 3.3976863566098108, 3.8885447731938174, 3.983073505599192, 3.863605830493509, 3.789876988994499, 3.842258271221112, 4.029835872158761, 4.062907975339885, 3.948827221814452, 4.315980949523237, 4.2660487366257485, 4.167685710749705, 3.981223654823439, 3.7185334873248985, 3.823944466112566, 3.516432060180244, 3.716187724608792, 3.7446895763276733, 3.5747758715383435, 4.03479352941855, 4.3455008998005615, 4.241772004893406, 3.6965160463856392, 3.478690263192018, 4.037851605543197, 3.8915883395270803, 4.22920664276123, 4.213310202804304, 4.003318701108314, 4.054905666688712, 3.9826593017869945, 3.477374748274103, 3.276634170288404, 3.4911239519335564, 3.6604262429522807, 3.535969773553126, 3.9896483529745255, 3.9879579568172465, 3.8162340452474295, 3.506353792302809, 3.277921363234645, 3.587357689924654, 3.9716736336824443, 3.821552169213904, 3.7686844409132974, 3.464198656718022, 3.1946124262073967, 3.3098931718754545, 3.6043740608867987, 4.068244168880929, 4.059454796198114, 4.088654364921515, 3.7591058826926993, 3.9328072727025294, 3.909368820682814, 3.887472272973355, 4.322332680897225, 4.042224292848877, 4.2498452382288265, 4.692232618560406, 4.515169877610497, 4.798698785580166, 4.56527442325958, 4.686640483636307, 4.698185750796565, 4.452950155705401, 4.1736828108320205, 3.9161085372544204, 4.1286227977010865, 4.364016982228214, 4.500530646770738, 4.196777075441831, 4.130927623609276, 3.709390850287474, 3.6631563738204864, 4.03294221825737, 3.9265490062630724, 3.8909873922774953, 4.126390850640989, 4.135639502153489, 3.963094939615633, 3.8380573533694156, 3.5294970798747465, 3.397427000367617, 3.3978655531046376, 3.479879826795797, 3.5360895853322734, 3.521341896107126, 3.400782746568902, 3.117260629993291, 3.2471713451803565, 3.453375468915472, 3.0964963419501954, 3.135290629024252, 3.2072011593326035, 3.454982491574162, 3.5084860790077226, 3.9617933240816776, 3.707674999695633, 3.4824548494403276, 3.683003853101561, 3.565789931328036, 3.1969227330656174, 3.2687022606444476, 3.359202058886189, 3.2910714710094195, 3.5901246426595987, 3.3567417671823603, 3.443540184854987, 3.500501409195355, 3.596111649686673, 3.979331758900177, 4.154315853444825, 4.136001649001567, 4.219421131946545, 3.9766952664407342, 4.301791245089147, 4.250393144469127, 4.036394046496332, 3.738881292871325, 3.922006417419988, 4.451388292560544, 4.763579361818909, 4.933486201324537, 5.1633325765479965, 4.972596158644303, 4.713395576350546, 4.763816434423434, 4.862049106675975, 5.445605543277484, 5.0752647435474945, 5.300771084855131, 5.630434393975417, 5.604919833217244, 5.4707884529561035, 5.635717604305516, 5.920018269130993, 6.472196626638729, 6.575304768479627, 5.975279931076737, 6.372723920580246, 6.935277587181952, 6.75655186387084, 6.513102477243765, 6.813831823305716, 7.006854278595765, 7.055283480595624, 7.222893313089049, 7.597890839144851, 7.3436905016558764, 7.33870059870349, 7.299293146098189, 7.220185162300007, 7.257009440929752, 7.543420862213672, 7.605568211281312, 7.540883585850899, 7.713875238505314, 8.011461345477022, 8.412029492359862, 8.58614839130597, 8.561222189413048, 8.32103727277032, 8.280494642719672, 8.505816617917061, 8.573345617574656, 8.329044025249896, 8.458542424530203, 8.797918506964423, 8.53134198666052, 8.573559404366954, 8.483895644174146, 8.196600899487828, 8.141876002272893, 7.885871114865354, 7.810281457535681, 7.804653087774234, 7.91359990868356, 8.143871683057135, 8.157709229017865, 8.115853549377052, 7.875458693117592, 7.6538293779492665, 7.370501440385002, 6.895667402277791, 6.796510483695193, 6.782735916012303, 7.57765565067939, 7.626986296351568, 8.157248443978409, 7.849227143545542, 7.623313065717561, 7.796634185782482, 8.102985838330023, 8.0847374677424, 7.9306513073004234, 7.943742588766064, 7.893141200255574, 7.640032769424392, 7.275599933522844, 7.663894932350145, 8.14448127493831, 8.304196655602295, 8.036749630627323, 7.917916087514518, 8.064348415225577, 8.342223683921974, 8.170645299433836, 8.134851098955851, 7.829822815410128, 8.064016019848477, 8.136443068507232, 7.996790233780783, 7.71037462158161, 7.507722653529035, 7.907324026124209, 7.84344279872477, 7.885963967127529, 8.113967549074351, 8.005461451330326, 8.113507400167704, 7.666766158533644, 7.697577616713862, 7.739762779834532, 7.862084740767887, 7.81528648225451, 7.568791994341813, 7.481939798542364, 7.410708363773908, 7.5982591196506455, 7.918360026009357, 7.784766436484177, 7.7173451295916395, 7.462435569220572, 7.687280393337702, 7.641161480286308, 7.540004653353409, 7.613230004946683, 7.553635999327797, 7.774528555781229, 8.102768899564111, 7.765908415049548, 7.564393078750655, 7.491993554383499, 7.528634900328806, 7.950326044726557, 7.690444176925193, 7.713215050580675, 7.572325658636759, 7.3062966597054, 7.441681620095272, 7.645435356602998, 7.864609533557815, 8.270986703118059, 8.19093344039983, 7.956809229183873, 8.265889834151784, 7.996941064676041, 7.903070980408466, 7.953008799665195, 7.983585459506946, 7.84256193517923, 7.93468753682792, 8.061483366070139, 8.026745919700412, 7.855628624536829, 7.79711946025906, 7.850157759870833, 7.658776525971277, 7.383943340689518, 7.570311139935204, 7.424914228695003, 7.321287602860921, 7.340502972100009, 7.276005464986639, 6.939673648934509, 6.900599137820399, 6.828598551764192, 7.115694056790084, 7.00021543474571, 6.91671895879285, 6.91430078347424, 6.8848035293326, 6.371619449470275, 6.369505232609931, 6.26402655447414, 6.043492819917911, 6.175766062233303, 5.806532150590822, 5.728165805591373, 5.926945818112534, 5.99207880653013, 5.805793369929123, 5.518510911253379, 5.244100084942843, 5.39527235092911, 5.096359249874075, 5.175991627113948, 4.9569964464822425, 4.540692851438524, 4.476568991874411, 4.234552795429674, 4.423876218569477, 4.200572335281259, 4.064779273030554, 4.17435599669309, 4.0489312837924, 3.8901812025191225, 3.6267398652556975, 3.0627271611629716, 3.192371425144812, 3.419965693458171, 3.0882582035601582, 2.8647302044243785, 2.9624667967222926, 2.7847662711490275, 2.553264374911525, 2.6223883810484425, 2.640486478346713, 2.573396031834608, 2.1067813323709585, 1.7294513700853866, 1.454900128142096, 1.729866345381547, 1.2284902776116502, 1.153896345852185, 1.2775590672904482, 0.9430287860211966, 0.937322522641701, 1.1864032472995798, 1.0022698925072768, 0.6070127951486647, 0.3639271008553483, 0.17406112629870696, 0.002388804854092098, 0.0, 0.0, 0.037553882763192756, 0.17565653170276213, 0.01643206919895648, 0.0, 0.0, 0.021188412902564343, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

convolution_2 = np.convolve(heights_sequence_2, heights_sequence_2)

In [ ]:
#@title Verification

# Calculate the 2-norm squared: ||f*f||_2^2
num_points = len(convolution_2)
x_points = np.linspace(-0.5, 0.5, num_points + 2)
x_intervals = np.diff(x_points) # Width of each interval
y_points = np.concatenate(([0], convolution_2, [0]))
l2_norm_squared = 0.0
for i in range(len(convolution_2) + 1):  # Iterate through intervals
  y1 = y_points[i]
  y2 = y_points[i+1]
  h = x_intervals[i]
  # Integral of (mx + c)^2 = h/3 * (y1^2 + y1*y2 + y2^2) where m = (y2-y1)/h, c = y1 - m*x1, interval is [x1, x2], y1 = mx1+c, y2=mx2+c
  interval_l2_squared = (h / 3) * (y1**2 + y1 * y2 + y2**2)
  l2_norm_squared += interval_l2_squared

# Calculate the 1-norm: ||f*f||_1
norm_1 = np.sum(np.abs(convolution_2)) / (len(convolution_2) + 1)

# Calculate the infinity-norm: ||f*f||_inf
norm_inf = np.max(np.abs(convolution_2))
C_lower_bound = l2_norm_squared / (norm_1 * norm_inf)

print(f"This step function shows that C2 >= {C_lower_bound}")

In [ ]:
#@title Visualization
import numpy as np
import matplotlib.pyplot as plt


def plot_step_function(step_heights_input: list[float], title=""):
  """Plots a step function with equally-spaced intervals on [-1/4,1/4]."""
  num_steps = len(step_heights_input)

  # Generate x values for plotting (need to plot steps properly).
  step_edges_plot = np.linspace(-0.25, 0.25, num_steps + 1)
  x_plot = np.array([])
  y_plot = np.array([])

  for i in range(num_steps):
    x_start = step_edges_plot[i]
    x_end = step_edges_plot[i+1]
    x_step_vals = np.linspace(x_start, x_end, 100)  # Points within each step.
    y_step_vals = np.full_like(x_step_vals, step_heights_input[i])
    x_plot = np.concatenate((x_plot, x_step_vals))
    y_plot = np.concatenate((y_plot, y_step_vals))

  # Plot the step function.
  plt.figure(figsize=(16, 10))
  plt.plot(x_plot, y_plot)
  plt.xlabel("x")
  plt.ylabel("f(x)")
  plt.title(title)
  plt.xlim([-0.3, 0.3])  # Adjust x-axis limits if needed.
  plt.ylim([-1, max(step_heights_input) * 1.2])  # Adjust y-axis limits.
  plt.grid(True)
  plt.step(step_edges_plot[:-1], step_heights_input, where='post', color='green', linewidth=2)  # Overlay with plt.step for clarity.
  plt.show()

plot_step_function(heights_sequence_2, title="AlphaEvolve's discovered step function")
plot_step_function(convolution_2, title="Autoconvolution of the discovered step function")